In [1]:
import sys
sys.path.append('../scripts')

import requests
import zipfile
import xml.etree.ElementTree as ET
import os
import pickle
import re
from utils import download, cleanCmpnyNm, cleanColNm, cleanSalesData, catOwn, catOwn2, \
getGrpCmpnyDict, cleanseTransWide, makeHeader
from selenium import webdriver
import chromedriver_autoinstaller
import numpy as np
import pandas as pd

from os import path
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import math

In [2]:
chromedriver_autoinstaller.install(path="../utils/")

'/Users/choidamian/Chaebol/utils/101/chromedriver'

In [8]:
def cleanColNm(colNm):
    colNm = colNm.str.replace(' ', '', regex=True)
    colNm = colNm.str.replace('㈜', '', regex=True)
    colNm = colNm.str.replace('\(주\)', '', regex=True)
    colNm = colNm.str.replace('\(유\)', '', regex=True)
    colNm = colNm.str.replace('주식회사', '', regex=True)
    colNm = colNm.str.replace('\(유\)', '', regex=True)
    colNm = colNm.str.replace('\(합\)', '', regex=True)
    colNm = colNm.str.replace('국내계열회사계', '국내계열사계', regex=True)
    colNm = colNm.str.replace('해외계열회사계', '해외계열사계', regex=True)
    colNm = colNm.str.replace('^해외계열사계$', '해외계열사계(매출액)', regex=True)
    colNm = colNm.str.replace('^국내계열사계$', '국내계열사계(매출액)', regex=True)
    colNm = colNm.str.replace('소속사', '', regex=True)
    colNm = colNm.str.replace('^\(매출액\)$', '해외계열사계(매출액)', regex=True)
    colNm = colNm.str.replace('^국내$', '국내매출액', regex=True)
    colNm = colNm.str.replace('^해외$', '해외매출액', regex=True)
    colNm = colNm.str.replace('국내매출액전체', '국내매출액', regex=True)
    colNm = colNm.str.replace('해외매출액전체', '해외매출액', regex=True)
    colNm = colNm.str.replace('\*주\d+', '', regex=True)
    colNm = colNm.str.replace('\(\*\d+\)', '', regex=True)
    colNm = colNm.str.replace('\*', '', regex=True)
    
    return colNm

def cleanCmpnyNm(df):
    df = df.replace(' ', '', regex=True)
    df = df.replace('합자회사', '', regex=True)
    df = df.replace('전문회사', '', regex=True)
    df = df.replace('유한회사', '', regex=True)
    df = df.replace('㈜', '', regex=True)
    df = df.replace('\(주\)', '', regex=True)
    df = df.replace('\(유\)', '', regex=True)
    df = df.replace('주식회사', '', regex=True)
    df = df.replace('\(유\)', '', regex=True)
    df = df.replace('\(합\)', '', regex=True)
    df = df.replace("\('18.10.10계열제외\)", '', regex=True)
    df = df.replace("\(2019.04.30흡수합병해산\)", '', regex=True)
    df = df.replace("\(2019.04.30해산\)", '', regex=True)
    df = df.replace("\(2020.03.31해산\)", '', regex=True)
    df = df.replace("\(2020.10.30해산\)", '', regex=True)
    df = df.replace('\(구,퍼니파우\)', '', regex=True)
    df = df.replace('\(구,넷마블블루\)', '', regex=True)
    df = df.replace('\(구,천백십일\)', '', regex=True)
    df = df.replace('\(舊[\uac00-\ud7a3]*\)', '', regex=True)
    df = df.replace('\(舊[\u3131-\u314e|\u314f-\u3163|\uac00-\ud7a3]*\)', '', regex=True)
    df = df.replace('\（舊[\u3131-\u314e|\u314f-\u3163|\uac00-\ud7a3]*\）', '', regex=True)
    df = df.replace('\(舊[\u3131-\u314e|\u314f-\u3163|\uac00-\ud7a3]*\）', '', regex=True)
    df = df.replace('\(구[\uac00-\ud7a3]*\)', '', regex=True)
    df = df.replace('\(주\d+\)', '', regex=True)
    df = df.replace('\*주\d+', '', regex=True)
    df = df.replace('\(\*\d+\)', '', regex=True)
    df = df.replace('\*', '', regex=True)

    return df

def convColNm(colNm):
    newColNm = []
    col_1 = None
    for colMul in list(colNm):
        for col in colMul:
            if (isinstance(col, float) and math.isnan(col)) or ('Unnamed:' in col):
                print(f"Null value found at {colMul}.")
                newColNm.append(colMul[-1])
                break
            col = re.sub(r"\s*", "", col)
            col = re.sub(r"\*", "", col)
            if ('매출회사' in col) or ('매도회사' in col) or ('매입회사' in col):
                newColNm.append('매출회사')
                col_1 = col
                break
            elif (col == '국내계열회사') or (col == '해외계열회사') or (col == '해외계열사회사') \
            or (col == '해외계열사') or (col == '매출액총계') or (col == '국내계열사계(매출액)') \
            or (col == '해외계열사계(매출액)') or (col == '계열회사') or (col == '매출액') \
            or (col == '국내계열사계') or (col == '해외계열사계') or (col == '해외계열회사계') \
            or (col == '기타'): # 유진기업(2020) - *기타: 매입자가 유진그룹 소속회사로 계열편입되기 전 직전사업연도 중 발생한 매출임.
                newColNm.append(colMul[-1])
                col_1 = col
                break
            elif (col == '금융회사') or (col == '비금융회사'):
                newColNm.append(colMul[-1])
                col_1 = col
                break
            elif (col == '(소속회사)'):
                newColNm.append('매출회사')
                break
            else:
                print(col)
    return newColNm

def cleanseTransWide(dfWide):

    # exclude a financial dummy variable for sales company
    dfWideT = dfWide.T
    dfWideV2 = dfWideT[~dfWideT[0].replace(' ', '', regex=True).isin(['금융회사', '비금융회사'])]
    dfWideV2 = dfWideV2.T

    # convert multiIndex column names to normal column names
    dfWideV2.columns = convColNm(dfWideV2.columns)

    # clean column names
    dfWideV2.columns = cleanColNm(dfWideV2.columns)
    
    # if there are multiple columns with the same name '매출회사' drop the others except the first one
    if isinstance(dfWideV2['매출회사'], pd.DataFrame):
        srSales = dfWideV2['매출회사'].iloc[:,0]
        dfWideV2 = dfWideV2.drop(columns=['매출회사'])
        dfWideV2['매출회사'] = srSales
    
    # exclude row that contains aggregate data
    dfWideV2 = dfWideV2[~dfWideV2['매출회사'].duplicated()]
    dfWideV2 = dfWideV2[dfWideV2['매출회사'] != '소계']
    dfWideV2 = dfWideV2[dfWideV2['매출회사'] != '합계']

    # exclude row that contains aggregate information
    dfWideV2T = dfWideV2.T.reset_index()
    dfWideV2T = dfWideV2T[~dfWideV2T['index'].isin(['소계', '합계', '계', '거래업체수', '해외계열사업체수'])]
    dfWideV3 = dfWideV2T.set_index('index').T

    # clean sales data
    dfWideV3 = cleanSalesData(dfWideV3)

    # clean company name
    cleanCmpnyNm(dfWideV3['매출회사'])
    dfWideV3 = dfWideV3[~dfWideV3['매출회사'].isna()]

    # split wide to rhs / lhs
    dfRhs = dfWideV3.loc[:, ['매출회사', '국내계열사계(매출액)', '해외계열사계(매출액)', '국내매출액', '해외매출액']]
    dfLhs = dfWideV3.drop(columns=['국내계열사계(매출액)', '해외계열사계(매출액)', '국내매출액', '해외매출액'])

    # convert wide to long
    dfLhs = dfLhs.set_index('매출회사').stack().reset_index()
    dfLhs.columns = ['매출회사', '매입회사', '매출액']

    # merge long with rhs
    dfMrgd = dfLhs.merge(dfRhs, on='매출회사', how='left')
    
    # clean company name once more
    dfMrgd.loc[:, '매출회사'] = cleanCmpnyNm(dfMrgd['매출회사'])
    dfMrgd.loc[:, '매입회사'] = cleanCmpnyNm(dfMrgd['매입회사'])

    return dfMrgd

def getGrpCmpnyDict(year):
    
    dfAppnGroupSttus = pd.read_excel(f'../data/grpSumry/appnGroupSttus/appnGroupSttus{year}.xlsx')
    dfAppnGroupSttus = dfAppnGroupSttus[dfAppnGroupSttus['smerNm'].str.len() <= 3]
    dfAppnGroupSttus = dfAppnGroupSttus.reset_index().drop(columns=['index'])
    dfAppnGroupSttus['repreCmpny'] = cleanCmpnyNm(dfAppnGroupSttus['repreCmpny'])
    
    dfAppnGroupSttus['repreCmpny'].replace('에스케이', 'SK', regex=True, inplace=True)
    dfAppnGroupSttus['repreCmpny'].replace('엘지', 'LG', regex=True, inplace=True)
    dfAppnGroupSttus['repreCmpny'].replace('지에스', 'GS', regex=True, inplace=True)
    dfAppnGroupSttus['repreCmpny'].replace('한진칼', '대한항공', regex=True, inplace=True)
    dfAppnGroupSttus['repreCmpny'].replace('씨제이', 'CJ', regex=True, inplace=True)
    dfAppnGroupSttus['repreCmpny'].replace('엘에스', 'LS', regex=True, inplace=True)
    dfAppnGroupSttus['repreCmpny'].replace('네이버', 'NAVER', regex=True, inplace=True)
    dfAppnGroupSttus['repreCmpny'].replace('에이치디씨', 'HDC', regex=True, inplace=True)
    dfAppnGroupSttus['repreCmpny'].replace('디엘', 'DL', regex=True, inplace=True)
    dfAppnGroupSttus['repreCmpny'].replace('에쓰-오일', 'S-Oil', regex=True, inplace=True)
    dfAppnGroupSttus['repreCmpny'].replace('오씨아이', 'OCI', regex=True, inplace=True)
    dfAppnGroupSttus['repreCmpny'].replace('한국투자금융지주', '한국금융지주', regex=True, inplace=True)
    dfAppnGroupSttus['repreCmpny'].replace('디비아이엔씨', 'DB', regex=True, inplace=True)
    dfAppnGroupSttus['repreCmpny'].replace('에이케이홀딩스', 'AK홀딩스', regex=True, inplace=True)
    dfAppnGroupSttus['repreCmpny'].replace('케이지케미칼', 'KG케미칼', regex=True, inplace=True)
    dfAppnGroupSttus['repreCmpny'].replace('현대해상화재보험', '현대해상', regex=True, inplace=True)
    
    dGrpCmpny = {}
    for i in range(len(dfAppnGroupSttus)):
        dGrpCmpny[dfAppnGroupSttus.loc[i, 'unityGrupNm']] = dfAppnGroupSttus.loc[i, 'repreCmpny']
    
    return dGrpCmpny

def makeHeader(df, firmNm, year):
    nHeaders = 0
    for data in df[0]:
        if ('매출회사' in data) or ('매도회사' in data) or ('매입회사' in data):
            nHeaders += 1
        else:
            break
    if nHeaders <= 0:
        if (firmNm == 'OCI' and year == 2020):
            nHeaders = 3
        elif (firmNm == 'OCI' and year == 2021):
            nHeaders = 2
        else:
            raise Exception("Sales company name column not found. Please check the original document.")
    df.columns = list(zip(*df.iloc[list(range(nHeaders)), :].values))
    df = df.drop(list(range(nHeaders)))
    df.reset_index(inplace=True)
    df.drop(columns=['index'], inplace=True)
    return df

In [6]:
class Api:
    
    def __init__(self, apiKey):
        
        self.apiKey = apiKey
        self.corpCode = None

    def getCorpCode(self, path):

        url = "https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key=" + self.apiKey
        download(url, path)

        with zipfile.ZipFile(path, 'r') as zipObj:
            fileList = zipObj.namelist()
            target = fileList[0]
        
        loc = os.path.normpath(path)
        loc = loc.split(os.sep)
        loc = '/'.join(loc[:-1])
        
        with zipfile.ZipFile(path, 'r') as zipRef:
            zipRef.extractall(loc)

        self.corpCode = loc + '/' + target
        print(f'Corporation Code File downloaded to {self.corpCode}')

    def findCorpCode(self, corpNm):
        
        if not self.corpCode:
            print('No corporation code file found. Downloading the code file...')
            self.getCorpCode('../data/raw/opendart/corpCode/CORPCODE.zip')

        xml = ET.parse(self.corpCode)
        root = xml.getroot()

        lists = root.findall('list')

        codeList = []
        for list in lists:
            if (list.find('corp_name').text == corpNm):
                codeList.append(list.find('corp_code').text)
                
        return codeList

    def findCorpCode(self, corpNm):
        
        if not self.corpCode:
            print('No corporation code file found. Downloading the code file...')
            self.getCorpCode('../data/raw/opendart/corpCode/CORPCODE.zip')

        xml = ET.parse(self.corpCode)
        root = xml.getroot()

        lists = root.findall('list')

        codeList = []
        for list in lists:
            if (list.find('corp_name').text == corpNm):
                codeList.append(list.find('corp_code').text)
        return codeList

    def findRceptNum(self, corpNm, year):

        codeList = self.findCorpCode(corpNm)
        pblntfTy = 'j'
        pblntfDetailTy = "J004"
        pgCnt = "50"
        bgnDe = str(year) + '0501'
        endDe = str(year+1) + '0430'
        if (corpNm in ['아모레퍼시픽그룹'] and year == 2020) or (corpNm in ['아이엠엠인베스트먼트'] and year == 2020):
            bgnDe = str(year) + '0501'
            endDe = str(year+1) + '0830'

        # find the firm that submitted reports to FTC whose name is <firmname>
        for corpCode in codeList:
            url = "https://opendart.fss.or.kr/api/list.json?corp_code=" + corpCode + "&crtfc_key=" + self.apiKey \
            + "&bgn_de=" + bgnDe + "&end_de=" + endDe + "&last_reprt_at=Y" + "&pblntf_ty=" + pblntfTy + \
            "&pblntf_detail_ty=" + pblntfDetailTy + "&page_count=" + pgCnt
            res = requests.get(url)
            content = res.json()
            if 'list' in content:
                break
        
        else:
            raise ValueError(f"{corpNm}: no report found for {bgnDe} ~ {endDe}")


        # find all the reports whose names contain the string '대규모기업집단현황공시[연1회공시및1/4분기용' and save as a list
        # and return rcept_no of the last report in the list
        receptNums = [report['rcept_no'] for report in content['list'] if ('대규모기업집단현황공시[연1회공시및1/4분기용(대표회사)]' \
                        in report['report_nm'])]
        
        if (corpNm in ['아모레퍼시픽그룹', '아이엠엠인베스트먼트'] and year == 2021) \
        or (corpNm in ['교보생명보험'] and year == 2019) \
        or (corpNm in ['한국앤컴퍼니'] and year == 2020) \
        or (corpNm in ['아모레퍼시픽그룹'] and year == 2019) \
        :
            receptNums = [receptNums[0]] # select the last report received
        elif (corpNm in ['아모레퍼시픽그룹', '아이엠엠인베스트먼트'] and year == 2020):
            receptNums = [receptNums[1]] # select the first report received
        
        assert len(receptNums) >= 1, f'{corpNm}: no 대규모기업집단현황공시[연1회공시및1/4분기용(대표회사)] found for {bgnDe} ~ {endDe}'
        assert len(receptNums) <= 1, f'{corpNm}: Too many 대규모기업집단현황공시[연1회공시및1/4분기용(대표회사)] report found'

        return receptNums[0]

    def getSharesUrl(self, corpNm, year):

        rceptNum = self.findRceptNum(corpNm, year)
        driver = webdriver.Chrome()
        driver.implicitly_wait(3)
        url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=" + rceptNum
        driver.get(url)

        # In the navigation menu click the link that contains the string '소유지분현황'
        driver.find_element_by_partial_link_text('소유지분현황').click()

        # get the url of the '소유지분현황' page and return it
        sharesURL = driver.find_element_by_id('ifrm').get_attribute('src')
        
        # quit
        driver.quit()

        return sharesURL

    def getSharesTable(self, corpNm, year):
        
        # get shares url
        url = self.getSharesUrl(corpNm, year)
        
        # parse html tables which includes string data '동일인'(the same person in Korean) with thousands delimiter ',' 
        tables = pd.read_html(url, match='동일인', thousands=',')
        
        # convert column names
        for table in tables:
            if len(table.columns) != 12:
                print(table.head())
                
            if len(table.columns) == 11 and corpNm == '미래에셋캐피탈' and year == 2021:
                table.insert(4, '금융회사', np.nan)
            elif len(table.columns) == 10 and corpNm == '미래에셋캐피탈' and year == 2021:
                table.insert(2, '동일인과의관계', '동일인측')
                table.insert(4, '동일인과의관계3', '계열회사(국내+해외)')
            elif len(table.columns) == 9 and corpNm == '미래에셋캐피탈' and year == 2021:
                table.insert(4, '금융회사', np.nan)
                table.insert(8, '우선주주식수', np.nan)
                table.insert(9, '우선주지분율', np.nan)
                
            table.columns = ['금융회사', '소속회사명', '동일인과의관계', '동일인과의관계2', '동일인과의관계3', '성명', \
                             '보통주주식수', '보통주지분율', '우선주주식수', '우선주지분율', '합계주식수', '합계지분율']
            
        # concat tables
        df = pd.concat(tables)
        df.reset_index(inplace=True)
        df.drop(columns=['index'], inplace=True)
        df = df[df['소속회사명'] != '소속회사명']
        
        # data cleaning
        clean(df['소속회사명'])
        clean(df['성명'])
        df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']] = df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']].replace(' ', '', regex=True)
        df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']] = df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']].replace('⑧', '', regex=True)
        df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']] = df[['동일인과의관계', '동일인과의관계2', '동일인과의관계3']].replace('⑨', '', regex=True)
        df['합계주식수'].replace('^-$', '', regex=True, inplace=True)
        df['합계주식수'] = df['합계주식수'].copy().replace('', np.nan,regex=True)
        df['합계주식수'] = df['합계주식수'].copy().astype(float)

        return df
    
    def getSharesTableAll(self, year):
        
        dGrpCmpny = getGrpCmpnyDict(year)
        
        for grp, cmpny in dGrpCmpny.items():
            dfShares = api.getSharesTable(cmpny, year)
            # save the data in the after_cleansing folder
            if not os.path.exists(f'../data/ownership-status/raw/{year}'):
                os.makedirs(f'../data/ownership-status/raw/{year}')
            dfShares.to_excel(f'../data/ownership-status/raw/{year}/{grp}.xlsx', index=False)
            print("="* 15 + cmpny + "=" * 15)
            print(dfShares.head())
            print(dfShares.tail())
            print(dfShares['동일인과의관계'].unique())
            print(dfShares['동일인과의관계2'].unique())
            print(dfShares['동일인과의관계3'].unique())
            print("="* 15 + cmpny + "=" * 15)
            
    def getTransUrl(self, firmNm, year):
        
        rceptNo = self.findRceptNum(firmNm, year)
        driver = webdriver.Chrome()
        driver.maximize_window()
        driver.implicitly_wait(3)
        url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=" + rceptNo

        driver.get(url)
        action = ActionChains(driver)
        action.move_to_element(driver.find_element(By.ID, '30')).perform()
        btn = driver.find_element(By.PARTIAL_LINK_TEXT, '계열회사간 상품ㆍ용역거래 현황')
        btn.click()

        # scrape the url of '계열회사간 상품ㆍ용역거래 현황' page
        transUrl = driver.find_element(By.ID, 'ifrm').get_attribute('src')
        driver.quit()
        
        return transUrl
    
    def getTransTableTest(self, firmNm, year):
        
        if not os.path.exists(f'./data/transactions{year}'):
            os.makedirs(f'./data/transactions{year}')
        
        rceptNo = self.findRceptNum(firmNm, year)
        driver = webdriver.Chrome()
        driver.maximize_window()
        driver.implicitly_wait(3)
        url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=" + rceptNo

        driver.get(url)
        action = ActionChains(driver)
        action.move_to_element(driver.find_element(By.ID, '30')).perform()
        btn = driver.find_element(By.PARTIAL_LINK_TEXT, '계열회사간 상품ㆍ용역거래 현황')
        btn.click()
        transUrl = driver.find_element_by_id('ifrm').get_attribute('src')
        
        driver.get(transUrl)
        
        
        # get table elements which are following siblings of p elements that contains text '대표회사'

        tables = driver.find_elements(By.XPATH, "/html/body/table[@border='1']")
        
#             dfTables = [pd.read_html(table.get_attribute('outerHTML'), header=[0, 1, 2])[0] for table in tables]
#         else:    
        dfTables = [pd.read_html(table.get_attribute('outerHTML'))[0] for table in tables]
        if ((firmNm == "SK") and (year == 2021)) or ((firmNm == "롯데지주") and (year in [2019, 2020, 2021]))\
        or ((firmNm == "미래에셋캐피탈") and (year in [2019, 2020, 2021])) \
        or ((firmNm == "NAVER") and (year in [2020, 2021])) \
        or ((firmNm == "효성") and (year in [2019, 2020, 2021])):
            dfTables = [makeHeader(table) for table in dfTables]
        dfTables = [dfTable for dfTable in dfTables if len(dfTable) > 5]
        dfWide = pd.concat(dfTables, axis=1)
        driver.quit()
        
        return dfWide

    def getTransTable(self, firmNm, year):

        if not os.path.exists(f'./data/transactions{year}'):
            os.makedirs(f'./data/transactions{year}')
        
        rceptNo = self.findRceptNum(firmNm, year)
        driver = webdriver.Chrome()
        driver.maximize_window()
        driver.implicitly_wait(3)
        url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=" + rceptNo

        driver.get(url)
        action = ActionChains(driver)
        action.move_to_element(driver.find_element(By.ID, '30')).perform()
        btn = driver.find_element(By.PARTIAL_LINK_TEXT, '계열회사간 상품ㆍ용역거래 현황')
        btn.click()
        transUrl = driver.find_element_by_id('ifrm').get_attribute('src')
        
        driver.get(transUrl)
        
        
        # get table elements which are following siblings of p elements that contains text '대표회사'

        tables = driver.find_elements(By.XPATH, "/html/body/table[@border='1']")
        
#             dfTables = [pd.read_html(table.get_attribute('outerHTML'), header=[0, 1, 2])[0] for table in tables]
#         else:    
        dfTables = [pd.read_html(table.get_attribute('outerHTML'))[0] for table in tables]
        if ((firmNm == "SK") and (year == 2021)) or ((firmNm == "롯데지주") and (year in [2019, 2020, 2021]))\
        or ((firmNm == "미래에셋캐피탈") and (year in [2019, 2020, 2021])) \
        or ((firmNm == "NAVER") and (year in [2020, 2021])) \
        or ((firmNm == "효성") and (year in [2019, 2020, 2021])) \
        or ((firmNm == "하림지주") and (year in [2019, 2020, 2021])) \
        or ((firmNm == "OCI") and (year in [2020, 2021])) \
        or ((firmNm == "티케이케미칼") and (year in [2019, 2020])) \
        or ((firmNm == "이랜드월드") and (year in [2019])) \
        or ((firmNm == "장금상선") and (year in [2020])):
            dfTables = [makeHeader(table, firmNm, year) for table in dfTables]
        dfTables = [dfTable for dfTable in dfTables if len(dfTable) > 5]
        dfWide = pd.concat(dfTables, axis=1)
        driver.quit()
        
        dfLong = cleanseTransWide(dfWide)
        
        dfLong = dfLong[~dfLong['매출회사'].isin(['소계', '합계', '계', '거래업체수', '해외계열사업체수', '계열회사합계매입액', \
                                             '비금융소계', '금융회사', '기타', '해외계열사수', '해외계열사계매출액한화', \
                                             '기타(루셈등)', '계열회사합계(매입액)', '⑩소계'])]
        dfLong = dfLong[~dfLong['매입회사'].isin(['소계', '합계', '계', '거래업체수', '해외계열사업체수', '해외계열회사업체수', \
                                             '비금융소계', '금융회사', '기타', '해외계열사수', '해외계열사계매출액한화', \
                                              '기타(루셈등)', '계열회사합계(매입액)', '⑩소계'])]
        
        return dfLong
    
    def getTransTableAll(self, year):
        
        dGrpCmpny = getGrpCmpnyDict(year)
        
        cont = True
        for grp, cmpny in dGrpCmpny.items():
            print("="* 15 + grp + "=" * 15)
            
            if grp == '삼성':
                cont = False
            
            if cont:
                continue
                
            dfTransLong = api.getTransTable(cmpny, year)
            # save the data in the after_cleansing folder
            if not os.path.exists(f'../data/transactions/{year}'):
                os.makedirs(f'../data/transactions/{year}')
            dfTransLong.to_excel(f'../data/transactions/{year}/{grp}.xlsx', index=False)
            
            print(dfTransLong['매출회사'].unique())
            print(dfTransLong['매입회사'].unique())
            print(dfTransLong.head())
            print(dfTransLong.tail())
            print("="* 15 + grp + "=" * 15)

# 2021

In [4]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTableAll(2021)

===============삼성===============
No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['삼성전자' '삼성물산' '삼우종합건축사사무소' '씨브이네트' '삼성바이오로직스' '삼성디스플레이' '삼성코닝어드밴스드글라스'
 '에스유머티리얼스' '스테코' '세메스' '삼성전자서비스' '삼성전자서비스씨에스' '삼성전자판매' '삼성전자로지텍' '수원삼성축'
 '삼성메디슨' '삼성에스디아이' '삼성전기' '삼성중공업' '에스디플렉스' '제일패션리테일' '삼성웰스토리' '시큐아이'
 '멀티캠퍼스' '에스티엠' '삼성에스디에스' '에스코어' '오픈핸즈' '미라콤아이앤씨' '삼성엔지니어링' '에스원'
 '휴먼티에스에스' '에스원씨알엠' '제일기획' '호텔신라' '신라스테이' '에스비티엠' '삼성경제연' '삼성라이온즈'
 '삼성화재해상보험' '삼성화재애니카손해사정' '삼성화재서비스손해사정' '삼성화재금융서비스보험대리점' '삼성생명보험' '삼성선물'
 '삼성자산운용' '삼성액티브자산운용' '삼성생명서비스손해사정' '삼성에스알에이자산운용' '삼성생명금융서비스보험대리점' '삼성증권'
 '삼성카드' '삼성카드고객서비스']
['삼성물산' '삼성바이오에피스' '삼성디스플레이' '삼성전자서비스' '삼성전자판매' '삼성메디슨' '삼성에스디아이' '삼성전기'
 '삼성웰스토리' '삼성에스디에스' '삼성엔지니어링' '에스원' '호텔신라' '삼성화재해상보험' '삼성증권' '삼성카드' '삼성전자'
 '서울레이크사이드' '삼우종합건축사사무소' '씨브이네트' '삼성바이오로직스' '삼성코닝어드밴스드글라스' '에스유머티리얼스'
 '스테코' '삼성전자로지텍' '수원삼성축' '삼성중공업' '제일패션리테일' '멀티캠퍼스' '에스티엠' '에스코어' '미라콤아이앤씨'
 '제일기획' '에이치디씨신라면세

['엘지' '엘지화학' '씨텍' '행복누리' '팜한농' '엘지하우시스' '그린누리' '엘지생활건강' '코카콜라음료' '한국음료'
 '해태에이치티비' '에프엠지' '엘지파루크' '미젠스토리' '밝은누리' '태극제약' '루치펠로코리아' '엘지엠엠에이' '엘지전자'
 '엘지디스플레이' '나눔누리' '엘지이노텍' '이노위드' '하이프라자' '하이엠솔루텍' '하이텔레서비스' '에이스냉동공조'
 '하누리' '로보스타' '로보메디' '실리콘웍스' '엘지유플러스' '씨에스리더' '아인텔레서비스' '미디어로그' '데이콤크로싱'
 '씨에스원파트너' '위드유' '엘지헬로비전' '유플러스홈서비스' '엘지씨엔에스' '비즈테크파트너스' '행복마루'
 '에스앤아이코퍼레이션' '곤지암예원' '드림누리' '에스앤아이씨엠' '엘지경영개발원' '엘지스포츠' '지투알' '에이치에스애드'
 '엘베스트' '엘지상사' '판토스' '판토스부산신항물류센터' '헬리스타항공' '한울타리']
['엘지화학' '엘지에너지솔루션' '엘지하우시스' '엘지토스템비엠' '엘지생활건강' '엘지파루크' '더페이스샵' '엘지엠엠에이'
 '엘지전자' '엘지디스플레이' '엘지이노텍' '하이프라자' '엘지유플러스' '엘지헬로비전' '엘지씨엔에스' '에스앤아이코퍼레이션'
 '엘지경영개발원' '엘지스포츠' '에이치에스애드' '엘지상사' '판토스' 'LGHoldingsJapanCo.,Ltd.' '엘지'
 '씨텍' '행복누리' '팜한농' '코카콜라음료' '해태에이치티비' '로보스타' '그린누리' '곤지암예원' '한국음료' '크린소울'
 '에프엠지' '미젠스토리' '밝은누리' '태극제약' '루치펠로코리아' '캐이앤아이' '씨앤피코스메틱스' '유플러스홈서비스'
 '이노위드' '씨에스리더' '비즈테크파트너스' '미래엠' '하이엠솔루텍' '하이텔레서비스' '하누리' '실리콘웍스'
 '아인텔레서비스' '미디어로그' '씨에스원파트너' '위드유' '지투알' '엘베스트' '판토스부산신항물류센터' '에이스냉동공조'
 '우지막코리아' '울릉샘물' '로아코

['지에스' '지에스에너지' '지에스파워' '보령엘엔지터미널' '지에스파크이십사' '지에스칼텍스' '지에스엠비즈' '이노폴리텍'
 '상지해운' '지에스바이오' '지에스에코메탈' '한울사랑' '지에스리테일' '후레쉬서브' '지에스넷비전' '지에스네트웍스'
 '후레쉬미트' '지에스홈쇼핑' '지에스텔레서비스' '지에스글로벌' '피엘에스' '지에스엔텍' '지에스스포츠' '지에스건설'
 '파르나스호텔' '자이에스앤디' '자이에스텍' '지씨에스' '비에스엠' '자이에너지운영' '피앤에쓰' '지앤엠에스테이트'
 '지에스네오텍' '삼양통상' '삼양인터내셔날' '보헌개발' '켐텍인터내셔날' '경원건설' '승산' '삼정건업' '지에스이앤알'
 '지에스풍력발전' '동해그린에너지']
['지에스에너지' '지에스파워' '지에스파크이십사' '지에스칼텍스' '지에스엠비즈' '지에스바이오' '지에스에코메탈' '지에스리테일'
 '지에스넷비전' '지에스네트웍스' '지에스홈쇼핑' '지에스텔레서비스' '지에스이피에스' '지에스글로벌' '지에스엔텍'
 '지에스스포츠' '지에스건설' '지에스네오텍' '위너셋' '지에스이앤알' '지에스동해전력' '지에스풍력발전' '지에스포천그린에너지'
 '인천종합에너지' 'KoreaGSE&PPte.Ltd' 'GSEnergyTradingSingaporePte.Ltd.'
 '삼양인터내셔날' '파르나스호텔' '자이에너지운영' '피앤에쓰' '지에스' '이노폴리텍' '상지해운' '켐텍인터내셔날'
 'GSCaltexChinaCo.,Ltd.' 'GSCCzechs.r.o.' 'GSC(Suzhou)PlasticsCo.,Ltd.'
 'GSCaltexIndiaPte.,Ltd.' 'GSCaltex(Langfang)PlasticsCo.,Ltd.'
 'GSCaltexSingaporePte.,Ltd.' 'GSLubricants(Tianjin)Co.,Ltd.'
 'GSCaltexMexicoS.deR.L.deC.V.' 'GSGLOBALSINGAPOREPTE.LTD.'
 'GSAromaticsPte.

['대한항공' '한진칼' '한진' '한국공항' '정석기업' '한진관광' '토파스여행정보' '한진정보통신' '진에어' '싸이버스카이'
 '에어코리아' '칼호텔네트워크' '한국글로발로지스틱스시스템' '한진인천컨테이너터미널' '항공종합서비스' '포항항7부두운영'
 '부산글로벌물류센터' '평택컨테이너터미날' '한진부산컨테이너터미널' '아이에이티' '한진울산신항운영' '서울복합물류자산관리'
 '서울복합물류프로젝트금융투자' '왕산레저개발' '태일통상' '태일캐터링' '청원냉장' '세계혼재항공화물' '더블유에이씨항공서비스']
['한진칼' '한진' '진에어' '정석기업' '칼호텔네트워크' '토파스여행정보' '한진관광' '한국공항' '한진정보통신'
 '항공종합서비스' '싸이버스카이' '한국글로발로지스틱스시스템' '아이에이티' '왕산레저개발' '에어코리아' '세계혼재항공화물'
 '더블유에이씨항공서비스' '제동레저' 'HanjinInternationalCorp.' "HanjinInt'lJapanCo.,Ltd"
 'TotalAviationServiceLLC' 'HanjinIntermodalAmericaInc.' '대한항공'
 '한진부산컨테이너터미널' '한진인천컨테이너터미널' '평택컨테이너터미날' '부산글로벌물류센터' '한진울산신항운영'
 '서울복합물류자산관리' '한국티비티' 'QINGDAOHANJINLUHAIINTERNATIONALLOGISTICSCO.,LTD'
 'HanjinGlobalLogistics(HongKong)Ltd.'
 'HANJINGLOBALLOGISTICS(SHANGHAI)LIMITED'
 'HanjinGlobalLogistics(Dalian)Ltd.'
 'HANJINGLOBALLOGISTICS(SHENZHEN)LIMITED'
 'HANJINGLOBALLOGISTICSEUROPEs.r.o.' 'HANJINGLOBALLOGISTICSMYANMARCO.,LTD'
 'HANJINGLOBALLOGISTICS(VIETNAM)LIMITED' '포항항7부두운영' '태일캐터링'


['디엘' '대림' '디엘건설' '디엘모터스' '글래드호텔앤리조트' '에이플러스디' '제주항공우주호텔' '디엘에너지' '하이웨이솔라'
 '포승그린파워' '청진이삼프로젝트' '청진이삼자산관리' '대림에이엠씨' '대림피앤피'
 '대림제7호마산회원기업형임대위탁관리부동산투자회사']
['대림' '디엘건설' '대림씨엔에스' '글래드호텔앤리조트' '송도파워' '에코술이홀' '험프리에스엘큐원' '제주항공우주호텔'
 '덕송내각고속화도로' '디엘에너지' '청진이삼자산관리' '대림에이엠씨' '대림오토바이' '대림피앤피'
 '대림제7호마산회원기업형임대위탁관리부동산투자회사' '디엘에프엔씨' '효제피에프브이' '디엘' '디엘모터스' '포천파워'
 '에이플러스디' '밀머란에스피씨' '포승그린파워' '코크레인에스피씨' '청진이삼프로젝트'
 '대림제6호부산우암동기업형임대위탁관리부동산투자회사']
  매출회사       매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)      국내매출액     해외매출액
0   디엘         대림   4843.0     677705.0      47040.0  6181343.0  936791.0
1   디엘       디엘건설  83395.0     677705.0      47040.0  6181343.0  936791.0
2   디엘     대림씨엔에스      3.0     677705.0      47040.0  6181343.0  936791.0
3   디엘  글래드호텔앤리조트   1809.0     677705.0      47040.0  6181343.0  936791.0
4   디엘       송도파워      2.0     677705.0      47040.0  6181343.0  936791.0
                         매출회사    매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)  \
55                      대림피앤피      디엘  2106.0   

...

['티케이케미칼' '남선알미늄' '남선홀딩스' '라도' '바로코사' '벡셀' '삼라' '삼라농원' '삼라마이다스' '우방'
 '울산방송' '이엔에이치' '케이엘홀딩스' '한통엔지니어링' '대한상선' '대한해운' '동아건설산업' '삼환기업'
 '에스엠글로벌유통' '에스엠레저산업' '에스엠상선' '에스엠스틸' '에스엠신용정보' '에스엠인더스트리' '에스엠중공업'
 '에스엠하이플러스' '에스엠화진' '에스엠화진인터내셔널' '우방산업' '이코사주류' '케이엘씨에스엠' '코리코엔터프라이스'
 '탑스텐동강시스타' '플러스매니지먼트' '한덕철광산업' '에스엠에이엠씨투자대부']
['삼라' '우방' '이엔에이치' '경남기업' '대한해운' '동아건설산업' '에스엠인더스트리' '에스엠중공업' '에스엠하이플러스'
 '티케이케미칼' '삼환기업' '에스엠상선' '벡셀' '에스엠글로벌유통' '코리코엔터프라이스' '남선알미늄' '바로코사' '우방산업'
 '라도' '케이엘홀딩스' '한통엔지니어링' '대한상선' '대한해운엘엔지' '신화디앤디' '에스엠스틸' '에스엠신용정보' '삼라농원'
 'SMVNCO.,LTD' '태길종합건설' '에스엠상선경인터미널' '케이엘씨에스엠' '한국선박금융' '한덕철광산업'
 '에스엠에이엠씨투자대부' '삼라마이다스' '에스엠화진' '이코사주류' '에스엠화진인터내셔널' '탑스텐동강시스타' '판교에스디쓰리'
 '서남바이오에너지' '에스엠레저산업' '화진고신장식재료' '대림종합건설더블유']
     매출회사   매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액    해외매출액
0  티케이케미칼     삼라    19.0       3976.0          NaN  331911.0  79116.0
1  티케이케미칼     우방     1.0       3976.0          NaN  331911.0  79116.0
2  티케이케미칼  이엔에이치  3683.0       3976.0          NaN  331911.0  7

['디엠씨미디어' '리앤에스스포츠' '무노스' '문고리닷컴' '블루원' '블루원레저' '에스비에스' '에스비에스네오파트너스'
 '에스비에스디지털뉴스랩' '에스비에스미디어넷' '에스비에스아이앤엠' '에스비에스에이앤티' '에스비에스엠앤씨' '에스비에스콘텐츠허브'
 '에스비에스플러스' '엠시에타' '이너시티개발' '인제스피디움' '태영건설' '태영그레인터미널' '티에스케이그린에너지'
 '티에스케이엔지니어링' '티에스케이엠엔에스' '티에스케이워터' '티에스케이이앤피' '티에스케이코퍼레이션' '티에스케이프리텍'
 '평택싸이로' '포맷티스트' '삼계개발' '스마트미디어렙' '스튜디오에스' '신경주지역개발' '에스비에스미디어홀딩스'
 '에스비에스바이아컴' '에코시스템']
['에스비에스' '스마트미디어렙' '에스비에스미디어넷' '에스비에스엠앤씨' 'SBS디지털뉴스랩' '문고리닷컴' '블루원'
 '에스비에스아이앤엠' 'SBS플러스' '에스비에스콘텐츠허브' 'SBS미디어홀딩스' '태영건설' '태영인더스트리' '블루원레저'
 '엠시에타' '에스비에스바이아컴' '스튜디오에스' '디엠씨미디어' 'SBS에이엔티' 'SBSinternational' 'KCP'
 '블렌딩' '리앤에스스포츠' '무노스' '콘텐츠웨이브' '엠시에타개발' 'SBS아프리카TV'
 'KoreaContentPlatfrom,LCC' '에코시티' '티에스케이워터' '대AMC' '에코시티개발' '포천바이오에너지'
 '경산에코에너지' '신경주역세권공영개발' '이너시티개발' '평택싸이로' '네오시티' '군포복합개발PFV' 'TSK이앤피'
 'TSK코퍼레이션' '울산이앤피' '휴비스워터' '여수엑스포환경' '여천이피에스' '부산바이오에너지' 'TSK엔지니어링'
 'TSK엔워터테크' 'TSK그린에너지' '태영그레인터미널' '에스비에스네오파트너스']
     매출회사       매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액  해외매출액
0  디엠씨미디어      에스비에스   896.0   

['한라홀딩스' '위코' '제이제이한라' '한라' '한라세라지오' '한라지엘에스' '한라오엠에스' '만도' '만도헬라일렉트로닉스'
 '만도브로제']
['위코' '제이제이한라' '한라' '한라세라지오' '한라지엘에스' '한라오엠에스' '목포신항만운영' '만도' '만도헬라일렉트로닉스'
 '만도브로제' '한라홀딩스' '배곧신도시지역특성화타운' '평택동부도로' '만도차이나홀딩스']
    매출회사    매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액     해외매출액
0  한라홀딩스      위코    13.0     103905.0     105208.0  440230.0  127369.0
1  한라홀딩스  제이제이한라    40.0     103905.0     105208.0  440230.0  127369.0
2  한라홀딩스      한라  9789.0     103905.0     105208.0  440230.0  127369.0
3  한라홀딩스  한라세라지오    17.0     103905.0     105208.0  440230.0  127369.0
4  한라홀딩스  한라지엘에스   446.0     103905.0     105208.0  440230.0  127369.0
          매출회사        매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액  \
39  만도헬라일렉트로닉스          만도  436042.0     436281.0      58337.0  466144.0   
40  만도헬라일렉트로닉스       만도브로제      31.0     436281.0      58337.0  466144.0   
41       만도브로제       한라홀딩스    6677.0      69819.0       8457.0   81326.0   
42       만도브로제          만도   36745.0      69819.0       845

['장금상선' '와이제이씨' '한성라인' '경평물류' '광양항서부컨테이너터미널' '국양로지텍' '국양해운' '부산항터미널'
 '시노코쉽매니지먼트' '에스엠상선광양터미널' '장금마리타임' '조강해운' '퍼시픽엔지니어링' '흥아라인']
['와이제이씨' '한성라인' '경평물류' '국양로지텍' '국양해운' '슈퍼랙' '시노코쉽매니지먼트' '시노코탱커'
 '시노코페트로케미컬' '장금마리타임' '퍼시픽엔지니어링' '흥아라인'
 'TIANJINYUJIACHANGINTERNATIONALFORWARDING' '장금상선' '에스엠상선광양터미널'
 '광양항서부컨테이너터미널' '부산항터미널']
   매출회사   매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액     해외매출액
0  장금상선  와이제이씨   529.0     120650.0       3140.0  690953.0  518181.0
1  장금상선   한성라인  9807.0     120650.0       3140.0  690953.0  518181.0
2  장금상선   경평물류    23.0     120650.0       3140.0  690953.0  518181.0
3  장금상선  국양로지텍   566.0     120650.0       3140.0  690953.0  518181.0
4  장금상선   국양해운   754.0     120650.0       3140.0  690953.0  518181.0
    매출회사    매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액     해외매출액
69  흥아라인   와이제이씨    41.0      74404.0          NaN  248521.0  386468.0
70  흥아라인    한성라인  2050.0      74404.0          NaN  248521.0  386468.0
71  흥아라인    경평물류    39.0      74404.0          Na

['대방건설' '대방산업개발' '대덕하우징씨스템' '지유인터내셔날' '디비일산' '디케이일산']
['대방하우징' '대방주택' '노블랜드' '디비건설' '디비산업개발' '대방이노베이션' '대방이엔씨' '대방개발기업' '디비개발'
 '엔비건설' '디비종합건설' '디비주택' '대방일산디엠시티' '대방덕은' '디엠개발' '대방산업개발' '대방산업개발동탄'
 '대방건설' '엘리움' '엘리움개발' '엘리움건설' '엘리움주텍' '엘리움주택개발']
   매출회사    매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)      국내매출액  해외매출액
0  대방건설   대방하우징  153182.0     970711.0          NaN  1557457.0    NaN
1  대방건설    대방주택  103392.0     970711.0          NaN  1557457.0    NaN
2  대방건설    노블랜드   86208.0     970711.0          NaN  1557457.0    NaN
3  대방건설    디비건설  261352.0     970711.0          NaN  1557457.0    NaN
4  대방건설  디비산업개발   78658.0     970711.0          NaN  1557457.0    NaN
        매출회사      매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액  해외매출액
33  대덕하우징씨스템    대방산업개발    332.0       3063.0          NaN   8119.0    NaN
34   지유인터내셔날      대방건설    144.0        196.0          NaN    391.0    NaN
35   지유인터내셔날    대방산업개발     52.0        196.0          NaN    391.0    NaN
36      디비일산  대방일산디엠시티  10107.0      10

# 2020

In [5]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTableAll(2020)

===============삼성===============
===============현대자동차===============
===============에스케이===============
===============엘지===============
===============롯데===============
===============한화===============
===============지에스===============
===============현대중공업===============
===============신세계===============
===============씨제이===============
===============한진===============
===============두산===============
===============엘에스===============
===============부영===============
===============대림===============
===============미래에셋===============
===============금호아시아나===============
===============현대백화점===============
===============카카오===============
===============한국투자금융===============
===============교보생명보험===============
===============효성===============
===============하림===============
===============영풍===============
===============에이치디씨===============
===============케이씨씨===============
===============코오롱===============
===============오씨아이===============
===============이랜드===============
====

# 2019

'(구)롯데디에프'가 ''로 대체됨

In [7]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTableAll(2019)

===============삼성===============
No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['삼성전자' '삼성물산' '서울레이크사이드' '삼우종합건축사사무소' '씨브이네트' '삼성바이오로직스' '삼성디스플레이'
 '삼성코닝어드밴스드글라스' '에스유머티리얼스' '스테코' '세메스' '삼성전자서비스' '삼성전자서비스씨에스' '삼성전자판매'
 '삼성전자로지텍' '수원삼성축' '삼성메디슨' '삼성에스디아이' '삼성전기' '삼성중공업' '에스디플렉스' '제일패션리테일'
 '삼성웰스토리' '시큐아이' '멀티캠퍼스' '에스티엠' '삼성에스디에스' '에스코어' '오픈핸즈' '미라콤아이앤씨'
 '삼성엔지니어링' '에스원' '휴먼티에스에스' '에스원씨알엠' '제일기획' '호텔신라' '신라스테이' '에이치디씨신라면세점'
 '에스비티엠' '삼성경제연' '삼성라이온즈' '삼성화재해상보험' '삼성화재애니카손해사정' '삼성화재서비스손해사정'
 '삼성화재금융서비스보험대리점' '삼성생명보험' '삼성선물' '삼성자산운용' '생보부동산신탁' '삼성액티브자산운용'
 '삼성생명서비스손해사정' '삼성에스알에이자산운용' '삼성생명금융서비스보험대리점' '삼성증권' '삼성카드' '삼성카드고객서비스']
['삼성물산' '삼성바이오에피스' '삼성디스플레이' '세메스' '삼성전자서비스' '삼성전자판매' '삼성메디슨' '삼성에스디아이'
 '삼성전기' '삼성웰스토리' '시큐아이' '삼성에스디에스' '삼성엔지니어링' '에스원' '제일기획' '호텔신라' '삼성화재해상보험'
 '삼성생명보험' '삼성자산운용' '삼성증권' '삼성카드' '삼성전자' '서울레이크사이드' '삼우종합건축사사무소' '씨브이네트'
 '삼성바이오로직스' '삼성코닝어드밴스드글라스' '에스유머티리얼스' '스테코' '삼성전자로지텍' '수원삼

['엘지' '엘지화학' '씨텍' '행복누리' '팜한농' '엘지하우시스' '엘지토스템비엠' '그린누리' '엘지생활건강' '코카콜라음료'
 '한국음료' '해태에이치티비' '더페이스샵' '크린소울' '씨앤피코스메틱스' '캐이엔아이' '에프엠지' '엘지파루크' '밝은누리'
 '태극제약' '제이에스제약' '엘지엠엠에이' '엘지전자' '엘지디스플레이' '나눔누리' '엘지이노텍' '이노위드' '하이프라자'
 '하이엠솔루텍' '하이텔레서비스' '에이스냉동공조' '하이엔텍' '엘지히타치워터솔루션' '하누리' '로보스타' '실리콘웍스'
 '엘지유플러스' '씨에스리더' '아인텔레서비스' '미디어로그' '데이콤크로싱' '씨에스원파트너' '위드유' '엘지씨엔에스'
 '비즈테크파트너스' '행복마루' '에스앤아이코퍼레이션' '곤지암예원' '미래엠' '드림누리' '서브원' '엘지경영개발원'
 '엘지스포츠' '지투알' '에이치에스애드' '엘베스트' '엘지상사' '판토스' '판토스부산신항물류센터' '헬리스타항공' '한울타리']
['엘지화학' '엘지하우시스' '엘지토스템비엠' '엘지생활건강' '더페이스샵' '엘지파루크' '엘지엠엠에이' '엘지전자'
 '엘지디스플레이' '엘지이노텍' '하이프라자' '엘지히타치워터솔루션' '엘지유플러스' '엘지씨엔에스' '에스앤아이코퍼레이션'
 '엘지경영개발원' '엘지스포츠' '에이치에스애드' '엘지상사' '판토스' '엘지엔시스'
 'LGHoldingsJapanCo.,Ltd.' '엘지' '씨텍' '행복누리' '팜한농' '코카콜라음료' '하이엔텍' '서브원'
 '팜화옹' '곤지암예원' '하우시스이엔지' '한국음료' '해태에이치티비' '크린소울' '씨앤피코스메틱스' '캐이엔아이' '에프엠지'
 '밝은누리' '태극제약' '하이엠솔루텍' '씨에스리더' '비즈테크파트너스' '미래엠' '제이에스제약' '하이텔레서비스'
 '에이스냉동공조' '하누리' '엘지퓨얼셀시스템즈코리아' '실리콘웍스' '아인텔레서비스' '미디어로그' '씨에스원파트너' '위드유'
 '지투알' '엘베스트' 

['지에스' '지에스에너지' '지에스파워' '보령엘엔지터미널' '지에스파크이십사' '지에스칼텍스' '지에스엠비즈' '이노폴리텍'
 '상지해운' '지에스바이오' '지에스에코메탈' '지에스리테일' '후레쉬서브' '지에스넷비전' '씨브이에스넷' '지에스네트웍스'
 '지에스홈쇼핑' '지에스텔레서비스' '텐바이텐' '지에스글로벌' '피엘에스' '지에스엔텍' '지에스스포츠' '지에스건설'
 '파르나스호텔' '자이에스앤디' '자이에스텍' '지씨에스' '비에스엠' '자이에너지운영' '피앤에쓰' '지앤엠에스테이트'
 '지에스네오텍' '삼양통상' '삼양인터내셔날' '보헌개발' '켐텍인터내셔날' '옥산유통' '경원건설' '승산' '센트럴모터스'
 '삼정건업' '지에스이앤알' '지에스풍력발전']
['지에스에너지' '지에스파워' '해양도시가스' '서라벌도시가스' '지에스이엠' '지에스파크이십사' '지에스칼텍스' '지에스엠비즈'
 '지에스바이오' '지에스에코메탈' '지에스리테일' '지에스넷비전' '지에스네트웍스' '지에스홈쇼핑' '지에스텔레서비스'
 '지에스이피에스' '지에스글로벌' '지에스엔텍' '지에스스포츠' '지에스건설' '지에스네오텍' '지에스아이티엠' '센트럴모터스'
 '위너셋' '지에스이앤알' '지에스동해전력' '지에스풍력발전' '와이지이' '보령엘엔지터미널' '지에스' '인천종합에너지'
 '파르나스호텔' '자이에너지운영' '이노폴리텍' '상지해운' '삼양인터내셔날' '켐텍인터내셔날'
 'GSCaltexChinaCo.,Ltd.' 'GSCCzechs.r.o.' 'GSC(Suzhou)PlasticsCo.,Ltd.'
 'GSCaltexIndiaPte.,Ltd.' 'GSCaltex(Langfang)PlasticsCo.,Ltd.'
 'GSCaltexSingaporePte.,Ltd.' 'LurodaLubricants(Tianjin)Co.,Ltd.'
 'GSCaltexMexicoS.deR.L.deC.V.' 'GSGLOBALSINGAPOREPTE.LTD.'
 'GSGlobalJapanCo

['씨제이' '씨제이제일제당' '원지' '돈돈팜' '씨제이씨푸드' '씨제이엠디원' '어업회사법인신의도천일염' '씨제이브리딩'
 '씨제이대한통운' '한국복합물류' '씨제이대한통운비엔디' '동석물류' '울산항만운영' '마산항제4부두운영' '대한통운에스비'
 '이앤씨인프라' '씨제이프레시웨이' '프레시원미트' '프레시원광주' '프레시원남서울' '프레시원중부' '프레시원강남'
 '프레시원동서울' '프레시원대' '프레시원부산' '에프앤디인프라' '송림푸드' '송림에프에스' '형제푸드' '씨제이푸드빌'
 '투썸플레이스' '씨제이씨지브이' '디시네마오브코리아' '씨제이포디플렉스' '씨제이이엔엠' '씨제이헬로' '씨제이헬로하나방송'
 '씨제이텔레닉스' '슈퍼레이스' '아트웍스코리아' '아트서비스' '메조미디어' '제이에스픽쳐스' '엠엠오엔터테인먼트'
 '하이라이트레코즈' '에이오엠지' '제이케이필름' '랭크웨이브' '젤리피쉬엔터테인먼트' '하이어뮤직레코즈' '스튜디오테이크원'
 '데이비드토이' '아메바컬쳐' '디베이스앤' '스톤뮤직엔터테인먼트' '스윙엔터테인먼트' '스튜디오드래곤' '화앤담픽쳐스'
 '문화창고' '케이피제이' '씨제이올리브네트웍스' '씨제이파워캐스트' '디아이웨어' '제이에이치투자' '에스지생활안전'
 '타임와이즈인베스트먼트']
['씨제이제일제당' '씨제이씨푸드' '씨제이브리딩' '씨제이대한통운' '한국복합물류' '씨제이대한통운비엔디' '씨제이프레시웨이'
 '씨제이푸드빌' '투썸플레이스' '씨제이씨지브이' '씨제이포디플렉스' '씨제이이엔엠' '씨제이헬로' '씨제이올리브네트웍스'
 '씨제이파워캐스트' '씨제이건설' '씨제이헬스케어' '씨제이이앤엠' '씨제이디지털뮤직'
 'CJVIETNAMCOMPANYLIMITED' '씨제이' '원지' '돈돈팜' '씨제이엠디원' '씨제이텔레닉스' '슈퍼레이스'
 '메조미디어' '스튜디오드래곤' '씨앤아이레저산업' '에스지생활안전' '영우냉동식품' '타임와이즈인베스트먼트' '프레시원대'
 '씨제이라이브시티' '어업회사법인신의도천일

['미래에셋캐피탈' '미래에셋대우' '미래에셋생명보험' '미래에셋자산운용' '미래에셋벤처투자' '미래에셋펀드서비스'
 '미래에셋금융서비스' '멀티에셋자산운용' '미래에셋파트너스제9호사모투자' '오딘제9차' '미래에셋컨설팅' '브랜드무브'
 '와이케이디벨롭먼트']
['미래에셋대우' '미래에셋자산운용' '미래에셋생명보험' '멀티에셋자산운용' '미래에셋금융서비스' '미래에셋글로벌유니콘사모투자'
 '미래에셋네이버아시아그로쓰사모투자' 'MiraeAssetFinanceCompany(Vietnam)' '미래에셋캐피탈'
 '미래에셋벤처투자' 'PCA생명보험' '미래에셋모바일' '미래에셋펀드서비스' '에너지인프라자산운용' '미래에셋파트너스5호'
 '미래에셋파트너스9호' '미래에셋대우코리아제일호사모투자' '미래에셋대우글로벌케이그로쓰사모투자' '미래에셋대우오퍼튜너티사모투자'
 '미래에셋대우헌터스사모투자' '미래에셋대우글로벌제일호사모투자' '미래에셋대우코리아제이호사모투자' '미래에셋컨설팅' '부동산일일사'
 '브랜드무브' '수원학교사랑' 'MiraeAssetSecurities(HK)Limited' '와이케이디벨롭먼트'
 '코에프씨미래에셋그로쓰챔프2010의4호사모투자' '미래에셋파트너스칠호사모투자'
 'MiraeAssetGlobalInvestments(HK)Limited'
 'MiraeAssetGlobalInvestments(USA)INC.'
 'MiraeAssetGlobalInvestments(Taiwan)'
 'MiraeAssetGlobalETFsHodlignsLimited' '오딘제9차' '서울공항리무진' '미래비아이']
      매출회사       매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액   해외매출액
0  미래에셋캐피탈     미래에셋대우  29421.0      37850.0       2231.0  174685.0  2231.0
1  미래에셋캐피탈   미래에셋자산운용     12.0      37850.0       22

['하림지주' '그린바이텍' '동림건설' '디디에프엔비' '맥시칸' '산과들에프앤씨' '선진' '선진브릿지랩' '선진에프에스'
 '선진팜' '선진햄' '싱그린FS' '애그리로보텍' '엔디' '엔바이콘' '엔에스쇼핑' '엔에스홈쇼핑미디어센터' '올품'
 '참트레이딩' '청운농업회사법인' '팜스코' '포스에스엠' '하림' '하림산업' '하림애니멀클리닉' '하림엠에스' '하림펫푸드'
 '농업회사법인에이치비씨' '농업회사법인봉산지피' '농업회사법인선진한마을' '농업회사법인순우리한우' '농업회사법인주원산오리'
 '농업회사법인팜스코바이오인티' '농업회사법인팜앤코' '농업회사법인포크랜드' '농업회사법인한사랑' '농업회사법인피그앤그린'
 '제일사료' '팬오션' '한강씨엠' '한국썸벧' '에코캐피탈']
['그린바이텍' '산과들에프앤씨' '선진' '선진에프에스' '애그리로보텍' '엔바이콘' '엔에스쇼핑' '참트레이딩' '팜스코'
 '하림' '하림유통' '하림펫푸드' '농업회사법인에이치비씨' '농업회사법인주원산오리' '제일사료' '싱그린FS' '하림지주'
 '농업회사법인선진한마을' '하림엠에스' '선진미트농업회사법인' '선진브릿지랩' '선진팜' '선진햄' '청운농업회사법인'
 '농업회사법인봉산지피' '농업회사법인봉화' '농업회사법인한사랑' '한국썸벧' '팬오션' '포스에스엠' '하림산업' '하림식품'
 '올품' 'SinggreenVinaCOMPANYLIMITED' '엔에스홈쇼핑미디어센터' '농업회사법인순우리한우' '디디에프엔비'
 '한강씨엠' '동림건설' '에코캐피탈' '농업회사법인팜스코바이오인티' '농업회사법인팜앤코' '농업회사법인포크랜드' '에이치에스푸드'
 '농업회사법인피그앤그린' 'PanOcean(America)Inc.'
 'PanOceanSingaporeBulkCarrierPte.Ltd.' '하림애니멀클리닉'
 'SunjinMyanmarCompanyLimited']
   매출회사     매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액 

['에이치디씨' '에이치디씨스포츠' '에이치디씨아이서비스' '에이치디씨아이콘트롤스' '에이치디씨아이파크몰' '에이치디씨영창'
 '에이치디씨현대산업개발' '에이치디씨현대피씨이' '호텔에이치디씨' '에이치디씨자산운용']
['아이파크마리나' '에이치디씨스포츠' '에이치디씨신라면세점' '에이치디씨아이서비스' '에이치디씨아이앤콘스' '에이치디씨아이콘트롤스'
 '에이치디씨아이파크몰' '에이치디씨영창' '에이치디씨현대산업개발' '에이치디씨현대이피' '에이치디씨현대피씨이' '호텔에이치디씨'
 '에이치디씨자산운용' '에이치디씨' '미래비아이' '북항아이브리지' '엠엔큐투자파트너스.' '부동산일일사'
 '에이치디씨민간임대주택제1호위탁관리부동산투자회사' '부산컨테이너터미널' '지개남산도시고속화도로']
    매출회사        매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액  해외매출액
0  에이치디씨     아이파크마리나    21.0      13534.0          0.0  49614.0    0.0
1  에이치디씨    에이치디씨스포츠    14.0      13534.0          0.0  49614.0    0.0
2  에이치디씨  에이치디씨신라면세점   542.0      13534.0          0.0  49614.0    0.0
3  에이치디씨  에이치디씨아이서비스  1293.0      13534.0          0.0  49614.0    0.0
4  에이치디씨  에이치디씨아이앤콘스   322.0      13534.0          0.0  49614.0    0.0
         매출회사         매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액  해외매출액
76  에이치디씨자산운용   에이치디씨아이서비스  11.0        287.0          0.0  7902.0    0.0
77  에이치디씨자산운용   에이치디씨아이앤콘스  11.0        

['이랜드월드' '이랜드리테일' '이랜드파크' '이월드' '이랜드건설' '이랜드시스템스' '와인캐슬' '이랜드제주리조트'
 '농업회사법인맛누리' '이랜드크루즈' '리드온' '이랜드서비스' '올리브스튜디오' '이앤씨월드' '이랜드중국패션디자인'
 '이랜드위시디자인' '예지실업' '이랜드스포츠' '이랜드인재원' '이리츠코크렙기업' '케이비와이즈스타제6호위탁관리부동산투자회사'
 '리드']
['이랜드리테일' '이랜드파크' '이월드' '이랜드건설' '이랜드시스템스' '이랜드제주리조트' '농업회사법인맛누리' '고운조경'
 '애월국제문화복합단지' '이랜드크루즈' '투어몰' '리드온' '이랜드서비스' '올리브스튜디오' '이앤씨월드' '이랜드중국패션디자인'
 '이랜드위시디자인' '예지실업' '이랜드스포츠' '이에셋투자개발' '이랜드인재원' '리드' '해외거래처수' '이랜드월드'
 '이랜드공덕' '와팝' '이리츠코크렙기업' '와인캐슬']
    매출회사     매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)      국내매출액     해외매출액
0  이랜드월드   이랜드리테일  146733.0     170578.0      35018.0  1258076.0  180558.0
1  이랜드월드    이랜드파크    9447.0     170578.0      35018.0  1258076.0  180558.0
2  이랜드월드      이월드       4.0     170578.0      35018.0  1258076.0  180558.0
3  이랜드월드    이랜드건설      84.0     170578.0      35018.0  1258076.0  180558.0
4  이랜드월드  이랜드시스템스    1948.0     170578.0      35018.0  1258076.0  180558.0
                       매출회사     매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)  \
185  케이비와이즈

['태영건설' '디엠씨미디어' '리앤에스스포츠' '블렌딩' '블루원' '에스비에스' '에스비에스네오파트너스' '에스비에스디지털뉴스랩'
 '에스비에스아이앤엠' '에스비에스아프리카티비' '에스비에스에이앤티' '에스비에스엠앤씨' '에스비에스콘텐츠허브' '에스비에스플러스'
 '엠시에타' '인제스피디움' '티에스케이그린에너지' '티에스케이엠엔에스' '티에스케이워터' '티에스케이코퍼레이션' '더스토리웍스'
 '스마트미디어렙' '신경주지역개발' '에스비에스미디어홀딩스' '에스비에스바이아컴' '에코시스템' '콘텐츠연합플랫폼']
['태영인더스트리' '유니시티' '에코시티개발' '엠시에타개발' '포천바이오에너지' '블루원' '경산에코에너지' '에코시티'
 '신경주역세권공영개발' '이너시티개발' '동부권푸른물' '에스비에스' '스마트미디어렙' '에스비에스플러스' '에스비에스엠앤씨'
 '에스비에스콘텐츠허브' '에스비에스미디어홀딩스' '태영건설' '인제스피디움' '에스비에스바이아컴' '콘텐츠연합플랫폼'
 '디엠씨미디어' '에스비에스에이엔티' '에스비에스아이앤엠' 'SBS아프리카티비' 'SBSinternationalInc' 'KCP'
 '에스비에스디지털뉴스랩' '에스비에스네오파트너스' '더스토리웍스' '블렌딩' 'PLANKEntertainment' '리앤에스스포츠'
 '울산이앤피' '티에스케이그린에너지' '티에스케이코퍼레이션' '여수엑스포환경' '티에스케이엔워터테크' '부산바이오에너지'
 '티에스케이워터']
   매출회사      매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)      국내매출액   해외매출액
0  태영건설   태영인더스트리     148.0     850294.0          NaN  1991815.0  4234.0
1  태영건설      유니시티  476421.0     850294.0          NaN  1991815.0  4234.0
2  태영건설    에코시티개발  133049.0     850294.0          Na

['유진기업' '동양' '유진에너팜' '유진홈데이' '이에이치씨' '지' '티와이강원' '한성레미콘' '한일합섬' '굿앤파트너스'
 '남부산업' '당진기업' '동화기업' '바이오코엔' '우진레미콘' '유진로텍' '유진아이티서비스' '유진에이엠씨' '유진엠'
 '유진엠플러스' '유진프라이빗에쿼티' '이순산업' '천안기업' '한국로지스틱스' '한국통운' '현대개발' '양지지앤알홀딩스'
 '유진에버베스트턴어라운드기업재무안정사모투자' '유진자산운용' '유진제사호헤라클레스사모투자' '유진제삼호오벨리스크기업재무안정사모투자'
 '유진투자선물' '유진투자증권']
['나눔로또' '동양' '유진에너팜' '유진홈데이' '이에이치씨' '지' '한성레미콘' '한일합섬' '남부산업' '당진기업'
 '동화기업' '우진레미콘' '유진로텍' '유진아이티서비스' '유진에이엠씨' '유진엠' '유진엠플러스' '유진초저온' '이순산업'
 '천안기업' '한국통운' '현대산업' '선진엔티에스' '현대개발' '유진저축은행' '유진투자증권' '유진기업'
 'PT.HanilIndonesia' '한국로지스틱스' '유진프라이빗에쿼티' '유진자산운용' '유진투자선물' '성인산업'
 '파인리조트' '유진스타제2호사모투자' '유진에이스제일호사모투자' '유진제사호헤라클레스사모투자'
 '유진제삼호오벨리스크기업재무안정사모투자' '유진제오호타이탄기업재무안정사모투자' '부산로지스틱스' '대전로지스틱스' '경인로지스틱스'
 '중부로지스틱스' '케이비에프' '유진에버베스트턴어라운드기업재무안정사모투자' '유진에스비홀딩스' '양지지앤알홀딩스']
   매출회사   매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액  해외매출액
0  유진기업   나눔로또    183.0      40359.0          NaN  780324.0  663.0
1  유진기업     동양  13231.0      40359.0          NaN  780324.0  663.0
2  유진기업  유진에너팜    

# 중앙

In [671]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('중앙홀딩스', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['중앙홀딩스' '더패키지문화산업' '이태원클라쓰문화산업' '제이씨엔' '미디어링크' '비에이엔터테인먼트' '올마루' '제이콘텐트리'
 '중앙멀티플렉스개발' '차이나랩' '차이코퍼레이션' '필름몬스터' '메가박스중앙' '미디어프린팅넷' '스튜디오버드' '우성홀딩스'
 '제이티비씨' '제이티비씨미디어컴' '제이티비씨미디어텍' '제이티비씨스튜디오' '제이티비씨스포츠' '제이티비씨포'
 '제이티비씨플러스' '조인디' '조인스중앙' '중앙데일리' '중앙일보' '중앙일보에스' '중앙일보엠앤피' '중앙프라퍼티'
 '허스트중앙' '휘닉스중앙' '휘닉스중앙에프앤비' '휘닉스중앙제주' '휘닉스평창' '베이스인베스트먼트']
['다보중앙' '미디어링크' '제이콘텐트리' '차이나랩' '필름몬스터' '메가박스중앙' '미디어프린팅넷' '제이티비씨'
 '제이티비씨미디어컴' '제이티비씨미디어텍' '제이티비씨스튜디오' '제이티비씨스포츠' '제이티비씨포' '제이티비씨플러스' '조인디'
 '조인스중앙' '중앙데일리' '중앙일보' '중앙일보에스' '중앙일보엠앤피' '허스트중앙' '휘닉스중앙' '휘닉스중앙에프앤비'
 '휘닉스중앙제주' '휘닉스평창' '중앙홀딩스' '제이씨엔' '스튜디오버드' '소셜클럽' '아임포트' '비에이엔터테인먼트'
 '드라마하우스스튜디오' '그린터틀' '이태원클라쓰문화산업' 'JoongangilboUSA,Inc' '중앙리조트투자'
 '베이스에이스트링창업벤처전문사모투자' '베이스이스트링창업벤처전문사모투자' '스케일업플랫폼창업벤처전문사모투자']


매출회사                매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)  국내매출액 해외매출액
0        중앙홀딩스                다보중앙      6       36498         NaN  37257   NaN
1        중앙홀딩스               미디어링크    174       36498         NaN  37257   NaN
2        중앙홀딩스              제이콘텐트리    656       36498         NaN  37257   NaN
3        중앙홀딩스                차이나랩     75       36498         NaN  37257   NaN
4        중앙홀딩스               필름몬스터     20       36498         NaN  37257   NaN
..         ...                 ...    ...         ...         ...    ...   ...
235    휘닉스중앙제주               휘닉스중앙   1352        1853         NaN  23446   NaN
236      휘닉스평창               휘닉스중앙  17265       17265         NaN  17308   NaN
237  베이스인베스트먼트  베이스에이스트링창업벤처전문사모투자    711        1371         NaN   1709   NaN
238  베이스인베스트먼트   베이스이스트링창업벤처전문사모투자    572        1371         NaN   1709   NaN
239  베이스인베스트먼트   스케일업플랫폼창업벤처전문사모투자     88        1371         NaN   1709   NaN

[240 rows x 7 columns]

# 대방건설

In [672]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('대방건설', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['대방건설' '대방산업개발' '대덕하우징씨스템' '지유인터내셔날' '디비일산' '디케이일산']
['대방하우징' '대방주택' '노블랜드' '디비건설' '디비산업개발' '대방이노베이션' '대방이엔씨' '대방개발기업' '디비개발'
 '엔비건설' '디비종합건설' '디비주택' '대방일산디엠시티' '대방덕은' '디엠개발' '대방산업개발' '대방산업개발동탄'
 '대방건설' '엘리움' '엘리움개발' '엘리움건설' '엘리움주텍' '엘리움주택개발']


매출회사      매입회사     매출액 국내계열사계(매출액)  해외계열사계(매출액)    국내매출액  해외매출액
0       대방건설     대방하우징  153182      970711          NaN  1557457    NaN
1       대방건설      대방주택  103392      970711          NaN  1557457    NaN
2       대방건설      노블랜드   86208      970711          NaN  1557457    NaN
3       대방건설      디비건설  261352      970711          NaN  1557457    NaN
4       대방건설    디비산업개발   78658      970711          NaN  1557457    NaN
5       대방건설   대방이노베이션       2      970711          NaN  1557457    NaN
6       대방건설     대방이엔씨       1      970711          NaN  1557457    NaN
7       대방건설    대방개발기업     655      970711          NaN  1557457    NaN
8       대방건설      디비개발       5      970711          NaN  1557457    NaN
9       대방건설      엔비건설   73890      970711          NaN  1557457    NaN
10      대방건설    디비종합건설   49110      970711          NaN  1557457    NaN
11      대방건설      디비주택   31269      970711          NaN  1557457    NaN
12      대방건설  대방일산디엠시티   39058      970711          NaN  1557457    NaN
13      대방건설      대방덕은   54864      970711          NaN  1557457    NaN
14      대방건설      디엠개발   36557      970711          NaN  1557457    NaN
15      대방건설    대방산업개발     125      970711          NaN  1557457    NaN
16      대방건설  대방산업개발동탄    2383      970711          NaN  1557457    NaN
17    대방산업개발      대방건설    5833       61693          NaN    74660    NaN
18    대방산업개발      디엠개발       0       61693          NaN    74660    NaN
19    대방산업개발  대방산업개발동탄   45128       61693          NaN    74660    NaN
20    대방산업개발       엘리움    1984       61693          NaN    74660    NaN
21    대방산업개발     엘리움개발    1984       61693          NaN    74660    NaN
22    대방산업개발     엘리움건설    2886       61693          NaN    74660    NaN
23    대방산업개발     엘리움주텍     992       61693          NaN    74660    NaN
24    대방산업개발   엘리움주택개발    2886       61693          NaN    74660    NaN
25  대덕하우징씨스템      대방건설    1929        3063          NaN     8119    NaN
26  대덕하우징씨스템     대방하우징      20        3063          NaN     8119    NaN
27  대덕하우징씨스템      대방주택     411        3063          NaN     8119    NaN
28  대덕하우징씨스템      노블랜드      86        3063          NaN     8119    NaN
29  대덕하우징씨스템      디비건설     222        3063          NaN     8119    NaN
30  대덕하우징씨스템    디비종합건설      14        3063          NaN     8119    NaN
31  대덕하우징씨스템      대방덕은       2        3063          NaN     8119    NaN
32  대덕하우징씨스템      디엠개발      47        3063          NaN     8119    NaN
33  대덕하우징씨스템    대방산업개발     332        3063          NaN     8119    NaN
34   지유인터내셔날      대방건설     144         196          NaN      391    NaN
35   지유인터내셔날    대방산업개발      52         196          NaN      391    NaN
36      디비일산  대방일산디엠시티   10107       10107          NaN    10107    NaN
37     디케이일산  대방일산디엠시티   10107       10107          NaN    10107    NaN

# 현대해상화재보험

In [670]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('현대해상', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['현대해상화재보험' '에이치지이니셔티브' '현대인베스트먼트글로벌전문투자형사모부동산1호투자' '현대인베스트먼트자산운용'
 '현대하이라이프손해사정' '현대하이카손해사정' '엠지알브이바이브' '솔로몬엠' '솔로몬테크노서플라이' '현대씨앤알'
 '현대에이치디에스']
['현대하이카손해사정' '현대하이라이프손해사정' '현대인베스트자산운용' '현대씨앤알' '현대에이치디에스' '솔로몬테크노서플라이'
 '밀텍' '현대재산보험(중국)유한공사' '에이치지이니셔티브시몬느임팩트제1호창업벤처전문사모투자' 'SASCornillon'
 '현대해상화재보험' '엠지알브이' 'SolomonEuropeGmbH']


매출회사                          매입회사     매출액  \
0                    현대해상화재보험                     현대하이카손해사정    3732   
1                    현대해상화재보험                   현대하이라이프손해사정    1659   
2                    현대해상화재보험                    현대인베스트자산운용     102   
3                    현대해상화재보험                         현대씨앤알    7558   
4                    현대해상화재보험                      현대에이치디에스     324   
5                    현대해상화재보험                    솔로몬테크노서플라이     128   
6                    현대해상화재보험                            밀텍       1   
7                    현대해상화재보험                현대재산보험(중국)유한공사     372   
8                   에이치지이니셔티브  에이치지이니셔티브시몬느임팩트제1호창업벤처전문사모투자      78   
9   현대인베스트먼트글로벌전문투자형사모부동산1호투자                  SASCornillon    9198   
10               현대인베스트먼트자산운용                      현대해상화재보험    1893   
11                현대하이라이프손해사정                      현대해상화재보험   68461   
12                현대하이라이프손해사정                         현대씨앤알      10   
13                현대하이라이프손해사정                현대재산보험(중국)유한공사      31   
14                  현대하이카손해사정                      현대해상화재보험  120114   
15                   엠지알브이바이브                         엠지알브이      21   
16                       솔로몬엠                    솔로몬테크노서플라이    1410   
17                 솔로몬테크노서플라이                      현대에이치디에스    1842   
18                 솔로몬테크노서플라이             SolomonEuropeGmbH     234   
19                      현대씨앤알                      현대해상화재보험   90535   
20                      현대씨앤알                     현대하이카손해사정     598   
21                      현대씨앤알                   현대하이라이프손해사정     507   
22                      현대씨앤알                    현대인베스트자산운용       3   
23                      현대씨앤알                      현대에이치디에스      75   
24                   현대에이치디에스                      현대해상화재보험   72327   
25                   현대에이치디에스                     현대하이카손해사정     712   
26                   현대에이치디에스                   현대하이라이프손해사정     687   
27                   현대에이치디에스                    현대인베스트자산운용      94   
28                   현대에이치디에스                         현대씨앤알    1401   

   국내계열사계(매출액) 해외계열사계(매출액)     국내매출액    해외매출액  
0        13503         372  16131807  1416841  
1        13503         372  16131807  1416841  
2        13503         372  16131807  1416841  
3        13503         372  16131807  1416841  
4        13503         372  16131807  1416841  
5        13503         372  16131807  1416841  
6        13503         372  16131807  1416841  
7        13503         372  16131807  1416841  
8           78         NaN      1192      953  
9          NaN        9198       NaN     9198  
10        1893         NaN     20516      NaN  
11       68471          31     68708       31  
12       68471          31     68708       31  
13       68471          31     68708       31  
14      120114         NaN    125694      NaN  
15          21         NaN        21      NaN  
16        1410         NaN      4703      NaN  
17        1842         234     42191     1249  
18        1842         234     42191     1249  
19       91718         NaN    173424      NaN  
20       91718         NaN    173424      NaN  
21       91718         NaN    173424      NaN  
22       91718         NaN    173424      NaN  
23       91718         NaN    173424      NaN  
24       75221         NaN     93886      NaN  
25       75221         NaN     93886      NaN  
26       75221         NaN     93886      NaN  
27       75221         NaN     93886      NaN  
28       75221         NaN     93886      NaN

# 엠디엠

In [668]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('엠디엠', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['엠디엠' '엠디엠에프엔씨' '엠디엠투자운용' '엠디엠플러스' '더엠리테일' '엠앤케이' '한국자산신탁' '한국자산에셋운용'
 '한국자산캐피탈']
['엠디엠플러스' '엠디엠' '엠디엠투자운용' '엠앤케이' '한국자산신탁' '한국자산에셋운용' '한국자산캐피탈'
 '엠프라퍼티제1호위탁관리부동산투자회사' '원트웬티파이브피에프브이' '엠디엠에프엔씨' '엠스페이스한남피에프브이'
 '엠스페이스대명피에프브이' '카이트제십육호위탁관리부동산투자회사' '카이트제십팔호위탁관리부동산투자회사' '더엠리테일'
 '에스비씨피에프브이']


매출회사                 매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액  \
0        엠디엠               엠디엠플러스    66           66          NaN     6420   
1    엠디엠에프엔씨                  엠디엠    40          535          NaN     6795   
2    엠디엠에프엔씨               엠디엠플러스   180          535          NaN     6795   
3    엠디엠에프엔씨              엠디엠투자운용     9          535          NaN     6795   
4    엠디엠에프엔씨                 엠앤케이    52          535          NaN     6795   
5    엠디엠에프엔씨               한국자산신탁   211          535          NaN     6795   
6    엠디엠에프엔씨             한국자산에셋운용    28          535          NaN     6795   
7    엠디엠에프엔씨              한국자산캐피탈    15          535          NaN     6795   
8    엠디엠투자운용  엠프라퍼티제1호위탁관리부동산투자회사   150          363          NaN     1948   
9    엠디엠투자운용         원트웬티파이브피에프브이   213          363          NaN     1948   
10    엠디엠플러스                  엠디엠    37         1112          NaN  1232656   
11    엠디엠플러스              엠디엠투자운용     7         1112          NaN  1232656   
12    엠디엠플러스              엠디엠에프엔씨   689         1112          NaN  1232656   
13    엠디엠플러스               한국자산신탁   355         1112          NaN  1232656   
14    엠디엠플러스             한국자산에셋운용    15         1112          NaN  1232656   
15    엠디엠플러스              한국자산캐피탈     9         1112          NaN  1232656   
16     더엠리테일                  엠디엠  1596         3136          NaN     3141   
17     더엠리테일               엠디엠플러스  1540         3136          NaN     3141   
18      엠앤케이               엠디엠플러스  6008         6008          NaN    29436   
19    한국자산신탁                  엠디엠    73         6878          NaN   218451   
20    한국자산신탁               엠디엠플러스  3874         6878          NaN   218451   
21    한국자산신탁                 엠앤케이   958         6878          NaN   218451   
22    한국자산신탁         엠스페이스한남피에프브이    30         6878          NaN   218451   
23    한국자산신탁         엠스페이스대명피에프브이     1         6878          NaN   218451   
24    한국자산신탁         원트웬티파이브피에프브이    72         6878          NaN   218451   
25    한국자산신탁   카이트제십육호위탁관리부동산투자회사   466         6878          NaN   218451   
26    한국자산신탁   카이트제십팔호위탁관리부동산투자회사   477         6878          NaN   218451   
27    한국자산신탁                더엠리테일   114         6878          NaN   218451   
28    한국자산신탁            에스비씨피에프브이   245         6878          NaN   218451   
29    한국자산신탁              한국자산캐피탈   568         6878          NaN   218451   
30  한국자산에셋운용         엠스페이스대명피에프브이  1400         1400          NaN     8325   
31   한국자산캐피탈                  엠디엠     6          327          NaN    28950   
32   한국자산캐피탈               엠디엠플러스    19          327          NaN    28950   
33   한국자산캐피탈         엠스페이스대명피에프브이   302          327          NaN    28950   

    해외매출액  
0     NaN  
1     NaN  
2     NaN  
3     NaN  
4     NaN  
5     NaN  
6     NaN  
7     NaN  
8     NaN  
9     NaN  
10    NaN  
11    NaN  
12    NaN  
13    NaN  
14    NaN  
15    NaN  
16    NaN  
17    NaN  
18    NaN  
19    NaN  
20    NaN  
21    NaN  
22    NaN  
23    NaN  
24    NaN  
25    NaN  
26    NaN  
27    NaN  
28    NaN  
29    NaN  
30    NaN  
31    NaN  
32    NaN  
33    NaN

# 아이에스지주

In [667]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('아이에스지주', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['아이에스지주' '아이에스동서' '영풍파일' '중앙레미콘' '서현관리' '바운스' '오션디앤씨' '일신홀딩스' '케이알에너지'
 '티씨이' '인선이엔티' '인선기업' '인선모터스' '파주비앤알' '영흥산업환경' '조치원환경' '영흥토건' '씨에이씨자산운용']
['일신홀딩스' '아이에스지주' '아이에스해운' '영풍파일' '다온이앤씨' '디에스이앤씨' '새빛이앤씨' '유라이앤씨' '도담이앤씨'
 '초아건설' '다솜건설' '아람이앤씨' '서현관리' '해니건설' '나래이앤씨' '티와이건설' '바운스' '일신개발' '아크로건설'
 '인하우스건설' '도원건설' '동서건설' '이누스건설' '트리엔파트너스' '아모르소프트' '아이에스동서' '씨에이씨자산운용'
 '부산블루코스트' '케이알에너지' '인선모터스' 'TCEVINADENIMJOINTSTOCKCOMPANY'
 'TCEJEANSCO.LTD' '인선기업' '골든에코' '영흥산업환경' '인선이엔티']


매출회사                           매입회사   매출액 국내계열사계(매출액) 해외계열사계(매출액)  \
0     아이에스지주                          일신홀딩스     2           2         NaN   
1     아이에스동서                         아이에스지주    12         409         NaN   
2     아이에스동서                         아이에스해운     1         409         NaN   
3     아이에스동서                           영풍파일   243         409         NaN   
4     아이에스동서                          다온이앤씨     1         409         NaN   
5     아이에스동서                         디에스이앤씨     1         409         NaN   
6     아이에스동서                          새빛이앤씨     1         409         NaN   
7     아이에스동서                          유라이앤씨    32         409         NaN   
8     아이에스동서                          도담이앤씨     1         409         NaN   
9     아이에스동서                           초아건설     1         409         NaN   
10    아이에스동서                           다솜건설     1         409         NaN   
11    아이에스동서                          아람이앤씨     1         409         NaN   
12    아이에스동서                           서현관리    19         409         NaN   
13    아이에스동서                           해니건설     1         409         NaN   
14    아이에스동서                          나래이앤씨     1         409         NaN   
15    아이에스동서                          티와이건설     1         409         NaN   
16    아이에스동서                            바운스    78         409         NaN   
17    아이에스동서                           일신개발     2         409         NaN   
18    아이에스동서                          아크로건설     1         409         NaN   
19    아이에스동서                         인하우스건설     1         409         NaN   
20    아이에스동서                           도원건설     1         409         NaN   
21    아이에스동서                           동서건설     1         409         NaN   
22    아이에스동서                          이누스건설     1         409         NaN   
23    아이에스동서                        트리엔파트너스     6         409         NaN   
24    아이에스동서                         아모르소프트     1         409         NaN   
25      영풍파일                         아이에스동서  2341        2341         NaN   
26     중앙레미콘                           영풍파일    13          13         NaN   
27      서현관리                         아이에스동서   196         350         NaN   
28      서현관리                            바운스   154         350         NaN   
29       바운스                         아이에스지주     6           7         NaN   
30       바운스                       씨에이씨자산운용     1           7         NaN   
31     오션디앤씨                         아이에스지주    13         337         NaN   
32     오션디앤씨                         아이에스동서   322         337         NaN   
33     오션디앤씨                        부산블루코스트     1         337         NaN   
34     오션디앤씨                          일신홀딩스     1         337         NaN   
35     일신홀딩스                         케이알에너지   589         589         NaN   
36    케이알에너지                          인선모터스  2837        2837         NaN   
37       티씨이                         아이에스동서     2           7        1323   
38       티씨이                          일신홀딩스     5           7        1323   
39       티씨이  TCEVINADENIMJOINTSTOCKCOMPANY   533           7        1323   
40       티씨이                 TCEJEANSCO.LTD   790           7        1323   
41     인선이엔티                         아이에스동서    98        4449         NaN   
42     인선이엔티                         케이알에너지  1008        4449         NaN   
43     인선이엔티                           인선기업     4        4449         NaN   
44     인선이엔티                          인선모터스  1910        4449         NaN   
45     인선이엔티                           골든에코     4        4449         NaN   
46     인선이엔티                         영흥산업환경  1425        4449         NaN   
47      인선기업                         아이에스동서   409         420         NaN   
48      인선기업                          인선이엔티    11         420         NaN   
49     인선모터스                          인선이엔티    61         141         NaN   
50     인선모터스                           인선기업    16         141         NaN   
51  

# 장금상선

In [665]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('장금상선', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['장금상선' '와이제이씨' '피엔씨티' '한성라인' '경평물류' '광양항서부컨테이너터미널' '국양로지텍' '국양해운'
 '부산항터미널' '슈퍼랙' '시노코쉽매니지먼트' '시노코탱커' '시노코페트로케미컬' '장금마리타임' '조강해운' '퍼시픽엔지니어링'
 '흥아라인']
['장금상선' '와이제이씨' '한성라인' '경평물류' '광양항서부컨테이너터미널' '국양로지텍' '국양해운' '부산항터미널' '슈퍼랙'
 '시노코쉽매니지먼트' '시노코탱커' '시노코페트로케미컬' '장금마리타임' '퍼시픽엔지니어링' '흥아라인'
 'TIANJINYUJIACHANGINTERNATIONALFORWARDING' 'SINOKORBULKPTE.LTD']


매출회사                                      매입회사    매출액 국내계열사계(매출액)  \
0    장금상선                                      장금상선      0       33269   
1    장금상선                                     와이제이씨    296       33269   
2    장금상선                                      한성라인  11735       33269   
3    장금상선                                      경평물류     14       33269   
4    장금상선                              광양항서부컨테이너터미널      0       33269   
..    ...                                       ...    ...         ...   
284  흥아라인                                 시노코페트로케미컬      0        5266   
285  흥아라인                                    장금마리타임      0        5266   
286  흥아라인                                  퍼시픽엔지니어링      0        5266   
287  흥아라인                                      흥아라인      0        5266   
289  흥아라인  TIANJINYUJIACHANGINTERNATIONALFORWARDING      0        5266   

    해외계열사계(매출액)   국내매출액   해외매출액  
0          3421  595391  565416  
1          3421  595391  565416  
2          3421  595391  565416  
3          3421  595391  565416  
4          3421  595391  565416  
..          ...     ...     ...  
284           0   34910   49054  
285           0   34910   49054  
286           0   34910   49054  
287           0   34910   49054  
289           0   34910   49054  

[273 rows x 7 columns]

In [666]:
dfLong = api.getTransTable('장금상선', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['장금상선' '와이제이씨' '한성라인' '경평물류' '광양항서부컨테이너터미널' '국양로지텍' '국양해운' '부산항터미널'
 '시노코쉽매니지먼트' '에스엠상선광양터미널' '장금마리타임' '조강해운' '퍼시픽엔지니어링' '흥아라인']
['와이제이씨' '한성라인' '경평물류' '국양로지텍' '국양해운' '슈퍼랙' '시노코쉽매니지먼트' '시노코탱커'
 '시노코페트로케미컬' '장금마리타임' '퍼시픽엔지니어링' '흥아라인'
 'TIANJINYUJIACHANGINTERNATIONALFORWARDING' '장금상선' '에스엠상선광양터미널'
 '광양항서부컨테이너터미널' '부산항터미널']


매출회사    매입회사   매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액   해외매출액
0   장금상선   와이제이씨   529      120650        3140  690953  518181
1   장금상선    한성라인  9807      120650        3140  690953  518181
2   장금상선    경평물류    23      120650        3140  690953  518181
3   장금상선   국양로지텍   566      120650        3140  690953  518181
4   장금상선    국양해운   754      120650        3140  690953  518181
..   ...     ...   ...         ...         ...     ...     ...
69  흥아라인   와이제이씨    41       74404         NaN  248521  386468
70  흥아라인    한성라인  2050       74404         NaN  248521  386468
71  흥아라인    경평물류    39       74404         NaN  248521  386468
72  흥아라인   국양로지텍   146       74404         NaN  248521  386468
73  흥아라인  장금마리타임   177       74404         NaN  248521  386468

[74 rows x 7 columns]

# 삼양

In [647]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('삼양홀딩스', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['삼양홀딩스' '삼양사' '삼양바이오팜' '삼양화성' '삼양이노켐' '삼양화인테크놀로지' '삼양데이타시스템' '삼양에프앤비'
 '메디켐']
['삼양사' '삼양패키징' '케이씨아이' '삼양바이오팜' '삼남석유화학' '삼양화성' '삼양이노켐' '삼양화인테크놀로지'
 '삼양데이타시스템' '삼양에프앤비' '메디켐' 'SamyangEngineeringPlastics(Shanghai)Co.,Ltd.'
 'SamyangEPHungaryCo.,Ltd.' '삼양홀딩스' 'SamyangEPvietnamLLC']


매출회사                                          매입회사     매출액  \
0       삼양홀딩스                                           삼양사   24632   
1       삼양홀딩스                                         삼양패키징    5277   
2       삼양홀딩스                                         케이씨아이      89   
3       삼양홀딩스                                        삼양바이오팜    3247   
4       삼양홀딩스                                        삼남석유화학    1096   
5       삼양홀딩스                                          삼양화성    1032   
6       삼양홀딩스                                         삼양이노켐    1114   
7       삼양홀딩스                                     삼양화인테크놀로지     571   
8       삼양홀딩스                                      삼양데이타시스템    2086   
9       삼양홀딩스                                        삼양에프앤비     404   
10      삼양홀딩스                                           메디켐      53   
11      삼양홀딩스  SamyangEngineeringPlastics(Shanghai)Co.,Ltd.     248   
12      삼양홀딩스                      SamyangEPHungaryCo.,Ltd.     297   
13        삼양사                                         삼양홀딩스     214   
14        삼양사                                         삼양패키징    9429   
15        삼양사                                         케이씨아이      41   
16        삼양사                                        삼양바이오팜     229   
17        삼양사                                        삼남석유화학     156   
18        삼양사                                          삼양화성    1323   
19        삼양사                                         삼양이노켐     578   
20        삼양사                                     삼양화인테크놀로지    8926   
21        삼양사                                      삼양데이타시스템      23   
22        삼양사                                        삼양에프앤비    4328   
23        삼양사  SamyangEngineeringPlastics(Shanghai)Co.,Ltd.   20972   
24        삼양사                      SamyangEPHungaryCo.,Ltd.   14599   
25     삼양바이오팜                                           삼양사     771   
26     삼양바이오팜                                           메디켐      86   
27       삼양화성                                           삼양사  182475   
28      삼양이노켐                                          삼양화성  140114   
29  삼양화인테크놀로지                                           삼양사   23863   
30   삼양데이타시스템                                         삼양홀딩스    1154   
31   삼양데이타시스템                                           삼양사    9501   
32   삼양데이타시스템                                         삼양패키징    1511   
33   삼양데이타시스템                                         케이씨아이      79   
34   삼양데이타시스템                                        삼양바이오팜    2120   
35   삼양데이타시스템                                        삼남석유화학     997   
36   삼양데이타시스템                                          삼양화성     562   
37   삼양데이타시스템                                         삼양이노켐     254   
38   삼양데이타시스템                                     삼양화인테크놀로지     240   
39   삼양데이타시스템                                        삼양에프앤비      91   
40   삼양데이타시스템  SamyangEngineeringPlastics(Shanghai)Co.,Ltd.       7   
41   삼양데이타시스템                      SamyangEPHungaryCo.,Ltd.       2   
42   삼양데이타시스템                           SamyangEPvietnamLLC     138   
43     삼양에프앤비                                         삼양홀딩스      71   
44     삼양에프앤비                                           삼양사     247   
45     삼양에프앤비                                         삼양패키징       7   
46     삼양에프앤비                                        삼양바이오팜      52   
47     삼양에프앤비                                      삼양데이타시스템      15   
48        메디켐                                        삼양바이오팜    2319   
49        메디켐                                          삼양화성     165   

   국내계열사계(매출액) 해외계열사계(매출액)    국내매출액   해외매출액  
0        39601         545    57562     558  
1        39601         545    57562     558  
2        39601         545    57562     558  
3        39601         545    57562     558  
4        39601         545    57562     558  
5        39601         545    57562     558  
6        39601         545    57562     558  
7        39601     

In [648]:
dfLong = api.getTransTable('삼양홀딩스', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['삼양홀딩스' '삼양사' '삼양바이오팜' '삼양화성' '삼양이노켐' '삼양화인테크놀로지' '삼양데이타시스템' '삼양에프앤비'
 '메디켐']
['삼양사' '삼양패키징' '케이씨아이' '삼양바이오팜' '삼남석유화학' '삼양화성' '삼양이노켐' '삼양화인테크놀로지'
 '삼양데이타시스템' '삼양에프앤비' '메디켐' '삼양공정소료(상해)유한공사' '삼양EP헝가리' '삼양홀딩스'
 'SamyangEPvietnamLLC']


매출회사                 매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  \
0       삼양홀딩스                  삼양사   23977       38412         530   113610   
1       삼양홀딩스                삼양패키징    5331       38412         530   113610   
2       삼양홀딩스                케이씨아이     136       38412         530   113610   
3       삼양홀딩스               삼양바이오팜    3340       38412         530   113610   
4       삼양홀딩스               삼남석유화학    1110       38412         530   113610   
5       삼양홀딩스                 삼양화성     837       38412         530   113610   
6       삼양홀딩스                삼양이노켐    1313       38412         530   113610   
7       삼양홀딩스            삼양화인테크놀로지     581       38412         530   113610   
8       삼양홀딩스             삼양데이타시스템    1690       38412         530   113610   
9       삼양홀딩스               삼양에프앤비      37       38412         530   113610   
10      삼양홀딩스                  메디켐      60       38412         530   113610   
11      삼양홀딩스       삼양공정소료(상해)유한공사     248       38412         530   113610   
12      삼양홀딩스              삼양EP헝가리     282       38412         530   113610   
13        삼양사                삼양홀딩스     164        8693       40154  1109688   
14        삼양사                삼양패키징     880        8693       40154  1109688   
15        삼양사                케이씨아이     182        8693       40154  1109688   
16        삼양사               삼양바이오팜     199        8693       40154  1109688   
17        삼양사               삼남석유화학     139        8693       40154  1109688   
18        삼양사                 삼양화성     851        8693       40154  1109688   
19        삼양사                삼양이노켐     742        8693       40154  1109688   
20        삼양사            삼양화인테크놀로지    5011        8693       40154  1109688   
21        삼양사             삼양데이타시스템      28        8693       40154  1109688   
22        삼양사               삼양에프앤비     495        8693       40154  1109688   
23        삼양사                  메디켐       2        8693       40154  1109688   
24        삼양사       삼양공정소료(상해)유한공사   19590        8693       40154  1109688   
25        삼양사              삼양EP헝가리   17390        8693       40154  1109688   
26        삼양사  SamyangEPvietnamLLC    3174        8693       40154  1109688   
27     삼양바이오팜                  삼양사     772         832         NaN    30754   
28     삼양바이오팜                  메디켐      60         832         NaN    30754   
29       삼양화성                  삼양사  170100      170100         NaN    53303   
30      삼양이노켐                 삼양화성  156924      156924         NaN     3075   
31  삼양화인테크놀로지                  삼양사   20840       20840         NaN    10599   
32   삼양데이타시스템                삼양홀딩스    1659       16796         482    71509   
33   삼양데이타시스템                  삼양사    9884       16796         482    71509   
34   삼양데이타시스템                삼양패키징    1632       16796         482    71509   
35   삼양데이타시스템                케이씨아이      35       16796         482    71509   
36   삼양데이타시스템               삼양바이오팜    1629       16796         482    71509   
37   삼양데이타시스템               삼남석유화학     881       16796         482    71509   
38   삼양데이타시스템                 삼양화성     550       16796         482    71509   
39   삼양데이타시스템                삼양이노켐     267       16796         482    71509   
40   삼양데이타시스템            삼양화인테크놀로지     216       16796         482    71509   
41   삼양데이타시스템               삼양에프앤비      43       16796         482    71509   
42   삼양데이타시스템       삼양공정소료(상해)유한공사      36       16796         482    71509   
43   삼양데이타시스템              삼양EP헝가리       6       16796         482    71509   
44   삼양데이타시스템  SamyangEPvietnamLLC     440       16796         482    71509   
45     삼양에프앤비                삼양홀딩스      55         369         NaN     2677   
46     삼양에프앤비                  삼양사     247         369         NaN     2677   
47     삼양에프앤비                삼양패키징       6         369         NaN     2677   
48     삼양에프앤비               삼양바이오팜      50         369         NaN     2677   
49     삼양에프앤비             삼양데이타시스템      11         369     

# IMM인베스트먼트

In [642]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('아이엠엠인베스트먼트', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['아이엠엠인베스트먼트' '아이엠엠스페셜시츄에이션1호사모투자' '아이엠엠인베스트먼트4호사모투자' '아이엠엠인프라제5호사모투자'
 '아이엠엠인프라제7호사모투자' '에트라사모투자' '에트라제2호사모투자' '에트라제3의1호사모투자' '페트라6의1호사모투자'
 '페트라6의알파사모투자' '페트라6호사모투자' '페트라7호사모투자' '하논사모투자' '깜포' '브루넬' '메티스원' '카보드'
 '아이엠엠' '그린에너지' '씨앤비인터내셔날' '이미인' '강동냉장' '나진산업' '농업회사법인수직' '농업회사법인팜에이트'
 '미래원엘름' '미래원티팜' '신대한정유산업' '에코매니지먼트코리아홀딩스' '제이콥헬스케어' '한국환경개발']
['아이엠엠글로벌사모투자' '아이엠엠스페셜시츄에이션1의1호사모투자' '아이엠엠스페셜시츄에이션1호사모투자'
 '아이엠엠인베스트먼트4호사모투자' '아이엠엠인프라제4호사모투자' '아이엠엠인프라제5의1호사모투자' '아이엠엠인프라제5호사모투자'
 '아이엠엠인프라제7호사모투자' '아이엠엠인프라제8호사모투자' '에코리아사모투자' '에트라사모투자' '에트라제2호사모투자'
 '에트라제3의1호사모투자' '페트라6의1호사모투자' '페트라6의알파사모투자' '페트라6호사모투자' '페트라7의베타사모투자'
 '페트라7의알파사모투자' '페트라7호사모투자' '하논사모투자' 'IMMInvestmentGroupJapan' '브루넬'
 'CrianzaAviationLtd.' '베수비오' '에스메랄다' '에트루리' '에끌레르' '조반니' 'PetriniLtd'
 'Petrini2Ltd' '메티스원' '벨리즈투' '이펫' '카베자' '레트로원' '케렌시아원' '롱샹' '나진산업' '강동냉장'
 '에코매니지먼트코리아홀딩스' '아이엠엠인베스트먼트'

매출회사                  매입회사   매출액 국내계열사계(매출액) 해외계열사계(매출액)  국내매출액  \
0      아이엠엠인베스트먼트           아이엠엠글로벌사모투자    57       18214          98  41462   
1      아이엠엠인베스트먼트  아이엠엠스페셜시츄에이션1의1호사모투자    90       18214          98  41462   
2      아이엠엠인베스트먼트    아이엠엠스페셜시츄에이션1호사모투자  1255       18214          98  41462   
3      아이엠엠인베스트먼트      아이엠엠인베스트먼트4호사모투자   176       18214          98  41462   
4      아이엠엠인베스트먼트        아이엠엠인프라제4호사모투자  1261       18214          98  41462   
..            ...                   ...   ...         ...         ...    ...   
73  에코매니지먼트코리아홀딩스               신대한정유산업  1269        3578         NaN  13525   
74  에코매니지먼트코리아홀딩스                한국환경개발   475        3578         NaN  13525   
75        제이콥헬스케어             솔트라이트아이엔씨     2           2         NaN   2869   
76         한국환경개발                   비노텍    13          30         NaN  16296   
77         한국환경개발               신대한정유산업    17          30         NaN  16296   

   해외매출액  
0     98  
1     98  
2     98  
3     98  
4     98  
..   ...  
73   NaN  
74   NaN  
75   NaN  
76   NaN  
77   NaN  

[78 rows x 7 columns]

In [645]:
dfLong = api.getTransTable('아이엠엠인베스트먼트', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['아이엠엠인베스트먼트' '아이씨에이코리아' '아이엠엠글로벌사모투자' '아이엠엠인베스트먼트4호사모투자' '아이엠엠인프라제5호사모투자'
 '아이엠엠인프라제7호사모투자' '아이엠엠인프라제8호사모투자' '하논사모투자' '페트라6호사모투자' '페트라6의1호사모투자'
 '페트라7호사모투자' '에트라사모투자' '에트라제2호사모투자' '에트라제3의1호사모투자' '아이엠엠티이피' '깜포' '브루넬'
 '조반니' '조반니투' '카보드' '메티스원' '바이슨' '아이엠엠' '에코매니지먼트코리아홀딩스' '한국환경개발' '비노텍'
 '신대한정유산업' '에스티에코' '그린에너지' '탑에코' '강동냉장' '나진산업' '드림라인' '드림마크원' '씨앤비인터내셔날'
 '제이콥헬스케어' '이미인' '농업회사법인팜에이트' '농업회사법인플랜티팜']
['아이엠엠글로벌사모투자' '아이엠엠스페셜시츄에이션1호사모투자' '아이엠엠스페셜시츄에이션1의2호사모투자'
 '아이엠엠인베스트먼트4호사모투자' '아이엠엠인프라제4호사모투자' '아이엠엠인프라제5호사모투자' '아이엠엠인프라제5의1호사모투자'
 '아이엠엠인프라제7호사모투자' '아이엠엠인프라제8호사모투자' '하논사모투자' '에코리아사모투자' '페트라6호사모투자'
 '페트라6의1호사모투자' '페트라6의알파사모투자' '페트라7호사모투자' '페트라7의알파사모투자' '페트라7의베타사모투자'
 '에트라사모투자' '에트라제2호사모투자' '에트라제3의1호사모투자' '케이티씨유글로벌파트너쉽사모투자'
 'IMMInvestmentGroupJapan' '아이엠엠티이피' 'CrianzaAviationLtd.' '에스메랄다' '에트루리'
 '베수비오' '조반니' '에끌레르' '롱샹' '발레겐' '메티스원' '카베자' '이펫' '스쿼드원' '구엘' '레트로원'
 '케렌시아원' 'PetriniLtd' 'Petrini2Ltd' 'IMMTEPUSLLC' '나진산업' '강동냉장' '조반니투'
 '팜양주' '에코매니지먼트코리아홀딩스' 'TexasBison,LLC' '아이엠엠인베스트먼트' '한

매출회사                  매입회사   매출액 국내계열사계(매출액) 해외계열사계(매출액)  국내매출액  \
0   아이엠엠인베스트먼트           아이엠엠글로벌사모투자  1795       23626          33  39341   
1   아이엠엠인베스트먼트    아이엠엠스페셜시츄에이션1호사모투자  2421       23626          33  39341   
2   아이엠엠인베스트먼트  아이엠엠스페셜시츄에이션1의2호사모투자    96       23626          33  39341   
3   아이엠엠인베스트먼트      아이엠엠인베스트먼트4호사모투자   115       23626          33  39341   
4   아이엠엠인베스트먼트        아이엠엠인프라제4호사모투자  1255       23626          33  39341   
..         ...                   ...   ...         ...         ...    ...   
90         이미인        이미인(상해)화장품유한공사    39         NaN          39  38678   
91  농업회사법인팜에이트            농업회사법인플랜티팜    35          35         NaN  53164   
92  농업회사법인플랜티팜                  나진산업   800        2116         NaN   8171   
93  농업회사법인플랜티팜            농업회사법인팜에이트  1297        2116         NaN   8171   
94  농업회사법인플랜티팜                 미래원엘름    19        2116         NaN   8171   

    해외매출액  
0     131  
1     131  
2     131  
3     131  
4     131  
..    ...  
90  16116  
91    NaN  
92    NaN  
93    NaN  
94    NaN  

[95 rows x 7 columns]

# 키움증권

In [634]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('키움증권', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['키움증권' '키움예스저축은행' '키움인베스트먼트' '키움투자자산운용' '키움프라이빗에쿼티' '포스코우리이아이지글로벌사모투자'
 '한국전력우리스프랏글로벌사모투자' '한국전력우리스프랏글로벌해외자원개발사모투자' '한국전력우리스프랏글로벌해외자원개발제3차'
 '게티이미지코리아' '다우기술' '다우데이타' '더트루메틱' '동부고속' '리노미디어' '리노시스템즈' '미래테크놀로지'
 '사람인에이치알' '사람인에이치에스' '와이즈버즈' '이매진스' '이머니' '지인21' '키다리스튜디오' '키다리이엔티'
 '키움이앤에스' '에스지서비스' '한국정보인증']
['다우데이타' '다우기술' '키움이앤에스' '사람인에이치알' '한국정보인증' '이머니' '키다리스튜디오' '미래테크놀로지'
 '이매진스' '게티이미지코리아' '키움인베스트먼트' '키움저축은행' '키움투자자산운용' '키움예스저축은행' '키움프라이빗에쿼티'
 '키움증권' '키움뉴마진글로벌파트너쉽사모투자' '키움히어로제2호사모투자' '포스코우리이아이지글로벌사모투자'
 '한국전력우리스프랏글로벌사모투자' '한국전력우리스프랏글로벌해외자원개발사모투자' '키움프라이빗에쿼티코리아와이드파트너스사모투자'
 '키움케이아이피헬스케어플랫폼사모투자' '키움아이온코스닥스케일업창업벤처전문사모투자' 'PWEFCooperatiefU.A'
 '한국전력우리스프랏글로벌해외자원개발제3차' 'COPAEurekaInc.' '와이즈버즈' '리노미디어' '더트루메틱'
 '키움에셋플래너' '사람인에이치에스' '키다리이엔티' '키움캐피탈' '다우재팬' 'WOSMUSAS' '씨알디' '큰나무기술'
 '엔비메디컬']


매출회사      매입회사 매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액 해외매출액
0      키움증권     다우데이타   6         149         NaN  1851506   NaN
1      키움증권      다우기술  22         149         NaN  1851506   NaN
2      키움증권    키움이앤에스   1         149         NaN  1851506   NaN
3      키움증권   사람인에이치알  24         149         NaN  1851506   NaN
4      키움증권    한국정보인증   4         149         NaN  1851506   NaN
..      ...       ...  ..         ...         ...      ...   ...
176  한국정보인증  키움인베스트먼트   0         142         NaN    38913   NaN
177  한국정보인증    키움저축은행  24         142         NaN    38913   NaN
178  한국정보인증  키움투자자산운용   0         142         NaN    38913   NaN
179  한국정보인증  키움예스저축은행   6         142         NaN    38913   NaN
180  한국정보인증      키움증권  15         142         NaN    38913   NaN

[181 rows x 7 columns]

In [635]:
dfLong = api.getTransTable('키움증권', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['키움증권' '키움예스저축은행' '키움인베스트먼트' '키움캐피탈' '키움투자자산운용' '키움프라이빗에쿼티'
 '포스코우리이아이지글로벌사모투자' '게티이미지코리아' '다우기술' '다우데이타' '동부고속' '미래테크놀로지' '사람인에이치알'
 '사람인에이치에스' '와이즈버즈' '이매진스' '이머니' '키다리스튜디오' '키움이앤에스' '에스지서비스' '한국정보인증']
['다우기술' '다우데이타' '미래테크놀로지' '이머니' '키움이앤에스' '키움예스저축은행' '키움저축은행'
 '글로벌강소기업키움엠앤에이전략창업벤처전문사모투자' '키움에셋플래너' '키움인베스트먼트' '키움캐피탈' '키움투자자산운용'
 '키움프라이빗에쿼티' '키움증권' '에스케이에스키움파이오니어사모투자' '키움뉴마진글로벌파트너쉽사모투자' '키움히어로제2호사모투자'
 '포스코우리이아이지글로벌사모투자' '한국전력우리스프랏글로벌사모투자' '한국전력우리스프랏글로벌해외자원개발사모투자'
 '키움아이온코스닥스케일업창업벤처전문사모투자' '키움케이아이피헬스케어플랫폼사모투자' '키움프라이빗에쿼티자이언트사모투자'
 '키움프라이빗에쿼티코리아와이드파트너스사모투자' 'PWEFCooperatiefU.A' '사람인에이치알' '와이즈버즈' '이매진스'
 '키다리스튜디오' '한국정보인증' '리노미디어' '게티이미지코리아' '동부고속' '사람인에이치에스' '키다리이엔티' '다우재팬'
 'DELITOONSAS' '씨알디' '엔비메디컬' 'DAOUSIGNVIETNAMCOMPANYLIMITED']


매출회사                           매입회사 매출액 국내계열사계(매출액) 해외계열사계(매출액)  \
0      키움증권                           다우기술   0        1344         NaN   
1      키움증권                          다우데이타  84        1344         NaN   
2      키움증권                        미래테크놀로지   4        1344         NaN   
3      키움증권                            이머니   0        1344         NaN   
4      키움증권                         키움이앤에스  15        1344         NaN   
..      ...                            ...  ..         ...         ...   
170  한국정보인증                       키움예스저축은행  15         142          37   
171  한국정보인증                         키움저축은행  30         142          37   
172  한국정보인증                        키움에셋플래너   1         142          37   
173  한국정보인증                       키움투자자산운용   0         142          37   
174  한국정보인증  DAOUSIGNVIETNAMCOMPANYLIMITED  37         142          37   

       국내매출액 해외매출액  
0    2628220   NaN  
1    2628220   NaN  
2    2628220   NaN  
3    2628220   NaN  
4    2628220   NaN  
..       ...   ...  
170    41396  1194  
171    41396  1194  
172    41396  1194  
173    41396  1194  
174    41396  1194  

[175 rows x 7 columns]

In [636]:
dfLong = api.getTransTable('키움증권', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['키움증권' '키움인베스트먼트' '키움캐피탈' '키움투자자산운용' '키움프라이빗에쿼티' '포스코우리이아이지글로벌사모투자'
 '게티이미지코리아' '다우기술' '다우데이타' '레진엔터테인먼트' '미래테크놀로지' '사람인에이치알' '사람인에이치에스'
 '와이즈버즈' '이매진스' '이머니' '키다리스튜디오' '키다리이엔티' '키움이앤에스' '에스지서비스' '한국정보인증']
['게티이미지코리아' '다우기술' '다우데이타' '사람인에이치알' '이매진스' '이머니' '키움이앤에스' '키움예스저축은행'
 '키움저축은행' '글로벌강소기업키움엠앤에이전략창업벤처전문사모투자' '키움뉴마진글로벌파트너쉽사모투자' '키움에셋플래너'
 '키움에프앤아이' '키움인베스트먼트' '키움캐피탈' '키움투자자산운용' '키움프라이빗에쿼티' '키움증권'
 '에스케이에스키움파이오니어사모투자' '키움히어로제2호사모투자' '키움히어로제3호사모투자' '키움히어로제4호사모투자'
 '키움프라이빗에쿼티자이언트사모투자' '포스코우리이아이지글로벌사모투자' '한국전력우리스프랏글로벌사모투자'
 '한국전력우리스프랏글로벌해외자원개발사모투자' '키움아이온코스닥스케일업창업벤처전문사모투자' '키움케이아이피헬스케어플랫폼사모투자'
 '키움프라이빗에쿼티앤트맨창업벤처전문사모투자' '키움프라이빗에쿼티에스테틱사모투자' '키움프라이빗에쿼티코리아와이드파트너스사모투자'
 'PWEFCooperatiefU.A' '미래테크놀로지' '와이즈버즈' '키다리스튜디오' '한국정보인증' '동부고속'
 '사람인에이치에스' '키다리이엔티' '다우재팬' 'LezhinEntertainment,LLC'
 'LezhinEntertainment,Corp' 'DELITOONSAS' 'DAOUSIGNVIETNAMCOMPANYLIMITED']


매출회사                           매입회사 매출액 국내계열사계(매출액) 해외계열사계(매출액)  \
0      키움증권                       게티이미지코리아   0        1473         NaN   
1      키움증권                           다우기술   3        1473         NaN   
2      키움증권                          다우데이타  80        1473         NaN   
3      키움증권                        사람인에이치알   1        1473         NaN   
4      키움증권                           이매진스   0        1473         NaN   
..      ...                            ...  ..         ...         ...   
177  한국정보인증                         키움저축은행   2         263          65   
178  한국정보인증                        키움에셋플래너   0         263          65   
179  한국정보인증                          키움캐피탈   0         263          65   
180  한국정보인증                       키움투자자산운용   0         263          65   
181  한국정보인증  DAOUSIGNVIETNAMCOMPANYLIMITED  65         263          65   

       국내매출액 해외매출액  
0    3844513   NaN  
1    3844513   NaN  
2    3844513   NaN  
3    3844513   NaN  
4    3844513   NaN  
..       ...   ...  
177    45105   620  
178    45105   620  
179    45105   620  
180    45105   620  
181    45105   620  

[182 rows x 7 columns]

# AK홀딩스

In [631]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('AK홀딩스', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['에이케이홀딩스' '애경유화' '에이케이앤엠엔바이오팜' '에이케이켐텍' '에이케이에셋' '애경화학' '애경산업' '애경에스티'
 '모두락애경산업' '에이제이피' '제주항공' '모두락' '퍼시픽제3호전문사모부동산투자' '제이에이에스' '에이케이에스앤디'
 '수원애경역사' '평택역사' '마포애경타운' '에이엠플러스자산개발' '에이케이아이에스' '애경개발' '에이케이레저' '애경피앤티'
 '에이텍' '코스파' '한국특수소재' '애드미션' '우영운수' '비컨로지스틱스' '에이엘오']
['애경유화' '에이케이앤엠엔바이오팜' '에이케이켐텍' '애경화학' '애경산업' '애경에스티' '에이제이피' '제주항공'
 '제이에이에스' '에이케이에스앤디' '수원애경역사' '평택역사' '마포애경타운' '에이케이네크워크' '에이엠플러스자산개발'
 '에이엠플러스피에프브이기흥' '에이엠플러스피에프브이위례' '에이케이아이에스' '애경개발' '에이케이레저' '애경피앤티' '에이텍'
 '코스파' '한국특수소재' '애드미션' 'Aekyung(Ningbo)ChemicalCo.,Ltd.'
 'AEKYUNGHONGKONGCOMPANYLIMITED'
 'AEKYUNG(SHANGHAI)INVESTMENT&TRADINGCO.,LTD.' 'AKVINACO.,LTD.'
 'AEKYUNG(FOSHAN)PAINTCO.,LTD.' 'AEKYUNG(TIANJIN)PAINTCO.,LTD.'
 'AKCHEMTECHHANOICO.,Ltd' 'AEKYUNGCHEMTECHINDIAPRIVATELIMITED' '애경레지콘'
 'AEKYUNGCHEMICAL(QINGDAO)CO.,LTD' '에이케이홀딩스' '우영운수' '에이엘오'
 'Aekyung(Shanghai)TradeCo.,Lt

매출회사         매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)  국내매출액 해외매출액
0    에이케이홀딩스         애경유화  2159.0        9388         NaN  33693   NaN
1    에이케이홀딩스  에이케이앤엠엔바이오팜    17.0        9388         NaN  33693   NaN
2    에이케이홀딩스       에이케이켐텍   894.0        9388         NaN  33693   NaN
3    에이케이홀딩스         애경화학   751.0        9388         NaN  33693   NaN
4    에이케이홀딩스         애경산업  2010.0        9388         NaN  33693   NaN
..       ...          ...     ...         ...         ...    ...   ...
192  비컨로지스틱스         애경산업  5190.0        5190         NaN   5190   NaN
193     에이엘오         애경산업  4177.0        4369         NaN   9804   NaN
194     에이엘오        애경피앤티    65.0        4369         NaN   9804   NaN
195     에이엘오          코스파    36.0        4369         NaN   9804   NaN
196     에이엘오           인셋    91.0        4369         NaN   9804   NaN

[197 rows x 7 columns]

In [632]:
dfLong = api.getTransTable('AK홀딩스', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['애경유화' '에이케이앤엠엔바이오팜' '에이케이켐텍' '에이케이에셋' '애경화학' '애경산업' '애경에스티' '모두락애경산업'
 '에이제이피' '제주항공' '모두락' '퍼시픽제3호전문사모부동산투자' '제이에이에스' '에이케이에스앤디' '수원애경역사'
 '평택역사' '마포애경타운' '에이엠플러스자산개발' '에이케이아이에스' '애경개발' '에이케이레저' '애경피앤티' '에이텍'
 '코스파' '애드미션' '우영운수' '비컨로지스틱스']
['에이케이앤엠엔바이오팜' '에이케이켐텍' '애경화학' '애경산업' 'Aekyung(Ningbo)ChemicalCo.,Ltd.'
 'AEKYUNGHONGKONGCOMPANYLIMITED'
 'AEKYUNG(SHANGHAI)INVESTMENT&TRADINGCO.,LTD.' 'AKVINACO.,LTD.' '애경유화'
 '에이텍' 'AEKYUNG(FOSHAN)PAINTCO.,LTD.' 'AKCHEMTECHHANOICO.,Ltd'
 'AEKYUNGCHEMTECHINDIAPRIVATELIMITED' '애경개발' '애경레지콘'
 'AEKYUNGCHEMICAL(QINGDAO)CO.,LTD' '에이케이홀딩스' '한국특수소재' '제주항공' '에이케이에스앤디'
 '수원애경역사' '평택역사' '에이엠플러스자산개발' '에이케이아이에스' '에이케이레저' '애경피앤티' '코스파'
 'Aekyung(Shanghai)TradeCo.,Ltd' '애경에스티' '마포애경타운' '퍼시픽제3호전문사모부동산투자'
 '제이에이에스' '모두락' '에이엠플러스피에프브이기흥' '에이제이피' '애드미션' '에이엠플러스피에프브이산곡' '에이엘오'
 '에이케이에셋']


매출회사                             매입회사     매출액  국내계열사계(매출액)  \
0       애경유화                      에이케이앤엠엔바이오팜  3658.0      12917.0   
1       애경유화                           에이케이켐텍   135.0      12917.0   
2       애경유화                             애경화학  9007.0      12917.0   
3       애경유화                             애경산업   117.0      12917.0   
4       애경유화  Aekyung(Ningbo)ChemicalCo.,Ltd.  1136.0      12917.0   
..       ...                              ...     ...          ...   
151      코스파                              에이텍    98.0         98.0   
152     애드미션                          에이케이홀딩스    12.0         14.0   
153     애드미션                       에이엠플러스자산개발     2.0         14.0   
154     우영운수                             애경산업  1538.0       1538.0   
155  비컨로지스틱스                             애경산업  1765.0       1765.0   

     해외계열사계(매출액)     국내매출액   해외매출액  
0        44272.0  634823.0  172641  
1        44272.0  634823.0  172641  
2        44272.0  634823.0  172641  
3        44272.0  634823.0  172641  
4        44272.0  634823.0  172641  
..           ...       ...     ...  
151          NaN   30179.0     454  
152          NaN    8211.0     NaN  
153          NaN    8211.0     NaN  
154          NaN    1538.0     NaN  
155          NaN    1765.0     NaN  

[156 rows x 7 columns]

In [633]:
dfLong = api.getTransTable('AK홀딩스', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['에이케이홀딩스' '애경유화' '에이케이앤엠엔바이오팜' '에이케이켐텍' '애경특수도료' '애경중부컨트리클럽' '애경화학'
 '애경산업' '애경에스티' '모두락애경산업' '에이제이피' '제주항공' '모두락' '퍼시픽제3호전문사모부동산투자' '제이에이에스'
 '에이케이에스앤디' '수원애경역사' '마포애경타운' '에이엠플러스자산개발' '에이케이아이에스' '애경개발' '에이케이레저'
 '에이텍' '코스파' '우영운수' '비컨로지스틱스']
['애경유화' '에이케이앤엠엔바이오팜' '에이케이켐텍' '애경특수도료' '애경화학' '애경산업' '에이제이피' '제주항공'
 '퍼시픽제3호전문사모부동산투자' '제이에이에스' '에이케이에스앤디' '수원애경역사' '마포애경타운' '에이엠플러스자산개발'
 '에이엠플러스피에프브이기흥' '에이엠플러스피에프브이산곡' '에이케이아이에스' '애경개발' '에이케이레저' '에이텍' '코스파'
 'Aekyung(Ningbo)ChemicalCo.,Ltd.' 'AEKYUNGHONGKONGCOMPANYLIMITED'
 'AEKYUNG(SHANGHAI)INVESTMENT&TRADINGCO.,LTD.' 'AKVINACO.,LTD.'
 'AKCHEMTECHHANOICO.,Ltd' 'AEKYUNGCHEMTECHINDIAPRIVATELIMITED'
 'AEKYUNGCHEMICAL(QINGDAO)CO.,LTD' '에이케이홀딩스' '애경에스티' '모두락'
 'Aekyung(Shanghai)TradeCo.,Ltd' '엘로지스틱스피에프브이' '애드미션' '애경중부컨트리클럽']


매출회사         매입회사      매출액  국내계열사계(매출액) 해외계열사계(매출액)    국내매출액 해외매출액
0    에이케이홀딩스         애경유화   2028.0       9013.0         NaN  45515.0   NaN
1    에이케이홀딩스  에이케이앤엠엔바이오팜     53.0       9013.0         NaN  45515.0   NaN
2    에이케이홀딩스       에이케이켐텍    675.0       9013.0         NaN  45515.0   NaN
3    에이케이홀딩스       애경특수도료    124.0       9013.0         NaN  45515.0   NaN
4    에이케이홀딩스         애경화학    608.0       9013.0         NaN  45515.0   NaN
..       ...          ...      ...          ...         ...      ...   ...
159   에이케이레저         애경개발    255.0        429.0         NaN   4319.0   NaN
160      에이텍         애경산업  20748.0      20748.0         NaN  57528.0   NaN
161      코스파          에이텍    242.0        242.0         NaN  22908.0  1160
162     우영운수         애경산업    212.0        212.0         NaN    368.0   NaN
163  비컨로지스틱스         애경산업   1199.0       1199.0         NaN   1199.0   NaN

[164 rows x 7 columns]

# 넷마블

In [623]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('넷마블', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['넷마블' '마켓프레스' '미디어웹' '에브리플레이' '체리벅스' '퍼니파우' '포플랫' '넷마블네오' '넷마블넥서스'
 '넷마블몬스터' '넷마블앤파크' '넷마블엔투' '아이지에스' '엔탑자산관리' '엠엔비프로덕션' '이데아게임즈' '이츠게임즈'
 '지스퀘어피에프브이' '천백십일']
['체리벅스' '크레노바' '퍼니파우' '포플랫' '넷마블네오' '넷마블넥서스' '넷마블몬스터' '넷마블앤파크' '넷마블엔투'
 '아이지에스' '엠엔비프로덕션' '이데아게임즈' '이츠게임즈' '천백십일' 'EURIGSs.r.o.'
 'IGS&CPhilippinesInc.' 'IGSChina' 'JamCity,Inc' 'JoybombHongKongLtd.'
 'JOYGAMEInteractiveServicesLtd.' 'Netmarble(Thailand)Co.,Ltd.'
 'NetmarbleEMEAFZLLC' 'NetmarbleJapanInc.' 'NetmarbleJoybombInc.'
 'NetmarbleUS,Inc.' 'Netmarble(Beijing)Limited'
 'NetmarbleTurkeySanayiveTicaretA.S.' 'NetmarbleGamesIndonesia,PT' '넷마블'
 '에브리플레이' '미디어웹' '지스퀘어피에프브이']


매출회사                                매입회사     매출액 국내계열사계(매출액)  \
0         넷마블                                체리벅스     268       28628   
1         넷마블                                크레노바       1       28628   
2         넷마블                                퍼니파우     252       28628   
3         넷마블                                 포플랫     201       28628   
4         넷마블                               넷마블네오    1128       28628   
5         넷마블                              넷마블넥서스     876       28628   
6         넷마블                              넷마블몬스터     671       28628   
7         넷마블                              넷마블앤파크     277       28628   
8         넷마블                               넷마블엔투     863       28628   
9         넷마블                               아이지에스      42       28628   
10        넷마블                             엠엔비프로덕션       0       28628   
11        넷마블                              이데아게임즈     250       28628   
12        넷마블                              이데아게임즈     157       28628   
13        넷마블                               이츠게임즈     631       28628   
14        넷마블                                천백십일   23011       28628   
15        넷마블                        EURIGSs.r.o.       0       28628   
16        넷마블                IGS&CPhilippinesInc.       1       28628   
17        넷마블                            IGSChina       1       28628   
18        넷마블                         JamCity,Inc       5       28628   
19        넷마블                 JoybombHongKongLtd.      92       28628   
20        넷마블      JOYGAMEInteractiveServicesLtd.     174       28628   
21        넷마블         Netmarble(Thailand)Co.,Ltd.     107       28628   
22        넷마블                  NetmarbleEMEAFZLLC    1283       28628   
23        넷마블                  NetmarbleJapanInc.       2       28628   
24        넷마블                NetmarbleJoybombInc.      28       28628   
25        넷마블                    NetmarbleUS,Inc.       1       28628   
26        넷마블           Netmarble(Beijing)Limited      19       28628   
27        넷마블  NetmarbleTurkeySanayiveTicaretA.S.       2       28628   
28        넷마블          NetmarbleGamesIndonesia,PT       1       28628   
29      마켓프레스                                 넷마블      13          22   
30      마켓프레스                                체리벅스       8          22   
31      마켓프레스                              넷마블앤파크       1          22   
32       미디어웹                                 넷마블      19         190   
33       미디어웹                              에브리플레이      40         190   
34       미디어웹                              넷마블앤파크      30         190   
35       미디어웹                                천백십일     101         190   
36     에브리플레이                                미디어웹      27          27   
37       체리벅스                                 넷마블    5369        5369   
38       퍼니파우                                 넷마블     257         257   
39        포플랫                                 넷마블    6253        6253   
40      넷마블네오                                 넷마블  144858      144858   
41     넷마블넥서스                                 넷마블   29522       29522   
42     넷마블몬스터                                 넷마블   41621       41621   
43     넷마블앤파크                                 넷마블    6730        6730   
44      넷마블엔투                                 넷마블   37237       37237   
45      아이지에스                                 넷마블   29556       29792   
46      아이지에스                              넷마블몬스터       8       29792   
47      아이지에스                               넷마블엔투      38       29792   
48      아이지에스                                천백십일     190       29792   
49      아이지에스                    NetmarbleUS,Inc.      27       29792   
50     엔탑자산관리                           지스퀘어피에프브이    1127        1127   
51    엠엔비프로덕션                                 넷마블    1383        1383   
52     이데아게임즈                                 넷마블    2131        2131   
53      이츠게임즈                                 넷마블      -1          

In [626]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('넷마블', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['넷마블' '마켓프레스' '미디어웹' '에브리플레이' '구로발게임즈' '넷마블네오' '넷마블넥서스' '넷마블몬스터' '넷마블앤파크'
 '넷마블엔투' '넷마블체리' '넷마블펀' '아이지에스' '엔탑자산관리' '엠엔비프로덕션' '이데아게임즈' '지스퀘어피에프브이']
['포플랫' '구로발게임즈' '넷마블네오' '넷마블넥서스' '넷마블몬스터' '넷마블앤파크' '넷마블엔투' '넷마블체리' '넷마블펀'
 '아이지에스' '엔탑자산관리' '엠엔비프로덕션' '이데아게임즈' '잼팟' 'DigiparkSingaporePTE.LTD.'
 'JamCity,Inc.' 'JOYGAMEInteractiveServicesLtd.'
 'Netmarble(Beijing)Limited' 'Netmarble(Thailand)Co.,Ltd.'
 'NetmarbleEMEAFZLLC' 'NETMARBLEHONGKONGLIMITED' 'NetmarbleJapanInc.'
 'NetmarbleJoyBombInc.' 'NetmarbleTurkeySanayiveTicaretA.S.'
 'NetmarbleUS,Inc.' 'PTNETMARBLEGAMESINDONESIA' 'EURIGSs.r.o.' 'IGSChina'
 'IGS&CPhilippinesInc.' '넷마블' '에브리플레이' '미디어웹' '지스퀘어피에프브이' '지타운피에프브이']


매출회사                                매입회사     매출액 국내계열사계(매출액)  \
0         넷마블                                 포플랫      35       21487   
1         넷마블                              구로발게임즈      71       21487   
2         넷마블                               넷마블네오     185       21487   
3         넷마블                              넷마블넥서스     174       21487   
4         넷마블                              넷마블몬스터     894       21487   
5         넷마블                              넷마블앤파크     355       21487   
6         넷마블                               넷마블엔투     178       21487   
7         넷마블                               넷마블체리      52       21487   
8         넷마블                                넷마블펀      75       21487   
9         넷마블                               아이지에스     273       21487   
10        넷마블                              엔탑자산관리      56       21487   
11        넷마블                             엠엔비프로덕션      17       21487   
12        넷마블                              이데아게임즈      67       21487   
13        넷마블                              이데아게임즈      56       21487   
14        넷마블                                  잼팟   18999       21487   
15        넷마블           DigiparkSingaporePTE.LTD.      12       21487   
16        넷마블                        JamCity,Inc.       0       21487   
17        넷마블      JOYGAMEInteractiveServicesLtd.      49       21487   
18        넷마블           Netmarble(Beijing)Limited    1074       21487   
19        넷마블         Netmarble(Thailand)Co.,Ltd.      48       21487   
20        넷마블                  NetmarbleEMEAFZLLC     333       21487   
21        넷마블            NETMARBLEHONGKONGLIMITED      12       21487   
22        넷마블                  NetmarbleJapanInc.      33       21487   
23        넷마블                NetmarbleJoyBombInc.      68       21487   
24        넷마블  NetmarbleTurkeySanayiveTicaretA.S.      30       21487   
25        넷마블                    NetmarbleUS,Inc.      23       21487   
26        넷마블           PTNETMARBLEGAMESINDONESIA      17       21487   
27        넷마블                        EURIGSs.r.o.       6       21487   
28        넷마블                            IGSChina      37       21487   
29        넷마블                IGS&CPhilippinesInc.      27       21487   
30      마켓프레스                                 넷마블      13          18   
31      마켓프레스                               넷마블네오       2          18   
32      마켓프레스                               넷마블체리       3          18   
33       미디어웹                                 넷마블      66         117   
34       미디어웹                              에브리플레이      31         117   
35       미디어웹                              넷마블앤파크      20         117   
36     에브리플레이                                미디어웹      80          80   
37     구로발게임즈                                 넷마블       0           0   
38      넷마블네오                                 넷마블  105720      105720   
39     넷마블넥서스                                 넷마블   15981       15981   
40     넷마블몬스터                                 넷마블   35511       35511   
41     넷마블앤파크                                 넷마블    6539        6539   
42      넷마블엔투                                 넷마블   27717       27717   
43      넷마블체리                                 넷마블   47475       47475   
44       넷마블펀                                 넷마블   37496       37496   
45      아이지에스                                 넷마블   28467       29228   
46      아이지에스                                 포플랫     137       29228   
47      아이지에스                              넷마블몬스터      56       29228   
48      아이지에스                               넷마블엔투     116       29228   
49      아이지에스                               넷마블체리      24       29228   
50      아이지에스                                  잼팟     428       29228   
51     엔탑자산관리                           지스퀘어피에프브이    1127        2240   
52     엔탑자산관리                            지타운피에프브이    1113        2240   
53    엠엔비프로덕션                                 넷마블    3450        35

In [630]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('넷마블', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['넷마블' '미디어웹' '키링' '구로발게임즈' '넷마블네오' '넷마블넥서스' '넷마블몬스터' '넷마블앤파크' '넷마블에프앤씨'
 '넷마블엔투' '아이지에스' '엔탑자산관리' '엠엔비' '지스퀘어피에프브이' '코웨이' '코웨이엔텍']
['구로발게임즈' '넷마블네오' '넷마블넥서스' '넷마블몬스터' '넷마블앤파크' '넷마블에프앤씨' '넷마블엔투' '아이지에스'
 '엔탑자산관리' '엠엔비' '잼팟' '코웨이' '이데아게임즈' '포플랫' '넷마블체리'
 'DigiparkSingaporePTE.LTD.' 'JoybombHongKongLimited'
 'Netmarble(Beijing)Limited' 'Netmarble(Thailand)Co.,Ltd.'
 'NetmarbleEMEAFZLLC' 'NETMARBLEHONGKONGLIMITED' 'NetmarbleJapanInc.'
 'NetmarbleJoyBombInc.' 'NetmarbleTurkeySanayiveTicaretA.S.'
 'NetmarbleUS,Inc.' 'PT.NETMARBLEGAMESINDONESIA' 'IGSChina'
 'IGS&CPhilippinesInc.' '넷마블' '에브리플레이' '지스퀘어피에프브이' '지타운피에프브이' '미디어웹' '키링'
 '코웨이엔텍' 'CowayChinaCo.,Ltd.' 'COWAY(M)SDN.BHD' 'CowayUSA,Inc.'
 'Coway(Thailand)CompanyLimited' 'PTCOWAYINTERNATIONALINDONESIA'
 'CowayVinaCo.,Ltd.' '포천맑은물']


매출회사                           매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)  \
0     넷마블                         구로발게임즈     42       16698        3310   
1     넷마블                          넷마블네오    355       16698        3310   
2     넷마블                         넷마블넥서스    172       16698        3310   
3     넷마블                         넷마블몬스터    867       16698        3310   
4     넷마블                         넷마블앤파크    340       16698        3310   
..    ...                            ...    ...         ...         ...   
61    코웨이                  CowayUSA,Inc.  59565          99      296384   
62    코웨이  Coway(Thailand)CompanyLimited   7864          99      296384   
63    코웨이  PTCOWAYINTERNATIONALINDONESIA   1815          99      296384   
64    코웨이              CowayVinaCo.,Ltd.    419          99      296384   
65  코웨이엔텍                          포천맑은물   1798        1798         NaN   

      국내매출액   해외매출액  
0    601143  777287  
1    601143  777287  
2    601143  777287  
3    601143  777287  
4    601143  777287  
..      ...     ...  
61  2187064  405483  
62  2187064  405483  
63  2187064  405483  
64  2187064  405483  
65    40132    4004  

[66 rows x 7 columns]

# 하이트진로홀딩스

In [608]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('하이트진로홀딩스', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['하이트진로홀딩스' '하이트진로' '진로소주' '하이트진로산업' '하이트진로음료' '진로양조' '강원물류' '수양물류' '천주물류'
 '블루헤런' '서영이앤티' '연암' '송정' '대우패키지' '대우화학' '대우컴바인']
['하이트진로' '하이트진로음료' '하이트진로홀딩스' '진로소주' '하이트진로산업' '서영이앤티' '연암' '수양물류' '천주물류'
 '강원물류' '진로양조' '송정' '대우패키지' '대우컴바인']


매출회사      매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  해외매출액
0   하이트진로홀딩스     하이트진로   4252        4297         NaN    33230    NaN
1   하이트진로홀딩스   하이트진로음료     45        4297         NaN    33230    NaN
2      하이트진로  하이트진로홀딩스     22        6540       53252  1560537  97085
3      하이트진로      진로소주   1169        6540       53252  1560537  97085
4      하이트진로   하이트진로산업     40        6540       53252  1560537  97085
5      하이트진로   하이트진로음료   3122        6540       53252  1560537  97085
6      하이트진로     서영이앤티   2187        6540       53252  1560537  97085
7       진로소주     하이트진로  20726       20726       21268    20726  21273
8    하이트진로산업     하이트진로  53340       61063         NaN    61222    NaN
9    하이트진로산업      진로소주   7709       61063         NaN    61222    NaN
10   하이트진로산업        연암     14       61063         NaN    61222    NaN
11   하이트진로음료     하이트진로    791         791          35    77255    761
12      진로양조     하이트진로   2318        2318         NaN     2318     80
13      강원물류     하이트진로  14848       14863         NaN    15408    NaN
14      강원물류      수양물류      3       14863         NaN    15408    NaN
15      강원물류      천주물류      1       14863         NaN    15408    NaN
16      강원물류     서영이앤티     11       14863         NaN    15408    NaN
17      수양물류     하이트진로  16201       16207         NaN    16696    NaN
18      수양물류      강원물류      5       16207         NaN    16696    NaN
19      수양물류      천주물류      1       16207         NaN    16696    NaN
20      천주물류     하이트진로   9842        9861         NaN    10114    NaN
21      천주물류      강원물류     17        9861         NaN    10114    NaN
22      천주물류      수양물류      2        9861         NaN    10114    NaN
23      블루헤런     하이트진로    400         400         NaN    12078    NaN
24     서영이앤티     하이트진로  19933       20059         NaN    69704   4879
25     서영이앤티      진로소주     23       20059         NaN    69704   4879
26     서영이앤티   하이트진로산업      7       20059         NaN    69704   4879
27     서영이앤티   하이트진로음료     75       20059         NaN    69704   4879
28     서영이앤티      진로양조      0       20059         NaN    69704   4879
29     서영이앤티      강원물류      7       20059         NaN    69704   4879
30     서영이앤티      수양물류      7       20059         NaN    69704   4879
31     서영이앤티      천주물류      5       20059         NaN    69704   4879
32     서영이앤티        연암      2       20059         NaN    69704   4879
33        연암     하이트진로   3155        3659         NaN    18440    NaN
34        연암   하이트진로산업      2        3659         NaN    18440    NaN
35        연암   하이트진로음료    340        3659         NaN    18440    NaN
36        연암        송정      4        3659         NaN    18440    NaN
37        연암     대우패키지     78        3659         NaN    18440    NaN
38        연암     대우컴바인     80        3659         NaN    18440    NaN
39        송정        연암    492         492         NaN     6552    NaN
40     대우패키지     하이트진로   3458        4244         NaN     8195    NaN
41     대우패키지      진로소주    669        4244         NaN     8195    NaN
42     대우패키지   하이트진로음료    117        4244         NaN     8195    NaN
43      대우화학     하이트진로   3236        4853          13     8770     13
44      대우화학      진로소주    111        4853          13     8770     13
45      대우화학   하이트진로음료    860        4853          13     8770     13
46      대우화학     대우패키지    639        4853          13     8770     13
47      대우화학     대우컴바인      7        4853          13     8770     13
48     대우컴바인     하이트진로   2689       10331         NaN    10365    NaN
49     대우컴바인   하이트진로음료   6907       10331         NaN    10365    NaN
50     대우컴바인     대우패키지    735       10331         NaN    10365    NaN

In [609]:
dfLong = api.getTransTable('하이트진로홀딩스', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['하이트진로홀딩스' '하이트진로' '진로소주' '하이트진로산업' '하이트진로음료' '진로양조' '강원물류' '수양물류' '천주물류'
 '블루헤런' '서영이앤티' '연암' '송정' '대우화학' '대우패키지' '대우컴바인']
['하이트진로' '하이트진로음료' '하이트진로홀딩스' '진로소주' '하이트진로산업' '서영이앤티' '수양물류' '천주물류'
 '강원물류' '연암' '대우화학' '송정' '대우패키지' '대우컴바인']


매출회사      매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  해외매출액
0   하이트진로홀딩스     하이트진로   4738        4797         NaN    33791    NaN
1   하이트진로홀딩스   하이트진로음료     59        4797         NaN    33791    NaN
2      하이트진로  하이트진로홀딩스     21        6284       51953  1732692  97455
3      하이트진로      진로소주    952        6284       51953  1732692  97455
4      하이트진로   하이트진로산업    103        6284       51953  1732692  97455
5      하이트진로   하이트진로음료   3480        6284       51953  1732692  97455
6      하이트진로     서영이앤티   1728        6284       51953  1732692  97455
7       진로소주     하이트진로  21560       21560       17363    21560  17366
8    하이트진로산업     하이트진로  82289       90030          41    90250     41
9    하이트진로산업      진로소주   7733       90030          41    90250     41
10   하이트진로산업   하이트진로음료      8       90030          41    90250     41
11   하이트진로음료     하이트진로    852         852          17    96818    497
12      진로양조     하이트진로   2551        2551         NaN     2551    121
13      강원물류     하이트진로  19465       19469         NaN    19868    NaN
14      강원물류      수양물류      3       19469         NaN    19868    NaN
15      강원물류      천주물류      1       19469         NaN    19868    NaN
16      수양물류     하이트진로  19968       19979         NaN    20546    NaN
17      수양물류      강원물류      8       19979         NaN    20546    NaN
18      수양물류      천주물류      3       19979         NaN    20546    NaN
19      천주물류     하이트진로  10974       10995         NaN    11354    NaN
20      천주물류      강원물류      0       10995         NaN    11354    NaN
21      천주물류      수양물류     21       10995         NaN    11354    NaN
22      블루헤런     하이트진로    400         400         NaN    13243    NaN
23     서영이앤티     하이트진로  19948       20262         NaN    86809   3956
24     서영이앤티      진로소주     44       20262         NaN    86809   3956
25     서영이앤티   하이트진로산업      5       20262         NaN    86809   3956
26     서영이앤티   하이트진로음료    244       20262         NaN    86809   3956
27     서영이앤티      강원물류      7       20262         NaN    86809   3956
28     서영이앤티      수양물류      7       20262         NaN    86809   3956
29     서영이앤티      천주물류      4       20262         NaN    86809   3956
30     서영이앤티        연암      1       20262         NaN    86809   3956
31     서영이앤티      대우화학      2       20262         NaN    86809   3956
32        연암     하이트진로   4495        5307         NaN    20042   1150
33        연암   하이트진로산업      1        5307         NaN    20042   1150
34        연암   하이트진로음료    644        5307         NaN    20042   1150
35        연암        송정      4        5307         NaN    20042   1150
36        연암     대우패키지     62        5307         NaN    20042   1150
37        연암     대우컴바인    101        5307         NaN    20042   1150
38        송정        연암    473         473         NaN     8270    NaN
39      대우화학     하이트진로  25694       26414          13    30317     13
40      대우화학      진로소주    110       26414          13    30317     13
41      대우화학   하이트진로음료    454       26414          13    30317     13
42      대우화학     대우패키지    156       26414          13    30317     13
43     대우패키지     하이트진로   1317        1949          11     8458     11
44     대우패키지      진로소주    622        1949          11     8458     11
45     대우패키지   하이트진로음료     10        1949          11     8458     11
46     대우컴바인     하이트진로   4396       14378         NaN    15455    NaN
47     대우컴바인   하이트진로음료   9856       14378         NaN    15455    NaN
48     대우컴바인     대우패키지    126       14378         NaN    15455    NaN

In [610]:
dfLong = api.getTransTable('하이트진로홀딩스', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['하이트진로홀딩스' '하이트진로' '진로소주' '하이트진로산업' '하이트진로음료' '진로양조' '강원물류' '수양물류' '천주물류'
 '블루헤런' '서영이앤티' '연암' '송정' '대우화학' '대우패키지' '대우컴바인']
['하이트진로' '하이트진로음료' '하이트진로홀딩스' '진로소주' '하이트진로산업' '서영이앤티' '연암' '수양물류' '천주물류'
 '강원물류' '송정' '대우컴바인' '대우패키지']


매출회사      매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액   해외매출액
0   하이트진로홀딩스     하이트진로   5345        5403         NaN    35812     NaN
1   하이트진로홀딩스   하이트진로음료     58        5403         NaN    35812     NaN
2      하이트진로  하이트진로홀딩스     22        5569       72013  1939357  109931
3      하이트진로      진로소주    875        5569       72013  1939357  109931
4      하이트진로   하이트진로산업    192        5569       72013  1939357  109931
5      하이트진로   하이트진로음료   4629        5569       72013  1939357  109931
6      하이트진로     서영이앤티   -148        5569       72013  1939357  109931
7      하이트진로        연암      0        5569       72013  1939357  109931
8       진로소주     하이트진로  21987       21987       15828    21987   15828
9    하이트진로산업     하이트진로  79246       87604          21    87941      21
10   하이트진로산업      진로소주   8357       87604          21    87941      21
11   하이트진로음료     하이트진로    830         831         NaN   104806     715
12   하이트진로음료     서영이앤티      2         831         NaN   104806     715
13      진로양조     하이트진로   2253        2253         NaN     2255     135
14      강원물류     하이트진로  17979       17997         NaN    18367     NaN
15      강원물류      수양물류      2       17997         NaN    18367     NaN
16      강원물류      천주물류      0       17997         NaN    18367     NaN
17      강원물류     서영이앤티     16       17997         NaN    18367     NaN
18      수양물류     하이트진로  21738       21744         NaN    22384     NaN
19      수양물류      강원물류      4       21744         NaN    22384     NaN
20      수양물류      천주물류      2       21744         NaN    22384     NaN
21      천주물류     하이트진로  15037       15113         NaN    15394     NaN
22      천주물류      강원물류      0       15113         NaN    15394     NaN
23      천주물류      수양물류     76       15113         NaN    15394     NaN
24      블루헤런     하이트진로    426         426         NaN    13182     NaN
25     서영이앤티     하이트진로  22350       22599         NaN   100505    4161
26     서영이앤티      진로소주     53       22599         NaN   100505    4161
27     서영이앤티   하이트진로산업      6       22599         NaN   100505    4161
28     서영이앤티   하이트진로음료    170       22599         NaN   100505    4161
29     서영이앤티      강원물류      7       22599         NaN   100505    4161
30     서영이앤티      수양물류      7       22599         NaN   100505    4161
31     서영이앤티      천주물류      4       22599         NaN   100505    4161
32     서영이앤티        연암      4       22599         NaN   100505    4161
33        연암     하이트진로   5812        6743         NaN    19403    1586
34        연암   하이트진로음료    804        6743         NaN    19403    1586
35        연암        송정      4        6743         NaN    19403    1586
36        연암     대우컴바인    123        6743         NaN    19403    1586
37        송정        연암    359         359         NaN     8520     NaN
38      대우화학     하이트진로  14083       14731         NaN    16547     NaN
39      대우화학      진로소주     98       14731         NaN    16547     NaN
40      대우화학   하이트진로음료    464       14731         NaN    16547     NaN
41      대우화학     대우패키지     82       14731         NaN    16547     NaN
42      대우화학     대우컴바인      5       14731         NaN    16547     NaN
43     대우패키지     하이트진로   1033        1917         NaN    11020     NaN
44     대우패키지      진로소주    349        1917         NaN    11020     NaN
45     대우패키지     대우컴바인    534        1917         NaN    11020     NaN
46     대우컴바인     하이트진로   5397       10943         NaN    11856     NaN
47     대우컴바인   하이트진로음료   5546       10943         NaN    11856     NaN

# 금호석유화학

In [604]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('금호석유화학', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['금호석유화학' '금호피앤비화학' '금호미쓰이화학' '금호폴리켐' '금호티앤엘' '금호개발상사' '여수페트로']
['금호피앤비화학' '금호미쓰이화학' '금호폴리켐' '금호개발상사' '금호티앤엘' '코리아에너지발전소' '상해금호일려소료유한공사'
 '광동금호일려고분자재료유한공사' '남경금포금호화공유한공사' '금호석유화학' '강소금호양농화공유한공사']


매출회사             매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액    해외매출액
0    금호석유화학          금호피앤비화학  161794      199963       22370  1336599  2562974
1    금호석유화학          금호미쓰이화학    7583      199963       22370  1336599  2562974
2    금호석유화학            금호폴리켐   28109      199963       22370  1336599  2562974
3    금호석유화학           금호개발상사     501      199963       22370  1336599  2562974
4    금호석유화학            금호티앤엘    1970      199963       22370  1336599  2562974
5    금호석유화학        코리아에너지발전소       6      199963       22370  1336599  2562974
6    금호석유화학     상해금호일려소료유한공사   13802      199963       22370  1336599  2562974
7    금호석유화학  광동금호일려고분자재료유한공사    7643      199963       22370  1336599  2562974
8    금호석유화학     남경금포금호화공유한공사     925      199963       22370  1336599  2562974
9   금호피앤비화학           금호석유화학   23596       24760       16619   700092  1100072
10  금호피앤비화학          금호미쓰이화학      22       24760       16619   700092  1100072
11  금호피앤비화학            금호폴리켐     545       24760       16619   700092  1100072
12  금호피앤비화학           금호개발상사     596       24760       16619   700092  1100072
13  금호피앤비화학     강소금호양농화공유한공사   16619       24760       16619   700092  1100072
14  금호미쓰이화학           금호석유화학    4256        4554         NaN   442270   465771
15  금호미쓰이화학           금호개발상사     298        4554         NaN   442270   465771
16    금호폴리켐           금호석유화학     130         171         NaN   166115   310681
17    금호폴리켐           금호개발상사      41         171         NaN   166115   310681
18    금호티앤엘           금호석유화학   30087       30130         NaN    55995      NaN
19    금호티앤엘           금호개발상사      44       30130         NaN    55995      NaN
20   금호개발상사           금호석유화학    8039       11862         NaN    46471     1673
21   금호개발상사          금호피앤비화학    2091       11862         NaN    46471     1673
22   금호개발상사          금호미쓰이화학     166       11862         NaN    46471     1673
23   금호개발상사            금호폴리켐       3       11862         NaN    46471     1673
24   금호개발상사            금호티앤엘    1563       11862         NaN    46471     1673
25    여수페트로           금호석유화학      65         279         NaN     1290      NaN
26    여수페트로          금호피앤비화학     214         279         NaN     1290      NaN

In [606]:
dfLong = api.getTransTable('유진기업', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['유진기업' '동양' '유진에너팜' '유진홈데이' '이에이치씨' '지구레미콘' '한성레미콘' '한일합섬' '굿앤파트너스'
 '남부산업' '당진기업' '동화기업' '성인산업' '우진레미콘' '유진로텍' '유진아이티서비스' '유진에이엠씨' '유진엠'
 '유진엠플러스' '유진프라이빗에쿼티' '이순산업' '천안기업' '한국로지스틱스' '한국통운' '현대개발'
 '유진에버베스트턴어라운드기업재무안정사모투자' '유진에스비홀딩스' '유진자산운용' '유진제사호헤라클레스사모투자' '유진투자선물'
 '유진투자증권']
['나눔로또' '동양' '유진에너팜' '유진홈데이' '이에이치씨' '지구레미콘' '한성레미콘' '한일합섬' '굿앤파트너스'
 '남부산업' '당진기업' '동화기업' '성인산업' '우진레미콘' '농업회사법인유진개발' '유진로텍' '유진아이티서비스'
 '유진에이엠씨' '유진엠' '유진엠플러스' '이순산업' '천안기업' '한국통운' '현대개발' '유진초저온' '유진저축은행'
 '유진투자증권' '유진기업' '한국특수형강' '유진자산운용' '유진투자선물' 'PT.HanilIndonesia' '한국로지스틱스'
 '유진프라이빗에쿼티' '파인리조트' '유진스타제이호사모투자' '유진제사호헤라클레스사모투자' '인프라이니셔티브제1호사모투자'
 '유진에이스제일호사모투자' '유진제삼호오벨리스크기업재무안정사모투자' '유진제오호타이탄기업재무안정사모투자'
 '인프라이니셔티브1호사모투자' '유진스마트모빌리티사모투자' '씨아이에스' '케이비에프' '유진에버베스트턴어라운드기업재무안정사모투자'
 '유진에스비홀딩스']


매출회사      매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액 해외매출액
0              유진기업      나눔로또      9       26724         NaN  803952  3046
1              유진기업        동양  12498       26724         NaN  803952  3046
2              유진기업     유진에너팜     12       26724         NaN  803952  3046
3              유진기업     유진홈데이    226       26724         NaN  803952  3046
4              유진기업     이에이치씨   6895       26724         NaN  803952  3046
..              ...       ...    ...         ...         ...     ...   ...
193  유진제사호헤라클레스사모투자  유진에스비홀딩스  11501       11501         NaN   11503   NaN
194          유진투자선물    유진저축은행    1.0         647         NaN   73669   NaN
195          유진투자선물    유진투자증권  646.0         647         NaN   73669   NaN
196          유진투자증권    유진자산운용    1.0           3         NaN  877548   NaN
197          유진투자증권    유진투자선물    2.0           3         NaN  877548   NaN

[198 rows x 7 columns]

In [607]:
dfLong = api.getTransTable('유진기업', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['유진기업' '동양' '유진디랩' '유진로지스틱스' '유진홈센터' '지구레미콘' '한성레미콘' '굿앤파트너스' '동화기업'
 '성인산업' '우진레미콘' '유진레저' '유진아이티서비스' '유진에이엠씨' '유진엠' '유진엠플러스' '유진한일합섬' '이순산업'
 '천안기업' '한국로지스틱스' '현대개발' '유진에버베스트턴어라운드기업재무안정사모투자' '유진에스비홀딩스' '유진자산운용'
 '유진제사호헤라클레스사모투자' '유진투자선물' '유진투자증권' '유진프라이빗에쿼티' '인프라이니셔티브제1호사모투자' '클린메디']
['나눔로또' '동양' '유진디랩' '유진로지스틱스' '유진에너팜' '유진홈센터' '지구레미콘' '한성레미콘' '굿앤파트너스'
 '남부산업' '농업회사법인유진팜앤바이오' '당진기업' '동화기업' '성인산업' '우진레미콘' '유진레저' '유진아이티서비스'
 '유진에이엠씨' '유진엠' '유진엠플러스' '유진한일합섬' '이순산업' '천안기업' '현대개발' '유진저축은행' '유진투자증권'
 '유진기업' '유진자산운용' '한국로지스틱스' '유진투자선물' '유진프라이빗에쿼티' '한국초저온'
 'PT.HANILINDONESIA' '씨아이에스케미칼' '테크스퀘어' '유진에버베스트턴어라운드기업재무안정사모투자'
 '유진에스비홀딩스' '유진스마트모빌리티사모투자' '유진스타제이호사모투자' '유진제사호헤라클레스사모투자'
 '인프라이니셔티브제1호사모투자' '유진제오호타이탄기업재무안정사모투자' '클린메디' '디디에스']


매출회사                매입회사    매출액 국내계열사계(매출액)  해외계열사계(매출액)  \
0               유진기업                나눔로또      1       43198            0   
1               유진기업                  동양  24756       43198            0   
2               유진기업                유진디랩    172       43198            0   
3               유진기업             유진로지스틱스   1052       43198            0   
4               유진기업               유진에너팜      5       43198            0   
..               ...                 ...    ...         ...          ...   
187        유진프라이빗에쿼티      유진제사호헤라클레스사모투자   1326        2525            0   
188        유진프라이빗에쿼티     인프라이니셔티브제1호사모투자    367        2525            0   
189        유진프라이빗에쿼티  유진제오호타이탄기업재무안정사모투자    -16        2525            0   
190  인프라이니셔티브제1호사모투자                클린메디    391         391            0   
191             클린메디                디디에스    431         431            0   

      국내매출액 해외매출액  
0    747452  1703  
1    747452  1703  
2    747452  1703  
3    747452  1703  
4    747452  1703  
..      ...   ...  
187    2619     0  
188    2619     0  
189    2619     0  
190    1074     0  
191     431     0  

[192 rows x 7 columns]

# 유진기업

In [598]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('유진기업', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['유진기업' '동양' '유진에너팜' '유진홈데이' '이에이치씨' '지구레미콘' '티와이강원' '한성레미콘' '한일합섬'
 '굿앤파트너스' '남부산업' '당진기업' '동화기업' '바이오코엔' '우진레미콘' '유진로텍' '유진아이티서비스' '유진에이엠씨'
 '유진엠' '유진엠플러스' '유진프라이빗에쿼티' '이순산업' '천안기업' '한국로지스틱스' '한국통운' '현대개발'
 '양지지앤알홀딩스' '유진에버베스트턴어라운드기업재무안정사모투자' '유진자산운용' '유진제사호헤라클레스사모투자'
 '유진제삼호오벨리스크기업재무안정사모투자' '유진투자선물' '유진투자증권']
['나눔로또' '동양' '유진에너팜' '유진홈데이' '이에이치씨' '지구레미콘' '한성레미콘' '한일합섬' '남부산업' '당진기업'
 '동화기업' '우진레미콘' '유진로텍' '유진아이티서비스' '유진에이엠씨' '유진엠' '유진엠플러스' '유진초저온' '이순산업'
 '천안기업' '한국통운' '현대산업' '선진엔티에스' '현대개발' '유진저축은행' '유진투자증권' '유진기업'
 'PT.HanilIndonesia' '한국로지스틱스' '유진프라이빗에쿼티' '유진자산운용' '유진투자선물' '성인산업'
 '파인리조트' '유진스타제2호사모투자' '유진에이스제일호사모투자' '유진제사호헤라클레스사모투자'
 '유진제삼호오벨리스크기업재무안정사모투자' '유진제오호타이탄기업재무안정사모투자' '부산로지스틱스' '대전로지스틱스' '경인로지스틱스'
 '중부로지스틱스' '케이비에프' '유진에버베스트턴어라운드기업재무안정사모투자' '유진에스비홀딩스' '양지지앤알홀딩스']


매출회사      매입회사      매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액  \
0                    유진기업      나눔로또    183.0       40359         NaN  780324   
1                    유진기업        동양  13231.0       40359         NaN  780324   
2                    유진기업     유진에너팜     20.0       40359         NaN  780324   
3                    유진기업     유진홈데이    165.0       40359         NaN  780324   
4                    유진기업     이에이치씨    179.0       40359         NaN  780324   
..                    ...       ...      ...         ...         ...     ...   
179  유진제삼호오벨리스크기업재무안정사모투자  양지지앤알홀딩스      314         314         NaN     314   
180                유진투자선물    유진저축은행      5.0         668         NaN   74270   
181                유진투자선물    유진투자증권    663.0         668         NaN   74270   
182                유진투자증권    유진자산운용      3.0           7         NaN  724057   
183                유진투자증권    유진투자선물      4.0           7         NaN  724057   

    해외매출액  
0     663  
1     663  
2     663  
3     663  
4     663  
..    ...  
179   NaN  
180   NaN  
181   NaN  
182   NaN  
183   NaN  

[184 rows x 7 columns]

In [602]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('유진기업', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['유진기업' '동양' '유진에너팜' '유진홈데이' '이에이치씨' '지구레미콘' '한성레미콘' '한일합섬' '굿앤파트너스'
 '남부산업' '당진기업' '동화기업' '성인산업' '우진레미콘' '유진로텍' '유진아이티서비스' '유진에이엠씨' '유진엠'
 '유진엠플러스' '유진프라이빗에쿼티' '이순산업' '천안기업' '한국로지스틱스' '한국통운' '현대개발'
 '유진에버베스트턴어라운드기업재무안정사모투자' '유진에스비홀딩스' '유진자산운용' '유진제사호헤라클레스사모투자' '유진투자선물'
 '유진투자증권']
['나눔로또' '동양' '유진에너팜' '유진홈데이' '이에이치씨' '지구레미콘' '한성레미콘' '한일합섬' '굿앤파트너스'
 '남부산업' '당진기업' '동화기업' '성인산업' '우진레미콘' '농업회사법인유진개발' '유진로텍' '유진아이티서비스'
 '유진에이엠씨' '유진엠' '유진엠플러스' '이순산업' '천안기업' '한국통운' '현대개발' '유진초저온' '유진저축은행'
 '유진투자증권' '유진기업' '한국특수형강' '유진자산운용' '유진투자선물' 'PT.HanilIndonesia' '한국로지스틱스'
 '유진프라이빗에쿼티' '파인리조트' '유진스타제이호사모투자' '유진제사호헤라클레스사모투자' '인프라이니셔티브제1호사모투자'
 '유진에이스제일호사모투자' '유진제삼호오벨리스크기업재무안정사모투자' '유진제오호타이탄기업재무안정사모투자'
 '인프라이니셔티브1호사모투자' '유진스마트모빌리티사모투자' '씨아이에스' '케이비에프' '유진에버베스트턴어라운드기업재무안정사모투자'
 '유진에스비홀딩스']


매출회사      매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액 해외매출액
0              유진기업      나눔로또      9       26724         NaN  803952  3046
1              유진기업        동양  12498       26724         NaN  803952  3046
2              유진기업     유진에너팜     12       26724         NaN  803952  3046
3              유진기업     유진홈데이    226       26724         NaN  803952  3046
4              유진기업     이에이치씨   6895       26724         NaN  803952  3046
..              ...       ...    ...         ...         ...     ...   ...
193  유진제사호헤라클레스사모투자  유진에스비홀딩스  11501       11501         NaN   11503   NaN
194          유진투자선물    유진저축은행    1.0         647         NaN   73669   NaN
195          유진투자선물    유진투자증권  646.0         647         NaN   73669   NaN
196          유진투자증권    유진자산운용    1.0           3         NaN  877548   NaN
197          유진투자증권    유진투자선물    2.0           3         NaN  877548   NaN

[198 rows x 7 columns]

In [603]:
dfLong = api.getTransTable('유진기업', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['유진기업' '동양' '유진디랩' '유진로지스틱스' '유진홈센터' '지구레미콘' '한성레미콘' '굿앤파트너스' '동화기업'
 '성인산업' '우진레미콘' '유진레저' '유진아이티서비스' '유진에이엠씨' '유진엠' '유진엠플러스' '유진한일합섬' '이순산업'
 '천안기업' '한국로지스틱스' '현대개발' '유진에버베스트턴어라운드기업재무안정사모투자' '유진에스비홀딩스' '유진자산운용'
 '유진제사호헤라클레스사모투자' '유진투자선물' '유진투자증권' '유진프라이빗에쿼티' '인프라이니셔티브제1호사모투자' '클린메디']
['나눔로또' '동양' '유진디랩' '유진로지스틱스' '유진에너팜' '유진홈센터' '지구레미콘' '한성레미콘' '굿앤파트너스'
 '남부산업' '농업회사법인유진팜앤바이오' '당진기업' '동화기업' '성인산업' '우진레미콘' '유진레저' '유진아이티서비스'
 '유진에이엠씨' '유진엠' '유진엠플러스' '유진한일합섬' '이순산업' '천안기업' '현대개발' '유진저축은행' '유진투자증권'
 '유진기업' '유진자산운용' '한국로지스틱스' '유진투자선물' '유진프라이빗에쿼티' '한국초저온'
 'PT.HANILINDONESIA' '씨아이에스케미칼' '테크스퀘어' '유진에버베스트턴어라운드기업재무안정사모투자'
 '유진에스비홀딩스' '유진스마트모빌리티사모투자' '유진스타제이호사모투자' '유진제사호헤라클레스사모투자'
 '인프라이니셔티브제1호사모투자' '유진제오호타이탄기업재무안정사모투자' '클린메디' '디디에스']


매출회사                매입회사    매출액 국내계열사계(매출액)  해외계열사계(매출액)  \
0               유진기업                나눔로또      1       43198            0   
1               유진기업                  동양  24756       43198            0   
2               유진기업                유진디랩    172       43198            0   
3               유진기업             유진로지스틱스   1052       43198            0   
4               유진기업               유진에너팜      5       43198            0   
..               ...                 ...    ...         ...          ...   
187        유진프라이빗에쿼티      유진제사호헤라클레스사모투자   1326        2525            0   
188        유진프라이빗에쿼티     인프라이니셔티브제1호사모투자    367        2525            0   
189        유진프라이빗에쿼티  유진제오호타이탄기업재무안정사모투자    -16        2525            0   
190  인프라이니셔티브제1호사모투자                클린메디    391         391            0   
191             클린메디                디디에스    431         431            0   

      국내매출액 해외매출액  
0    747452  1703  
1    747452  1703  
2    747452  1703  
3    747452  1703  
4    747452  1703  
..      ...   ...  
187    2619     0  
188    2619     0  
189    2619     0  
190    1074     0  
191     431     0  

[192 rows x 7 columns]

# 동국제강

In [595]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('동국제강', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['동국제강' '페럼인프라' '당진고대부두운영' '인터지스' '인터지스신항센터' '아이앤케이신항만' '인터지스웅동센터'
 '디케이유엔씨' '인터지스중앙부두' '제이비당진페럼빌전문투자형사모투자']
['페럼인프라' '당진고대부두운영' '인터지스' '디케이유엔씨' '인터지스중앙부두' '동국제강' '인터지스신항센터'
 '인터지스웅동센터' '아이앤케이신항만' '제주로해운물류']


매출회사      매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  \
0                동국제강     페럼인프라     311        2288      496583  4139994   
1                동국제강  당진고대부두운영      35        2288      496583  4139994   
2                동국제강      인터지스    1123        2288      496583  4139994   
3                동국제강    디케이유엔씨     816        2288      496583  4139994   
4                동국제강  인터지스중앙부두       3        2288      496583  4139994   
5               페럼인프라      동국제강     429         429         NaN    12665   
6            당진고대부두운영      인터지스     254         254         NaN     5693   
7                인터지스      동국제강  206648      207496        4350   449093   
8                인터지스  당진고대부두운영     208      207496        4350   449093   
9                인터지스  인터지스신항센터     123      207496        4350   449093   
10               인터지스  인터지스웅동센터     193      207496        4350   449093   
11               인터지스    디케이유엔씨     180      207496        4350   449093   
12               인터지스  인터지스중앙부두     144      207496        4350   449093   
13           인터지스신항센터      인터지스    1116        1179         NaN     2993   
14           인터지스신항센터  인터지스웅동센터      63        1179         NaN     2993   
15           아이앤케이신항만      인터지스   17785       17785         NaN    35571   
16           인터지스웅동센터      인터지스    2703        2757         NaN     5484   
17           인터지스웅동센터  인터지스신항센터      54        2757         NaN     5484   
18             디케이유엔씨      동국제강   28999       35348         112   163086   
19             디케이유엔씨     페럼인프라      31       35348         112   163086   
20             디케이유엔씨  당진고대부두운영      11       35348         112   163086   
21             디케이유엔씨      인터지스    5951       35348         112   163086   
22             디케이유엔씨  인터지스신항센터      15       35348         112   163086   
23             디케이유엔씨  아이앤케이신항만     248       35348         112   163086   
24             디케이유엔씨  인터지스웅동센터      43       35348         112   163086   
25             디케이유엔씨  인터지스중앙부두      49       35348         112   163086   
26             디케이유엔씨   제주로해운물류       1       35348         112   163086   
27           인터지스중앙부두      동국제강     300        1825         NaN    15820   
28           인터지스중앙부두      인터지스    1525        1825         NaN    15820   
29  제이비당진페럼빌전문투자형사모투자      동국제강     296         296         NaN      NaN   

      해외매출액  
0   1224852  
1   1224852  
2   1224852  
3   1224852  
4   1224852  
5       NaN  
6       NaN  
7       NaN  
8       NaN  
9       NaN  
10      NaN  
11      NaN  
12      NaN  
13      NaN  
14      NaN  
15      NaN  
16      NaN  
17      NaN  
18      229  
19      229  
20      229  
21      229  
22      229  
23      229  
24      229  
25      229  
26      229  
27      NaN  
28      NaN  
29      NaN

In [596]:
dfLong = api.getTransTable('동국제강', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['동국제강' '페럼인프라' '당진고대부두운영' '인터지스' '인터지스신항센터' '아이앤케이신항만' '인터지스웅동센터'
 '동국시스템즈' '인터지스중앙부두']
['페럼인프라' '당진고대부두운영' '인터지스' '인터지스웅동센터' '동국시스템즈' '동국제강' '인터지스신항센터'
 '아이앤케이신항만' '인터지스중앙부두' '제주로해운물류']


매출회사      매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액    해외매출액
0       동국제강     페럼인프라     175        1013      514822  3851349  1204060
1       동국제강  당진고대부두운영      37        1013      514822  3851349  1204060
2       동국제강      인터지스     631        1013      514822  3851349  1204060
3       동국제강  인터지스웅동센터       1        1013      514822  3851349  1204060
4       동국제강    동국시스템즈     169        1013      514822  3851349  1204060
5      페럼인프라      동국제강     430         431         NaN    12968      NaN
6      페럼인프라      인터지스       1         431         NaN    12968      NaN
7   당진고대부두운영      인터지스     442         442         NaN     5741      NaN
8       인터지스      동국제강  222403      223639         241   431257      NaN
9       인터지스     페럼인프라      10      223639         241   431257      NaN
10      인터지스  당진고대부두운영     262      223639         241   431257      NaN
11      인터지스  인터지스신항센터     463      223639         241   431257      NaN
12      인터지스  아이앤케이신항만      61      223639         241   431257      NaN
13      인터지스  인터지스웅동센터     262      223639         241   431257      NaN
14      인터지스    동국시스템즈     125      223639         241   431257      NaN
15      인터지스  인터지스중앙부두      51      223639         241   431257      NaN
16      인터지스   제주로해운물류       2      223639         241   431257      NaN
17  인터지스신항센터      인터지스     834         872         NaN     3244      NaN
18  인터지스신항센터  인터지스웅동센터      38         872         NaN     3244      NaN
19  아이앤케이신항만      인터지스   19312       19312         NaN    38624      NaN
20  인터지스웅동센터      인터지스    1594        1660         NaN     5516      NaN
21  인터지스웅동센터  인터지스신항센터      66        1660         NaN     5516      NaN
22    동국시스템즈      동국제강   28647       34137         109   192438      645
23    동국시스템즈     페럼인프라      41       34137         109   192438      645
24    동국시스템즈  당진고대부두운영       6       34137         109   192438      645
25    동국시스템즈      인터지스    5302       34137         109   192438      645
26    동국시스템즈  인터지스신항센터      28       34137         109   192438      645
27    동국시스템즈  아이앤케이신항만       3       34137         109   192438      645
28    동국시스템즈  인터지스웅동센터      43       34137         109   192438      645
29    동국시스템즈  인터지스중앙부두      67       34137         109   192438      645
30  인터지스중앙부두      인터지스    1448        1448         NaN    16023      NaN

In [597]:
dfLong = api.getTransTable('동국제강', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['동국제강' '페럼인프라' '당진고대부두운영' '인터지스' '인터지스신항센터' '아이앤케이신항만' '인터지스웅동센터'
 '동국시스템즈' '인터지스중앙부두']
['페럼인프라' '당진고대부두운영' '인터지스' '인터지스웅동센터' '동국시스템즈' '인터지스중앙부두' '동국제강'
 '인터지스신항센터' '아이앤케이신항만']


매출회사      매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액    해외매출액
0       동국제강     페럼인프라     126         931      517638  3463381  1150275
1       동국제강  당진고대부두운영      30         931      517638  3463381  1150275
2       동국제강      인터지스     577         931      517638  3463381  1150275
3       동국제강  인터지스웅동센터       1         931      517638  3463381  1150275
4       동국제강    동국시스템즈     196         931      517638  3463381  1150275
5       동국제강  인터지스중앙부두       1         931      517638  3463381  1150275
6      페럼인프라      동국제강   27469       27889         NaN    42729      NaN
7      페럼인프라  당진고대부두운영       4       27889         NaN    42729      NaN
8      페럼인프라      인터지스     301       27889         NaN    42729      NaN
9      페럼인프라  인터지스신항센터      18       27889         NaN    42729      NaN
10     페럼인프라  인터지스웅동센터      19       27889         NaN    42729      NaN
11     페럼인프라    동국시스템즈       2       27889         NaN    42729      NaN
12     페럼인프라  인터지스중앙부두      76       27889         NaN    42729      NaN
13  당진고대부두운영      인터지스     719         719         NaN     6724      NaN
14      인터지스      동국제강  226280      229668         708   418388      NaN
15      인터지스  당진고대부두운영     285      229668         708   418388      NaN
16      인터지스  인터지스신항센터     989      229668         708   418388      NaN
17      인터지스  아이앤케이신항만      81      229668         708   418388      NaN
18      인터지스  인터지스웅동센터    1812      229668         708   418388      NaN
19      인터지스    동국시스템즈     138      229668         708   418388      NaN
20      인터지스  인터지스중앙부두      83      229668         708   418388      NaN
21  인터지스신항센터      인터지스     945         996         NaN     3367      NaN
22  인터지스신항센터  인터지스웅동센터      51         996         NaN     3367      NaN
23  아이앤케이신항만      인터지스   21179       21179         NaN    42358      NaN
24  인터지스웅동센터      인터지스    1798        1863         NaN     5396      NaN
25  인터지스웅동센터  인터지스신항센터      65        1863         NaN     5396      NaN
26    동국시스템즈      동국제강   16928       20405         157   175443      794
27    동국시스템즈     페럼인프라     580       20405         157   175443      794
28    동국시스템즈  당진고대부두운영       1       20405         157   175443      794
29    동국시스템즈      인터지스    2855       20405         157   175443      794
30    동국시스템즈  인터지스신항센터       4       20405         157   175443      794
31    동국시스템즈  인터지스웅동센터      36       20405         157   175443      794
32    동국시스템즈  인터지스중앙부두       1       20405         157   175443      794
33  인터지스중앙부두      인터지스    3161        3161         NaN    15902      NaN

# 삼천리

In [592]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('삼천리', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['삼천리' '삼천리이엔지' '삼천리이에스' '삼천리모터스' '에스파워' '안산도시개발' '삼천리열처리' '삼탄']
['삼천리이엔지' '삼천리이에스' '삼천리모터스' '휴세스' '에스파워' '안산도시개발' '삼천리자산운용' '삼천리' '신승에너지'
 'PT.SAMINDOUTAMAKALTIM' 'PT.TRASINDOMURNIPERKASA']


매출회사                     매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  \
0      삼천리                   삼천리이엔지  11297       18838         NaN  2534707   
1      삼천리                   삼천리이에스    424       18838         NaN  2534707   
2      삼천리                   삼천리모터스      4       18838         NaN  2534707   
3      삼천리                      휴세스      5       18838         NaN  2534707   
4      삼천리                     에스파워   4929       18838         NaN  2534707   
5      삼천리                   안산도시개발   2065       18838         NaN  2534707   
6      삼천리                  삼천리자산운용    114       18838         NaN  2534707   
7   삼천리이엔지                      삼천리  46071       52813         NaN   103356   
8   삼천리이엔지                   삼천리이에스     80       52813         NaN   103356   
9   삼천리이엔지                   삼천리모터스      9       52813         NaN   103356   
10  삼천리이엔지                      휴세스   4816       52813         NaN   103356   
11  삼천리이엔지                     에스파워      1       52813         NaN   103356   
12  삼천리이엔지                   안산도시개발   1816       52813         NaN   103356   
13  삼천리이엔지                  삼천리자산운용     20       52813         NaN   103356   
14  삼천리이에스                      삼천리    673       22439         NaN   221439   
15  삼천리이에스                   삼천리이엔지     33       22439         NaN   221439   
16  삼천리이에스                   안산도시개발     24       22439         NaN   221439   
17  삼천리이에스                    신승에너지  21709       22439         NaN   221439   
18  삼천리모터스                      삼천리      9           9         NaN    85643   
19    에스파워                      삼천리    620       27589         NaN   551378   
20    에스파워                   안산도시개발  26969       27589         NaN   551378   
21  안산도시개발                   삼천리이엔지      2        4947         NaN    71411   
22  안산도시개발                      휴세스   4945        4947         NaN    71411   
23  삼천리열처리                      삼천리    126         126         NaN      400   
24      삼탄    PT.SAMINDOUTAMAKALTIM   3669         NaN        6902    79895   
25      삼탄  PT.TRASINDOMURNIPERKASA   3233         NaN        6902    79895   

     해외매출액  
0      NaN  
1      NaN  
2      NaN  
3      NaN  
4      NaN  
5      NaN  
6      NaN  
7      NaN  
8      NaN  
9      NaN  
10     NaN  
11     NaN  
12     NaN  
13     NaN  
14     NaN  
15     NaN  
16     NaN  
17     NaN  
18     NaN  
19     NaN  
20     NaN  
21     NaN  
22     NaN  
23     NaN  
24  533789  
25  533789

In [593]:
dfLong = api.getTransTable('삼천리', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['삼천리' '삼천리이엔지' '삼천리이에스' '삼천리모터스' '에스파워' '안산도시개발' '신승에너지' '삼천리열처리'
 '에스티인터내셔널코퍼레이션' '영덕해파랑풍력발전']
['삼천리이엔지' '삼천리이에스' '삼천리모터스' '휴세스' '에스파워' '안산도시개발' '삼천리자산운용' '삼천리' '신승에너지'
 'PT.SAMINDOUTAMAKALTIM' 'PT.TRASINDOMURNIPERKASA' '영덕풍력발전']


매출회사                     매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)  \
0             삼천리                   삼천리이엔지  11685       25595         NaN   
1             삼천리                   삼천리이에스    419       25595         NaN   
2             삼천리                   삼천리모터스      1       25595         NaN   
3             삼천리                      휴세스    910       25595         NaN   
4             삼천리                     에스파워  10224       25595         NaN   
5             삼천리                   안산도시개발   2242       25595         NaN   
6             삼천리                  삼천리자산운용    114       25595         NaN   
7          삼천리이엔지                      삼천리  49131       53736         NaN   
8          삼천리이엔지                   삼천리이에스     88       53736         NaN   
9          삼천리이엔지                   삼천리모터스     14       53736         NaN   
10         삼천리이엔지                      휴세스    639       53736         NaN   
11         삼천리이엔지                     에스파워     12       53736         NaN   
12         삼천리이엔지                   안산도시개발   3836       53736         NaN   
13         삼천리이엔지                    신승에너지      1       53736         NaN   
14         삼천리이엔지                  삼천리자산운용     15       53736         NaN   
15         삼천리이에스                      삼천리   1068       16338         NaN   
16         삼천리이에스                   삼천리이엔지     91       16338         NaN   
17         삼천리이에스                   삼천리모터스    295       16338         NaN   
18         삼천리이에스                   안산도시개발   7796       16338         NaN   
19         삼천리이에스                    신승에너지   7088       16338         NaN   
20         삼천리모터스                      삼천리     22          22         NaN   
21           에스파워                      삼천리    619       30294         NaN   
22           에스파워                   안산도시개발  29675       30294         NaN   
23         안산도시개발                   삼천리이엔지      1        7881         NaN   
24         안산도시개발                      휴세스   7880        7881         NaN   
25          신승에너지                     에스파워    332         332         NaN   
26         삼천리열처리                      삼천리     64          64         NaN   
27  에스티인터내셔널코퍼레이션    PT.SAMINDOUTAMAKALTIM   3612         NaN        5717   
28  에스티인터내셔널코퍼레이션  PT.TRASINDOMURNIPERKASA   2105         NaN        5717   
29      영덕해파랑풍력발전                   영덕풍력발전      5           5         NaN   

      국내매출액   해외매출액  
0   2538242     NaN  
1   2538242     NaN  
2   2538242     NaN  
3   2538242     NaN  
4   2538242     NaN  
5   2538242     NaN  
6   2538242     NaN  
7    108692     NaN  
8    108692     NaN  
9    108692     NaN  
10   108692     NaN  
11   108692     NaN  
12   108692     NaN  
13   108692     NaN  
14   108692     NaN  
15   151367     NaN  
16   151367     NaN  
17   151367     NaN  
18   151367     NaN  
19   151367     NaN  
20   105461     NaN  
21   576554     NaN  
22   576554     NaN  
23    76816     NaN  
24    76816     NaN  
25     9259     NaN  
26      246     NaN  
27    42358  322800  
28    42358  322800  
29        5     NaN

In [594]:
dfLong = api.getTransTable('삼천리', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['삼천리' '삼천리이엔지' '삼천리이에스' '삼천리모터스' '에스파워' '안산도시개발' '신승에너지' '에스티인터내셔널코퍼레이션'
 '영덕해파랑풍력발전' '유니슨' '삼천리자산운용' '비티에스제1호사모투자' '아네모이']
['삼천리이엔지' '삼천리이에스' '삼천리모터스' '휴세스' '에스파워' '안산도시개발' '삼천리자산운용' '삼천리'
 'PT.SAMINDOUTAMAKALTIM' 'PT.TRASINDOMURNIPERKASA' '영덕풍력발전' '윈앤피' '의령풍력발전'
 '아이오' '비티에스제1호사모투자' '아폴로' '아네모이' '유니슨']


매출회사                     매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)  \
0             삼천리                   삼천리이엔지   8995       15996         NaN   
1             삼천리                   삼천리이에스    512       15996         NaN   
2             삼천리                   삼천리모터스      4       15996         NaN   
3             삼천리                      휴세스     11       15996         NaN   
4             삼천리                     에스파워   3699       15996         NaN   
5             삼천리                   안산도시개발   2662       15996         NaN   
6             삼천리                  삼천리자산운용    113       15996         NaN   
7          삼천리이엔지                      삼천리  46825       49736         NaN   
8          삼천리이엔지                   삼천리이에스     57       49736         NaN   
9          삼천리이엔지                   삼천리모터스     93       49736         NaN   
10         삼천리이엔지                      휴세스   2205       49736         NaN   
11         삼천리이엔지                     에스파워     14       49736         NaN   
12         삼천리이엔지                   안산도시개발    515       49736         NaN   
13         삼천리이엔지                  삼천리자산운용     27       49736         NaN   
14         삼천리이에스                      삼천리   1427        2987         NaN   
15         삼천리이에스                   삼천리이엔지     62        2987         NaN   
16         삼천리이에스                   삼천리모터스    975        2987         NaN   
17         삼천리이에스                   안산도시개발    523        2987         NaN   
18         삼천리모터스                      삼천리      8         143         NaN   
19         삼천리모터스                   삼천리이엔지    135         143         NaN   
20           에스파워                      삼천리    283       25823         NaN   
21           에스파워                   안산도시개발  25540       25823         NaN   
22         안산도시개발                   삼천리이엔지      1        5113         NaN   
23         안산도시개발                      휴세스   5112        5113         NaN   
24          신승에너지                     에스파워    297         297         NaN   
25  에스티인터내셔널코퍼레이션    PT.SAMINDOUTAMAKALTIM   1962         NaN        3978   
26  에스티인터내셔널코퍼레이션  PT.TRASINDOMURNIPERKASA   2016         NaN        3978   
27      영덕해파랑풍력발전                   영덕풍력발전      4           4         NaN   
28            유니슨                      윈앤피  65323       65778         NaN   
29            유니슨                   의령풍력발전    237       65778         NaN   
30            유니슨                      아이오    218       65778         NaN   
31        삼천리자산운용              비티에스제1호사모투자   1303        1303         NaN   
32    비티에스제1호사모투자                      아폴로    402       97847         NaN   
33    비티에스제1호사모투자                     아네모이  97445       97847         NaN   
34           아네모이                      유니슨  97446       97446         NaN   

      국내매출액   해외매출액  
0   2394992     NaN  
1   2394992     NaN  
2   2394992     NaN  
3   2394992     NaN  
4   2394992     NaN  
5   2394992     NaN  
6   2394992     NaN  
7    107461     NaN  
8    107461     NaN  
9    107461     NaN  
10   107461     NaN  
11   107461     NaN  
12   107461     NaN  
13   107461     NaN  
14   141844     NaN  
15   141844     NaN  
16   141844     NaN  
17   141844     NaN  
18   169760     NaN  
19   169760     NaN  
20   401814     NaN  
21   401814     NaN  
22    80489     NaN  
23    80489     NaN  
24    10659     NaN  
25     4879  255178  
26     4879  255178  
27        4     NaN  
28    75832    2680  
29    75832    2680  
30    75832    2680  
31     5846     NaN  
32    97854     NaN  
33    97854     NaN  
34    97446     NaN

# 아모레퍼시픽그룹

In [585]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('아모레퍼시픽그룹', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['아모레퍼시픽그룹' '아모레퍼시픽' '이니스프리' '에뛰드' '아모스프로페셔널' '에스쁘아' '에스트라' '퍼시픽글라스'
 '퍼시픽패키지' '농업회사법인오설록농장' '코스비전' '위드림']
['아모레퍼시픽' '이니스프리' '에뛰드' '아모스프로페셔널' '에스쁘아' '에스트라' '퍼시픽글라스' '퍼시픽패키지'
 '농업회사법인오설록농장' '코스비전' '아모레퍼시픽그룹' '위드림']


매출회사         매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액   해외매출액
0      아모레퍼시픽그룹       아모레퍼시픽   17806       21409         NaN    73340     NaN
1      아모레퍼시픽그룹        이니스프리    1149       21409         NaN    73340     NaN
2      아모레퍼시픽그룹          에뛰드     512       21409         NaN    73340     NaN
3      아모레퍼시픽그룹     아모스프로페셔널      27       21409         NaN    73340     NaN
4      아모레퍼시픽그룹         에스쁘아     227       21409         NaN    73340     NaN
5      아모레퍼시픽그룹         에스트라     568       21409         NaN    73340     NaN
6      아모레퍼시픽그룹       퍼시픽글라스     179       21409         NaN    73340     NaN
7      아모레퍼시픽그룹       퍼시픽패키지     137       21409         NaN    73340     NaN
8      아모레퍼시픽그룹  농업회사법인오설록농장      86       21409         NaN    73340     NaN
9      아모레퍼시픽그룹         코스비전     718       21409         NaN    73340     NaN
10       아모레퍼시픽     아모레퍼시픽그룹    2545       64120      316547  3166609  504097
11       아모레퍼시픽        이니스프리   18797       64120      316547  3166609  504097
12       아모레퍼시픽          에뛰드    9013       64120      316547  3166609  504097
13       아모레퍼시픽     아모스프로페셔널   27726       64120      316547  3166609  504097
14       아모레퍼시픽         에스쁘아    2019       64120      316547  3166609  504097
15       아모레퍼시픽         에스트라    1993       64120      316547  3166609  504097
16       아모레퍼시픽       퍼시픽글라스     424       64120      316547  3166609  504097
17       아모레퍼시픽       퍼시픽패키지     334       64120      316547  3166609  504097
18       아모레퍼시픽  농업회사법인오설록농장     135       64120      316547  3166609  504097
19       아모레퍼시픽         코스비전    1120       64120      316547  3166609  504097
20       아모레퍼시픽          위드림      14       64120      316547  3166609  504097
21        이니스프리       아모레퍼시픽    7511        7744       97851   494535  104380
22        이니스프리          에뛰드     206        7744       97851   494535  104380
23        이니스프리         에스쁘아      27        7744       97851   494535  104380
24          에뛰드       아모레퍼시픽    1333        1582       27752   188259   30023
25          에뛰드        이니스프리     236        1582       27752   188259   30023
26          에뛰드         에스쁘아      13        1582       27752   188259   30023
27     아모스프로페셔널       아모레퍼시픽    2374        2374         191    84166     289
28         에스쁘아       아모레퍼시픽    5646        5671         NaN    42141     NaN
29         에스쁘아        이니스프리      18        5671         NaN    42141     NaN
30         에스쁘아          에뛰드       7        5671         NaN    42141     NaN
31         에스트라       아모레퍼시픽   76120       76124         576    99561     576
32         에스트라         코스비전       4       76124         576    99561     576
33       퍼시픽글라스       아모레퍼시픽   45301       46820          19    55125    8175
34       퍼시픽글라스     아모스프로페셔널       6       46820          19    55125    8175
35       퍼시픽글라스         코스비전    1513       46820          19    55125    8175
36       퍼시픽패키지       아모레퍼시픽   45172       52348           1    55140       1
37       퍼시픽패키지        이니스프리      71       52348           1    55140       1
38       퍼시픽패키지          에뛰드      21       52348           1    55140       1
39       퍼시픽패키지         에스쁘아       1       52348           1    55140       1
40       퍼시픽패키지         에스트라    2722       52348           1    55140       1
41       퍼시픽패키지       퍼시픽글라스       5       52348           1    55140       1
42       퍼시픽패키지  농업회사법인오설록농장       1       52348           1    55140       1
43       퍼시픽패키지         코스비전    4355       52348           1    55140       1
44  농업회사법인오설록농장       아모레퍼시픽    1842        1855         NaN    20346     921
45  농업회사법인오설록농장        이니스프리      13        1855         NaN    20346     921
46         코스비전       아모레퍼시픽   28278      170207         NaN   170272     NaN
47         코스비전        이니스프리  103123      170207         NaN   170272     NaN
48         코스비전          에뛰드   32954      170207         NaN   170272     NaN
49         코스비전         에스쁘아    5331      170207         NaN   170272     NaN
50         코스비전         에스트라     

In [586]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('아모레퍼시픽그룹', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['아모레퍼시픽그룹' '아모레퍼시픽' '이니스프리' '에뛰드' '아모스프로페셔널' '에스쁘아' '오설록' '에스트라' '퍼시픽글라스'
 '퍼시픽패키지' '농업회사법인오설록농장' '코스비전' '위드림' '그린파트너즈' '코어테크놀로지']
['아모레퍼시픽' '이니스프리' '에뛰드' '아모스프로페셔널' '에스쁘아' '오설록' '에스트라' '퍼시픽글라스' '퍼시픽패키지'
 '농업회사법인오설록농장' '코스비전' '아모레퍼시픽그룹' '위드림' '그린파트너즈']


매출회사         매입회사     매출액  국내계열사계(매출액) 해외계열사계(매출액)    국내매출액   해외매출액
0      아모레퍼시픽그룹       아모레퍼시픽   21508        25350         NaN   153702     NaN
1      아모레퍼시픽그룹        이니스프리    2075        25350         NaN   153702     NaN
2      아모레퍼시픽그룹          에뛰드     864        25350         NaN   153702     NaN
3      아모레퍼시픽그룹     아모스프로페셔널     233        25350         NaN   153702     NaN
4      아모레퍼시픽그룹         에스쁘아     107        25350         NaN   153702     NaN
5      아모레퍼시픽그룹          오설록      31        25350         NaN   153702     NaN
6      아모레퍼시픽그룹         에스트라     161        25350         NaN   153702     NaN
7      아모레퍼시픽그룹       퍼시픽글라스     150        25350         NaN   153702     NaN
8      아모레퍼시픽그룹       퍼시픽패키지      87        25350         NaN   153702     NaN
9      아모레퍼시픽그룹  농업회사법인오설록농장      35        25350         NaN   153702     NaN
10     아모레퍼시픽그룹         코스비전      99        25350         NaN   153702     NaN
11       아모레퍼시픽     아모레퍼시픽그룹    1889        62521      362186  3359969  553861
12       아모레퍼시픽        이니스프리   20503        62521      362186  3359969  553861
13       아모레퍼시픽          에뛰드    7902        62521      362186  3359969  553861
14       아모레퍼시픽     아모스프로페셔널   21467        62521      362186  3359969  553861
15       아모레퍼시픽         에스쁘아    1898        62521      362186  3359969  553861
16       아모레퍼시픽          오설록    3230        62521      362186  3359969  553861
17       아모레퍼시픽         에스트라    2767        62521      362186  3359969  553861
18       아모레퍼시픽       퍼시픽글라스     436        62521      362186  3359969  553861
19       아모레퍼시픽       퍼시픽패키지     205        62521      362186  3359969  553861
20       아모레퍼시픽  농업회사법인오설록농장     152        62521      362186  3359969  553861
21       아모레퍼시픽         코스비전    2007        62521      362186  3359969  553861
22       아모레퍼시픽          위드림      40        62521      362186  3359969  553861
23       아모레퍼시픽       그린파트너즈      25        62521      362186  3359969  553861
24        이니스프리       아모레퍼시픽    3540         3540      143653   407316  144546
25          에뛰드       아모레퍼시픽    1083         1191       29943   147240   32721
26          에뛰드         에스쁘아     108         1191       29943   147240   32721
27     아모스프로페셔널       아모레퍼시픽     641          641         252    83152     284
28         에스쁘아       아모레퍼시픽    5937         6040         882    45344    1377
29         에스쁘아     아모스프로페셔널     103         6040         882    45344    1377
30          오설록       아모레퍼시픽      47           58         NaN    12962     NaN
31          오설록        이니스프리      11           58         NaN    12962     NaN
32         에스트라       아모레퍼시픽   81487        81498         227   110912     227
33         에스트라         코스비전      11        81498         227   110912     227
34       퍼시픽글라스       아모레퍼시픽   52590        54794          26    64050    8436
35       퍼시픽글라스         코스비전    2204        54794          26    64050    8436
36       퍼시픽패키지       아모레퍼시픽   43244        52471         NaN    54620     NaN
37       퍼시픽패키지        이니스프리     110        52471         NaN    54620     NaN
38       퍼시픽패키지          에뛰드       8        52471         NaN    54620     NaN
39       퍼시픽패키지         에스쁘아       1        52471         NaN    54620     NaN
40       퍼시픽패키지          오설록      23        52471         NaN    54620     NaN
41       퍼시픽패키지         에스트라    2318        52471         NaN    54620     NaN
42       퍼시픽패키지  농업회사법인오설록농장       1        52471         NaN    54620     NaN
43       퍼시픽패키지         코스비전    6766        52471         NaN    54620     NaN
44  농업회사법인오설록농장       아모레퍼시픽    2431         2447         NaN    18593     827
45  농업회사법인오설록농장        이니스프리       6         2447         NaN    18593     827
46  농업회사법인오설록농장          오설록      10         2447         NaN    18593     827
47         코스비전       아모레퍼시픽   27133       175849         NaN   175878     NaN
48         코스비전        이니스프리  111208       175849         NaN   175878     NaN
49         코스비전          에뛰드   25943       175849         NaN

In [588]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('아모레퍼시픽그룹', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['아모레퍼시픽그룹' '아모레퍼시픽' '이니스프리' '에뛰드' '아모스프로페셔널' '에스쁘아' '오설록' '에스트라' '퍼시픽패키지'
 '농업회사법인오설록농장' '코스비전' '위드림' '코어테크놀로지' '그린파트너즈']
['아모레퍼시픽' '이니스프리' '아모스프로페셔널' '에스쁘아' '오설록' '에스트라' '퍼시픽글라스' '퍼시픽패키지'
 '농업회사법인오설록농장' '코스비전' '아모레퍼시픽그룹' '에뛰드' '위드림' '코어테크놀로지' '그린파트너즈']


매출회사      매입회사    매출액  국내계열사계(매출액) 해외계열사계(매출액)  국내매출액 해외매출액
0   아모레퍼시픽그룹    아모레퍼시픽  12839        13381         NaN  52398   NaN
1   아모레퍼시픽그룹     이니스프리    140        13381         NaN  52398   NaN
2   아모레퍼시픽그룹  아모스프로페셔널     34        13381         NaN  52398   NaN
3   아모레퍼시픽그룹      에스쁘아     40        13381         NaN  52398   NaN
4   아모레퍼시픽그룹       오설록     67        13381         NaN  52398   NaN
..       ...       ...    ...          ...         ...    ...   ...
68       위드림       오설록    381         2921         NaN   3510   NaN
69   코어테크놀로지    아모레퍼시픽   3418         3614          12   3648    12
70   코어테크놀로지      코스비전    196         3614          12   3648    12
71    그린파트너즈    아모레퍼시픽   1135         5389         NaN   5389   NaN
72    그린파트너즈       오설록   4254         5389         NaN   5389   NaN

[73 rows x 7 columns]

# 한라홀딩스

In [577]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('한라홀딩스', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['한라홀딩스' '위코' '제이제이한라' '한라' '한라세라지오' '한라오엠에스' '한라개발' '만도' '만도차이나홀딩스'
 '만도헬라일렉트로닉스' '만도브로제']
['위코' '제이제이한라' '한라' '한라세라지오' '한라지엘에스' '한라오엠에스' '한라개발' '목포신항만운영' '한라엔컴'
 '만도' '만도차이나홀딩스' '만도헬라일렉트로닉스' '만도브로제' '한라홀딩스' '배곧신도시지역특성화타운' '한라엠티스']


매출회사          매입회사     매출액  국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  \
0        한라홀딩스            위코      27       199614      133618   517512   
1        한라홀딩스        제이제이한라      82       199614      133618   517512   
2        한라홀딩스            한라   26421       199614      133618   517512   
3        한라홀딩스        한라세라지오      28       199614      133618   517512   
4        한라홀딩스        한라지엘에스      73       199614      133618   517512   
5        한라홀딩스        한라오엠에스     223       199614      133618   517512   
6        한라홀딩스          한라개발      17       199614      133618   517512   
7        한라홀딩스       목포신항만운영     131       199614      133618   517512   
8        한라홀딩스          한라엔컴    1804       199614      133618   517512   
9        한라홀딩스            만도  166766       199614      133618   517512   
10       한라홀딩스      만도차이나홀딩스       2       199614      133618   517512   
11       한라홀딩스    만도헬라일렉트로닉스    3643       199614      133618   517512   
12       한라홀딩스         만도브로제     397       199614      133618   517512   
13          위코         한라홀딩스   13155        25159         NaN    25919   
14          위코            만도   12004        25159         NaN    25919   
15      제이제이한라         한라홀딩스    1850         1850         NaN    16647   
16          한라         한라홀딩스   22000       355284       30641  1091003   
17          한라        제이제이한라    2576       355284       30641  1091003   
18          한라  배곧신도시지역특성화타운  212774       355284       30641  1091003   
19          한라        한라지엘에스  117431       355284       30641  1091003   
20          한라          한라엔컴      25       355284       30641  1091003   
21          한라            만도     478       355284       30641  1091003   
22      한라세라지오         한라홀딩스    1200         1660         NaN     8243   
23      한라세라지오            한라     460         1660         NaN     8243   
24      한라오엠에스         한라홀딩스    5500         5500         NaN    52447   
25        한라개발         한라홀딩스    2316        14826         NaN    24622   
26        한라개발        제이제이한라    1142        14826         NaN    24622   
27        한라개발            한라    2657        14826         NaN    24622   
28        한라개발        한라세라지오     710        14826         NaN    24622   
29        한라개발          한라엔컴     318        14826         NaN    24622   
30        한라개발            만도    6612        14826         NaN    24622   
31        한라개발    만도헬라일렉트로닉스     831        14826         NaN    24622   
32        한라개발         만도브로제     240        14826         NaN    24622   
33          만도         한라홀딩스    8115        24026      462879  1001779   
34          만도            위코     179        24026      462879  1001779   
35          만도            한라     849        24026      462879  1001779   
36          만도        한라오엠에스       8        24026      462879  1001779   
37          만도          한라개발      13        24026      462879  1001779   
38          만도       목포신항만운영      36        24026      462879  1001779   
39          만도          한라엔컴     168        24026      462879  1001779   
40          만도      만도차이나홀딩스       1        24026      462879  1001779   
41          만도    만도헬라일렉트로닉스     295        24026      462879  1001779   
42          만도         만도브로제   14362        24026      462879  1001779   
43    만도차이나홀딩스            만도    3941         3941        3434     3941   
44  만도헬라일렉트로닉스         한라홀딩스      22       361659       64585   388219   
45  만도헬라일렉트로닉스         한라엠티스     142       361659       64585   388219   
46  만도헬라일렉트로닉스            만도  361495       361659       64585   388219   
47  만도헬라일렉트로닉스         만도브로제       0       361659       64585   388219   
48       만도브로제         한라홀딩스   16283        78157        8972    62224   
49       만도브로제            만도   46345        78157        8972    62224   
50       만도브로제    만도헬라일렉트로닉스   15529        78157        8972    62224   

      해외매출액  
0    159123  
1    159123  
2    159123  
3    159123  
4    159123  
5    159123  
6    159123  
7    159123  
8    159123  
9    159123  
10   15

In [578]:
dfLong = api.getTransTable('한라홀딩스', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['한라홀딩스' '위코' '제이제이한라' '한라' '한라세라지오' '한라지엘에스' '한라오엠에스' '만도' '만도차이나홀딩스'
 '만도헬라일렉트로닉스' '만도브로제']
['위코' '제이제이한라' '한라' '한라세라지오' '한라지엘에스' '한라오엠에스' '한라개발' '목포신항만운영' '만도'
 '만도헬라일렉트로닉스' '만도브로제' '한라홀딩스' '배곧신도시지역특성화타운' '평택동부도로' '만도차이나홀딩스']


매출회사          매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액    해외매출액
0        한라홀딩스            위코      15      148526      106564   564680    55923
1        한라홀딩스        제이제이한라      52      148526      106564   564680    55923
2        한라홀딩스            한라   20895      148526      106564   564680    55923
3        한라홀딩스        한라세라지오      25      148526      106564   564680    55923
4        한라홀딩스        한라지엘에스     441      148526      106564   564680    55923
5        한라홀딩스        한라오엠에스     172      148526      106564   564680    55923
6        한라홀딩스          한라개발      18      148526      106564   564680    55923
7        한라홀딩스       목포신항만운영     112      148526      106564   564680    55923
8        한라홀딩스            만도  122155      148526      106564   564680    55923
9        한라홀딩스    만도헬라일렉트로닉스    4150      148526      106564   564680    55923
10       한라홀딩스         만도브로제     491      148526      106564   564680    55923
11          위코         한라홀딩스   12788       23747          21    30046       21
12          위코            만도   10959       23747          21    30046       21
13      제이제이한라         한라홀딩스    1800        1800         NaN    14320      NaN
14          한라        제이제이한라     318      213617         310  1184614    13877
15          한라  배곧신도시지역특성화타운  159425      213617         310  1184614    13877
16          한라        한라지엘에스   50300      213617         310  1184614    13877
17          한라        평택동부도로    2679      213617         310  1184614    13877
18          한라            만도     895      213617         310  1184614    13877
19      한라세라지오         한라홀딩스    1200        1660         NaN     8405      NaN
20      한라세라지오            한라     460        1660         NaN     8405      NaN
21      한라지엘에스         한라홀딩스       7          15         NaN     7124      NaN
22      한라지엘에스            한라       8          15         NaN     7124      NaN
23      한라오엠에스            한라      89          89         NaN    35503      NaN
24          만도         한라홀딩스    7944        9421      521168  1057407  2265967
25          만도            위코     265        9421      521168  1057407  2265967
26          만도            한라     747        9421      521168  1057407  2265967
27          만도        한라세라지오       0        9421      521168  1057407  2265967
28          만도        한라오엠에스       9        9421      521168  1057407  2265967
29          만도          한라개발      12        9421      521168  1057407  2265967
30          만도       목포신항만운영      35        9421      521168  1057407  2265967
31          만도      만도차이나홀딩스      36        9421      521168  1057407  2265967
32          만도    만도헬라일렉트로닉스     264        9421      521168  1057407  2265967
33          만도         만도브로제     107        9421      521168  1057407  2265967
34    만도차이나홀딩스            만도    3531        3531        2616     3531   115451
35  만도헬라일렉트로닉스            만도  422963      422986       54147   454975    55914
36  만도헬라일렉트로닉스         만도브로제      23      422986       54147   454975    55914
37       만도브로제         한라홀딩스   14679       83176        6679    69627    21417
38       만도브로제            만도   47910       83176        6679    69627    21417
39       만도브로제    만도헬라일렉트로닉스   20587       83176        6679    69627    21417

In [579]:
dfLong = api.getTransTable('한라홀딩스', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['한라홀딩스' '위코' '제이제이한라' '한라' '한라세라지오' '한라지엘에스' '한라오엠에스' '만도' '만도헬라일렉트로닉스'
 '만도브로제']
['위코' '제이제이한라' '한라' '한라세라지오' '한라지엘에스' '한라오엠에스' '목포신항만운영' '만도' '만도헬라일렉트로닉스'
 '만도브로제' '한라홀딩스' '배곧신도시지역특성화타운' '평택동부도로' '만도차이나홀딩스']


매출회사          매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액    해외매출액
0        한라홀딩스            위코      13      103905      105208   440230   127369
1        한라홀딩스        제이제이한라      40      103905      105208   440230   127369
2        한라홀딩스            한라    9789      103905      105208   440230   127369
3        한라홀딩스        한라세라지오      17      103905      105208   440230   127369
4        한라홀딩스        한라지엘에스     446      103905      105208   440230   127369
5        한라홀딩스        한라오엠에스     221      103905      105208   440230   127369
6        한라홀딩스       목포신항만운영     108      103905      105208   440230   127369
7        한라홀딩스            만도   89244      103905      105208   440230   127369
8        한라홀딩스    만도헬라일렉트로닉스    3487      103905      105208   440230   127369
9        한라홀딩스         만도브로제     540      103905      105208   440230   127369
10          위코         한라홀딩스   11504       18557         102    23903      102
11          위코            만도    7054       18557         102    23903      102
12      제이제이한라         한라홀딩스    1900        1900         NaN    15778      NaN
13          한라        제이제이한라    1480      161645         NaN  1421190    20879
14          한라  배곧신도시지역특성화타운   86730      161645         NaN  1421190    20879
15          한라        한라지엘에스   57681      161645         NaN  1421190    20879
16          한라        평택동부도로    2022      161645         NaN  1421190    20879
17          한라        한라오엠에스      11      161645         NaN  1421190    20879
18          한라            만도   13721      161645         NaN  1421190    20879
19      한라세라지오         한라홀딩스    1295        2027         NaN    15366      NaN
20      한라세라지오            한라     662        2027         NaN    15366      NaN
21      한라세라지오            만도      62        2027         NaN    15366      NaN
22      한라세라지오         만도브로제       8        2027         NaN    15366      NaN
23      한라지엘에스         한라홀딩스       5          55         NaN     7687      NaN
24      한라지엘에스            한라      50          55         NaN     7687      NaN
25      한라오엠에스            한라      92         567         NaN    49810      NaN
26      한라오엠에스        한라지엘에스     475         567         NaN    49810      NaN
27          만도         한라홀딩스    7678       13359      370298  1202920  1806845
28          만도            위코     246       13359      370298  1202920  1806845
29          만도        제이제이한라       0       13359      370298  1202920  1806845
30          만도            한라     737       13359      370298  1202920  1806845
31          만도        한라세라지오       0       13359      370298  1202920  1806845
32          만도        한라지엘에스       0       13359      370298  1202920  1806845
33          만도        한라오엠에스      10       13359      370298  1202920  1806845
34          만도       목포신항만운영      14       13359      370298  1202920  1806845
35          만도      만도차이나홀딩스     120       13359      370298  1202920  1806845
36          만도    만도헬라일렉트로닉스    4457       13359      370298  1202920  1806845
37          만도         만도브로제      97       13359      370298  1202920  1806845
38  만도헬라일렉트로닉스         한라홀딩스     207      436281       58337   466144    59242
39  만도헬라일렉트로닉스            만도  436042      436281       58337   466144    59242
40  만도헬라일렉트로닉스         만도브로제      31      436281       58337   466144    59242
41       만도브로제         한라홀딩스    6677       69819        8457    81326     8903
42       만도브로제            만도   36745       69819        8457    81326     8903
43       만도브로제    만도헬라일렉트로닉스   26397       69819        8457    81326     8903

# 동원엔터프라이즈

In [574]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('동원엔터프라이즈', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['동원엔터프라이즈' '동원에프앤비' '동원산업' '동원시스템즈' '동원냉장' '동원건설산업' '동원팜스' '동원홈푸드'
 '동원씨앤에스' '동부익스프레스' '동원로엑스' '테크팩솔루션' '농업회사법인어석' '동원와인플러스' '동부인천항만'
 '동부광양물류센터' '동부부산컨테이너터미널' '동원티엘에스' '부산신항다목적터미널']
['동원에프앤비' '동원산업' '동원시스템즈' '동원냉장' '동원건설산업' '동원팜스' '동원홈푸드' '동원씨앤에스'
 '동부익스프레스' '동원로엑스' '테크팩솔루션' '동원와인플러스' '동부인천항만' '동부광양물류센터' '동부부산컨테이너터미널'
 '동원티엘에스' 'StarKistCo.' 'SCASA' 'TTP' '동원엔터프라이즈' 'DWGlobalInc.'
 'DongwonJapanCo.,Ltd.' '동원식품(상해)유한공사' 'StarKistSamoaCo.' 'GalapescaS.A.'
 'STSC' 'CAPSEN' 'KKT' 'TALOFA' 'MVP' '농업회사법인어석' '수도권동북부물류단지' '위해삼조유한공사'
 '부산신항다목적터미널' 'DI' '상해동부' 'DV' 'DU']


매출회사     매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)  국내매출액 해외매출액
0       동원엔터프라이즈   동원에프앤비  12885       41738        2422  42961  2422
1       동원엔터프라이즈     동원산업   4355       41738        2422  42961  2422
2       동원엔터프라이즈   동원시스템즈   4466       41738        2422  42961  2422
3       동원엔터프라이즈     동원냉장    227       41738        2422  42961  2422
4       동원엔터프라이즈   동원건설산업   1661       41738        2422  42961  2422
..           ...      ...    ...         ...         ...    ...   ...
139     동부광양물류센터  동부익스프레스    432         434         NaN   5083   NaN
140  동부부산컨테이너터미널     동원산업     92        2449         NaN  49574   NaN
141  동부부산컨테이너터미널  동부익스프레스   2357        2449         NaN  49574   NaN
142       동원티엘에스  동부익스프레스     36          36         NaN   6979   NaN
143   부산신항다목적터미널  동부익스프레스    482         482         NaN  18681   NaN

[144 rows x 7 columns]

In [575]:
dfLong = api.getTransTable('동원엔터프라이즈', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['동원엔터프라이즈' '동원에프앤비' '동원산업-1' '동원시스템즈' '동원건설산업' '동원팜스' '동원홈푸드' '동원씨앤에스'
 '동원로엑스' '동원로엑스냉장' '테크팩솔루션' '농업회사법인어석' '동원와인플러스' '동원로엑스인천' '동원로엑스광양'
 '동부부산컨테이너터미널' '부산신항다목적터미널' '비아이디씨']
['동원에프앤비' '동원산업' '동원시스템즈' '동원건설산업' '동원팜스' '동원홈푸드' '동원씨앤에스' '동원로엑스'
 '동원로엑스냉장' '테크팩솔루션' '농업회사법인어석' '동원와인플러스' '동원로엑스인천' '동원로엑스광양' '동부부산컨테이너터미널'
 '동원티엘에스' '비아이디씨' '동원냉장' 'StarKistCo.' 'SCASA' 'TTP' '동원엔터프라이즈'
 'DWGlobalInc.' 'DongwonJapanCo.,Ltd.' '동원식품(상해)유한공사'
 'DongwonF&BVietnamCo.,Ltd.' 'StarKistSamoaCo.' 'GalapescaS.A.' 'CAPSEN'
 'KKT' 'TALOFA' 'MVP' '수도권동북부물류단지' '남춘천산업단지개발' '위해삼조유한공사' '부산신항다목적터미널'
 '상해동부' 'DU' 'DV']


매출회사    매입회사    매출액 국내계열사계(매출액)  해외계열사계(매출액)  국내매출액 해외매출액
0       동원엔터프라이즈  동원에프앤비  12252       42154       1949.0  43943  1949
1       동원엔터프라이즈    동원산업   4892       42154       1949.0  43943  1949
2       동원엔터프라이즈  동원시스템즈   4301       42154       1949.0  43943  1949
3       동원엔터프라이즈  동원건설산업   1958       42154       1949.0  43943  1949
4       동원엔터프라이즈    동원팜스   2018       42154       1949.0  43943  1949
..           ...     ...    ...         ...          ...    ...   ...
146  동부부산컨테이너터미널    동원산업     38        1826          NaN  53650   NaN
147  동부부산컨테이너터미널   동원로엑스   1788        1826          NaN  53650   NaN
148   부산신항다목적터미널   동원로엑스     49          49          NaN  17739   NaN
149        비아이디씨    동원산업    215         688          NaN  47364  2754
150        비아이디씨   동원로엑스    473         688          NaN  47364  2754

[151 rows x 7 columns]

In [576]:
dfLong = api.getTransTable('동원엔터프라이즈', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['동원엔터프라이즈' '동원에프앤비' '동원산업' '동원시스템즈' '동원건설산업' '동원팜스' '동원홈푸드' '동원씨앤에스'
 '동원로엑스' '동원로엑스냉장' '테크팩솔루션' '농업회사법인어석' '동원와인플러스' '동원로엑스인천' '동원부산컨테이너터미널'
 '동원티엘에스' '부산신항다목적터미널' '비아이디씨']
['동원에프앤비' '동원산업' '동원시스템즈' '동원건설산업' '동원팜스' '동원홈푸드' '동원씨앤에스' '동원로엑스'
 '동원로엑스냉장' '동원로엑스냉장투' '테크팩솔루션' '농업회사법인어석' '동원와인플러스' '동원로엑스인천'
 '동원부산컨테이너터미널' '동원티엘에스' '비아이디씨' '동화' '동원로엑스광양' 'StarKistCo.' 'SCASA' 'TTP'
 '동원엔터프라이즈' 'DWGlobalInc.' 'DongwonJapanCo.,Ltd.' '동원식품(상해)유한공사'
 'DongwonF&BVietnamCo.,Ltd.' 'STSC' 'StarKistSamoaCo.' 'GalapescaS.A'
 'CAPSEN' 'KKT' 'TUDO' 'TALOFA' 'MVP' '남춘천산업단지개발' '위해삼조유한공사' '부산신항다목적터미널'
 'DU']


매출회사    매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)  국내매출액 해외매출액
0      동원엔터프라이즈  동원에프앤비  10780       38687        2156  39402  2156
1      동원엔터프라이즈    동원산업   2863       38687        2156  39402  2156
2      동원엔터프라이즈  동원시스템즈   3228       38687        2156  39402  2156
3      동원엔터프라이즈  동원건설산업   2434       38687        2156  39402  2156
4      동원엔터프라이즈    동원팜스   2237       38687        2156  39402  2156
..          ...     ...    ...         ...         ...    ...   ...
155  부산신항다목적터미널   동원로엑스     26          26         NaN  15489   NaN
156       비아이디씨  동원에프앤비     40        3068         NaN  43578   262
157       비아이디씨    동원산업     63        3068         NaN  43578   262
158       비아이디씨  동원건설산업     13        3068         NaN  43578   262
159       비아이디씨   동원로엑스   2952        3068         NaN  43578   262

[160 rows x 7 columns]

# 엔엑스씨

In [564]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('엔엑스씨', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['엔엑스씨' '엔엑스씨엘' '소호브릭스' '와이즈키즈' '엔엑스프로퍼티스' '넥슨코리아' '넥슨네트웍스' '넥슨스페이스' '네오플'
 '넥슨지티' '띵소프트' '넥슨커뮤니케이션즈' '불리언게임즈' '넥슨레드' '엔미디어플랫폼' '중앙판교개발']
['엔엑스씨엘' '소호브릭스' '넥슨코리아' '네오플' '넥슨지티' '넥슨레드' '브이아이피사모주식형펀드일호' '엔엑스씨'
 '넥슨스페이스' '엔엑스프로퍼티스' '스토케코리아' '와이즈키즈' '엔엑스브이피' '넥슨네트웍스' '띵소프트' '넥슨커뮤니케이션즈'
 '불리언게임즈' '엔미디어플랫폼' '엔도어즈']


매출회사            매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액    해외매출액
0        엔엑스씨           엔엑스씨엘     65       10279        5781   15787     6195
1        엔엑스씨           소호브릭스    409       10279        5781   15787     6195
2        엔엑스씨           넥슨코리아   2841       10279        5781   15787     6195
3        엔엑스씨             네오플   1333       10279        5781   15787     6195
4        엔엑스씨            넥슨지티      0       10279        5781   15787     6195
5        엔엑스씨            넥슨레드      2       10279        5781   15787     6195
6        엔엑스씨  브이아이피사모주식형펀드일호   5629       10279        5781   15787     6195
7       엔엑스씨엘            엔엑스씨      3          34         NaN     421      NaN
8       엔엑스씨엘           넥슨코리아      0          34         NaN     421      NaN
9       엔엑스씨엘             네오플     31          34         NaN     421      NaN
10      소호브릭스            엔엑스씨      8          12         NaN     650       10
11      소호브릭스           넥슨코리아      4          12         NaN     650       10
12      소호브릭스          넥슨스페이스      0          12         NaN     650       10
13      와이즈키즈            엔엑스씨      1          25         NaN     145       68
14      와이즈키즈        엔엑스프로퍼티스     24          25         NaN     145       68
15   엔엑스프로퍼티스          스토케코리아    474         604         NaN     910      NaN
16   엔엑스프로퍼티스           와이즈키즈     89         604         NaN     910      NaN
17   엔엑스프로퍼티스          엔엑스브이피     41         604         NaN     910      NaN
18      넥슨코리아            엔엑스씨     11        1800       20514  717326   229527
19      넥슨코리아          넥슨네트웍스    190        1800       20514  717326   229527
20      넥슨코리아          넥슨스페이스     25        1800       20514  717326   229527
21      넥슨코리아             네오플   1098        1800       20514  717326   229527
22      넥슨코리아            넥슨지티      1        1800       20514  717326   229527
23      넥슨코리아            띵소프트     32        1800       20514  717326   229527
24      넥슨코리아       넥슨커뮤니케이션즈     24        1800       20514  717326   229527
25      넥슨코리아          불리언게임즈     -2        1800       20514  717326   229527
26      넥슨코리아            넥슨레드      0        1800       20514  717326   229527
27      넥슨코리아         엔미디어플랫폼    421        1800       20514  717326   229527
28     넥슨네트웍스           넥슨코리아  33707       34220        1447   34219     1447
29     넥슨네트웍스             네오플    387       34220        1447   34219     1447
30     넥슨네트웍스         엔미디어플랫폼    126       34220        1447   34219     1447
31     넥슨스페이스           넥슨코리아   9384       12326         NaN   12538      NaN
32     넥슨스페이스          넥슨네트웍스   1221       12326         NaN   12538      NaN
33     넥슨스페이스            넥슨지티    547       12326         NaN   12538      NaN
34     넥슨스페이스            띵소프트    201       12326         NaN   12538      NaN
35     넥슨스페이스          불리언게임즈     79       12326         NaN   12538      NaN
36     넥슨스페이스            넥슨레드    717       12326         NaN   12538      NaN
37     넥슨스페이스         엔미디어플랫폼    157       12326         NaN   12538      NaN
38     넥슨스페이스            엔도어즈     20       12326         NaN   12538      NaN
39        네오플           와이즈키즈      0       51043        1429   57167  1248432
40        네오플           넥슨코리아  51043       51043        1429   57167  1248432
41       넥슨지티           넥슨코리아  27638       27648         113   27675      234
42       넥슨지티            넥슨레드     10       27648         113   27675      234
43       띵소프트           넥슨코리아   1249        5002         NaN    5002      NaN
44       띵소프트             네오플   3753        5002         NaN    5002      NaN
45  넥슨커뮤니케이션즈           넥슨코리아   3600        3600         NaN    3600      NaN
46     불리언게임즈           넥슨코리아  12146       12146         NaN   12348      NaN
47       넥슨레드           넥슨코리아   2581       11162         NaN   11459      496
48       넥슨레드            넥슨지티   8581       11162         NaN   11459      496
49    엔미디어플랫폼           넥슨코리아   4939        5117         NaN   17065      NaN
50    엔미디어플랫폼             네오플  

In [566]:
dfLong = api.getTransTable('엔엑스씨', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['엔엑스씨' '아퀴스코리아' '스토케코리아' '유한책임회사와이즈키즈' '유한책임회사엔엑스프로퍼티스' '넥슨코리아' '넥슨네트웍스'
 '넥슨스페이스' '네오플' '넥슨지티' '띵소프트' '넥슨커뮤니케이션즈' '엔미디어플랫폼' '중앙판교개발']
['아퀴스코리아' '가승개발' '넥슨코리아' '네오플' '넥슨지티' '소호브릭스' '넥슨레드' 'ATMediaInvestor,LLC'
 'NEXONCo.,Ltd.' 'NEXONAmerica,Inc.' 'NEXONMInc.' 'NEXONThailandCo.,Ltd.'
 'NEXONTAIWANLIMITED' '엔엑스씨' 'STOKKEAS' '유한책임회사엔엑스프로퍼티스' '스토케코리아'
 '유한책임회사와이즈키즈' '넥슨네트웍스' '넥슨스페이스' '띵소프트' '넥슨커뮤니케이션즈' '엔미디어플랫폼' '불리언게임즈'
 'BigHugeGames,Inc.']


매출회사    매입회사   매출액 국내계열사계(매출액) 해외계열사계(매출액)  국내매출액 해외매출액
0      엔엑스씨  아퀴스코리아    65        4836        7766  13737  8548
1      엔엑스씨    가승개발     7        4836        7766  13737  8548
2      엔엑스씨   넥슨코리아  2774        4836        7766  13737  8548
3      엔엑스씨     네오플  1568        4836        7766  13737  8548
4      엔엑스씨    넥슨지티     0        4836        7766  13737  8548
..      ...     ...   ...         ...         ...    ...   ...
60  엔미디어플랫폼     네오플    10        4507         NaN  17566    17
61  엔미디어플랫폼    넥슨지티    95        4507         NaN  17566    17
62   중앙판교개발    넥슨지티  2167        2400         NaN   2400   NaN
63   중앙판교개발    띵소프트    74        2400         NaN   2400   NaN
64   중앙판교개발    넥슨레드   159        2400         NaN   2400   NaN

[65 rows x 7 columns]

In [567]:
dfLong = api.getTransTable('엔엑스씨', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['엔엑스씨' '아퀴스코리아' '스토케코리아' '유한책임회사와이즈키즈' '유한책임회사엔엑스프로퍼티스' '넥슨코리아' '넥슨네트웍스'
 '넥슨스페이스' '네오플' '넥슨지티' '넥슨커뮤니케이션즈' '엔미디어플랫폼' '중앙판교개발' '티디에프' '니트로스튜디오']
['아퀴스코리아' '넥슨코리아' '네오플' '넥슨지티' '소호브릭스' '넥스레드' '가승개발' 'NEXONCo.,Ltd.'
 'NEXONAmerica,Inc.' 'NEXONThailandCo.,Ltd.' 'NEXONTAIWANLIMITED'
 'NXC,LLC' '엔엑스씨' '넥슨스페이스' 'STOKKEAS' 'StokkeJapanCo.,Ltd.'
 '유한책임회사엔엑스프로퍼티스' '유한책임회사와이즈키즈' '넥슨네트웍스' '넥슨커뮤니케이션즈' '엔미디어플랫폼' '데브캣'
 '니트로스튜디오' 'BigHugeGames,Inc.' 'NEXONNETWORKSVINACOMPANYLIMITED' '띵소프트'
 '불리언게임즈']


매출회사     매입회사   매출액 국내계열사계(매출액) 해외계열사계(매출액)  국내매출액  해외매출액
0      엔엑스씨   아퀴스코리아    10        6887       84111  12806  85166
1      엔엑스씨    넥슨코리아  4906        6887       84111  12806  85166
2      엔엑스씨      네오플  1887        6887       84111  12806  85166
3      엔엑스씨     넥슨지티     0        6887       84111  12806  85166
4      엔엑스씨    소호브릭스    73        6887       84111  12806  85166
..      ...      ...   ...         ...         ...    ...    ...
56   중앙판교개발     넥슨지티  1051        2367         NaN   2452    NaN
57   중앙판교개발  엔미디어플랫폼    23        2367         NaN   2452    NaN
58   중앙판교개발     넥스레드   234        2367         NaN   2452    NaN
59     티디에프    넥슨코리아    14          14         NaN     14    NaN
60  니트로스튜디오    넥슨코리아   948         948         NaN    948    NaN

[61 rows x 7 columns]

# 태영건설

In [556]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('태영건설', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['태영건설' '디엠씨미디어' '리앤에스스포츠' '블렌딩' '블루원' '에스비에스' '에스비에스네오파트너스' '에스비에스디지털뉴스랩'
 '에스비에스아이앤엠' '에스비에스아프리카티비' '에스비에스에이앤티' '에스비에스엠앤씨' '에스비에스콘텐츠허브' '에스비에스플러스'
 '엠시에타' '인제스피디움' '티에스케이그린에너지' '티에스케이엠엔에스' '티에스케이워터' '티에스케이코퍼레이션' '더스토리웍스'
 '스마트미디어렙' '신경주지역개발' '에스비에스미디어홀딩스' '에스비에스바이아컴' '에코시스템' '콘텐츠연합플랫폼']
['태영인더스트리' '유니시티' '에코시티개발' '엠시에타개발' '포천바이오에너지' '블루원' '경산에코에너지' '에코시티'
 '신경주역세권공영개발' '이너시티개발' '동부권푸른물' '에스비에스' '스마트미디어렙' '에스비에스플러스' '에스비에스엠앤씨'
 '에스비에스콘텐츠허브' '에스비에스미디어홀딩스' '태영건설' '인제스피디움' '에스비에스바이아컴' '콘텐츠연합플랫폼'
 '디엠씨미디어' '에스비에스에이엔티' '에스비에스아이앤엠' 'SBS아프리카티비' 'SBSinternationalInc' 'KCP'
 '에스비에스디지털뉴스랩' '에스비에스네오파트너스' '더스토리웍스' '블렌딩' 'PLANKEntertainment' '리앤에스스포츠'
 '울산이앤피' '티에스케이그린에너지' '티에스케이코퍼레이션' '여수엑스포환경' '티에스케이엔워터테크' '부산바이오에너지'
 '티에스케이워터']


매출회사        매입회사       매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액 해외매출액
0        태영건설     태영인더스트리     148.0      850294         NaN  1991815  4234
1        태영건설        유니시티  476421.0      850294         NaN  1991815  4234
2        태영건설      에코시티개발  133049.0      850294         NaN  1991815  4234
3        태영건설      엠시에타개발  207263.0      850294         NaN  1991815  4234
4        태영건설    포천바이오에너지    1049.0      850294         NaN  1991815  4234
..        ...         ...       ...         ...         ...      ...   ...
118     에코시스템  티에스케이그린에너지       676        1570         NaN    70279   NaN
119     에코시스템  티에스케이코퍼레이션       802        1570         NaN    70279   NaN
120     에코시스템     티에스케이워터        92        1570         NaN    70279   NaN
121  콘텐츠연합플랫폼       에스비에스      81.0        1004         NaN    65099   NaN
122  콘텐츠연합플랫폼     스마트미디어렙     923.0        1004         NaN    65099   NaN

[123 rows x 7 columns]

In [557]:
dfLong = api.getTransTable('태영건설', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['태영건설' '디엠씨미디어' '리앤에스스포츠' '블렌딩' '블루원' '에스비에스' '에스비에스네오파트너스' '에스비에스디지털뉴스랩'
 '에스비에스미디어넷' '에스비에스아이앤엠' '에스비에스아프리카티비' '에스비에스에이앤티' '에스비에스엠앤씨' '에스비에스콘텐츠허브'
 '에스비에스플러스' '엠시에타' '인제스피디움' '티에스케이그린에너지' '티에스케이엠엔에스' '티에스케이워터'
 '티에스케이코퍼레이션' '포맷티스트' '휴비스워터' '스마트미디어렙' '스튜디오에스' '신경주지역개발' '에스비에스미디어홀딩스'
 '에스비에스바이아컴' '에코시스템']
['태영인더스트리' '유니시티' '에코시티개발' '엠시에타개발' '포천바이오에너지' '블루원' '경산에코에너지' '에코시티'
 '신경주역세권공영개발' '이너시티개발' '동부권푸른물' '평택싸이로' '양주동서도로' '군포복합개발PFV' '에스비에스'
 '스마트미디어렙' '에스비에스미디어넷' '에스비에스엠앤씨' '에스비에스콘텐츠허브' '에스비에스네오파트너스' 'SBS디지털뉴스랩'
 'SBS미디어홀딩스' '태영건설' '인제스피디움' '엠시에타' '에코시스템' '에스비에스아이앤엠' '에스비에스디지털뉴스랩'
 '에스비에스바이아컴' '디엠씨미디어' '에스비에스에이엔티' 'SBS플러스' 'SBS아프리카티비' 'SBSinternational'
 'KCP' '리앤에스스포츠' '콘텐츠웨이브' '스튜디오에스' '블렌딩' '티에스케이코퍼레이션' '티에스케이워터' '티에스케이이앤피'
 '티에스케이엔워터테크' '티에스케이그린에너지' '휴비스워터' '여수엑스포환경' '부산바이오에너지']


매출회사        매입회사       매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  해외매출액
0         태영건설     태영인더스트리     127.0      838651         NaN  2157078  18607
1         태영건설        유니시티  351826.0      838651         NaN  2157078  18607
2         태영건설      에코시티개발  146633.0      838651         NaN  2157078  18607
3         태영건설      엠시에타개발  254646.0      838651         NaN  2157078  18607
4         태영건설    포천바이오에너지    2293.0      838651         NaN  2157078  18607
..         ...         ...       ...         ...         ...      ...    ...
149  에스비에스바이아컴   에스비에스아이앤엠       3.0         127         NaN     7647    NaN
150  에스비에스바이아컴      콘텐츠웨이브        81         127         NaN     7647    NaN
151      에코시스템  티에스케이그린에너지      1224        2722         NaN   111027    NaN
152      에코시스템  티에스케이코퍼레이션       904        2722         NaN   111027    NaN
153      에코시스템     티에스케이워터       594        2722         NaN   111027    NaN

[154 rows x 7 columns]

In [559]:
dfLong = api.getTransTable('티와이홀딩스', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['디엠씨미디어' '리앤에스스포츠' '무노스' '문고리닷컴' '블루원' '블루원레저' '에스비에스' '에스비에스네오파트너스'
 '에스비에스디지털뉴스랩' '에스비에스미디어넷' '에스비에스아이앤엠' '에스비에스에이앤티' '에스비에스엠앤씨' '에스비에스콘텐츠허브'
 '에스비에스플러스' '엠시에타' '이너시티개발' '인제스피디움' '태영건설' '태영그레인터미널' '티에스케이그린에너지'
 '티에스케이엔지니어링' '티에스케이엠엔에스' '티에스케이워터' '티에스케이이앤피' '티에스케이코퍼레이션' '티에스케이프리텍'
 '평택싸이로' '포맷티스트' '삼계개발' '스마트미디어렙' '스튜디오에스' '신경주지역개발' '에스비에스미디어홀딩스'
 '에스비에스바이아컴' '에코시스템']
['에스비에스' '스마트미디어렙' '에스비에스미디어넷' '에스비에스엠앤씨' '에스비에스엠앤씨.1' 'SBS디지털뉴스랩' '문고리닷컴'
 '블루원' '에스비에스아이앤엠' 'SBS플러스' '에스비에스콘텐츠허브' 'SBS미디어홀딩스' '태영건설' '태영인더스트리'
 '블루원레저' '엠시에타' '에스비에스바이아컴' '스튜디오에스' '디엠씨미디어' 'SBS에이엔티' 'SBSinternational'
 'KCP' '블렌딩' '리앤에스스포츠' '무노스' '콘텐츠웨이브' '엠시에타개발' 'SBS아프리카TV'
 'KoreaContentPlatfrom,LCC' '에코시티' '티에스케이워터' '대구남부AMC' '에코시티개발' '포천바이오에너지'
 '경산에코에너지' '신경주역세권공영개발' '이너시티개발' '평택싸이로' '네오시티' '군포복합개발PFV' 'TSK이앤피'
 'TSK코퍼레이션' '울산이앤피' '휴비스워터' '여수엑스포환경' '여천이피에스' '부산바이오에너지' 'TSK엔지니어링'
 'TSK엔워터테크' 'TSK그린에너지' '태영그레인터미널' '에스비에스네오파트너스']


매출회사        매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액 해외매출액
0       디엠씨미디어       에스비에스   896.0        2978         NaN   22732   NaN
1       디엠씨미디어     스마트미디어렙  1651.0        2978         NaN   22732   NaN
2       디엠씨미디어   에스비에스미디어넷     1.0        2978         NaN   22732   NaN
3       디엠씨미디어    에스비에스엠앤씨    27.0        2978         NaN   22732   NaN
4       디엠씨미디어  에스비에스엠앤씨.1    27.0        2978         NaN   22732   NaN
..         ...         ...     ...         ...         ...     ...   ...
149  에스비에스바이아컴      콘텐츠웨이브      19          20         NaN    6643   207
150      에코시스템      TSK이앤피     785        2999         NaN  117330   NaN
151      에코시스템    TSK그린에너지    1821        2999         NaN  117330   NaN
152      에코시스템    TSK엔지니어링     162        2999         NaN  117330   NaN
153      에코시스템     티에스케이워터     231        2999         NaN  117330   NaN

[153 rows x 7 columns]

# 호반건설

In [548]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('호반건설', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['호반건설' '호반산업' '호반티비엠' '호반호텔앤리조트' '그린산업' '퍼시픽마리나' '광주방송' '케이비씨플러스' '아브뉴프랑'
 '호반스카이밸리']
['호반자산개발' '스카이리빙' '스카이하우징' '스카이주택' '스카이건설' '호반산업' '배곧랜드마크피에프브이' '광주방송'
 '티에스주택' '티에스개발' '티에스건설' '티에스자산개발' '베르디움리빙' '호반티비엠' '호반건설' '호반베르디움'
 '리솜웨이하이골프리조트' '퍼시픽랜드' '아브뉴프랑' '호반스카이밸리' '코너스톤투자파트너스' '호반호텔앤리조트' '호반'
 '티에스리빙' '케이비씨플러스' '리젠시빌주택' '리젠시빌건설']


매출회사         매입회사    매출액 국내계열사계(매출액)  해외계열사계(매출액)    국내매출액 해외매출액
0       호반건설       호반자산개발   4745      279702          NaN  1174440   NaN
1       호반건설        스카이리빙  87514      279702          NaN  1174440   NaN
2       호반건설       스카이하우징  69942      279702          NaN  1174440   NaN
3       호반건설        스카이주택  75926      279702          NaN  1174440   NaN
4       호반건설        스카이건설  14079      279702          NaN  1174440   NaN
5       호반건설         호반산업    897      279702          NaN  1174440   NaN
6       호반건설  배곧랜드마크피에프브이  26171      279702          NaN  1174440   NaN
7       호반건설         광주방송    428      279702          NaN  1174440   NaN
8       호반산업        티에스주택   9637      166729          NaN   589358  3475
9       호반산업        티에스개발  26027      166729          NaN   589358  3475
10      호반산업        티에스건설  47697      166729          NaN   589358  3475
11      호반산업      티에스자산개발  79565      166729          NaN   589358  3475
12      호반산업       베르디움리빙   3492      166729          NaN   589358  3475
13      호반산업        호반티비엠    311      166729          NaN   589358  3475
14     호반티비엠         호반건설   1566        2983          NaN    17205   NaN
15     호반티비엠         호반산업   1417        2983          NaN    17205   NaN
16  호반호텔앤리조트         호반건설     15          26          NaN    45534   NaN
17  호반호텔앤리조트        스카이리빙      0          26          NaN    45534   NaN
18  호반호텔앤리조트       스카이하우징      0          26          NaN    45534   NaN
19  호반호텔앤리조트         호반산업      4          26          NaN    45534   NaN
20  호반호텔앤리조트        티에스주택      0          26          NaN    45534   NaN
21  호반호텔앤리조트        티에스개발      0          26          NaN    45534   NaN
22  호반호텔앤리조트        티에스건설      0          26          NaN    45534   NaN
23  호반호텔앤리조트        호반티비엠      0          26          NaN    45534   NaN
24  호반호텔앤리조트       호반베르디움      1          26          NaN    45534   NaN
25  호반호텔앤리조트  리솜웨이하이골프리조트      5          26          NaN    45534   NaN
26  호반호텔앤리조트        퍼시픽랜드      0          26          NaN    45534   NaN
27  호반호텔앤리조트        아브뉴프랑      1          26          NaN    45534   NaN
28  호반호텔앤리조트      호반스카이밸리      0          26          NaN    45534   NaN
29  호반호텔앤리조트   코너스톤투자파트너스      0          26          NaN    45534   NaN
30      그린산업     호반호텔앤리조트    761         761          NaN      761   NaN
31    퍼시픽마리나        퍼시픽랜드   1050        1050          NaN     1050   NaN
32      광주방송         호반건설    118         571          NaN    73769   NaN
33      광주방송           호반     96         571          NaN    73769   NaN
34      광주방송        티에스주택      1         571          NaN    73769   NaN
35      광주방송        티에스개발      1         571          NaN    73769   NaN
36      광주방송        티에스건설      1         571          NaN    73769   NaN
37      광주방송      티에스자산개발      1         571          NaN    73769   NaN
38      광주방송        티에스리빙      1         571          NaN    73769   NaN
39      광주방송       호반베르디움      7         571          NaN    73769   NaN
40      광주방송      케이비씨플러스    241         571          NaN    73769   NaN
41      광주방송       리젠시빌주택     92         571          NaN    73769   NaN
42      광주방송       리젠시빌건설     12         571          NaN    73769   NaN
43   케이비씨플러스         호반건설     63       35390          NaN    38142   NaN
44   케이비씨플러스         광주방송  35327       35390          NaN    38142   NaN
45     아브뉴프랑         호반건설    936        3324          NaN     3561   NaN
46     아브뉴프랑       호반베르디움   2388        3324          NaN     3561   NaN
47   호반스카이밸리         호반건설    719        2049          NaN    18937   NaN
48   호반스카이밸리           호반   1059        2049          NaN    18937   NaN
49   호반스카이밸리         호반산업    251        2049          NaN    18937   NaN
50   호반스카이밸리       호반베르디움      2        2049          NaN    18937   NaN
51   호반스카이밸리        퍼시픽랜드      2        2049          NaN    18937   NaN
52   호반스카이밸리         광주방송     16        2049          NaN    18937   NaN

In [550]:
dfLong = api.getTransTable('호반건설', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['호반건설' '호반자산개발' '호반산업' '티에스자산개발' '호반티비엠' '호반프라퍼티' '호반호텔앤리조트' '그린산업'
 '퍼시픽마리나' '플랜에이치벤처스' '호반서서울' '광주방송' '케이비씨플러스' '호반써밋' '아브뉴프랑' '호반스카이밸리'
 '코너스톤투자파트너스']
['호반자산개발' '대전용산개발' '스카이리빙' '스카이하우징' '호반주택' '호반산업' '배곧랜드마크피에프브이' '호반호텔앤리조트'
 '광주방송' '호반건설' '에이치비탕정' '티에스주택' '티에스개발' '티에스건설' '티에스자산개발' '티에스리빙' '호반티비엠'
 '호반프라퍼티' '플랜에이치벤처스' '호반써밋' '아브뉴프랑' '호반스카이밸리' '코너스톤투자파트너스' '호반서서울'
 '케이비씨플러스' '대아청과' '리젠시빌주택' '리젠시빌건설']


매출회사        매입회사    매출액 국내계열사계(매출액)  해외계열사계(매출액)    국내매출액 해외매출액
0          호반건설      호반자산개발  28291      284472          NaN  1977191   NaN
1          호반건설      대전용산개발    132      284472          NaN  1977191   NaN
2          호반건설       스카이리빙  16258      284472          NaN  1977191   NaN
3          호반건설      스카이하우징  16454      284472          NaN  1977191   NaN
4          호반건설        호반주택  10466      284472          NaN  1977191   NaN
..          ...         ...    ...         ...          ...      ...   ...
106     호반스카이밸리        광주방송     16        1457          NaN    21097   NaN
107     호반스카이밸리        호반써밋      7        1457          NaN    21097   NaN
108     호반스카이밸리       아브뉴프랑     58        1457          NaN    21097   NaN
109     호반스카이밸리  코너스톤투자파트너스     28        1457          NaN    21097   NaN
110  코너스톤투자파트너스        호반건설  218.0         218          NaN     1719   NaN

[111 rows x 7 columns]

In [552]:
dfLong = api.getTransTable('호반건설', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['호반건설' '호반자산개발' '호반산업' '호반티비엠' '호반프라퍼티' '호반호텔앤리조트' '그린산업' '퍼시픽마리나'
 '플랜에이치벤처스' '호반서서울' '광주방송' '케이비씨플러스' '호반써밋' '호반' '삼성금거래소' '삼인기업' '열린개발'
 '코너스톤투자파트너스']
['호반자산개발' '대전용산개발' '스카이리빙' '스마트시티더원피에프브이' '배곧랜드마크피에프브이' '호반호텔앤리조트' '호반건설'
 '호반주택' '호반산업' '티에스주택' '티에스개발' '티에스건설' '티에스자산개발' '티에스리빙' '호반티비엠' '호반프라퍼티'
 '플랜에이치벤처스' '호반써밋' '아브뉴프랑' '삼성금거래소' '코너스톤투자파트너스' '마륵파크' '그린산업' '호반서서울'
 '광주방송' '케이비씨플러스' '호반' '대아청과' '영암마트운남점']


매출회사          매입회사    매출액 국내계열사계(매출액)  해외계열사계(매출액)    국내매출액   해외매출액
0          호반건설        호반자산개발  17554      123239          NaN   895802     NaN
1          호반건설        대전용산개발  35115      123239          NaN   895802     NaN
2          호반건설         스카이리빙   6436      123239          NaN   895802     NaN
3          호반건설  스마트시티더원피에프브이   4109      123239          NaN   895802     NaN
4          호반건설   배곧랜드마크피에프브이    207      123239          NaN   895802     NaN
..          ...           ...    ...         ...          ...      ...     ...
100      삼성금거래소          대아청과     31         162          NaN  1136032  147599
101        삼인기업          호반건설  896.0        1794          NaN     2034     NaN
102        삼인기업          호반산업    898        1794          NaN     2034     NaN
103        열린개발       영암마트운남점    144         144          NaN      168     NaN
104  코너스톤투자파트너스          호반건설  778.0         778          NaN    10928     NaN

[105 rows x 7 columns]

# DB

In [545]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('DB', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['DBInc.' 'DB메탈' 'DB월드' 'DB하이텍' '동부철구' 'DB스탁인베스트' 'DB에프아이에스' 'DB인베스트'
 'DB저축은행' 'DB금융서비스' 'DB금융투자' 'DB생명보험' 'DB손해보험' 'DB씨앤에스자동차손해사정'
 'DB씨에스아이손해사정' 'DB씨에이에스손해사정' 'DB엠앤에스' 'DB자동차보험손해사정' 'DB자산운용' 'DB캐피탈']
['DBInc.' 'DB메탈' 'DB월드' 'DB하이텍' 'DB에프아이에스' '대우전자' '대우전자서비스' 'DB라이텍'
 'DB저축은행' 'DB금융서비스' 'DB금융투자' 'DB생명보험' 'DB손해보험' 'DB씨앤에스자동차손해사정'
 'DB씨에스아이손해사정' 'DB씨에이에스손해사정' 'DB엠앤에스' 'DB자동차보험손해사정' 'DB자산운용' 'DB캐피탈'
 'DBMetalUSAINC.' 'DBMetalEuropeGmbH']


매출회사               매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액  해외매출액
0    DBInc.             DBInc.      0        46302            0  122170  84669
1    DBInc.               DB메탈   6651        46302            0  122170  84669
2    DBInc.               DB월드    158        46302            0  122170  84669
3    DBInc.              DB하이텍  10506        46302            0  122170  84669
4    DBInc.           DB에프아이에스   1138        46302            0  122170  84669
..      ...                ...    ...          ...          ...     ...    ...
435   DB캐피탈        DB자동차보험손해사정      0          667            0   15007      0
436   DB캐피탈             DB자산운용      0          667            0   15007      0
437   DB캐피탈              DB캐피탈      0          667            0   15007      0
438   DB캐피탈     DBMetalUSAINC.      0          667            0   15007      0
439   DB캐피탈  DBMetalEuropeGmbH      0          667            0   15007      0

[440 rows x 7 columns]

In [546]:
dfLong = api.getTransTable('DB', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['DBInc.' 'DB메탈' 'DB월드' 'DB하이텍' '동부철구' 'DB스탁인베스트' 'DB에프아이에스' 'DB인베스트'
 'DB저축은행' 'DB금융서비스' 'DB금융투자' 'DB생명보험' 'DB손해보험' 'DB씨앤에스자동차손해사정'
 'DB씨에스아이손해사정' 'DB씨에이에스손해사정' 'DB엠앤에스' 'DB자동차보험손해사정' 'DB자산운용' 'DB캐피탈']
['DBInc.' 'DB메탈' 'DB월드' 'DB하이텍' 'DB에프아이에스' 'DB저축은행' 'DB금융서비스' 'DB금융투자'
 'DB생명보험' 'DB손해보험' 'DB씨앤에스자동차손해사정' 'DB씨에스아이손해사정' 'DB씨에이에스손해사정' 'DB엠앤에스'
 'DB자동차보험손해사정' 'DB자산운용' 'DB캐피탈' 'DBMetalUSAINC.' 'DBMetalEuropeGmbH'
 'PostandTelecomunicationJointStock\xa0InsuranceCorporation']


매출회사                                               매입회사    매출액  \
0    DBInc.                                             DBInc.      0   
1    DBInc.                                               DB메탈   9024   
2    DBInc.                                               DB월드    114   
3    DBInc.                                              DB하이텍  11206   
4    DBInc.                                           DB에프아이에스   1600   
..      ...                                                ...    ...   
395   DB캐피탈                                             DB자산운용      0   
396   DB캐피탈                                              DB캐피탈      0   
397   DB캐피탈                                     DBMetalUSAINC.      0   
398   DB캐피탈                                  DBMetalEuropeGmbH      0   
399   DB캐피탈  PostandTelecomunicationJointStock InsuranceCor...      0   

     국내계열사계(매출액)  해외계열사계(매출액)   국내매출액  해외매출액  
0          56333            0  165401  27583  
1          56333            0  165401  27583  
2          56333            0  165401  27583  
3          56333            0  165401  27583  
4          56333            0  165401  27583  
..           ...          ...     ...    ...  
395          114            0   17327      0  
396          114            0   17327      0  
397          114            0   17327      0  
398          114            0   17327      0  
399          114            0   17327      0  

[400 rows x 7 columns]

In [547]:
dfLong = api.getTransTable('이랜드월드', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['이랜드월드' '이랜드리테일' '이랜드파크' '이월드' '이랜드건설' '이랜드시스템스' '와인캐슬' '이랜드이츠' '켄싱턴평창'
 '농업회사법인켄싱턴팜' '농업회사법인맛누리' '켄싱턴월드' '리드온' '이네스트' '올리브스튜디오' '이앤씨월드'
 '이랜드중국패션디자인' '이랜드위시디자인' '예지실업' '이랜드스포츠' '이랜드인재원' '이키즈랜드' '이랜드서비스'
 '이리츠코크렙기업구조조정부동산투자회사' '케이비와이즈스타제6호위탁관리부동산투자회사']
['이랜드리테일' '이랜드파크' '이월드' '이랜드건설' '이랜드시스템스' '와인캐슬' '이랜드이츠' '켄싱턴평창'
 '농업회사법인맛누리' '이랜드테마파크제주' '이크루즈' '켄싱턴월드' '이네스트' '올리브스튜디오' '이앤씨월드'
 '이랜드중국패션디자인' '이랜드위시디자인' '예지실업' '이랜드스포츠' '이랜드인재원' '이키즈랜드' '이랜드서비스' '리드'
 '해외거래처수' '이랜드월드' '리드온' '농업회사법인켄싱턴팜' '이랜드공덕' '이랜드투자일임']


매출회사     매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  \
0                     이랜드월드   이랜드리테일  90388      104152       25036  1026036   
1                     이랜드월드    이랜드파크   5518      104152       25036  1026036   
2                     이랜드월드      이월드   1241      104152       25036  1026036   
3                     이랜드월드    이랜드건설    337      104152       25036  1026036   
4                     이랜드월드  이랜드시스템스     24      104152       25036  1026036   
..                      ...      ...    ...         ...         ...      ...   
202                  이랜드서비스     예지실업      3       43072          54    58553   
203                  이랜드서비스   이랜드스포츠      1       43072          54    58553   
204                  이랜드서비스   해외거래처수      3       43072          54    58553   
205     이리츠코크렙기업구조조정부동산투자회사   이랜드리테일  17632       17632         NaN    17632   
206  케이비와이즈스타제6호위탁관리부동산투자회사   이랜드리테일   6799        6799         NaN     6799   

     해외매출액  
0    28861  
1    28861  
2    28861  
3    28861  
4    28861  
..     ...  
202     54  
203     54  
204     54  
205    NaN  
206    NaN  

[207 rows x 7 columns]

# 이랜드월드

In [542]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('이랜드월드', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['이랜드월드' '이랜드리테일' '이랜드파크' '이월드' '이랜드건설' '이랜드시스템스' '와인캐슬' '이랜드제주리조트'
 '농업회사법인맛누리' '이랜드크루즈' '리드온' '이랜드서비스' '올리브스튜디오' '이앤씨월드' '이랜드중국패션디자인'
 '이랜드위시디자인' '예지실업' '이랜드스포츠' '이랜드인재원' '이리츠코크렙기업구조조정부동산투자회사'
 '케이비와이즈스타제6호위탁관리부동산투자회사' '리드']
['이랜드리테일' '이랜드파크' '이월드' '이랜드건설' '이랜드시스템스' '이랜드제주리조트' '농업회사법인맛누리' '고운조경'
 '애월국제문화복합단지' '이랜드크루즈' '투어몰' '리드온' '이랜드서비스' '올리브스튜디오' '이앤씨월드' '이랜드중국패션디자인'
 '이랜드위시디자인' '예지실업' '이랜드스포츠' '이에셋투자개발' '이랜드인재원' '리드' '해외거래처수' '이랜드월드'
 '이랜드공덕' '와팝' '이리츠코크렙기업구조조정부동산투자회사' '와인캐슬']


매출회사     매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  \
0                     이랜드월드   이랜드리테일  146733      170578       35018  1258076   
1                     이랜드월드    이랜드파크    9447      170578       35018  1258076   
2                     이랜드월드      이월드       4      170578       35018  1258076   
3                     이랜드월드    이랜드건설      84      170578       35018  1258076   
4                     이랜드월드  이랜드시스템스    1948      170578       35018  1258076   
..                      ...      ...     ...         ...         ...      ...   
185  케이비와이즈스타제6호위탁관리부동산투자회사   이랜드리테일    6650        6650         NaN     6650   
186                      리드    이랜드월드     146         209         NaN     1672   
187                      리드   이랜드리테일      55         209         NaN     1672   
188                      리드    이랜드파크       4         209         NaN     1672   
189                      리드  올리브스튜디오       4         209         NaN     1672   

      해외매출액  
0    180558  
1    180558  
2    180558  
3    180558  
4    180558  
..      ...  
185     NaN  
186     NaN  
187     NaN  
188     NaN  
189     NaN  

[190 rows x 7 columns]

In [543]:
dfLong = api.getTransTable('이랜드월드', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['이랜드월드' '이랜드리테일' '이랜드파크' '이월드' '이랜드건설' '이랜드시스템스' '와인캐슬' '이랜드이츠' '켄싱턴코리아'
 '농업회사법인맛누리' '이랜드크루즈' '리드온' '이네스트' '올리브스튜디오' '이앤씨월드' '이랜드중국패션디자인'
 '이랜드위시디자인' '예지실업' '이랜드스포츠' '이랜드인재원' '이키즈랜드' '이리츠코크렙기업구조조정부동산투자회사'
 '케이비와이즈스타제6호위탁관리부동산투자회사' '리드']
['이랜드리테일' '이랜드파크' '이월드' '이랜드건설' '이랜드시스템스' '와인캐슬' '이랜드이츠' '켄싱턴코리아'
 '농업회사법인맛누리' '이랜드테마파크제주' '이랜드크루즈' '켄싱턴월드' '리드온' '이네스트' '올리브스튜디오' '이앤씨월드'
 '이랜드중국패션디자인' '이랜드위시디자인' '예지실업' '이랜드스포츠' '이랜드인재원' '이키즈랜드' '리드' '해외거래처수'
 '이랜드월드' '이랜드제주리조트' '이랜드서비스' '와팝' '이랜드공덕']


매출회사     매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  해외매출액
0    이랜드월드   이랜드리테일  129448      191182       28830  1121351  55590
1    이랜드월드    이랜드파크    7317      191182       28830  1121351  55590
2    이랜드월드      이월드   48289      191182       28830  1121351  55590
3    이랜드월드    이랜드건설     122      191182       28830  1121351  55590
4    이랜드월드  이랜드시스템스    1432      191182       28830  1121351  55590
..     ...      ...     ...         ...         ...      ...    ...
199     리드    이랜드월드     146         209         NaN     1646    NaN
200     리드   이랜드리테일      55         209         NaN     1646    NaN
201     리드    이랜드파크       2         209         NaN     1646    NaN
202     리드    이랜드이츠       2         209         NaN     1646    NaN
203     리드  올리브스튜디오       4         209         NaN     1646    NaN

[204 rows x 7 columns]

In [544]:
dfLong = api.getTransTable('이랜드월드', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['이랜드월드' '이랜드리테일' '이랜드파크' '이월드' '이랜드건설' '이랜드시스템스' '와인캐슬' '이랜드이츠' '켄싱턴평창'
 '농업회사법인켄싱턴팜' '농업회사법인맛누리' '켄싱턴월드' '리드온' '이네스트' '올리브스튜디오' '이앤씨월드'
 '이랜드중국패션디자인' '이랜드위시디자인' '예지실업' '이랜드스포츠' '이랜드인재원' '이키즈랜드' '이랜드서비스'
 '이리츠코크렙기업구조조정부동산투자회사' '케이비와이즈스타제6호위탁관리부동산투자회사']
['이랜드리테일' '이랜드파크' '이월드' '이랜드건설' '이랜드시스템스' '와인캐슬' '이랜드이츠' '켄싱턴평창'
 '농업회사법인맛누리' '이랜드테마파크제주' '이크루즈' '켄싱턴월드' '이네스트' '올리브스튜디오' '이앤씨월드'
 '이랜드중국패션디자인' '이랜드위시디자인' '예지실업' '이랜드스포츠' '이랜드인재원' '이키즈랜드' '이랜드서비스' '리드'
 '해외거래처수' '이랜드월드' '리드온' '농업회사법인켄싱턴팜' '이랜드공덕' '이랜드투자일임']


매출회사     매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  \
0                     이랜드월드   이랜드리테일  90388      104152       25036  1026036   
1                     이랜드월드    이랜드파크   5518      104152       25036  1026036   
2                     이랜드월드      이월드   1241      104152       25036  1026036   
3                     이랜드월드    이랜드건설    337      104152       25036  1026036   
4                     이랜드월드  이랜드시스템스     24      104152       25036  1026036   
..                      ...      ...    ...         ...         ...      ...   
202                  이랜드서비스     예지실업      3       43072          54    58553   
203                  이랜드서비스   이랜드스포츠      1       43072          54    58553   
204                  이랜드서비스   해외거래처수      3       43072          54    58553   
205     이리츠코크렙기업구조조정부동산투자회사   이랜드리테일  17632       17632         NaN    17632   
206  케이비와이즈스타제6호위탁관리부동산투자회사   이랜드리테일   6799        6799         NaN     6799   

     해외매출액  
0    28861  
1    28861  
2    28861  
3    28861  
4    28861  
..     ...  
202     54  
203     54  
204     54  
205    NaN  
206    NaN  

[207 rows x 7 columns]

# 태광산업

In [531]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('태광산업', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['태광산업' '대한화섬' '서한물산' '세광패션' '티브로드' '티브로드동대문방송' '티브로드노원방송' '한국디지털케이블미디어센터'
 '티캐스트' '이채널' '챔프비전' '한국케이블텔레콤' '티시스' '티알엔' '메르뱅' '큰희망' '흥국생명보험'
 '흥국화재해상보험' '흥국증권' '흥국자산운용']
['대한화섬' '서한물산' '세광패션' '티브로드' '티시스' '티알엔' '흥국생명보험' '흥국화재해상보험' '고려저축은행'
 '태광화섬상숙유한공사' '태광산업' '티브로드동대문방송' '티브로드노원방송' '한국디지털케이블미디어센터' '티캐스트' '이채널'
 '한국케이블텔레콤' '쇼핑엔티' '챔프비전' '흥국증권' '흥국자산운용' '예가람저축은행' '메르뱅' '동림건설' '큰희망'
 '대신흥국제일호사모투자']


매출회사         매입회사      매출액  국내계열사계(매출액) 해외계열사계(매출액)    국내매출액   해외매출액
0      태광산업         대한화섬  36541.0      37678.0        9029  1511706  657957
1      태광산업         서한물산      267      37678.0        9029  1511706  657957
2      태광산업         세광패션        7      37678.0        9029  1511706  657957
3      태광산업         티브로드    267.0      37678.0        9029  1511706  657957
4      태광산업          티시스    111.0      37678.0        9029  1511706  657957
..      ...          ...      ...          ...         ...      ...     ...
175  흥국자산운용       흥국생명보험   2999.0       3823.0         NaN    23673     NaN
176  흥국자산운용     흥국화재해상보험    546.0       3823.0         NaN    23673     NaN
177  흥국자산운용  대신흥국제일호사모투자       16       3823.0         NaN    23673     NaN
178  흥국자산운용       고려저축은행     68.0       3823.0         NaN    23673     NaN
179  흥국자산운용      예가람저축은행    194.0       3823.0         NaN    23673     NaN

[180 rows x 7 columns]

In [532]:
dfLong = api.getTransTable('태광산업', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['태광산업' '대한화섬' '서한물산' '세광패션' '티캐스트' '이채널' '챔프비전' '한국케이블텔레콤' '티시스' '티알엔'
 '메르뱅' '큰희망' '흥국생명보험' '흥국화재해상보험' '흥국증권' '흥국자산운용']
['대한화섬' '서한물산' '세광패션' '티브로드' '티시스' '티알엔' '흥국생명보험' '흥국화재해상보험' '고려저축은행'
 '태광화섬상숙유한공사' '태광산업' '티브로드동대문방송' '티브로드노원방송' '이채널' '챔프비전' '한국케이블텔레콤' '티캐스트'
 '한국디지털케이블미디어센터' '흥국증권' '흥국자산운용' '예가람저축은행' '메르뱅' '큰희망']


매출회사      매입회사      매출액  국내계열사계(매출액) 해외계열사계(매출액)    국내매출액   해외매출액
0      태광산업      대한화섬  34429.0      35878.0        7610  1447935  579224
1      태광산업      서한물산      277      35878.0        7610  1447935  579224
2      태광산업      세광패션        6      35878.0        7610  1447935  579224
3      태광산업      티브로드    267.0      35878.0        7610  1447935  579224
4      태광산업       티시스    237.0      35878.0        7610  1447935  579224
..      ...       ...      ...          ...         ...      ...     ...
124    흥국증권   예가람저축은행     10.0         70.0         NaN    33735     NaN
125  흥국자산운용    흥국생명보험   2778.0       3839.0         NaN    23886     NaN
126  흥국자산운용  흥국화재해상보험    732.0       3839.0         NaN    23886     NaN
127  흥국자산운용    고려저축은행    114.0       3839.0         NaN    23886     NaN
128  흥국자산운용   예가람저축은행    215.0       3839.0         NaN    23886     NaN

[129 rows x 7 columns]

In [533]:
dfLong = api.getTransTable('태광산업', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['태광산업' '대한화섬' '서한물산' '세광패션' '티캐스트' '이채널' '챔프비전' '한국케이블텔레콤' '티시스' '티알엔'
 '메르뱅' '큰희망' '흥국생명보험' '흥국화재해상보험' '흥국증권' '흥국자산운용']
['대한화섬' '서한물산' '세광패션' '티브로드' '티시스' '티알엔' '흥국생명보험' '흥국화재해상보험' '고려저축은행'
 '태광화섬상숙유한공사' '태광산업' '티브로드동대문방송' '티브로드노원방송' '이채널' '챔프비전' '한국케이블텔레콤' '티캐스트'
 '한국디지털케이블미디어센터' '흥국증권' '흥국자산운용' '예가람저축은행' '메르뱅' '큰희망']


매출회사      매입회사      매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액   해외매출액
0      태광산업      대한화섬  19038.0       20411        7084  1097702  397923
1      태광산업      서한물산    277.0       20411        7084  1097702  397923
2      태광산업      세광패션      6.0       20411        7084  1097702  397923
3      태광산업      티브로드     89.0       20411        7084  1097702  397923
4      태광산업       티시스      262       20411        7084  1097702  397923
..      ...       ...      ...         ...         ...      ...     ...
124    흥국증권  흥국화재해상보험     52.0         144         NaN    46556     NaN
125  흥국자산운용    흥국생명보험   3310.0        4673         NaN    25251     NaN
126  흥국자산운용  흥국화재해상보험   1040.0        4673         NaN    25251     NaN
127  흥국자산운용    고려저축은행    129.0        4673         NaN    25251     NaN
128  흥국자산운용   예가람저축은행    194.0        4673         NaN    25251     NaN

[129 rows x 7 columns]

# 세아홀딩스

In [528]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('세아홀딩스', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['세아홀딩스' '세아제강지주' '세아제강' '세아베스틸' '세아창원특수강' '세아특수강' '세아에프에스' '세아네트웍스'
 '세아에삽' '세아엔지니어링' '세아엘앤에스' '세아메탈' '세아엠앤에스' '에이치피피' '우진정공' '한국쿨러' '세아씨엠'
 '동아스틸' '에스에스아이케이대부']
['세아제강지주' '세아제강' '세아베스틸' '세아창원특수강' '세아특수강' '세아에프에스' '세아에삽' '세아메탈'
 '세아엔지니어링' '세아엠앤에스' '세아엘앤에스' '세아네트웍스' '세아알앤아이' '우진정공' '세아씨엠' '에이팩인베스터스'
 '동아스틸' '에이치피피' '한국쿨러' '세아홀딩스' '이앤비원']


매출회사     매입회사   매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액  해외매출액
0         세아홀딩스   세아제강지주   678        9140         NaN   10006    NaN
1         세아홀딩스     세아제강   779        9140         NaN   10006    NaN
2         세아홀딩스    세아베스틸  2575        9140         NaN   10006    NaN
3         세아홀딩스  세아창원특수강  1472        9140         NaN   10006    NaN
4         세아홀딩스    세아특수강   821        9140         NaN   10006    NaN
..          ...      ...   ...         ...         ...     ...    ...
113        세아씨엠   세아네트웍스    28        6970        1730  297702  89143
115        동아스틸     세아제강  4873        6182         NaN   75396  16952
116        동아스틸   세아엘앤에스  1022        6182         NaN   75396  16952
117        동아스틸     세아씨엠   287        6182         NaN   75396  16952
140  에스에스아이케이대부     동아스틸  1619        1619         NaN    1652    NaN

[106 rows x 7 columns]

In [529]:
dfLong = api.getTransTable('세아홀딩스', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['세아홀딩스' '세아제강지주' '세아제강' '세아베스틸' '세아창원특수강' '세아특수강' '세아에프에스' '세아네트웍스'
 '세아에삽' '세아엔지니어링' '세아엘앤에스' '세아알앤아이' '세아메탈' '세아엠앤에스' '이앤비원' '에이치피피' '우진정공'
 '한국쿨러' '세아씨엠' '동아스틸' '씨티씨']
['세아제강지주' '세아제강' '세아베스틸' '세아창원특수강' '세아특수강' '세아에프에스' '세아에삽' '세아메탈'
 '세아엔지니어링' '세아엠앤에스' '세아엘앤에스' '세아네트웍스' '세아알앤아이' '우진정공' '세아씨엠' '에스에스아이케이'
 '세아스틸인터내셔날' '동아스틸' '에이치피피' '씨티씨' '한국쿨러' '세아홀딩스' '이앤비원' '에이팩인베스터스' '윈트러스트']


매출회사     매입회사   매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액  해외매출액
0    세아홀딩스   세아제강지주  1024       11543         NaN   12749    NaN
1    세아홀딩스     세아제강   645       11543         NaN   12749    NaN
2    세아홀딩스    세아베스틸  3875       11543         NaN   12749    NaN
3    세아홀딩스  세아창원특수강  2485       11543         NaN   12749    NaN
4    세아홀딩스    세아특수강   789       11543         NaN   12749    NaN
..     ...      ...   ...         ...         ...     ...    ...
119   세아씨엠   세아엘앤에스  1226        6212        3798  290484  91669
122   동아스틸     세아제강  9331       11647        8963   88618  12630
123   동아스틸   세아엘앤에스  2316       11647        8963   88618  12630
125    씨티씨   세아에프에스     1          72          29    1797   4112
126    씨티씨     한국쿨러    71          72          29    1797   4112

[110 rows x 7 columns]

In [530]:
dfLong = api.getTransTable('세아홀딩스', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['세아홀딩스' '세아제강지주' '세아제강' '세아베스틸' '세아창원특수강' '세아특수강' '세아에프에스' '세아네트웍스'
 '세아에삽' '세아엔지니어링' '세아엘앤에스' '아이언그레이' '세아메탈' '세아엠앤에스' '우진정공' '세아씨엠' '동아스틸'
 '씨티씨' '브이엔티지']
['세아제강지주' '세아제강' '세아베스틸' '세아창원특수강' '세아특수강' '세아에프에스' '세아에삽' '세아메탈'
 '세아엔지니어링' '세아엠앤에스' '세아엘앤에스' '세아네트웍스' '아이언그레이' '우진정공' '브이엔티지' '세아씨엠'
 '세아항공방산소재' '세아홀딩스' '에스에스아이케이' '세아스틸인터내셔날' '동아스틸' '씨티씨' '한국쿨러' '이앤비원'
 '에이팩인베스터스' '에이치피피' '탈리다쿰' '윈트러스트' '빈티지랩']


매출회사      매입회사   매출액 국내계열사계(매출액) 해외계열사계(매출액)  국내매출액 해외매출액
0    세아홀딩스    세아제강지주   464       11768          20  13019    20
1    세아홀딩스      세아제강  1413       11768          20  13019    20
2    세아홀딩스     세아베스틸  2488       11768          20  13019    20
3    세아홀딩스   세아창원특수강  2007       11768          20  13019    20
4    세아홀딩스     세아특수강  1438       11768          20  13019    20
..     ...       ...   ...         ...         ...    ...   ...
148  브이엔티지      세아씨엠   381       14497         329  14497   329
149  브이엔티지      동아스틸   139       14497         329  14497   329
150  브이엔티지       씨티씨    31       14497         329  14497   329
151  브이엔티지  세아항공방산소재   283       14497         329  14497   329
152  브이엔티지      빈티지랩   153       14497         329  14497   329

[136 rows x 7 columns]

# 한국앤컴퍼니

In [523]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('한국앤컴퍼니', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['한국테크놀로지그룹' '한국타이어앤테크놀로지' '한국아트라스비엑스' '한국카앤라이프' '한국네트웍스' '한국엔지니어링웍스'
 '한국프리시전웍스' '한국동그라미파트너스' '에이치케이모터즈' '작스모터스' '아트라스비엑스모터스포츠' '신양관광개발'
 '아노텐금산']
['한국타이어앤테크놀로지' '한국카앤라이프' '한국네트웍스' '신양월드레저' '한국테크놀로지그룹' '한국아트라스비엑스'
 '한국엔지니어링웍스' '한국프리시전웍스' '한국동그라미파트너스' '아노텐금산' '에이치케이모터즈' '작스모터스' '모델솔루션']


매출회사         매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액    해외매출액
0      한국테크놀로지그룹  한국타이어앤테크놀로지  57504       57553       57553   74874      NaN
1      한국테크놀로지그룹      한국카앤라이프     35       57553       57553   74874      NaN
2      한국테크놀로지그룹       한국네트웍스      8       57553       57553   74874      NaN
3      한국테크놀로지그룹       신양월드레저      6       57553       57553   74874      NaN
4    한국타이어앤테크놀로지    한국테크놀로지그룹   1658        2844     2154809  915864  2277365
5    한국타이어앤테크놀로지    한국아트라스비엑스    938        2844     2154809  915864  2277365
6    한국타이어앤테크놀로지      한국카앤라이프    114        2844     2154809  915864  2277365
7    한국타이어앤테크놀로지    한국엔지니어링웍스     42        2844     2154809  915864  2277365
8    한국타이어앤테크놀로지     한국프리시전웍스     25        2844     2154809  915864  2277365
9    한국타이어앤테크놀로지   한국동그라미파트너스      2        2844     2154809  915864  2277365
10   한국타이어앤테크놀로지        아노텐금산      6        2844     2154809  915864  2277365
11     한국아트라스비엑스  한국타이어앤테크놀로지  21194       21194       56491   68205   589198
12       한국카앤라이프  한국타이어앤테크놀로지      2        1706         NaN    1706      NaN
13       한국카앤라이프     에이치케이모터즈    906        1706         NaN    1706      NaN
14       한국카앤라이프        작스모터스    798        1706         NaN    1706      NaN
15        한국네트웍스    한국테크놀로지그룹     46       25359        3323   32590     5671
16        한국네트웍스  한국타이어앤테크놀로지  22781       25359        3323   32590     5671
17        한국네트웍스    한국아트라스비엑스   1572       25359        3323   32590     5671
18        한국네트웍스    한국엔지니어링웍스    672       25359        3323   32590     5671
19        한국네트웍스     한국프리시전웍스    205       25359        3323   32590     5671
20        한국네트웍스   한국동그라미파트너스      8       25359        3323   32590     5671
21        한국네트웍스        모델솔루션     75       25359        3323   32590     5671
22     한국엔지니어링웍스  한국타이어앤테크놀로지  42701       42706       25643   42962    27033
23     한국엔지니어링웍스    한국아트라스비엑스      5       42706       25643   42962    27033
24      한국프리시전웍스  한국타이어앤테크놀로지  30680       30711       30474   55499    31722
25      한국프리시전웍스    한국아트라스비엑스     31       30711       30474   55499    31722
26    한국동그라미파트너스    한국테크놀로지그룹    108        3916         NaN    3916      NaN
27    한국동그라미파트너스  한국타이어앤테크놀로지   3762        3916         NaN    3916      NaN
28    한국동그라미파트너스    한국엔지니어링웍스     28        3916         NaN    3916      NaN
29    한국동그라미파트너스     한국프리시전웍스     18        3916         NaN    3916      NaN
30      에이치케이모터즈    한국테크놀로지그룹     10         104         NaN     104      NaN
31      에이치케이모터즈  한국타이어앤테크놀로지     92         104         NaN     104      NaN
32      에이치케이모터즈      한국카앤라이프      2         104         NaN     104      NaN
33         작스모터스      한국카앤라이프     24          24         NaN      24      NaN
34  아트라스비엑스모터스포츠  한국타이어앤테크놀로지    500        1571         NaN    1571      NaN
35  아트라스비엑스모터스포츠    한국아트라스비엑스   1071        1571         NaN    1571      NaN
36        신양관광개발    한국테크놀로지그룹   1315        1463         NaN    1463      NaN
37        신양관광개발  한국타이어앤테크놀로지    148        1463         NaN    1463      NaN
38         아노텐금산  한국타이어앤테크놀로지   2495        2495         NaN    3002      NaN

In [524]:
dfLong = api.getTransTable('한국앤컴퍼니', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['한국테크놀로지그룹' '한국타이어앤테크놀로지' '한국아트라스비엑스' '한국카앤라이프' '한국네트웍스' '한국엔지니어링웍스'
 '한국프리시전웍스' '한국동그라미파트너스' '에이치케이모터즈' '아트라스비엑스모터스포츠' '신양관광개발' '아노텐금산'
 '모델솔루션' '에스아이카본']
['한국타이어앤테크놀로지' '한국아트라스비엑스' '한국카앤라이프' '한국네트웍스' '에이치케이모터즈' '신양월드레저'
 '한국테크놀로지그룹' '한국엔지니어링웍스' '한국프리시전웍스' '한국동그라미파트너스' '아트라스비엑스모터스포츠' '모델솔루션'
 '작스모터스' '아노텐금산']


매출회사          매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액    해외매출액
0      한국테크놀로지그룹   한국타이어앤테크놀로지  61758       62980         NaN   62980      NaN
1      한국테크놀로지그룹     한국아트라스비엑스   1058       62980         NaN   62980      NaN
2      한국테크놀로지그룹       한국카앤라이프      7       62980         NaN   62980      NaN
3      한국테크놀로지그룹        한국네트웍스      8       62980         NaN   62980      NaN
4      한국테크놀로지그룹      에이치케이모터즈    143       62980         NaN   62980      NaN
5      한국테크놀로지그룹        신양월드레저      6       62980         NaN   62980      NaN
6    한국타이어앤테크놀로지     한국테크놀로지그룹   1453        2953     2177821  846076  2326981
7    한국타이어앤테크놀로지     한국아트라스비엑스   1014        2953     2177821  846076  2326981
8    한국타이어앤테크놀로지       한국카앤라이프    178        2953     2177821  846076  2326981
9    한국타이어앤테크놀로지     한국엔지니어링웍스     32        2953     2177821  846076  2326981
10   한국타이어앤테크놀로지      한국프리시전웍스     38        2953     2177821  846076  2326981
11   한국타이어앤테크놀로지    한국동그라미파트너스      1        2953     2177821  846076  2326981
12   한국타이어앤테크놀로지      에이치케이모터즈      2        2953     2177821  846076  2326981
13   한국타이어앤테크놀로지  아트라스비엑스모터스포츠    113        2953     2177821  846076  2326981
14   한국타이어앤테크놀로지         모델솔루션     70        2953     2177821  846076  2326981
15     한국아트라스비엑스   한국타이어앤테크놀로지   4752        4752       57330   67247   573603
16       한국카앤라이프      에이치케이모터즈   1074        2379         NaN    3462      NaN
17       한국카앤라이프         작스모터스   1305        2379         NaN    3462      NaN
18        한국네트웍스     한국테크놀로지그룹     63       25636        5507   38309     5653
19        한국네트웍스   한국타이어앤테크놀로지  22609       25636        5507   38309     5653
20        한국네트웍스     한국아트라스비엑스   2324       25636        5507   38309     5653
21        한국네트웍스     한국엔지니어링웍스    475       25636        5507   38309     5653
22        한국네트웍스      한국프리시전웍스    165       25636        5507   38309     5653
23     한국엔지니어링웍스   한국타이어앤테크놀로지  35225       35227       10191   36573    10333
24     한국엔지니어링웍스     한국아트라스비엑스      2       35227       10191   36573    10333
25      한국프리시전웍스   한국타이어앤테크놀로지  27673       27673       28297   83282    28611
26    한국동그라미파트너스     한국테크놀로지그룹     94        4925         NaN    4925      NaN
27    한국동그라미파트너스   한국타이어앤테크놀로지   4768        4925         NaN    4925      NaN
28    한국동그라미파트너스     한국엔지니어링웍스     36        4925         NaN    4925      NaN
29    한국동그라미파트너스      한국프리시전웍스     17        4925         NaN    4925      NaN
30    한국동그라미파트너스         아노텐금산      1        4925         NaN    4925      NaN
31    한국동그라미파트너스         모델솔루션      9        4925         NaN    4925      NaN
32      에이치케이모터즈   한국타이어앤테크놀로지   4684        4684         NaN   13066      NaN
33  아트라스비엑스모터스포츠   한국타이어앤테크놀로지    500        1587         NaN    1587      NaN
34  아트라스비엑스모터스포츠     한국아트라스비엑스   1087        1587         NaN    1587      NaN
35        신양관광개발   한국타이어앤테크놀로지     50          50         NaN      50      NaN
36         아노텐금산   한국타이어앤테크놀로지   2727        2727         NaN    3376      NaN
37         모델솔루션   한국타이어앤테크놀로지    131         131        2734   22068    38496
38        에스아이카본   한국타이어앤테크놀로지     95          95         NaN     489      NaN

In [525]:
dfLong = api.getTransTable('한국앤컴퍼니', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['한국앤컴퍼니' '한국타이어앤테크놀로지' '한국카앤라이프' '한국네트웍스' '한국엔지니어링웍스' '한국프리시전웍스'
 '한국동그라미파트너스' '에이치케이모터즈' '아트라스비엑스모터스포츠' '신양관광개발' '리뉴에너지' '모델솔루션' '아름덴티스트리']
['한국타이어앤테크놀로지' '한국아트라스비엑스' '한국카앤라이프' '한국네트웍스' '한국엔지니어링웍스' '한국프리시전웍스'
 '에이치케이모터즈' '신양월드레저' '한국앤컴퍼니' '한국동그라미파트너스' '아트라스비엑스모터스포츠' '티앤에이' '한오토모빌레'
 '모델솔루션' '리뉴에너지']


매출회사          매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액    해외매출액
0         한국앤컴퍼니   한국타이어앤테크놀로지  39063       40256         NaN   64145      NaN
1         한국앤컴퍼니     한국아트라스비엑스   1058       40256         NaN   64145      NaN
2         한국앤컴퍼니       한국카앤라이프      1       40256         NaN   64145      NaN
3         한국앤컴퍼니        한국네트웍스     45       40256         NaN   64145      NaN
4         한국앤컴퍼니     한국엔지니어링웍스     31       40256         NaN   64145      NaN
5         한국앤컴퍼니      한국프리시전웍스     34       40256         NaN   64145      NaN
6         한국앤컴퍼니      에이치케이모터즈     18       40256         NaN   64145      NaN
7         한국앤컴퍼니        신양월드레저      6       40256         NaN   64145      NaN
8    한국타이어앤테크놀로지        한국앤컴퍼니    588        9150     1952963  798583  2063797
9    한국타이어앤테크놀로지     한국아트라스비엑스   1765        9150     1952963  798583  2063797
10   한국타이어앤테크놀로지       한국카앤라이프    229        9150     1952963  798583  2063797
11   한국타이어앤테크놀로지        한국네트웍스    270        9150     1952963  798583  2063797
12   한국타이어앤테크놀로지     한국엔지니어링웍스     32        9150     1952963  798583  2063797
13   한국타이어앤테크놀로지      한국프리시전웍스   4163        9150     1952963  798583  2063797
14   한국타이어앤테크놀로지    한국동그라미파트너스      1        9150     1952963  798583  2063797
15   한국타이어앤테크놀로지      에이치케이모터즈      9        9150     1952963  798583  2063797
16   한국타이어앤테크놀로지  아트라스비엑스모터스포츠    483        9150     1952963  798583  2063797
17   한국타이어앤테크놀로지          티앤에이   1610        9150     1952963  798583  2063797
18       한국카앤라이프   한국타이어앤테크놀로지     56        1613         NaN    2948      NaN
19       한국카앤라이프      에이치케이모터즈   1377        1613         NaN    2948      NaN
20       한국카앤라이프        한오토모빌레    180        1613         NaN    2948      NaN
21        한국네트웍스        한국앤컴퍼니     24       28155        4276   45977     4282
22        한국네트웍스   한국타이어앤테크놀로지  25549       28155        4276   45977     4282
23        한국네트웍스     한국아트라스비엑스   2064       28155        4276   45977     4282
24        한국네트웍스     한국엔지니어링웍스    224       28155        4276   45977     4282
25        한국네트웍스      한국프리시전웍스    177       28155        4276   45977     4282
26        한국네트웍스         모델솔루션    117       28155        4276   45977     4282
27     한국엔지니어링웍스   한국타이어앤테크놀로지  43796       43800        1399   44092     1933
28     한국엔지니어링웍스     한국아트라스비엑스      4       43800        1399   44092     1933
29      한국프리시전웍스   한국타이어앤테크놀로지  23430       23430       35675   69218    35804
30    한국동그라미파트너스        한국앤컴퍼니     69        4679         NaN    5174      NaN
31    한국동그라미파트너스   한국타이어앤테크놀로지   4552        4679         NaN    5174      NaN
32    한국동그라미파트너스     한국엔지니어링웍스     26        4679         NaN    5174      NaN
33    한국동그라미파트너스      한국프리시전웍스     15        4679         NaN    5174      NaN
34    한국동그라미파트너스         리뉴에너지      1        4679         NaN    5174      NaN
35    한국동그라미파트너스         모델솔루션     16        4679         NaN    5174      NaN
36      에이치케이모터즈   한국타이어앤테크놀로지    246         246         NaN   14941      NaN
37  아트라스비엑스모터스포츠   한국타이어앤테크놀로지   1079        2166         NaN    2617      NaN
38  아트라스비엑스모터스포츠     한국아트라스비엑스   1087        2166         NaN    2617      NaN
39        신양관광개발   한국타이어앤테크놀로지     50          50         NaN      50      NaN
40         리뉴에너지   한국타이어앤테크놀로지   3550        3550         NaN    3637       99
41         모델솔루션   한국타이어앤테크놀로지    243         384        2843   22507    32511
42         모델솔루션     한국아트라스비엑스    124         384        2843   22507    32511
43         모델솔루션     한국엔지니어링웍스     17         384        2843   22507    32511
44       아름덴티스트리      한국프리시전웍스    179         179         NaN    5770    14177

# 중흥건설

In [511]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('중흥건설', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['중흥건설' '중흥토건' '중흥엔지니어링' '세종이엔지']
['자중흥주택' '중흥건설산업' '중흥에스클래스' '순천에코밸리' '중흥산업개발' '중흥개발' '중봉건설' '중봉산업개발'
 '제이원산업개발' '세흥산업개발' '신세종' '그린세종' '다원개발' '새솔건설' '에코세종' '청원개발' '세종중흥건설'
 '청원산업개발' '청원건설산업' '영담']


매출회사     매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)      국내매출액  해외매출액
0      중흥건설    자중흥주택        -8     306459.0          NaN   867601.0    0.0
1      중흥건설   중흥건설산업        -6     306459.0          NaN   867601.0    0.0
2      중흥건설  중흥에스클래스     319.0     306459.0          NaN   867601.0    0.0
3      중흥건설   순천에코밸리       2.0     306459.0          NaN   867601.0    0.0
4      중흥건설   중흥산업개발  113147.0     306459.0          NaN   867601.0    0.0
5      중흥건설     중흥개발   78183.0     306459.0          NaN   867601.0    0.0
6      중흥건설     중봉건설      68.0     306459.0          NaN   867601.0    0.0
7      중흥건설   중봉산업개발      58.0     306459.0          NaN   867601.0    0.0
8      중흥건설  제이원산업개발       -21     306459.0          NaN   867601.0    0.0
9      중흥건설   세흥산업개발   34746.0     306459.0          NaN   867601.0    0.0
10     중흥건설      신세종       -88     306459.0          NaN   867601.0    0.0
11     중흥건설     그린세종   70122.0     306459.0          NaN   867601.0    0.0
12     중흥건설     다원개발       -36     306459.0          NaN   867601.0    0.0
13     중흥건설     새솔건설     127.0     306459.0          NaN   867601.0    0.0
14     중흥건설     에코세종     112.0     306459.0          NaN   867601.0    0.0
15     중흥건설     청원개발     259.0     306459.0          NaN   867601.0    0.0
16     중흥건설   세종중흥건설    3127.0     306459.0          NaN   867601.0    0.0
17     중흥건설   청원산업개발      94.0     306459.0          NaN   867601.0    0.0
18     중흥건설   청원건설산업    3127.0     306459.0          NaN   867601.0    0.0
19     중흥건설       영담    3127.0     306459.0          NaN   867601.0    0.0
20     중흥토건    자중흥주택        -8     775970.0          NaN  1770242.0    0.0
21     중흥토건   중흥건설산업        -6     775970.0          NaN  1770242.0    0.0
22     중흥토건  중흥에스클래스  243638.0     775970.0          NaN  1770242.0    0.0
23     중흥토건   중흥산업개발     206.0     775970.0          NaN  1770242.0    0.0
24     중흥토건     중흥개발      20.0     775970.0          NaN  1770242.0    0.0
25     중흥토건     중봉건설   81223.0     775970.0          NaN  1770242.0    0.0
26     중흥토건   중봉산업개발   79822.0     775970.0          NaN  1770242.0    0.0
27     중흥토건  제이원산업개발       -20     775970.0          NaN  1770242.0    0.0
28     중흥토건   세흥산업개발      30.0     775970.0          NaN  1770242.0    0.0
29     중흥토건      신세종       -88     775970.0          NaN  1770242.0    0.0
30     중흥토건     그린세종      72.0     775970.0          NaN  1770242.0    0.0
31     중흥토건     다원개발      1791     775970.0          NaN  1770242.0    0.0
32     중흥토건     새솔건설   74999.0     775970.0          NaN  1770242.0    0.0
33     중흥토건     에코세종   87393.0     775970.0          NaN  1770242.0    0.0
34     중흥토건     청원개발  163172.0     775970.0          NaN  1770242.0    0.0
35     중흥토건   청원산업개발   43726.0     775970.0          NaN  1770242.0    0.0
36  중흥엔지니어링     중봉건설     842.0        842.0          0.0     4870.0    0.0
37    세종이엔지  중흥에스클래스     702.0        702.0          NaN      708.0    0.0

In [512]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('중흥건설', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['중흥건설' '중흥토건' '중흥엔지니어링' '세종이엔지' '나주관광개발' '중흥종합건설' '헤럴드' '헤럴드에듀' '헤럴드팝']
['자중흥주택' '중흥에스클래스' '순천에코밸리' '중흥산업개발' '중흥개발' '중봉건설' '중봉산업개발' '제이원산업개발'
 '세흥산업개발' '그린세종' '다원개발' '새솔건설' '에코세종' '청원개발' '세종중흥건설' '청원산업개발' '청원건설산업'
 '영담' '중흥건설' '헤럴드' '중흥토건' '헤럴드에듀']


매출회사     매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)      국내매출액  해외매출액
0      중흥건설    자중흥주택       2.0     234896.0          0.0   916229.0    0.0
1      중흥건설  중흥에스클래스     268.0     234896.0          0.0   916229.0    0.0
2      중흥건설   순천에코밸리    9166.0     234896.0          0.0   916229.0    0.0
3      중흥건설   중흥산업개발    6499.0     234896.0          0.0   916229.0    0.0
4      중흥건설     중흥개발   50657.0     234896.0          0.0   916229.0    0.0
5      중흥건설     중봉건설      25.0     234896.0          0.0   916229.0    0.0
6      중흥건설   중봉산업개발     103.0     234896.0          0.0   916229.0    0.0
7      중흥건설  제이원산업개발       -14     234896.0          0.0   916229.0    0.0
8      중흥건설   세흥산업개발   41281.0     234896.0          0.0   916229.0    0.0
9      중흥건설     그린세종   59023.0     234896.0          0.0   916229.0    0.0
10     중흥건설     다원개발       3.0     234896.0          0.0   916229.0    0.0
11     중흥건설     새솔건설      50.0     234896.0          0.0   916229.0    0.0
12     중흥건설     에코세종      88.0     234896.0          0.0   916229.0    0.0
13     중흥건설     청원개발      86.0     234896.0          0.0   916229.0    0.0
14     중흥건설   세종중흥건설   22542.0     234896.0          0.0   916229.0    0.0
15     중흥건설   청원산업개발      27.0     234896.0          0.0   916229.0    0.0
16     중흥건설   청원건설산업   22542.0     234896.0          0.0   916229.0    0.0
17     중흥건설       영담   22542.0     234896.0          0.0   916229.0    0.0
18     중흥토건     중흥건설     113.0          0.0          NaN      721.0    0.0
19     중흥토건    자중흥주택       1.0          0.0          NaN      721.0    0.0
20     중흥토건  중흥에스클래스  238061.0          0.0          NaN      721.0    0.0
21     중흥토건   중흥산업개발     149.0          0.0          NaN      721.0    0.0
22     중흥토건     중흥개발     111.0          0.0          NaN      721.0    0.0
23     중흥토건     중봉건설   77643.0          0.0          NaN      721.0    0.0
24     중흥토건   중봉산업개발   35597.0          0.0          NaN      721.0    0.0
25     중흥토건  제이원산업개발       -13          0.0          NaN      721.0    0.0
26     중흥토건   세흥산업개발       8.0          0.0          NaN      721.0    0.0
27     중흥토건     그린세종      76.0          0.0          NaN      721.0    0.0
28     중흥토건     다원개발   20882.0          0.0          NaN      721.0    0.0
29     중흥토건     새솔건설     190.0          0.0          NaN      721.0    0.0
30     중흥토건     에코세종   13326.0          0.0          NaN      721.0    0.0
31     중흥토건     청원개발   37913.0          0.0          NaN      721.0    0.0
32     중흥토건   세종중흥건설       5.0          0.0          NaN      721.0    0.0
33     중흥토건   청원산업개발      27.0          0.0          NaN      721.0    0.0
34     중흥토건   청원건설산업       5.0          0.0          NaN      721.0    0.0
35     중흥토건       영담       5.0          0.0          NaN      721.0    0.0
36  중흥엔지니어링     중봉건설    1278.0          0.0          NaN      638.0    0.0
37    세종이엔지  중흥에스클래스    2459.0     435085.0          NaN  1473097.0    0.0
38   나주관광개발      헤럴드      39.0          0.0          NaN   453662.0    0.0
39   중흥종합건설  중흥에스클래스     240.0          0.0          NaN        4.0    0.0
40      헤럴드     중흥건설      20.0          0.0          NaN    18306.0    0.0
41      헤럴드     중흥토건       1.0          0.0          NaN    18306.0    0.0
42      헤럴드    헤럴드에듀       2.0          0.0          NaN    18306.0    0.0
43    헤럴드에듀      헤럴드     122.0          0.0          NaN    82380.0    0.0
44     헤럴드팝      헤럴드      23.0          0.0          NaN   109703.0    0.0

In [513]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('중흥건설', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['중흥건설' '중흥토건' '중봉건설' '헤럴드' '헤럴드에듀' '헤럴드아트데이' '헤럴드팝']
['자중흥주택' '중흥건설산업' '중흥토건' '중흥에스클래스' '순천에코밸리' '중흥산업개발' '중흥개발' '중봉건설'
 '중봉산업개발' '세흥산업개발' '다원개발' '새솔건설' '세종이엔지' '세종중흥건설' '청원건설산업' '영담' '중흥건설'
 '중흥엔지니어링' '브레인시티프로젝트금융투자' '헤럴드']


매출회사           매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)      국내매출액  \
0      중흥건설          자중흥주택        -1      76545.0          0.0   530963.0   
1      중흥건설         중흥건설산업        -1      76545.0          0.0   530963.0   
2      중흥건설           중흥토건      -136      76545.0          0.0   530963.0   
3      중흥건설        중흥에스클래스     183.0      76545.0          0.0   530963.0   
4      중흥건설         순천에코밸리    3544.0      76545.0          0.0   530963.0   
5      중흥건설         중흥산업개발       195      76545.0          0.0   530963.0   
6      중흥건설           중흥개발       7.0      76545.0          0.0   530963.0   
7      중흥건설           중봉건설      37.0      76545.0          0.0   530963.0   
8      중흥건설         중봉산업개발      15.0      76545.0          0.0   530963.0   
9      중흥건설         세흥산업개발     19820      76545.0          0.0   530963.0   
10     중흥건설           다원개발      23.0      76545.0          0.0   530963.0   
11     중흥건설           새솔건설       -73      76545.0          0.0   530963.0   
12     중흥건설          세종이엔지      18.0      76545.0          0.0   530963.0   
13     중흥건설         세종중흥건설   17638.0      76545.0          0.0   530963.0   
14     중흥건설         청원건설산업   17638.0      76545.0          0.0   530963.0   
15     중흥건설             영담   17638.0      76545.0          0.0   530963.0   
16     중흥토건           중흥건설     136.0          0.0          NaN   344644.0   
17     중흥토건          자중흥주택        -1          0.0          NaN   344644.0   
18     중흥토건         중흥건설산업        -1          0.0          NaN   344644.0   
19     중흥토건        중흥에스클래스  194514.0          0.0          NaN   344644.0   
20     중흥토건         중흥산업개발      -125          0.0          NaN   344644.0   
21     중흥토건           중흥개발       7.0          0.0          NaN   344644.0   
22     중흥토건           중봉건설  118432.0          0.0          NaN   344644.0   
23     중흥토건         중봉산업개발      14.0          0.0          NaN   344644.0   
24     중흥토건         세흥산업개발        -7          0.0          NaN   344644.0   
25     중흥토건           다원개발   41125.0          0.0          NaN   344644.0   
26     중흥토건           새솔건설     30967          0.0          NaN   344644.0   
27     중흥토건        중흥엔지니어링      29.0          0.0          NaN   344644.0   
28     중흥토건          세종이엔지   64731.0          0.0          NaN   344644.0   
29     중흥토건         세종중흥건설      29.0          0.0          NaN   344644.0   
30     중흥토건         청원건설산업      29.0          0.0          NaN   344644.0   
31     중흥토건             영담      29.0          0.0          NaN   344644.0   
32     중흥토건  브레인시티프로젝트금융투자   23009.0          0.0          NaN   344644.0   
33     중봉건설           중흥토건         2          0.0          NaN       70.0   
34      헤럴드           중흥건설      38.0     472917.0          NaN  1009733.0   
35      헤럴드           중흥토건        20     472917.0          NaN  1009733.0   
36      헤럴드         세흥산업개발        10     472917.0          NaN  1009733.0   
37      헤럴드          세종이엔지      30.0     472917.0          NaN  1009733.0   
38      헤럴드  브레인시티프로젝트금융투자       8.0     472917.0          NaN  1009733.0   
39    헤럴드에듀            헤럴드     300.0          0.0          NaN   349126.0   
40  헤럴드아트데이            헤럴드     403.0          0.0          NaN        0.0   
41     헤럴드팝            헤럴드      62.0          0.0          NaN        0.0   

    해외매출액  
0     0.0  
1     0.0  
2     0.0  
3     0.0  
4     0.0  
5     0.0  
6     0.0  
7     0.0  
8     0.0  
9     0.0  
10    0.0  
11    0.0  
12    0.0  
13    0.0  
14    0.0  
15    0.0  
16    0.0  
17    0.0  
18    0.0  
19    0.0  
20    0.0  
21    0.0  
22    0.0  
23    0.0  
24    0.0  
25    0.0  
26    0.0  
27    0.0  
28    0.0  
29    0.0  
30    0.0  
31    0.0  
32    0.0  
33    0.0  
34    0.0  
35    0.0  
36    0.0  
37    0.0  
38    0.0  
39    0.0  
40    0.0  
41    0.0

# 티케이케미칼

In [506]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('티케이케미칼', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['남선알미늄' '바로코사' '벡셀' '삼라' '신광' '신광하이메탈' '우방' '티케이케미칼' '한일개발' '경남모직' '대한상선'
 '대한해운' '동아건설산업' '삼환기업' '서림하이팩' '에스엠상선' '에스엠상선경인터미널' '에스엠상선광양터미널'
 '에스엠생명과학' '에스엠신용정보' '에스엠중공업' '에스엠케미칼' '에스엠하이플러스' '우방산업' '케이엘씨에스엠'
 '코리코엔터프라이스' '한국선박금융' '한덕철광산업' '에스엠에이엠씨투자대부']
['우방' '티케이케미칼' '경남모직' '동아건설산업' '에스엠상선' '에스엠중공업' '우방산업' '벡셀' '에스엠생명과학'
 '에스엠하이플러스' '남선알미늄' '바로코사' '삼라' '신광' '신광하이메탈' '태길종합건설' '경남기업' '대한상선'
 '대한해운' '삼환기업' '서림하이팩' '에스엠상선경인터미널' '에스엠상선광양터미널' '에스엠신용정보' '에스엠케미칼'
 '에이디엠이십일' '이코사주류' '케이엘씨에스엠' '코리코엔터프라이스' '한국선박금융' '한덕철광산업' '대원건설산업'
 '성우종합건설' '에스엠에이엠씨투자대부' '삼라홀딩스' '케이엘홀딩스' '케이엘홀딩스이호' '한일개발' '코리아엘엔지트레이딩'
 '판교에스디쓰리' '바로코사유통사하' '라도' '온양관광호텔' '한통엔지니어링' '신화디앤디' '삼라희망재단' '삼라자원개발'
 '대림종합건설더블유']


매출회사     매입회사   매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액   해외매출액
0          남선알미늄       우방  5864       16306         NaN  245704  102131
1          남선알미늄   티케이케미칼   547       16306         NaN  245704  102131
2          남선알미늄     경남모직   371       16306         NaN  245704  102131
3          남선알미늄   동아건설산업  5193       16306         NaN  245704  102131
4          남선알미늄    에스엠상선  3507       16306         NaN  245704  102131
..           ...      ...   ...         ...         ...     ...     ...
236  에스엠에이엠씨투자대부  에스엠생명과학   517        4456         NaN    8313     NaN
237  에스엠에이엠씨투자대부  에스엠신용정보    97        4456         NaN    8313     NaN
238  에스엠에이엠씨투자대부   에스엠케미칼     2        4456         NaN    8313     NaN
239  에스엠에이엠씨투자대부  에이디엠이십일    11        4456         NaN    8313     NaN
240  에스엠에이엠씨투자대부     우방산업   447        4456         NaN    8313     NaN

[241 rows x 7 columns]

In [520]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('티케이케미칼', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['남선알미늄' '남선홀딩스' '바로코사' '바로코사유통사하' '벡셀' '삼라' '삼라마이다스' '우방' '울산방송' '이엔에이치'
 '케이엘홀딩스' '티케이케미칼' '한통엔지니어링' '대한상선' '대한해운' '동아건설산업' '삼환기업' '에스엠레저산업'
 '에스엠상선' '에스엠상선광양터미널' '에스엠스틸' '에스엠신용정보' '에스엠인더스트리' '에스엠중공업' '에스엠하이플러스'
 '이코사주류' '케이엘씨에스엠' '코리코엔터프라이스' '플러스매니지먼트' '한국선박금융' '한덕철광산업' '에스엠에이엠씨투자대부']
['삼라' '우방' '티케이케미칼' '동아건설산업' '에스엠상선' '에스엠인더스트리' '에스엠티케미칼' '바로코사유통사하' '벡셀'
 '에스엠하이플러스' '에스엠생명과학' '바로코사' '남선알미늄' '태길종합건설' '경남기업' '대한상선' '대한해운' '삼환기업'
 '에스엠상선경인터미널' '에스엠스틸' '에스엠신용정보' '에스엠중공업' '이코사주류' '케이엘씨에스엠' '코리코엔터프라이스'
 '한덕철광산업' '신광' '서림하이팩' '에스엠케미칼' '삼라희망재단' '에스엠에이엠씨투자대부' '케이엘홀딩스' '삼라홀딩스'
 '케이엘홀딩스이호' '우방산업' '신촌역사' '이엔에이치' '삼라농원' '거래업체명' '코리아엘앤지트레이딩' '플러스매니지먼트'
 '한일개발' '라도' '신광하이메탈' '판교에스디쓰리' '한국선박금융' '기원토건' '서남바이오에너지' '에스엠레저산업'
 '동강시스타' '대림종합건설더블유']


매출회사      매입회사   매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액   해외매출액
0          남선알미늄        삼라   561       15021         NaN  216430  108352
1          남선알미늄        우방  6225       15021         NaN  216430  108352
2          남선알미늄    티케이케미칼    66       15021         NaN  216430  108352
3          남선알미늄    동아건설산업  4946       15021         NaN  216430  108352
4          남선알미늄     에스엠상선  2986       15021         NaN  216430  108352
..           ...       ...   ...         ...         ...     ...     ...
266  에스엠에이엠씨투자대부    동아건설산업    20         758         NaN    4112     NaN
267  에스엠에이엠씨투자대부     에스엠상선    86         758         NaN    4112     NaN
268  에스엠에이엠씨투자대부   에스엠신용정보    92         758         NaN    4112     NaN
269  에스엠에이엠씨투자대부  에스엠하이플러스    16         758         NaN    4112     NaN
270  에스엠에이엠씨투자대부    에스엠케미칼    15         758         NaN    4112     NaN

[221 rows x 7 columns]

In [510]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('티케이케미칼', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['티케이케미칼' '남선알미늄' '남선홀딩스' '라도' '바로코사' '벡셀' '삼라' '삼라농원' '삼라마이다스' '우방'
 '울산방송' '이엔에이치' '케이엘홀딩스' '한통엔지니어링' '대한상선' '대한해운' '동아건설산업' '삼환기업'
 '에스엠글로벌유통' '에스엠레저산업' '에스엠상선' '에스엠스틸' '에스엠신용정보' '에스엠인더스트리' '에스엠중공업'
 '에스엠하이플러스' '에스엠화진' '에스엠화진인터내셔널' '우방산업' '이코사주류' '케이엘씨에스엠' '코리코엔터프라이스'
 '탑스텐동강시스타' '플러스매니지먼트' '한덕철광산업' '에스엠에이엠씨투자대부']
['삼라' '우방' '이엔에이치' '경남기업' '대한해운' '동아건설산업' '에스엠인더스트리' '에스엠중공업' '에스엠하이플러스'
 '티케이케미칼' '삼환기업' '에스엠상선' '벡셀' '에스엠글로벌유통' '코리코엔터프라이스' '남선알미늄' '바로코사' '우방산업'
 '라도' '케이엘홀딩스' '한통엔지니어링' '대한상선' '대한해운엘엔지' '신화디앤디' '에스엠스틸' '에스엠신용정보' '삼라농원'
 'SMVNCO.,LTD' '태길종합건설' '에스엠상선경인터미널' '케이엘씨에스엠' '한국선박금융' '한덕철광산업'
 '에스엠에이엠씨투자대부' '삼라마이다스' '에스엠화진' '이코사주류' '에스엠화진인터내셔널' '탑스텐동강시스타' '판교에스디쓰리'
 '서남바이오에너지' '에스엠레저산업' '플러스매니지먼트' '동강시스타' '화진고신장식재료' '대림종합건설더블유']


매출회사     매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액  해외매출액
0         티케이케미칼       삼라   19.0        3976         NaN  331911  79116
1         티케이케미칼       우방    1.0        3976         NaN  331911  79116
2         티케이케미칼    이엔에이치   3683        3976         NaN  331911  79116
3         티케이케미칼     경남기업      0        3976         NaN  331911  79116
4         티케이케미칼     대한해운   10.0        3976         NaN  331911  79116
..           ...      ...    ...         ...         ...     ...    ...
265  에스엠에이엠씨투자대부       삼라  113.0         297         NaN     931    NaN
266  에스엠에이엠씨투자대부       우방  114.0         297         NaN     931    NaN
267  에스엠에이엠씨투자대부     대한해운   10.0         297         NaN     931    NaN
268  에스엠에이엠씨투자대부    에스엠상선   54.0         297         NaN     931    NaN
269  에스엠에이엠씨투자대부  에스엠신용정보    6.0         297         NaN     931    NaN

[224 rows x 7 columns]

# 케이씨씨

In [501]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('케이씨씨', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['케이씨씨' '케이씨씨건설' '코리아오토글라스' '금강레저' '세우실업' '동주상사' '동주' '대호포장' '티앤케이정보'
 '주령금속']
['케이씨씨건설' '코리아오토글라스' '세우실업' 'KCS' 'KCK' 'KCB' 'KCG' 'KCQ' 'KCT' 'KCI'
 'KCID' 'KCV' 'KCCHanoi' 'KCCPaint' 'Basildon' '케이씨씨' '동주' '대호포장']


매출회사      매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액   해외매출액
0       케이씨씨    케이씨씨건설   26247      141289      127147  2776961  570375
1       케이씨씨  코리아오토글라스  115037      141289      127147  2776961  570375
2       케이씨씨      세우실업       5      141289      127147  2776961  570375
3       케이씨씨       KCS    4812      141289      127147  2776961  570375
4       케이씨씨       KCK   28373      141289      127147  2776961  570375
5       케이씨씨       KCB   20221      141289      127147  2776961  570375
6       케이씨씨       KCG   12908      141289      127147  2776961  570375
7       케이씨씨       KCQ    2137      141289      127147  2776961  570375
8       케이씨씨       KCT   12931      141289      127147  2776961  570375
9       케이씨씨       KCI   18856      141289      127147  2776961  570375
10      케이씨씨      KCID    1191      141289      127147  2776961  570375
11      케이씨씨       KCV   10513      141289      127147  2776961  570375
12      케이씨씨  KCCHanoi    6492      141289      127147  2776961  570375
13      케이씨씨  KCCPaint     145      141289      127147  2776961  570375
14      케이씨씨  Basildon    8568      141289      127147  2776961  570375
15    케이씨씨건설      케이씨씨  110717      113966         NaN  1063373     213
16    케이씨씨건설  코리아오토글라스    3249      113966         NaN  1063373     213
17  코리아오토글라스    케이씨씨건설     979         979         NaN   208366  230343
18      금강레저      케이씨씨     699         733         NaN    10325     NaN
19      금강레저  코리아오토글라스      34         733         NaN    10325     NaN
20      세우실업      케이씨씨    5467        8703         NaN    17200     NaN
21      세우실업  코리아오토글라스    3103        8703         NaN    17200     NaN
22      세우실업        동주     134        8703         NaN    17200     NaN
23      동주상사        동주      37          59         NaN      350     NaN
24      동주상사      대호포장      22          59         NaN      350     NaN
25        동주      케이씨씨   10342       15176         NaN    21660     NaN
26        동주  코리아오토글라스     206       15176         NaN    21660     NaN
27        동주      세우실업    4628       15176         NaN    21660     NaN
28      대호포장        동주      36          36         NaN      317     NaN
29    티앤케이정보      케이씨씨    3449        3786         NaN     3840     NaN
30    티앤케이정보    케이씨씨건설     282        3786         NaN     3840     NaN
31    티앤케이정보  코리아오토글라스      55        3786         NaN     3840     NaN
32      주령금속      케이씨씨    2151        2151         NaN     2172     NaN

In [502]:
dfLong = api.getTransTable('케이씨씨', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['케이씨씨' '케이씨씨건설' '코리아오토글라스' '금강레저' '세우실업' '동주상사' '동주' '대호포장' '티앤케이정보'
 '주령금속' '모멘티브퍼포먼스머티리얼스코리아']
['케이씨씨건설' '코리아오토글라스' '세우실업' 'KCS' 'KCK' 'KCB' 'KCG' 'KCQ' 'KCT' 'KCI'
 'KCID' 'KCV' 'KCCHanoi' 'KCCPaint' 'Basildon' '케이씨씨' '동주'
 'MomentivePerformanceMaterialsAsiaPacificPteLtd'
 'MomentivePerformanceMaterialsGmbH'
 'MomentivePerformanceMaterialsIndiaPrivateLimited'
 'MomentivePerformanceMaterialsNanTongCO.,LTD'
 'MomentivePerformanceMaterialsThailandLtd.'
 'MomentivePerformanceMaterialsHongKongLtd.'
 'MomentivePerformanceMaterialsJapanLLC.' 'MPMSiliconesLLC'
 'MomentivePerformanceMaterialsUSALLC'
 'MomentivePerformanceMaterialsQuartzInc.']


매출회사                                              매입회사  \
0               케이씨씨                                            케이씨씨건설   
1               케이씨씨                                          코리아오토글라스   
2               케이씨씨                                              세우실업   
3               케이씨씨                                               KCS   
4               케이씨씨                                               KCK   
5               케이씨씨                                               KCB   
6               케이씨씨                                               KCG   
7               케이씨씨                                               KCQ   
8               케이씨씨                                               KCT   
9               케이씨씨                                               KCI   
10              케이씨씨                                              KCID   
11              케이씨씨                                               KCV   
12              케이씨씨                                          KCCHanoi   
13              케이씨씨                                          KCCPaint   
14              케이씨씨                                          Basildon   
15            케이씨씨건설                                              케이씨씨   
16          코리아오토글라스                                            케이씨씨건설   
17              금강레저                                              케이씨씨   
18              금강레저                                            케이씨씨건설   
19              금강레저                                          코리아오토글라스   
20              세우실업                                              케이씨씨   
21              세우실업                                          코리아오토글라스   
22              세우실업                                                동주   
23              동주상사                                                동주   
24                동주                                              케이씨씨   
25                동주                                              세우실업   
26              대호포장                                                동주   
27            티앤케이정보                                              케이씨씨   
28            티앤케이정보                                            케이씨씨건설   
29            티앤케이정보                                          코리아오토글라스   
30              주령금속                                              케이씨씨   
31  모멘티브퍼포먼스머티리얼스코리아    MomentivePerformanceMaterialsAsiaPacificPteLtd   
32  모멘티브퍼포먼스머티리얼스코리아                 MomentivePerformanceMaterialsGmbH   
33  모멘티브퍼포먼스머티리얼스코리아  MomentivePerformanceMaterialsIndiaPrivateLimited   
34  모멘티브퍼포먼스머티리얼스코리아       MomentivePerformanceMaterialsNanTongCO.,LTD   
35  모멘티브퍼포먼스머티리얼스코리아         MomentivePerformanceMaterialsThailandLtd.   
36  모멘티브퍼포먼스머티리얼스코리아         MomentivePerformanceMaterialsHongKongLtd.   
37  모멘티브퍼포먼스머티리얼스코리아            MomentivePerformanceMaterialsJapanLLC.   
38  모멘티브퍼포먼스머티리얼스코리아                                   MPMSiliconesLLC   
39  모멘티브퍼포먼스머티리얼스코리아               MomentivePerformanceMaterialsUSALLC   
40  모멘티브퍼포먼스머티리얼스코리아           MomentivePerformanceMaterialsQuartzInc.   

       매출액 국내계열사계(매출액) 해외계열사계(매출액) 해외계열사계(매출액)    국내매출액   해외매출액  
0    19326      134286       99421         NaN  1816508  580096  
1   114949      134286       99421         NaN  1816508  580096  
2       11      134286       99421         NaN  1816508  580096  
3       93      134286       99421         NaN  1816508  580096  
4    14311      134286       99421         NaN  1816508  580096  
5    11648      134286       99421         NaN  1816508  580096  
6     6287      134286       99421         NaN  1816508  580096  
7     8177      134286       99421         NaN  1816508  580096  
8    11317      134286       99421         NaN  1816508  580096  
9    18344      134286       99421         NaN  1816508  580096  
10    1404      134286       99421         NaN  1816508  580096  
11   16122      134286       99421         NaN  1816508  580096  
12    6184      134286       99421         NaN  1

In [503]:
dfLong = api.getTransTable('케이씨씨', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['케이씨씨' '케이씨씨건설' '케이씨씨글라스' '케이씨씨실리콘' '금강레저' '세우실업' '동주상사' '동주'
 '모멘티브퍼포먼스머티리얼스코리아' '세우에스아이']
['케이씨씨글라스' '코리아오토글라스' '케이씨씨실리콘' '세우실업' 'KCS' 'KCK' 'KCB' 'KCG' 'KCQ' 'KCT'
 'KCI' 'KCID' 'KCV' 'KCCHanoi' 'KCVN' 'KCCPaint' 'Basildon'
 'MOMHoldingCompany外' '케이씨씨' '케이씨씨건설' '동주' '동주피앤지'
 'MomentivePerformanceMaterialsAsiaPacificPteLtd'
 'MomentivePerformanceMaterialsGmbH'
 'MomentivePerformanceMaterialsIndiaPrivateLimited'
 'MomentivePerformanceMaterialsNanTongCO.,Ltd'
 'MomentivePerformanceMaterialsThailandLtd.'
 'MomentivePerformanceMaterialsHongKongLtd.'
 'MomentivePerformanceMaterialsJapanLLC.' 'MPMSilicones,LLC'
 'MomentivePerformanceMaterialsUSALLC'
 'MomentivePerformanceMaterialsShanghaiTradingCO.,Ltd']


매출회사                                               매입회사  \
0               케이씨씨                                            케이씨씨글라스   
1               케이씨씨                                           코리아오토글라스   
2               케이씨씨                                            케이씨씨실리콘   
3               케이씨씨                                               세우실업   
4               케이씨씨                                                KCS   
..               ...                                                ...   
59  모멘티브퍼포먼스머티리얼스코리아                                   MPMSilicones,LLC   
60  모멘티브퍼포먼스머티리얼스코리아                MomentivePerformanceMaterialsUSALLC   
61  모멘티브퍼포먼스머티리얼스코리아  MomentivePerformanceMaterialsShanghaiTradingCO...   
62            세우에스아이                                               케이씨씨   
63            세우에스아이                                            케이씨씨실리콘   

      매출액 국내계열사계(매출액) 해외계열사계(매출액) 해외계열사계(매출액)    국내매출액   해외매출액  
0   55773       56106       76989         NaN  1626330  445521  
1     319       56106       76989         NaN  1626330  445521  
2       6       56106       76989         NaN  1626330  445521  
3       8       56106       76989         NaN  1626330  445521  
4     357       56106       76989         NaN  1626330  445521  
..    ...         ...         ...         ...      ...     ...  
59    283        1071         NaN        3727    74264    4504  
60   2482        1071         NaN        3727    74264    4504  
61    104        1071         NaN        3727    74264    4504  
62   1146        1157         NaN         NaN     1937     NaN  
63     11        1157         NaN         NaN     1937     NaN  

[64 rows x 8 columns]

# HDC

In [498]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('HDC', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['에이치디씨' '에이치디씨스포츠' '에이치디씨아이서비스' '에이치디씨아이콘트롤스' '에이치디씨아이파크몰' '에이치디씨영창'
 '에이치디씨현대산업개발' '에이치디씨현대피씨이' '호텔에이치디씨' '에이치디씨자산운용']
['아이파크마리나' '에이치디씨스포츠' '에이치디씨신라면세점' '에이치디씨아이서비스' '에이치디씨아이앤콘스' '에이치디씨아이콘트롤스'
 '에이치디씨아이파크몰' '에이치디씨영창' '에이치디씨현대산업개발' '에이치디씨현대이피' '에이치디씨현대피씨이' '호텔에이치디씨'
 '에이치디씨자산운용' '에이치디씨' '미래비아이' '북항아이브리지' '엠엔큐투자파트너스.' '부동산일일사'
 '에이치디씨민간임대주택제1호위탁관리부동산투자회사' '부산컨테이너터미널' '지개남산도시고속화도로']


매출회사         매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)  국내매출액  해외매출액
0       에이치디씨      아이파크마리나    21        13534            0  49614      0
1       에이치디씨     에이치디씨스포츠    14        13534            0  49614      0
2       에이치디씨   에이치디씨신라면세점   542        13534            0  49614      0
3       에이치디씨   에이치디씨아이서비스  1293        13534            0  49614      0
4       에이치디씨   에이치디씨아이앤콘스   322        13534            0  49614      0
..        ...          ...   ...          ...          ...    ...    ...
76  에이치디씨자산운용   에이치디씨아이서비스    11          287            0   7902      0
77  에이치디씨자산운용   에이치디씨아이앤콘스    11          287            0   7902      0
78  에이치디씨자산운용  에이치디씨아이콘트롤스    12          287            0   7902      0
79  에이치디씨자산운용  에이치디씨현대산업개발    16          287            0   7902      0
80  에이치디씨자산운용    에이치디씨현대이피    11          287            0   7902      0

[81 rows x 7 columns]

In [499]:
dfLong = api.getTransTable('HDC', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['에이치디씨' '미래비아이' '부동산일일사' '에이치디씨리조트' '에이치디씨스포츠' '에이치디씨아이서비스' '에이치디씨아이콘트롤스'
 '에이치디씨아이파크몰' '에이치디씨영창' '에이치디씨현대산업개발' '에이치디씨현대피씨이' '호텔에이치디씨' '엠엔큐투자파트너스'
 '에이치디씨자산운용']
['북항아이브리지' '서울-춘천고속도로' '아이파크마리나' '에이치디씨리조트' '에이치디씨스포츠' '에이치디씨신라면세점'
 '에이치디씨아이서비스' '에이치디씨아이앤콘스' '에이치디씨아이콘트롤스' '에이치디씨아이파크몰' '에이치디씨영창'
 '에이치디씨현대산업개발' '에이치디씨현대이피' '에이치디씨현대피씨이' '통영에코파워' '호텔에이치디씨' '부동산일일사'
 '에이치디씨' '미래비아이' '에이치디씨민간임대주택제1호위탁관리부동산투자회사' '부산컨테이너터미널' '에이치디씨자산운용']


매출회사         매입회사 매출액  국내계열사계(매출액)  해외계열사계(매출액)  국내매출액  해외매출액
0        에이치디씨      북항아이브리지  44        14498            0  50099      0
1        에이치디씨    서울-춘천고속도로  77        14498            0  50099      0
2        에이치디씨      아이파크마리나  64        14498            0  50099      0
3        에이치디씨     에이치디씨리조트  42        14498            0  50099      0
4        에이치디씨     에이치디씨스포츠   4        14498            0  50099      0
..         ...          ...  ..          ...          ...    ...    ...
97   에이치디씨자산운용   에이치디씨아이서비스  11          299            0   8890      0
98   에이치디씨자산운용   에이치디씨아이앤콘스  11          299            0   8890      0
99   에이치디씨자산운용  에이치디씨아이콘트롤스  11          299            0   8890      0
100  에이치디씨자산운용  에이치디씨현대산업개발  15          299            0   8890      0
101  에이치디씨자산운용    에이치디씨현대이피  11          299            0   8890      0

[102 rows x 7 columns]

In [500]:
dfLong = api.getTransTable('HDC', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['에이치디씨' '에이치디씨민간임대주택제1호위탁관리부동산투자회사' '미래비아이' '부동산일일사' '에이치디씨리조트'
 '에이치디씨스포츠' '에이치디씨아이서비스' '에이치디씨아이콘트롤스' '에이치디씨아이파크몰' '에이치디씨영창'
 '에이치디씨현대산업개발' '에이치디씨현대피씨이' '호텔에이치디씨' '에이치디씨자산운용' '엠엔큐투자파트너스']
['부산컨테이너터미널' '북항아이브리지' '서울-춘천고속도로' '아이파크마리나' '에이치디씨리조트' '에이치디씨스포츠'
 '에이치디씨신라면세점' '에이치디씨아이서비스' '에이치디씨아이앤콘스' '에이치디씨아이콘트롤스' '에이치디씨아이파크몰'
 '에이치디씨영창' '에이치디씨현대산업개발' '에이치디씨현대이피' '통영에코파워' '호텔에이치디씨' '부동산일일사' '미래비아이'
 '에이치디씨' '에이치디씨현대피씨이' '에이치디씨자산운용' '에이치디씨민간임대주택제1호위탁관리부동산투자회사'
 '지개남산도시고속화도로']


매출회사         매입회사  매출액  국내계열사계(매출액) 해외계열사계(매출액)  국내매출액 해외매출액
0        에이치디씨    부산컨테이너터미널    5        13194         NaN  52839   NaN
1        에이치디씨      북항아이브리지    5        13194         NaN  52839   NaN
2        에이치디씨    서울-춘천고속도로    5        13194         NaN  52839   NaN
3        에이치디씨      아이파크마리나   39        13194         NaN  52839   NaN
4        에이치디씨     에이치디씨리조트  162        13194         NaN  52839   NaN
..         ...          ...  ...          ...         ...    ...   ...
108  에이치디씨자산운용  에이치디씨아이콘트롤스    8          369           0   8906     0
109  에이치디씨자산운용  에이치디씨현대산업개발   89          369           0   8906     0
110  에이치디씨자산운용    에이치디씨현대이피    8          369           0   8906     0
111  엠엔큐투자파트너스   에이치디씨아이서비스  450          570           0    720     0
112  엠엔큐투자파트너스   에이치디씨아이앤콘스  120          570           0    720     0

[113 rows x 7 columns]

# OCI

In [495]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('OCI', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['디씨알이' '유니드글로벌상사' '유니드' '이테크건설' '쿼츠테크' '군장에너지' '삼광글라스' '에스지개발' '오씨아이스페셜티'
 '오씨아이' '오씨아이파워' '오씨아이정보통신']
['오씨아이' '유니드' '오씨아이페로' '쿼츠테크' 'OCIHongKongLimited' 'OCIInternationalInc.'
 'OCIShanghaiInternationalTradingLtd.' 'UNIDGLOBALPHILS.,Inc.'
 'OCIUNIDEuropeB.V.' '유니드글로벌상사' 'KeystoneInvestmentHongKong' '삼광글라스'
 '군장에너지' '에스엠지에너지' 'eTECE&CShanghaiCo.,Ltd.' 'eTECARABIALTD'
 'eTECMalaysiaSdn.Bhd.' '오씨아이스페셜티' '디씨알이' '이테크건설' '상해개랑가구용품유한공사'
 'OCIMSdn.Bhd.' '유니온' '오씨아이정보통신' '오씨아이에스이' '오씨아이파워' '유니온머티리얼' '비앤오바이오'
 'OCIVietnam' 'PhilkoPeroxide' 'PhilkoChemLogistics'
 'TangshanOCIChemicalCo.,Ltd.' 'OCIChinaCo.,Ltd.' 'MaSteelOCIChemical'
 'ShandongOCI' 'JIANGSUOCICHEMICALCo.,Ltd.' 'OCISolarChinaCo.,Ltd.'
 'ELPIONSiliconSdn.Bhd.']


매출회사                         매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)  \
0       디씨알이                         오씨아이    492.0          492            0   
1   유니드글로벌상사                         오씨아이  19837.0        30583        84742   
2   유니드글로벌상사                          유니드  10354.0        30583        84742   
3   유니드글로벌상사                       오씨아이페로    227.0        30583        84742   
4   유니드글로벌상사                         쿼츠테크    165.0        30583        84742   
..       ...                          ...      ...          ...          ...   
74  오씨아이정보통신  TangshanOCIChemicalCo.,Ltd.     34.0         7765         1153   
75  오씨아이정보통신   JIANGSUOCICHEMICALCo.,Ltd.     34.0         7765         1153   
76  오씨아이정보통신             OCIChinaCo.,Ltd.     13.0         7765         1153   
77  오씨아이정보통신        OCISolarChinaCo.,Ltd.     17.0         7765         1153   
78  오씨아이정보통신        ELPIONSiliconSdn.Bhd.      8.0         7765         1153   

     국내매출액   해외매출액  
0      702       0  
1   186839  462331  
2   186839  462331  
3   186839  462331  
4   186839  462331  
..     ...     ...  
74    9813    1572  
75    9813    1572  
76    9813    1572  
77    9813    1572  
78    9813    1572  

[79 rows x 7 columns]

In [496]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('OCI', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['디씨알이' '유니드글로벌상사' '유니드' '이테크건설' '군장에너지' '삼광글라스' '에스지개발' '오씨아이스페셜티' '오씨아이'
 '비앤오바이오' '오씨아이파워' '행복도시태양광발전소' '오씨아이정보통신' '오씨아이페로' '오씨아이드림']
['오씨아이' '유니드' '오씨아이페로' 'OCIHongKongLimited' 'OCIInternationalInc.'
 'OCIShanghaiInternationalTradingLtd.' 'UNIDGLOBALPHILS.,Inc.'
 'OCIUNIDEuropeB.V.' '유니드글로벌상사' 'JIANGSUOCICHEMICALCo.,Ltd.' '군장에너지'
 '에스엠지에너지' '이테크건설' 'eTECARABIALTD' 'eTECMalaysiaSdn.Bhd.' '쿼츠테크' '디씨알이'
 '삼광글라스' '상해개랑가구용품유한공사' '오씨아이정보통신' '오씨아이에스이' '오씨아이파워' '오씨아이스페셜티' '비앤오바이오'
 '오씨아이드림' 'OCIMSdn.Bhd.' 'MaSteelOCIChemical' 'OCIVietnam'
 'PhilkoPeroxide' 'PhilkoChemLogistics' 'TangshanOCIChemicalCo.,Ltd.'
 'OCIChinaCo.,Ltd.' 'OCISOLARPOWERLLC' 'ShandongOCI'
 'ShandongOCI-Jianyangcarbonblack' 'OCISolarChinaCo.,Ltd.']


매출회사                         매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)  \
0       디씨알이                         오씨아이    129         129           0   
1   유니드글로벌상사                         오씨아이  12398       21913       57377   
2   유니드글로벌상사                          유니드   9195       21913       57377   
3   유니드글로벌상사                       오씨아이페로    320       21913       57377   
4   유니드글로벌상사           OCIHongKongLimited   7985       21913       57377   
..       ...                          ...    ...         ...         ...   
72    오씨아이페로  TangshanOCIChemicalCo.,Ltd.     34           0         961   
73    오씨아이페로             OCIChinaCo.,Ltd.     13           0         961   
74    오씨아이페로        OCISolarChinaCo.,Ltd.     17           0         961   
75    오씨아이드림                         오씨아이    302         302           0   
76    오씨아이드림                       오씨아이파워   0.05         302           0   

     국내매출액   해외매출액  
0      361       0  
1   149352  330591  
2   149352  330591  
3   149352  330591  
4   149352  330591  
..     ...     ...  
72   10961    1095  
73   10961    1095  
74   10961    1095  
75  226945       0  
76  226945       0  

[77 rows x 7 columns]

In [497]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('OCI', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['디씨알이' '유니드' '유니드글로벌상사' '삼광글라스' '에스지씨디벨롭먼트' '에스지씨솔루션' '에스지씨에너지'
 '에스지씨이테크건설' '오씨아이' '오씨아이드림' '오씨아이스페셜티' '오씨아이정보통신' '오씨아이파워']
['오씨아이' 'JIANGSUOCICHEMICALCo.,Ltd.' 'OCIInternationalInc.'
 'UNIDGLOBALPHILS.,Inc.' 'OCIUNIDEuropeB.V.' '유니드' '오씨아이페로'
 'OCIHongKongLimited' 'OCIShanghaiInternationalTradingLtd.'
 'SUNHAINTERNATIONALTRADINGCOMPANY' '디씨알이' '군장에너지' '에스지씨이테크건설'
 '상해개랑가구용품유한공사' '삼광글라스' '에스지씨에너지' '에스지씨솔루션' '에스지씨디벨롭먼트' '에스지씨그린파워'
 'UNIDSichuanNewMaterialsCO.,Ltd.' 'eTECArabiaLimited'
 'eTECMalaysiaSdn.Bhd.' '유니드글로벌상사' '오씨아이정보통신' '오씨아이에스이' '오씨아이파워'
 '오씨아이스페셜티' '비앤오바이오' '오씨아이드림' 'OCIMSdn.Bhd.' 'OCIVietnamCo.,Ltd.'
 'PhilkoPeroxideCorporation' 'PhilkoChemLogistics,Inc.'
 'ShandongOCICo.,Ltd' 'TangshanOCIChemicalCo.,Ltd.' 'OCIChinaCo.,Ltd.'
 'ShandongOCI-JianyangCarbonBlackCo.,Ltd' 'OCISolarChinaCo.,Ltd.'
 'OCISOLARPOWERLLC']


매출회사                        매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)  \
0       디씨알이                        오씨아이     55          55           0   
1        유니드                        오씨아이   1812        1812       15083   
2        유니드  JIANGSUOCICHEMICALCo.,Ltd.    265        1812       15083   
3        유니드        OCIInternationalInc.  13563        1812       15083   
4        유니드       UNIDGLOBALPHILS.,Inc.    762        1812       15083   
..       ...                         ...    ...         ...         ...   
91  오씨아이정보통신            OCIChinaCo.,Ltd.     13        8727         868   
92  오씨아이정보통신       OCISolarChinaCo.,Ltd.     17        8727         868   
93    오씨아이파워                        오씨아이     61          61         552   
94    오씨아이파워       OCISolarChinaCo.,Ltd.     22          61         552   
95    오씨아이파워            OCISOLARPOWERLLC    530          61         552   

     국내매출액   해외매출액  
0      362     NaN  
1   260123  306320  
2   260123  306320  
3   260123  306320  
4   260123  306320  
..     ...     ...  
91   10291     992  
92   10291     992  
93   41835    3025  
94   41835    3025  
95   41835    3025  

[96 rows x 7 columns]

# 코오롱

In [455]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('코오롱', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['코오롱' '코오롱인더스트리' '코오롱플라스틱' '코오롱바스프이노폼' '코오롱머티리얼' '코오롱글로텍' '그린나래'
 '코오롱데크컴퍼지트' '데크씨에스피' '스위트밀' '케이에프엔티' '셀빅개발' '코오롱글로벌' '네이처브리지' '테크비전'
 '덕평랜드' '코오롱하우스비전' '코오롱에코원' '코오롱환경서비스' '코오롱이엔지니어링' '코오롱생명과학' '코오롱베니트'
 '코오롱아우토' '엠오디' '코오롱엘에스아이' '코오롱제약' '코오롱오토모티브']
['코오롱인더스트리' '코오롱플라스틱' '코오롱머티리얼' '코오롱글로텍' '스위트밀' '씨에이텍' '코오롱글로벌' '코오롱하우스비전'
 '코오롱오토플랫폼' '코오롱에코원' '코오롱환경서비스' '코오롱이엔지니어링' '코오롱생명과학' '코오롱베니트' '코오롱엘에스아이'
 '코오롱제약' '코오롱오토케어서비스' '이노베이스' '더블유파트너스' '코오롱데크컴퍼지트' '코오롱인베스트먼트' '거래계열회사수'
 '코오롱' '코오롱바스프이노폼' '그린나래' '케이에프엔티' '네이처브리지' '테크비전' '엠오디' '리베토코리아' '셀빅개발'
 '코오롱아우토' '코오롱오토모티브' '데크씨에스피' '양양풍력발전' '코오롱하이드로제닉스' '피오르드프로세싱코리아']


매출회사      매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액 해외매출액
0         코오롱  코오롱인더스트리  19150       43413         169   58004   169
1         코오롱   코오롱플라스틱   2213       43413         169   58004   169
2         코오롱   코오롱머티리얼   2440       43413         169   58004   169
3         코오롱    코오롱글로텍   3859       43413         169   58004   169
4         코오롱      스위트밀     82       43413         169   58004   169
..        ...       ...    ...         ...         ...     ...   ...
245     코오롱제약      그린나래      1          76         276  104640  2168
246     코오롱제약      스위트밀      1          76         276  104640  2168
247     코오롱제약    코오롱글로벌     73          76         276  104640  2168
248     코오롱제약   거래계열회사수      1          76         276  104640  2168
249  코오롱오토모티브  코오롱오토플랫폼     57          57         NaN  102370   NaN

[250 rows x 7 columns]

In [456]:
dfLong = api.getTransTable('코오롱', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['코오롱' '코오롱인더스트리' '코오롱플라스틱' '코오롱바스프이노폼' '코오롱머티리얼' '코오롱글로텍' '그린나래'
 '코오롱데크컴퍼지트' '스위트밀' '케이에프엔티' '코오롱글로벌' '네이처브리지' '테크비전' '덕평랜드' '코오롱하우스비전'
 '코오롱환경에너지' '코오롱이엔지니어링' '코오롱생명과학' '코오롱베니트' '엠오디' '코오롱엘에스아이' '코오롱제약'
 '코오롱오토케어서비스' '코오롱오토모티브' '코오롱아우토' '리베토코리아']
['코오롱인더스트리' '코오롱플라스틱' '코오롱머티리얼' '코오롱글로텍' '그린나래' '코오롱데크컴퍼지트' '스위트밀' '씨에이텍'
 '코오롱글로벌' '코오롱하우스비전' '코오롱환경에너지' '코오롱이엔지니어링' '코오롱생명과학' '코오롱베니트' '이노베이스'
 '엠오디' '코오롱엘에스아이' '코오롱제약' '코오롱오토케어서비스' '코오롱오토모티브' '코오롱아우토' '더블유파트너스'
 '코오롱오토플랫폼' '코오롱화이버' '코오롱인베스트먼트' '거래계열회사수' '코오롱' '코오롱바스프이노폼' '케이에프엔티'
 '네이처브리지' '테크비전' '셀빅개발' '데크씨에스피' '양양풍력발전' '리베토코리아' '천안북부일반산업단지']


매출회사        매입회사      매출액 국내계열사계(매출액) 해외계열사계(매출액)  국내매출액 해외매출액
0       코오롱    코오롱인더스트리  16654.0       43174         130  60332   130
1       코오롱     코오롱플라스틱   2751.0       43174         130  60332   130
2       코오롱     코오롱머티리얼   2553.0       43174         130  60332   130
3       코오롱      코오롱글로텍   3129.0       43174         130  60332   130
4       코오롱        그린나래      1.0       43174         130  60332   130
..      ...         ...      ...         ...         ...    ...   ...
251  코오롱아우토  코오롱오토케어서비스     77.0         142         NaN  93716   NaN
252  코오롱아우토    코오롱오토플랫폼     32.0         142         NaN  93716   NaN
253  리베토코리아    코오롱인더스트리    546.0         599         NaN   3497   NaN
254  리베토코리아    코오롱하우스비전      9.0         599         NaN   3497   NaN
255  리베토코리아  코오롱오토케어서비스     44.0         599         NaN   3497   NaN

[256 rows x 7 columns]

In [457]:
dfLong = api.getTransTable('코오롱', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['코오롱' '코오롱인더스트리' '코오롱플라스틱' '코오롱바스프이노폼' '코오롱머티리얼' '코오롱글로텍' '그린나래'
 '코오롱데크컴퍼지트' '아토메탈테크코리아' '스위트밀' '케이에프엔티' '퍼플아이오' '코오롱글로벌' '네이처브리지' '테크비전'
 '덕평랜드' '코오롱하우스비전' '코오롱이앤씨' '코오롱오토케어서비스' '코오롱오토모티브' '코오롱아우토' '코오롱생명과학'
 '코오롱베니트' '엠오디' '코오롱엘에스아이' '코오롱제약' '아르텍스튜디오' '리베토코리아']
['코오롱인더스트리' '코오롱플라스틱' '코오롱머티리얼' '코오롱글로텍' '그린나래' '코오롱데크컴퍼지트' '스위트밀' '씨에이텍'
 '코오롱글로벌' '코오롱하우스비전' '코오롱이앤씨' '코오롱오토케어서비스' '코오롱오토모티브' '코오롱아우토' '코오롱생명과학'
 '코오롱베니트' '이노베이스' '엠오디' '코오롱엘에스아이' '코오롱제약' '더블유파트너스' '코오롱환경에너지'
 '코오롱이엔지니어링' '코오롱인베스트먼트' '거래계열회사수' '코오롱' '코오롱바스프이노폼' '케이에프엔티' '퍼플아이오'
 '네이처브리지' '테크비전' '한중문화타운' '코오롱오토플랫폼' '리베토코리아' '아토메탈테크코리아' '천안북부일반산업단지'
 '코오롱바이오텍' '아르텍스튜디오']


매출회사      매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)  국내매출액 해외매출액
0        코오롱  코오롱인더스트리  13339       38339          13  53980    20
1        코오롱   코오롱플라스틱   2335       38339          13  53980    20
2        코오롱   코오롱머티리얼   1451       38339          13  53980    20
3        코오롱    코오롱글로텍   3134       38339          13  53980    20
4        코오롱      그린나래      0       38339          13  53980    20
..       ...       ...    ...         ...         ...    ...   ...
252  아르텍스튜디오      그린나래     95         466         NaN    706   NaN
253  아르텍스튜디오    코오롱글로벌    183         466         NaN    706   NaN
254   리베토코리아  코오롱인더스트리    120         130         NaN   4632   NaN
255   리베토코리아    코오롱글로벌      8         130         NaN   4632   NaN
256   리베토코리아  코오롱하우스비전      2         130         NaN   4632   NaN

[257 rows x 7 columns]

# 금호건설

In [452]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('금호건설', 2019)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['금호산업' '아시아나항공' '금호고속관광[경기]' '금호고속관광[서울]' '금호고속관광[전남]' '금호고속' '금호리조트'
 '금호속리산고속' '금호티앤아이' '아시아나개발' '아시아나세이버' '아시아나아이디티' '아시아나에어포트' '에스티엠' '에어부산'
 '에어서울' '에이에이치' '에이오' '에이큐' '케이아이' '케이알' '케이에이' '케이에프' '케이오' '케이지']
['아시아나항공' '금호고속관광[경기]' '금호고속관광[서울]' '금호고속' '금호리조트' '금호속리산고속' '금호티앤아이'
 '아시아나개발' '아시아나세이버' '아시아나아이디티' '아시아나에어포트' '에스티엠' '에어부산' '에어서울' '에이에이치'
 '에이오' '에이큐' '케이아이' '케이알' '케이에이' '케이에프' '케이오' '케이지' '금호산업' '금호고속관광[전남]'
 'WEIHAIPOINTHOTEL&GOLFRESORT.CO.,LTD' '금호연건중국유한공사']


매출회사        매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  해외매출액
0    금호산업      아시아나항공  27699       37125         NaN  1360477  15758
1    금호산업  금호고속관광[경기]     15       37125         NaN  1360477  15758
2    금호산업  금호고속관광[서울]     19       37125         NaN  1360477  15758
3    금호산업        금호고속   5424       37125         NaN  1360477  15758
4    금호산업       금호리조트    359       37125         NaN  1360477  15758
..    ...         ...    ...         ...         ...      ...    ...
157  케이에프        에어서울     50       14347         NaN    17440    NaN
158   케이오      아시아나항공    172       23091         NaN    23091    NaN
159   케이오    아시아나에어포트  22919       23091         NaN    23091    NaN
160   케이지        금호고속   1083        2048         NaN     7392    NaN
161   케이지       금호리조트    965        2048         NaN     7392    NaN

[162 rows x 7 columns]

In [453]:
dfLong = api.getTransTable('금호건설', 2020)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['금호산업' '아시아나항공' '금호고속관광[경기]' '금호고속관광[서울]' '금호고속관광[전남]' '금호고속' '금호리조트'
 '금호속리산고속' '금호티앤아이' '아시아나개발' '아시아나세이버' '아시아나아이디티' '아시아나에어포트' '에스티엠' '에어부산'
 '에어서울' '에이에이치' '에이오' '에이큐' '케이아이' '케이알' '케이에이' '케이에프' '케이오' '케이지']
['아시아나항공' '금호고속관광[경기]' '금호고속관광[서울]' '금호고속관광[전남]' '금호고속' '금호리조트' '금호속리산고속'
 '금호티앤아이' '아시아나개발' '아시아나세이버' '아시아나아이디티' '아시아나에어포트' '에스티엠' '에어부산' '에어서울'
 '에이에이치' '에이오' '에이큐' '케이아이' '케이알' '케이에이' '케이에프' '케이오' '케이지' '금호산업'
 'AsianaStaffService,Inc.' '케이브이아이' 'KUMHOHOLDINGSH.K.CO,.LTD.'
 'WEIHAIPOINTHOTEL&GOLFRESORT.CO.,LTD']


매출회사        매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액 해외매출액
0    금호산업      아시아나항공  20238       31050         NaN  1587797  9475
1    금호산업  금호고속관광[경기]     12       31050         NaN  1587797  9475
2    금호산업  금호고속관광[서울]     15       31050         NaN  1587797  9475
3    금호산업  금호고속관광[전남]      7       31050         NaN  1587797  9475
4    금호산업        금호고속   1358       31050         NaN  1587797  9475
..    ...         ...    ...         ...         ...      ...   ...
156  케이에프    아시아나에어포트     44       14561         NaN    19292   NaN
157  케이에프        에어서울     54       14561         NaN    19292   NaN
158   케이오    아시아나에어포트  24406       24406         NaN    24406   NaN
159   케이지        금호고속   1132        2124         NaN     6657   NaN
160   케이지       금호리조트    992        2124         NaN     6657   NaN

[161 rows x 7 columns]

In [454]:
dfLong = api.getTransTable('금호건설', 2021)
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())
dfLong

['금호건설' '아시아나항공' '금호고속관광[경기]' '금호고속관광[서울]' '금호고속관광[전남]' '금호고속' '금호속리산고속'
 '금호익스프레스' '금호티앤아이' '아시아나개발' '아시아나세이버' '아시아나아이디티' '아시아나에어포트' '에스티엠' '에어부산'
 '에어서울' '에이에이치' '에이오' '에이큐' '충주보라매' '케이브이아이' '케이아이' '케이알' '케이에이' '케이에프'
 '케이오' '케이지']
['금호건설' '아시아나항공' '금호고속관광[경기]' '금호고속관광[서울]' '금호고속관광[전남]' '금호고속' '금호리조트'
 '금호속리산고속' '금호익스프레스' '금호티앤아이' '아시아나개발' '아시아나세이버' '아시아나아이디티' '아시아나에어포트'
 '에스티엠' '에어부산' '에어서울' '에이에이치' '에이오' '에이큐' '충주보라매' '케이브이아이' '케이아이' '케이알'
 '케이에이' '케이에프' '케이오' '케이지' 'AsianaStaffService,Inc.'
 'AsianaPhilippinesGSA,Inc.' 'KUMHOHOLDINGSH.K.CO,.LTD.'
 'WEIHAIPOINTHOTEL&GOLFRESORT.CO.,LTD']


매출회사                                 매입회사   매출액 국내계열사계(매출액) 해외계열사계(매출액)  \
0    금호건설                                 금호건설     0       10386           0   
1    금호건설                               아시아나항공  7552       10386           0   
2    금호건설                           금호고속관광[경기]    11       10386           0   
3    금호건설                           금호고속관광[서울]     7       10386           0   
4    금호건설                           금호고속관광[전남]     4       10386           0   
..    ...                                  ...   ...         ...         ...   
858   케이지                                  케이지     0        1193           0   
859   케이지              AsianaStaffService,Inc.     0        1193           0   
860   케이지            AsianaPhilippinesGSA,Inc.     0        1193           0   
861   케이지            KUMHOHOLDINGSH.K.CO,.LTD.     0        1193           0   
862   케이지  WEIHAIPOINTHOTEL&GOLFRESORT.CO.,LTD     0        1193           0   

       국내매출액  해외매출액  
0    1815504  13632  
1    1815504  13632  
2    1815504  13632  
3    1815504  13632  
4    1815504  13632  
..       ...    ...  
858     4061      0  
859     4061      0  
860     4061      0  
861     4061      0  
862     4061      0  

[863 rows x 7 columns]

# 하림지주

In [449]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('하림지주', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사                         매입회사  매출액 국내계열사계(매출액) 해외계열사계(매출액)  국내매출액  \
0     하림지주                        그린바이텍    1        2973         NaN  12378   
1     하림지주                      산과들에프앤씨    5        2973         NaN  12378   
2     하림지주                           선진  791        2973         NaN  12378   
3     하림지주                       선진에프에스   51        2973         NaN  12378   
4     하림지주                       애그리로보텍    1        2973         NaN  12378   
..     ...                          ...  ...         ...         ...    ...   
268   한국썸벧  SunjinMyanmarCompanyLimited  175       75089         175  95869   
269  에코캐피탈                           선진    2         875         NaN   7593   
270  에코캐피탈                        싱그린FS  103         875         NaN   7593   
271  에코캐피탈                  농업회사법인에이치비씨   21         875         NaN   7593   
272  에코캐피탈                  농업회사법인순우리한우  749         875         NaN   7593   

    해외매출액  
0     NaN  
1     NaN  
2     NaN  
3     NaN  
4     NaN  
..    ...  
268  1936  
269   NaN  
270   NaN  
271   NaN  
272   NaN  

[273 rows x 7 columns]

In [450]:
print(dfLong['매출회사'].unique())
print(dfLong['매입회사'].unique())

['하림지주' '그린바이텍' '동림건설' '디디에프엔비' '맥시칸' '산과들에프앤씨' '선진' '선진브릿지랩' '선진에프에스'
 '선진팜' '선진햄' '싱그린FS' '애그리로보텍' '엔디' '엔바이콘' '엔에스쇼핑' '엔에스홈쇼핑미디어센터' '올품'
 '참트레이딩' '청운농업회사법인' '팜스코' '포스에스엠' '하림' '하림산업' '하림애니멀클리닉' '하림엠에스' '하림펫푸드'
 '농업회사법인에이치비씨' '농업회사법인봉산지피' '농업회사법인선진한마을' '농업회사법인순우리한우' '농업회사법인주원산오리'
 '농업회사법인팜스코바이오인티' '농업회사법인팜앤코' '농업회사법인포크랜드' '농업회사법인한사랑' '농업회사법인피그앤그린'
 '제일사료' '팬오션' '한강씨엠' '한국썸벧' '에코캐피탈']
['그린바이텍' '산과들에프앤씨' '선진' '선진에프에스' '애그리로보텍' '엔바이콘' '엔에스쇼핑' '참트레이딩' '팜스코'
 '하림' '하림유통' '하림펫푸드' '농업회사법인에이치비씨' '농업회사법인선진한마을' '농업회사법인주원산오리' '제일사료'
 '싱그린FS' '하림지주' '하림엠에스' '선진미트농업회사법인' '선진브릿지랩' '선진팜' '선진햄' '청운농업회사법인'
 '농업회사법인봉산지피' '농업회사법인봉화' '농업회사법인한사랑' '한국썸벧' '팬오션' '포스에스엠' '하림산업' '하림식품'
 '올품' 'SinggreenVinaCOMPANYLIMITED' '엔에스홈쇼핑미디어센터' '농업회사법인순우리한우' '디디에프엔비'
 '한강씨엠' '동림건설' '에코캐피탈' '농업회사법인팜스코바이오인티' '농업회사법인팜앤코' '농업회사법인포크랜드' '에이치에스푸드'
 '농업회사법인피그앤그린' 'PanOceanAmericaInc.'
 'PanOceanSingaporeBulkCarrierPte.Ltd.' '하림애니멀클리닉'
 'SunjinMyanmarCompanyLimited']


In [451]:
dfLong = api.getTransTable('하림지주', 2019)
dfLong

매출회사                         매입회사  매출액 국내계열사계(매출액) 해외계열사계(매출액)  국내매출액  \
0     하림지주                        그린바이텍    1        2973         NaN  12378   
1     하림지주                      산과들에프앤씨    5        2973         NaN  12378   
2     하림지주                           선진  791        2973         NaN  12378   
3     하림지주                       선진에프에스   51        2973         NaN  12378   
4     하림지주                       애그리로보텍    1        2973         NaN  12378   
..     ...                          ...  ...         ...         ...    ...   
268   한국썸벧  SunjinMyanmarCompanyLimited  175       75089         175  95869   
269  에코캐피탈                           선진    2         875         NaN   7593   
270  에코캐피탈                        싱그린FS  103         875         NaN   7593   
271  에코캐피탈                  농업회사법인에이치비씨   21         875         NaN   7593   
272  에코캐피탈                  농업회사법인순우리한우  749         875         NaN   7593   

    해외매출액  
0     NaN  
1     NaN  
2     NaN  
3     NaN  
4     NaN  
..    ...  
268  1936  
269   NaN  
270   NaN  
271   NaN  
272   NaN  

[273 rows x 7 columns]

# 영풍

In [444]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('영풍', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사   매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액   해외매출액
0            영풍   영풍개발      72      139049       20123  671417  710439
1            영풍   영풍문고    5020      139049       20123  671417  710439
2            영풍   고려아연  114446      139049       20123  671417  710439
3            영풍   영풍정밀     550      139049       20123  671417  710439
4            영풍   서린상사   18495      139049       20123  671417  710439
..          ...    ...     ...         ...         ...     ...     ...
56         엑스메텍     영풍    1837        1925         NaN    1925     NaN
57         엑스메텍     켐코      88        1925         NaN    1925     NaN
58          알란텀  코리아니켈       5           5         506     694     751
59  징크옥사이드코퍼레이션   고려아연   85699       85699         NaN   85699     NaN
60           켐코  코리아니켈    3119        3119         NaN    5014   10513

[61 rows x 7 columns]

In [445]:
dfLong = api.getTransTable('영풍', 2020)
dfLong

매출회사                         매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)  \
0            영풍                         영풍개발      73      177867       52248   
1            영풍                         영풍문고    5093      177867       52248   
2            영풍                         고려아연  158989      177867       52248   
3            영풍                         영풍정밀     555      177867       52248   
4            영풍                         서린상사   12662      177867       52248   
5            영풍                       서린정보기술     271      177867       52248   
6            영풍                         유미개발      39      177867       52248   
7            영풍                         엑스메텍     185      177867       52248   
8            영풍  SorinShanghaiCorporationLtd   52248      177867       52248   
9          영풍문고                           영풍      16          16         NaN   
10         영풍전자                           영풍    1684        1862       45594   
11         영풍전자                       코리아써키트     178        1862       45594   
12       코리아써키트                           영풍     161       16499         NaN   
13       코리아써키트                         영풍전자    1319       16499         NaN   
14       코리아써키트                        시그네틱스   14867       16499         NaN   
15       코리아써키트                        인터플렉스     148       16499         NaN   
16       코리아써키트                         테라닉스       4       16499         NaN   
17        인터플렉스                       코리아써키트       6           6      162725   
18         테라닉스                         영풍전자    1181        4947         NaN   
19         테라닉스                       코리아써키트    3758        4947         NaN   
20         테라닉스                         고려아연       8        4947         NaN   
21        와이투북스                         영풍문고      65          65         NaN   
22         고려아연                           영풍    2124       43665      278435   
23         고려아연                         서린상사   40203       43665      278435   
24         고려아연                        코리아니켈     539       43665      278435   
25         고려아연                  징크옥사이드코퍼레이션     177       43665      278435   
26         고려아연                           켐코     622       43665      278435   
27         영풍정밀                           영풍    1560       10103        5499   
28         영풍정밀                         고려아연    8377       10103        5499   
29         영풍정밀                  징크옥사이드코퍼레이션      61       10103        5499   
30         영풍정밀                           켐코     105       10103        5499   
31         서린상사                           영풍    4141       16040        5537   
32         서린상사                         고려아연   11898       16040        5537   
33         서린상사                         영풍정밀       1       16040        5537   
34       서린정보기술                         고려아연    3043        3397         173   
35       서린정보기술                         서린상사     258        3397         173   
36       서린정보기술                        클린코리아       1        3397         173   
37       서린정보기술                        케이지엑스      12        3397         173   
38       서린정보기술                       케이지그린텍      17        3397         173   
39       서린정보기술                        코리아니켈      16        3397         173   
40       서린정보기술                          알란텀      23        3397         173   
41       서린정보기술                  징크옥사이드코퍼레이션      15        3397         173   
42       서린정보기술                           켐코      12        3397         173   
43        클린코리아                         고려아연     613         613         NaN   
44        케이지엑스                           영풍       2       27644         NaN   
45        케이지엑스                         고려아연   26125       27644         NaN   
46        케이지엑스                         영풍정밀       2       27644         NaN   
47        케이지엑스                         서린상사       3       27644         NaN   
48        케이지엑스                        코리아니켈       1       27644         NaN   
49        케

In [446]:
dfLong = api.getTransTable('영풍', 2021)
dfLong

매출회사                               매입회사     매출액  국내계열사계(매출액)  \
0            영풍                               영풍개발      74       211080   
1            영풍                            영풍문고홀딩스    2224       211080   
2            영풍                               영풍문고    1587       211080   
3            영풍                               고려아연  195359       211080   
4            영풍                               영풍정밀     563       211080   
5            영풍                               서린상사   10735       211080   
6            영풍                             서린정보기술     307       211080   
7            영풍                               유미개발      45       211080   
8            영풍                               엑스메텍     186       211080   
9            영풍  SorinCorporationSingaporePte.Ltd.    4062       211080   
10           영풍        SorinShanghaiCorporationLtd  109085       211080   
11      영풍문고홀딩스                                 영풍       7            8   
12      영풍문고홀딩스                               영풍전자       1            8   
13         영풍문고                                 영풍       2            2   
14         영풍전자                                 영풍    1681         1753   
15         영풍전자                              인터플렉스      72         1753   
16       코리아써키트                              시그네틱스   23295        23533   
17       코리아써키트                              인터플렉스     194        23533   
18       코리아써키트                               테라닉스      44        23533   
19        인터플렉스                             코리아써키트      10           10   
20         테라닉스                               영풍전자     956         4998   
21         테라닉스                             코리아써키트    4042         4998   
22         고려아연                                 영풍    6365        31302   
23         고려아연                               서린상사   23682        31302   
24         고려아연                              코리아니켈     615        31302   
25         고려아연                        징크옥사이드코퍼레이션     247        31302   
26         고려아연                                 켐코     393        31302   
27         영풍정밀                                 영풍    2423        15909   
28         영풍정밀                               고려아연   13444        15909   
29         영풍정밀                        징크옥사이드코퍼레이션      38        15909   
30         영풍정밀                                 켐코       4        15909   
31         서린상사                                 영풍    4182        16633   
32         서린상사                               고려아연   12451        16633   
33       서린정보기술                              시그네틱스       3         6186   
34       서린정보기술                               고려아연    4145         6186   
35       서린정보기술                               서린상사    1260         6186   
36       서린정보기술                              클린코리아       1         6186   
37       서린정보기술                              케이지엑스     305         6186   
38       서린정보기술                             케이지그린텍      33         6186   
39       서린정보기술                              코리아니켈      20         6186   
40       서린정보기술                                알란텀      26         6186   
41       서린정보기술                        징크옥사이드코퍼레이션      18         6186   
42       서린정보기술                                 켐코      54         6186   
43       서린정보기술                                케이잼     321         6186   
44        클린코리아                               고려아연     446          446   
45        케이지엑스                                 영풍       2        31329   
46        케이지엑스                               고려아연   31327        31329   
47       케이지그린텍                               고려아연    6593         6593   
48        코리아니켈                                알란텀       1            1   
49         엑스메텍                                 영풍    2008         2045   
50         엑스메텍                             코리아써키트      37         2045   
51          알란텀                              코리아니켈      43           43   
52  징크옥사이드코퍼레이션                     

# 효성

In [427]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('효성', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사       매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액    해외매출액
0            효성      효성티앤씨  15730      101578      593733  1478928  2531116
1            효성      효성중공업  32716      101578      593733  1478928  2531116
2            효성     효성첨단소재  15099      101578      593733  1478928  2531116
3            효성       효성화학  17930      101578      593733  1478928  2531116
4            효성    갤럭시아에스엠    227      101578      593733  1478928  2531116
..          ...        ...    ...         ...         ...      ...      ...
220      효성티앤에스  효성프리미어모터스     44       17808      261690   196528   306570
221      효성티앤에스      효성캐피탈     23       17808      261690   196528   306570
222      효성티앤에스   거래해외계열사수     27       17808      261690   196528   306570
223  프로베스트킹덤제일차         효성     45          47         NaN   202930      147
224  프로베스트킹덤제일차     더클래스효성      2          47         NaN   202930      147

[225 rows x 7 columns]

In [437]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('효성', 2020)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사                       매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액  \
0        효성                      효성티앤씨  27603      146940       13661  294983   
1        효성                      효성중공업  56332      146940       13661  294983   
2        효성                     효성첨단소재  25240      146940       13661  294983   
3        효성                       효성화학  29360      146940       13661  294983   
4        효성                      에프엠케이      5      146940       13661  294983   
..      ...                        ...    ...         ...         ...     ...   
347  효성티앤에스              GSTGlobalGmbh    141       14114      385719  230221   
348  효성티앤에스  GSTSafetyTextilesROS.R.L.    181       14114      385719  230221   
349  효성티앤에스                   거래해외계열사수     32       14114      385719  230221   
350   효성캐피탈                         효성    139         142         NaN  197963   
351   효성캐피탈                     더클래스효성      3         142         NaN  197963   

      해외매출액  
0     47867  
1     47867  
2     47867  
3     47867  
4     47867  
..      ...  
347  458735  
348  458735  
349  458735  
350     387  
351     387  

[352 rows x 7 columns]

In [438]:
dfLong = api.getTransTable('효성', 2021)
dfLong

매출회사              매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액   해외매출액
0        효성             효성티앤씨  25646      130400       10825  295010     481
1        효성             효성중공업  44614      130400       10825  295010     481
2        효성            효성첨단소재  23104      130400       10825  295010     481
3        효성              효성화학  27925      130400       10825  295010     481
4        효성           갤럭시아에스엠    293      130400       10825  295010     481
..      ...               ...    ...         ...         ...     ...     ...
179  효성티앤에스        효성인포메이션시스템     82       13744      427704  191658  521257
180  효성티앤에스             효성토요타     22       13744      427704  191658  521257
181  효성티앤에스            효성투자개발    103       13744      427704  191658  521257
182  효성티앤에스         효성프리미어모터스     29       13744      427704  191658  521257
183  효성티앤에스  거래해외계열사수*효성은주석참조     33       13744      427704  191658  521257

[184 rows x 7 columns]

# NAVER

In [383]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('NAVER', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
Null value found at ('Unnamed: 15_level_0', '금융회사', '스프링캠프').
Null value found at ('Unnamed: 16_level_0', '금융회사', '소계').
Null value found at ('Unnamed: 17_level_0', '국내계열사 계(매출액)', '국내계열사 계(매출액)').
['매출회사', '네이버(주)', '네이버비즈니스플랫폼(주)', '네이버아이앤에스', '라인플러스', '캠프모바일', '웍스모바일', '네이버랩스', '네이버웹툰', '서치솔루션', '네무스텍', '라인게임즈', '라인비즈플러스', '언블락', '하트잇', '오디언소리', '라인스튜디오', '매출회사', '라인업', '라인프렌즈', '라인플레이', '리코', '스노우', '엔아이티서비스', '그린웹서비스', '넥스트플로어', '에피폴라', '인컴즈', '컴파트너스', '스튜디오엔', '우주', '엔테크서비스', '피그', '컴패니에이아이', '매출회사', '플레이리스트', '에버영코리아', '넥스트플로어지하연구소', '엔비전스', '아이커넥트', '라인파이낸셜플러스', '스튜디오포립', '마크티', '드라마앤컴퍼니', '어뮤즈', '언체인', '엔스토어', '소계', '스프링캠프', '소계', '국내계열사 계(매출액)', '매출회사', 'LINE Corp.', 'Naver China', 'Naver France SAS', 'Works Mobile Japan', 'NAVER J.Hub', 'WAV Media Inc.', 'LINE VIETNAM JOINT STOCK COMPANY( LINE VN JSC)', 'NAVER Business Platform Asia Paci

매출회사                  매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  \
0      네이버            네이버비즈니스플랫폼  18089       20289        7620  3388705   
1      네이버              네이버아이앤에스      0       20289        7620  3388705   
2      네이버                 라인플러스    152       20289        7620  3388705   
3      네이버                 캠프모바일      2       20289        7620  3388705   
4      네이버                 웍스모바일    641       20289        7620  3388705   
..     ...                   ...    ...         ...         ...      ...   
214    언블락        LVCCorporation   1113          67        1992       77   
215    언블락  LINETECHPLUSPTE.LTD.    669          67        1992       77   
216    언체인             LINECorp.    485           0        2423      NaN   
217    언체인  LINETECHPLUSPTE.LTD.   1938           0        2423      NaN   
218  그레이해쉬            네이버비즈니스플랫폼      4           4         NaN     1045   

     해외매출액  
0    70309  
1    70309  
2    70309  
3    70309  
4    70309  
..     ...  
214   1992  
215   1992  
216   2423  
217   2423  
218    291  

[219 rows x 7 columns]

In [418]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('NAVER', 2020)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사        매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  해외매출액
0        네이버  네이버비즈니스플랫폼  21391       24006       13556  3856513  43501
1        네이버       네이버웹툰    473       24006       13556  3856513  43501
2        네이버      플레이리스트     25       24006       13556  3856513  43501
3        네이버         어뮤즈      6       24006       13556  3856513  43501
4        네이버         하트잇      2       24006       13556  3856513  43501
..       ...         ...    ...         ...         ...      ...    ...
179       피그       라인게임즈    531         531         NaN     1138    218
180   스튜디오포립       라인게임즈    105         105         NaN      105    NaN
181  네이버파이낸셜         네이버   7759        9110         NaN    83207   3581
182  네이버파이낸셜       네이버웹툰   1351        9110         NaN    83207   3581
183  네이버파이낸셜       스프링캠프      0        9110         NaN    83207   3581

[184 rows x 7 columns]

In [422]:
dfLong = api.getTransTable('NAVER', 2021)
dfLong

매출회사      매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  해외매출액
0         네이버   네이버클라우드  20214       42011       26525  4079865  46764
1         네이버     네이버웹툰   1902       42011       26525  4079865  46764
2         네이버  네이버웹툰컴퍼니    203       42011       26525  4079865  46764
3         네이버       스노우     51       42011       26525  4079865  46764
4         네이버    플레이리스트      9       42011       26525  4079865  46764
..        ...       ...    ...         ...         ...      ...    ...
199   네이버파이낸셜       네이버  65582       76813           0   704391      0
200   네이버파이낸셜     네이버웹툰   4955       76813           0   704391      0
201   네이버파이낸셜  네이버웹툰컴퍼니   6276       76813           0   704391      0
202  엔에프보험서비스   네이버파이낸셜     19          19           0       19      0
203     스프링캠프       네이버   4246        4246           0     9467      0

[204 rows x 7 columns]

# 교보생명보험

In [358]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('교보생명보험', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사          매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)     국내매출액 해외매출액
0      교보생명보험  교보라이프플래닛생명보험      8       16135         NaN  12505561   NaN
1      교보생명보험      교보악사자산운용   1362       16135         NaN  12505561   NaN
2      교보생명보험          교보증권    478       16135         NaN  12505561   NaN
3      교보생명보험       생보부동산신탁     52       16135         NaN  12505561   NaN
4      교보생명보험     케이씨에이손해사정    591       16135         NaN  12505561   NaN
5      교보생명보험         교보리얼코    314       16135         NaN  12505561   NaN
6      교보생명보험          교보문고   6697       16135         NaN  12505561   NaN
7      교보생명보험        교보정보통신     83       16135         NaN  12505561   NaN
8      교보생명보험        교보핫트랙스   3352       16135         NaN  12505561   NaN
9      교보생명보험       제일안전서비스     17       16135         NaN  12505561   NaN
10     교보생명보험      케이씨에이서비스   3181       16135         NaN  12505561   NaN
11   교보악사자산운용        교보생명보험   7656        7656           2     32558    41
12   교보악사자산운용   교보생명자산운용-미국      2        7656           2     32558    41
13       교보증권        교보생명보험   3143        3968         NaN   1246723   NaN
14       교보증권  교보라이프플래닛생명보험     68        3968         NaN   1246723   NaN
15       교보증권      교보악사자산운용    736        3968         NaN   1246723   NaN
16       교보증권       생보부동산신탁      6        3968         NaN   1246723   NaN
17       교보증권     케이씨에이손해사정      9        3968         NaN   1246723   NaN
18       교보증권       교보데이터센터      2        3968         NaN   1246723   NaN
19       교보증권         교보리얼코      3        3968         NaN   1246723   NaN
20       교보증권        교보정보통신      1        3968         NaN   1246723   NaN
21    생보부동산신탁        교보생명보험     55          55         NaN     66948   NaN
22  케이씨에이손해사정        교보생명보험  29375       29387         NaN     31822   NaN
23  케이씨에이손해사정  교보라이프플래닛생명보험     12       29387         NaN     31822   NaN
24    교보데이터센터        교보생명보험   2559        2672         NaN      5130   NaN
25    교보데이터센터  교보라이프플래닛생명보험    113        2672         NaN      5130   NaN
26      교보리얼코        교보생명보험  45518       50883         NaN    123118   NaN
27      교보리얼코          교보증권   2071       50883         NaN    123118   NaN
28      교보리얼코       교보데이터센터    895       50883         NaN    123118   NaN
29      교보리얼코          교보문고   2065       50883         NaN    123118   NaN
30      교보리얼코        교보정보통신     40       50883         NaN    123118   NaN
31      교보리얼코        교보핫트랙스    294       50883         NaN    123118   NaN
32     교보정보통신        교보생명보험  15681       24081         NaN     51991   NaN
33     교보정보통신  교보라이프플래닛생명보험   2073       24081         NaN     51991   NaN
34     교보정보통신      교보악사자산운용    232       24081         NaN     51991   NaN
35     교보정보통신          교보증권   1884       24081         NaN     51991   NaN
36     교보정보통신     케이씨에이손해사정    842       24081         NaN     51991   NaN
37     교보정보통신       교보데이터센터    185       24081         NaN     51991   NaN
38     교보정보통신         교보리얼코   1457       24081         NaN     51991   NaN
39     교보정보통신          교보문고   1187       24081         NaN     51991   NaN
40     교보정보통신        교보핫트랙스    271       24081         NaN     51991   NaN
41     교보정보통신      케이씨에이서비스    267       24081         NaN     51991   NaN
42     교보핫트랙스        교보생명보험   1944        8503         NaN    100886  4478
43     교보핫트랙스      교보악사자산운용      1        8503         NaN    100886  4478
44     교보핫트랙스          교보증권      7        8503         NaN    100886  4478
45     교보핫트랙스          교보문고   6550        8503         NaN    100886  4478
46    제일안전서비스         교보리얼코  17274       17274         NaN     17352   NaN
47       교보문고        교보생명보험   1728        3937         NaN    568448   NaN
48       교보문고  교보라이프플래닛생명보험     38        3937         NaN    568448   NaN
49       교보문고          교보증권      6        3937         NaN    568448   NaN
50       교보문고        교보핫트랙스   2166        3937         NaN    568448   NaN
51   케이씨에이서비스        교보생명보험  30569       30569         NaN     30569   NaN

In [359]:
dfLong = api.getTransTable('교보생명보험', 2020)
dfLong

매출회사          매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)     국내매출액 해외매출액
0      교보생명보험          교보증권    482       16351         NaN  12977036   NaN
1      교보생명보험      교보악사자산운용   1479       16351         NaN  12977036   NaN
2      교보생명보험  교보라이프플래닛생명보험     28       16351         NaN  12977036   NaN
3      교보생명보험     케이씨에이손해사정    575       16351         NaN  12977036   NaN
4      교보생명보험        교보자산신탁     90       16351         NaN  12977036   NaN
5      교보생명보험          교보문고   6689       16351         NaN  12977036   NaN
6      교보생명보험        교보핫트랙스   3397       16351         NaN  12977036   NaN
7      교보생명보험         교보리얼코    301       16351         NaN  12977036   NaN
8      교보생명보험     제일종합관리서비스     18       16351         NaN  12977036   NaN
9      교보생명보험        교보정보통신     80       16351         NaN  12977036   NaN
10     교보생명보험      케이씨에이서비스   3212       16351         NaN  12977036   NaN
11       교보증권        교보생명보험   3369        4058         NaN   1445369   NaN
12       교보증권      교보악사자산운용    659        4058         NaN   1445369   NaN
13       교보증권  교보라이프플래닛생명보험     15        4058         NaN   1445369   NaN
14       교보증권     케이씨에이손해사정     12        4058         NaN   1445369   NaN
15       교보증권         교보리얼코      1        4058         NaN   1445369   NaN
16       교보증권        교보정보통신      2        4058         NaN   1445369   NaN
17   교보악사자산운용        교보생명보험   7957        7957           2     34720    76
18   교보악사자산운용    교보생명자산운용미국      2        7957           2     34720    76
19  케이씨에이손해사정        교보생명보험  31213       31229         NaN     34651   NaN
20  케이씨에이손해사정  교보라이프플래닛생명보험     16       31229         NaN     34651   NaN
21     교보자산신탁        교보생명보험     72          72         NaN     71273   NaN
22       교보문고        교보생명보험   2120        8171         NaN    609989   NaN
23       교보문고          교보증권     20        8171         NaN    609989   NaN
24       교보문고  교보라이프플래닛생명보험     19        8171         NaN    609989   NaN
25       교보문고        교보핫트랙스   6012        8171         NaN    609989   NaN
26     교보핫트랙스        교보생명보험   1086        8241         NaN     96884   199
27     교보핫트랙스      교보악사자산운용      1        8241         NaN     96884   199
28     교보핫트랙스          교보문고   7154        8241         NaN     96884   199
29      교보리얼코        교보생명보험  44045       51601         NaN    177307   NaN
30      교보리얼코          교보증권   2217       51601         NaN    177307   NaN
31      교보리얼코      교보악사자산운용    725       51601         NaN    177307   NaN
32      교보리얼코          교보문고   3560       51601         NaN    177307   NaN
33      교보리얼코        교보정보통신     87       51601         NaN    177307   NaN
34      교보리얼코       교보데이터센터    967       51601         NaN    177307   NaN
35  제일종합관리서비스         교보리얼코  18603       18603         NaN     18677   NaN
36     교보정보통신        교보생명보험  22206       32776         NaN     49182   NaN
37     교보정보통신          교보증권   1954       32776         NaN     49182   NaN
38     교보정보통신      교보악사자산운용    221       32776         NaN     49182   NaN
39     교보정보통신  교보라이프플래닛생명보험   1915       32776         NaN     49182   NaN
40     교보정보통신     케이씨에이손해사정    799       32776         NaN     49182   NaN
41     교보정보통신          교보문고   1278       32776         NaN     49182   NaN
42     교보정보통신        교보핫트랙스    243       32776         NaN     49182   NaN
43     교보정보통신         교보리얼코   2862       32776         NaN     49182   NaN
44     교보정보통신     제일종합관리서비스     33       32776         NaN     49182   NaN
45     교보정보통신      케이씨에이서비스    288       32776         NaN     49182   NaN
46     교보정보통신       교보데이터센터    977       32776         NaN     49182   NaN
47   케이씨에이서비스        교보생명보험  31562       31562         NaN     31562   NaN
48    교보데이터센터        교보생명보험   2335        2461         NaN      4808   NaN
49    교보데이터센터  교보라이프플래닛생명보험    126        2461         NaN      4808   NaN

In [361]:
dfLong = api.getTransTable('교보생명보험', 2021)
dfLong

매출회사          매입회사    매출액  국내계열사계(매출액) 해외계열사계(매출액)     국내매출액 해외매출액
0         교보생명보험          교보증권    538        16403         NaN  15708927   NaN
1         교보생명보험      교보악사자산운용   1552        16403         NaN  15708927   NaN
2         교보생명보험  교보라이프플래닛생명보험     27        16403         NaN  15708927   NaN
3         교보생명보험        교보자산신탁     91        16403         NaN  15708927   NaN
4         교보생명보험     케이씨에이손해사정    731        16403         NaN  15708927   NaN
5         교보생명보험          교보문고   6492        16403         NaN  15708927   NaN
6         교보생명보험        교보핫트랙스   3337        16403         NaN  15708927   NaN
7         교보생명보험         교보리얼코    314        16403         NaN  15708927   NaN
8         교보생명보험     제일종합관리서비스     20        16403         NaN  15708927   NaN
9         교보생명보험        교보정보통신     71        16403         NaN  15708927   NaN
10        교보생명보험      케이씨에이서비스   3229        16403         NaN  15708927   NaN
11          교보증권        교보생명보험   4508         5514         NaN   1758939   NaN
12          교보증권      교보악사자산운용    966         5514         NaN   1758939   NaN
13          교보증권        교보자산신탁     16         5514         NaN   1758939   NaN
14          교보증권     케이씨에이손해사정     20         5514         NaN   1758939   NaN
15          교보증권        교보정보통신      4         5514         NaN   1758939   NaN
16      교보악사자산운용        교보생명보험   7960         7960           2     37588   142
17      교보악사자산운용    교보생명자산운용미국      2         7960           2     37588   142
18  교보라이프플래닛생명보험        교보생명보험    218          218         NaN    196261   NaN
19        교보자산신탁        교보생명보험    142          142         NaN     58628   NaN
20     케이씨에이손해사정        교보생명보험  33423        33460         NaN     36978   NaN
21     케이씨에이손해사정  교보라이프플래닛생명보험     37        33460         NaN     36978   NaN
22          교보문고        교보생명보험   2966         9557         NaN    694172   NaN
23          교보문고          교보증권     40         9557         NaN    694172   NaN
24          교보문고  교보라이프플래닛생명보험     18         9557         NaN    694172   NaN
25          교보문고     케이씨에이손해사정      8         9557         NaN    694172   NaN
26          교보문고        교보핫트랙스   6242         9557         NaN    694172   NaN
27          교보문고        교보정보통신    283         9557         NaN    694172   NaN
28        교보핫트랙스        교보생명보험   4368        13346         NaN     99893    11
29        교보핫트랙스          교보증권     77        13346         NaN     99893    11
30        교보핫트랙스      교보악사자산운용      2        13346         NaN     99893    11
31        교보핫트랙스  교보라이프플래닛생명보험      1        13346         NaN     99893    11
32        교보핫트랙스        교보자산신탁     34        13346         NaN     99893    11
33        교보핫트랙스     케이씨에이손해사정     11        13346         NaN     99893    11
34        교보핫트랙스          교보문고   8846        13346         NaN     99893    11
35        교보핫트랙스         교보리얼코      1        13346         NaN     99893    11
36        교보핫트랙스     제일종합관리서비스      5        13346         NaN     99893    11
37        교보핫트랙스        교보정보통신      2        13346         NaN     99893    11
38         교보리얼코        교보생명보험  43384        48147         NaN    118769   NaN
39         교보리얼코          교보증권   2349        48147         NaN    118769   NaN
40         교보리얼코  교보라이프플래닛생명보험     22        48147         NaN    118769   NaN
41         교보리얼코          교보문고   1319        48147         NaN    118769   NaN
42         교보리얼코        교보정보통신    482        48147         NaN    118769   NaN
43         교보리얼코       교보데이터센터    591        48147         NaN    118769   NaN
44     제일종합관리서비스         교보리얼코  16918        16918         NaN     17196   NaN
45        교보정보통신        교보생명보험  32840        43718         NaN     54272   NaN
46        교보정보통신          교보증권   1720        43718         NaN     54272   NaN
47        교보정보통신      교보악사자산운용    206        43718         NaN     54272   NaN
48        교보정보통신  교보라이프플래닛생명보험   4812        43718         NaN     54272   NaN
49        교보정보통신     케이씨에이손해사정    882        43718         NaN

# 한국금융지주

In [353]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('한국금융지주', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사          매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)  국내매출액  해외매출액
0   한국투자금융지주        한국투자증권  1461      16631.0       6803.0  20090   6803
1   한국투자금융지주      한국투자신탁운용   448      16631.0       6803.0  20090   6803
2   한국투자금융지주       한국투자캐피탈  2849      16631.0       6803.0  20090   6803
3   한국투자금융지주      한국투자저축은행  3560      16631.0       6803.0  20090   6803
4   한국투자금융지주      한국투자파트너스  8258      16631.0       6803.0  20090   6803
..       ...           ...   ...          ...          ...    ...    ...
64      드림라인  드림라인쏠라파워1호.1     9        763.0          NaN   4984    365
65      드림라인         드림마크원   587        763.0          NaN   4984    365
66      드림라인      한국투자저축은행   164        763.0          NaN   4984    365
67     드림마크원          드림라인  4000       4019.0          NaN      7  20905
68     드림마크원  드림라인쏠라파워1호.1    19       4019.0          NaN      7  20905

[69 rows x 7 columns]

In [354]:
dfLong = api.getTransTable('한국금융지주', 2020)
dfLong

매출회사        매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)  국내매출액  해외매출액
0   한국투자금융지주      한국투자증권  2246      14000.0       3517.0  17137   3517
1   한국투자금융지주    한국투자신탁운용   312      14000.0       3517.0  17137   3517
2   한국투자금융지주     한국투자캐피탈  2554      14000.0       3517.0  17137   3517
3   한국투자금융지주    한국투자저축은행  1320      14000.0       3517.0  17137   3517
4   한국투자금융지주    한국투자파트너스  7566      14000.0       3517.0  17137   3517
..       ...         ...   ...          ...          ...    ...    ...
70      드림라인  드림라인쏠라파워2호    42       2202.0          NaN   7055      5
71      드림라인    한국투자저축은행   184       2202.0          NaN   7055      5
72     드림마크원        드림라인   745        750.0          NaN     11  23341
73     드림마크원  드림라인쏠라파워2호     5        750.0          NaN     11  23341
74      이메디원        도시환경   981        981.0          NaN     51  12020

[75 rows x 7 columns]

In [355]:
dfLong = api.getTransTable('셀트리온홀딩스', 2021)
dfLong

매출회사        매입회사      매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액   해외매출액
0         셀트리온    셀트리온스킨큐어      201     1502555       42780  1392107  297680
1         셀트리온    셀트리온헬스케어  1478991     1502555       42780  1392107  297680
2         셀트리온      셀트리온제약    23363     1502555       42780  1392107  297680
3     셀트리온스킨큐어  셀트리온엔터테인먼트      124         552         607    53563    5005
4     셀트리온스킨큐어    셀트리온헬스케어       76         552         607    53563    5005
5     셀트리온스킨큐어        셀트리온      272         552         607    53563    5005
6     셀트리온스킨큐어      셀트리온제약       66         552         607    53563    5005
7     셀트리온스킨큐어     셀트리온홀딩스       12         552         607    53563    5005
8     셀트리온스킨큐어      티에스이엔씨        2         552         607    53563    5005
9       셀트리온제약        셀트리온    61604       61604         NaN   231982    1586
10  셀트리온엔터테인먼트        셀트리온      163         163         NaN    21804     NaN
11      티에스이엔씨        셀트리온     9029        9222         NaN    10137     NaN
12      티에스이엔씨      셀트리온제약      193        9222         NaN    10137     NaN

# 셀트리온홀딩스

In [350]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('셀트리온홀딩스', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사           매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액   해외매출액
0    셀트리온홀딩스     셀트리온엔터테인먼트     120         151         NaN     179     NaN
1    셀트리온홀딩스       셀트리온헬스케어      31         151         NaN     179     NaN
2       셀트리온       셀트리온스킨큐어     294      771739       15103  728307  133577
3       셀트리온       셀트리온헬스케어  771445      771739       15103  728307  133577
4   셀트리온헬스케어           셀트리온   21852       24200      626581   24200  913103
5   셀트리온헬스케어         셀트리온제약    2348       24200      626581   24200  913103
6   셀트리온스킨큐어     셀트리온엔터테인먼트      71         653         102   32135    6556
7   셀트리온스킨큐어       셀트리온헬스케어      70         653         102   32135    6556
8   셀트리온스킨큐어           셀트리온     330         653         102   32135    6556
9   셀트리온스킨큐어         셀트리온제약     180         653         102   32135    6556
10  셀트리온스킨큐어      셀트리온화학연구소       0         653         102   32135    6556
11  셀트리온스킨큐어   셀트리온충북대코스메틱스       1         653         102   32135    6556
12  셀트리온스킨큐어  셀트리온충북대바이오메딕스       1         653         102   32135    6556
13    셀트리온제약           셀트리온   22613       22613         NaN  138504     967
14    티에스이엔씨         셀트리온제약     184        2655         NaN    9659     NaN
15    티에스이엔씨         티에스이엔엠    2471        2655         NaN    9659     NaN
16    티에스이엔엠           셀트리온    7821        7821         NaN    7821     NaN

In [351]:
dfLong = api.getTransTable('셀트리온홀딩스', 2020)
dfLong

매출회사        매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액   해외매출액
0    셀트리온홀딩스        셀트리온     104         104         NaN     104     NaN
1       셀트리온    셀트리온스킨큐어     231      871775       21800  872097  109773
2       셀트리온    셀트리온헬스케어  862574      871775       21800  872097  109773
3       셀트리온      셀트리온제약    8970      871775       21800  872097  109773
4   셀트리온스킨큐어  셀트리온엔터테인먼트     140         749         163   35476    4475
5   셀트리온스킨큐어    셀트리온헬스케어      85         749         163   35476    4475
6   셀트리온스킨큐어        셀트리온     419         749         163   35476    4475
7   셀트리온스킨큐어      셀트리온제약     100         749         163   35476    4475
8   셀트리온스킨큐어     셀트리온홀딩스       5         749         163   35476    4475
9     셀트리온제약        셀트리온   21891       21891         NaN  171469    1214
10    티에스이엔씨        셀트리온    3033        3199         NaN    3779     NaN
11    티에스이엔씨      셀트리온제약     166        3199         NaN    3779     NaN

In [352]:
dfLong = api.getTransTable('셀트리온홀딩스', 2021)
dfLong

매출회사        매입회사      매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액   해외매출액
0         셀트리온    셀트리온스킨큐어      201     1502555       42780  1392107  297680
1         셀트리온    셀트리온헬스케어  1478991     1502555       42780  1392107  297680
2         셀트리온      셀트리온제약    23363     1502555       42780  1392107  297680
3     셀트리온스킨큐어  셀트리온엔터테인먼트      124         552         607    53563    5005
4     셀트리온스킨큐어    셀트리온헬스케어       76         552         607    53563    5005
5     셀트리온스킨큐어        셀트리온      272         552         607    53563    5005
6     셀트리온스킨큐어      셀트리온제약       66         552         607    53563    5005
7     셀트리온스킨큐어     셀트리온홀딩스       12         552         607    53563    5005
8     셀트리온스킨큐어      티에스이엔씨        2         552         607    53563    5005
9       셀트리온제약        셀트리온    61604       61604         NaN   231982    1586
10  셀트리온엔터테인먼트        셀트리온      163         163         NaN    21804     NaN
11      티에스이엔씨        셀트리온     9029        9222         NaN    10137     NaN
12      티에스이엔씨      셀트리온제약      193        9222         NaN    10137     NaN

# 금호건설

In [347]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('금호건설', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사        매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  해외매출액
0    금호산업      아시아나항공  27699       37125         NaN  1360477  15758
1    금호산업  금호고속관광[경기]     15       37125         NaN  1360477  15758
2    금호산업  금호고속관광[서울]     19       37125         NaN  1360477  15758
3    금호산업        금호고속   5424       37125         NaN  1360477  15758
4    금호산업       금호리조트    359       37125         NaN  1360477  15758
..    ...         ...    ...         ...         ...      ...    ...
157  케이에프        에어서울     50       14347         NaN    17440    NaN
158   케이오      아시아나항공    172       23091         NaN    23091    NaN
159   케이오    아시아나에어포트  22919       23091         NaN    23091    NaN
160   케이지        금호고속   1083        2048         NaN     7392    NaN
161   케이지       금호리조트    965        2048         NaN     7392    NaN

[162 rows x 7 columns]

In [348]:
dfLong = api.getTransTable('금호건설', 2020)
dfLong

매출회사        매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액 해외매출액
0    금호산업      아시아나항공  20238       31050         NaN  1587797  9475
1    금호산업  금호고속관광[경기]     12       31050         NaN  1587797  9475
2    금호산업  금호고속관광[서울]     15       31050         NaN  1587797  9475
3    금호산업  금호고속관광[전남]      7       31050         NaN  1587797  9475
4    금호산업        금호고속   1358       31050         NaN  1587797  9475
..    ...         ...    ...         ...         ...      ...   ...
156  케이에프    아시아나에어포트     44       14561         NaN    19292   NaN
157  케이에프        에어서울     54       14561         NaN    19292   NaN
158   케이오    아시아나에어포트  24406       24406         NaN    24406   NaN
159   케이지        금호고속   1132        2124         NaN     6657   NaN
160   케이지       금호리조트    992        2124         NaN     6657   NaN

[161 rows x 7 columns]

In [349]:
dfLong = api.getTransTable('금호건설', 2021)
dfLong

매출회사                                 매입회사   매출액 국내계열사계(매출액) 해외계열사계(매출액)  \
0    금호건설                                 금호건설     0       10386           0   
1    금호건설                               아시아나항공  7552       10386           0   
2    금호건설                           금호고속관광[경기]    11       10386           0   
3    금호건설                           금호고속관광[서울]     7       10386           0   
4    금호건설                           금호고속관광[전남]     4       10386           0   
..    ...                                  ...   ...         ...         ...   
858   케이지                                  케이지     0        1193           0   
859   케이지              AsianaStaffService,Inc.     0        1193           0   
860   케이지            AsianaPhilippinesGSA,Inc.     0        1193           0   
861   케이지            KUMHOHOLDINGSH.K.CO,.LTD.     0        1193           0   
862   케이지  WEIHAIPOINTHOTEL&GOLFRESORT.CO.,LTD     0        1193           0   

       국내매출액  해외매출액  
0    1815504  13632  
1    1815504  13632  
2    1815504  13632  
3    1815504  13632  
4    1815504  13632  
..       ...    ...  
858     4061      0  
859     4061      0  
860     4061      0  
861     4061      0  
862     4061      0  

[863 rows x 7 columns]

# 현대백화점

In [344]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('현대백화점', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사       매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액  해외매출액
0      현대백화점       한무쇼핑  95935.0       137411          NaN  1376064      0
1      현대백화점   현대백화점면세점    566.0       137411          NaN  1376064      0
2      현대백화점     현대그린푸드  19039.0       137411          NaN  1376064      0
3      현대백화점   현대캐터링시스템     36.0       137411          NaN  1376064      0
4      현대백화점  씨엔에스푸드시스템     12.0       137411          NaN  1376064      0
..       ...        ...      ...          ...          ...      ...    ...
195  현대아이티앤이      현대리바트    222.0        12987          NaN    13157      0
196  현대아이티앤이     현대렌탈케어    187.0        12987          NaN    13157      0
197  현대아이티앤이       에버다임      0.6        12987          NaN    13157      0
198  현대아이티앤이     현대드림투어    128.0        12987          NaN    13157      0
199  현대아이티앤이      한섬글로벌      2.0        12987          NaN    13157      0

[200 rows x 7 columns]

In [345]:
dfLong = api.getTransTable('현대백화점', 2020)
dfLong

매출회사      매입회사    매출액  국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  해외매출액
0      현대백화점      한무쇼핑  79719       193506         NaN  1385244      0
1      현대백화점      현대쇼핑    0.3       193506         NaN  1385244      0
2      현대백화점  현대백화점면세점   2112       193506         NaN  1385244      0
3      현대백화점    현대그린푸드  41276       193506         NaN  1385244      0
4      현대백화점  현대캐터링시스템     54       193506         NaN  1385244      0
..       ...       ...    ...          ...         ...      ...    ...
190  현대아이티앤이    현대렌탈케어    485        29431         NaN    31250      0
191  현대아이티앤이      에버다임      1        29431         NaN    31250      0
192  현대아이티앤이    현대드림투어    258        29431         NaN    31250      0
193  현대아이티앤이     한섬글로벌   0.03        29431         NaN    31250      0
194  현대아이티앤이    현대지앤에프     29        29431         NaN    31250      0

[195 rows x 7 columns]

In [346]:
dfLong = api.getTransTable('현대백화점', 2021)
dfLong

매출회사       매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액 해외매출액
0    현대백화점  씨엔에스푸드시스템     26      188596         NaN  1245063   NaN
1    현대백화점         한섬  51565      188596         NaN  1245063   NaN
2    현대백화점     한섬라이프앤      3      188596         NaN  1245063   NaN
3    현대백화점     현대그린푸드  39336      188596         NaN  1245063   NaN
4    현대백화점     현대드림투어    243      188596         NaN  1245063   NaN
..     ...        ...    ...         ...         ...      ...   ...
195   한무쇼핑      현대리바트   3827       73376         NaN   505370   NaN
196   한무쇼핑   현대백화점면세점  13993       73376         NaN   505370   NaN
197   한무쇼핑    현대에이치씨엔     10       73376         NaN   505370   NaN
198   한무쇼핑      현대퓨처넷    149       73376         NaN   505370   NaN
199   한무쇼핑      현대홈쇼핑    127       73376         NaN   505370   NaN

[200 rows x 7 columns]

# 미래에셋캐피탈

In [331]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('미래에셋캐피탈', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사       매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액 해외매출액
0      미래에셋캐피탈     미래에셋대우  29421       37850        2231  174685  2231
1      미래에셋캐피탈   미래에셋자산운용     12       37850        2231  174685  2231
2      미래에셋캐피탈   미래에셋생명보험   4692       37850        2231  174685  2231
3      미래에셋캐피탈   멀티에셋자산운용      5       37850        2231  174685  2231
4      미래에셋캐피탈  미래에셋금융서비스      2       37850        2231  174685  2231
..         ...        ...    ...         ...         ...     ...   ...
120  와이케이디벨롭먼트   멀티에셋자산운용     55        5944         NaN   31957   NaN
121  와이케이디벨롭먼트    미래에셋모바일      1        5944         NaN   31957   NaN
122  와이케이디벨롭먼트  미래에셋펀드서비스     35        5944         NaN   31957   NaN
123  와이케이디벨롭먼트  미래에셋금융서비스      1        5944         NaN   31957   NaN
124  와이케이디벨롭먼트    미래에셋컨설팅     39        5944         NaN   31957   NaN

[125 rows x 7 columns]

In [342]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('미래에셋캐피탈', 2020)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사                            매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)  \
0      미래에셋캐피탈                          미래에셋대우  31736       43772        3004   
1      미래에셋캐피탈                        미래에셋생명보험   4692       43772        3004   
2      미래에셋캐피탈                  미래에셋글로벌유니콘사모투자   3797       43772        3004   
3      미래에셋캐피탈               미래에셋네이버아시아그로쓰사모투자   3547       43772        3004   
4      미래에셋캐피탈  MiraeAssetSecuritiesVietnamLLC    562       43772        3004   
..         ...                             ...    ...         ...         ...   
109  와이케이디벨롭먼트                        멀티에셋자산운용     66        8577         NaN   
110  와이케이디벨롭먼트                       미래에셋펀드서비스     29        8577         NaN   
111  와이케이디벨롭먼트                       미래에셋금융서비스      6        8577         NaN   
112  와이케이디벨롭먼트                         미래에셋컨설팅     20        8577         NaN   
113       육공공구                         미래에셋컨설팅     20         NaN         NaN   

      국내매출액 해외매출액  
0    250214  3072  
1    250214  3072  
2    250214  3072  
3    250214  3072  
4    250214  3072  
..      ...   ...  
109   38774   NaN  
110   38774   NaN  
111   38774   NaN  
112   38774   NaN  
113       1   NaN  

[114 rows x 7 columns]

In [343]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('미래에셋캐피탈', 2021)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사                             매입회사    매출액 국내계열사계(매출액)  \
0      미래에셋캐피탈                           미래에셋증권  40302       53463   
1      미래에셋캐피탈                         미래에셋생명보험   4692       53463   
2      미래에셋캐피탈                   미래에셋글로벌유니콘사모투자   3797       53463   
3      미래에셋캐피탈                미래에셋네이버아시아그로쓰사모투자   4672       53463   
4      미래에셋캐피탈  MiraeAssetFinanceCompanyVietnam   2849       53463   
..         ...                              ...    ...         ...   
113  와이케이디벨롭먼트                         멀티에셋자산운용    130        8180   
114  와이케이디벨롭먼트                        미래에셋펀드서비스     15        8180   
115  와이케이디벨롭먼트                        미래에셋금융서비스     22        8180   
116  와이케이디벨롭먼트                       에너지인프라자산운용      3        8180   
117  와이케이디벨롭먼트                          미래에셋컨설팅     20        8180   

    해외계열사계(매출액)   국내매출액 해외매출액  
0          3155  259067  3155  
1          3155  259067  3155  
2          3155  259067  3155  
3          3155  259067  3155  
4          3155  259067  3155  
..          ...     ...   ...  
113         NaN   61687   NaN  
114         NaN   61687   NaN  
115         NaN   61687   NaN  
116         NaN   61687   NaN  
117         NaN   61687   NaN  

[118 rows x 7 columns]

# DL

In [317]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('DL', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사       매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액    해외매출액
0       대림산업    대림코퍼레이션  816117      960311       50887  7767478  1484397
1       대림산업       고려개발   57032      960311       50887  7767478  1484397
2       대림산업         삼호   54802      960311       50887  7767478  1484397
3       대림산업    대림자동차공업      12      960311       50887  7767478  1484397
4       대림산업  글래드호텔앤리조트    2489      960311       50887  7767478  1484397
..       ...        ...     ...         ...         ...      ...      ...
64    하이웨이솔라       포천파워     578         578         NaN     1503      NaN
65    포승그린파워       포천파워    7059        7059         NaN    42811      NaN
66  청진이삼프로젝트       대림산업   45479       45479         NaN    43511      NaN
67  청진이삼자산관리   청진이삼프로젝트     300         300         NaN      339      NaN
68    대림오토바이  글래드호텔앤리조트       5           5         NaN    63344     5801

[69 rows x 7 columns]

In [318]:
dfLong = api.getTransTable('DL', 2020)
dfLong

매출회사                        매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)  \
0       대림산업                     대림코퍼레이션  397782      881544       43272   
1       대림산업                        고려개발   57234      881544       43272   
2       대림산업                          삼호   58691      881544       43272   
3       대림산업                     대림자동차공업      11      881544       43272   
4       대림산업                   글래드호텔앤리조트    2709      881544       43272   
..       ...                         ...     ...         ...         ...   
56  청진이삼프로젝트                        대림산업   45524       45524         NaN   
57  청진이삼자산관리                    청진이삼프로젝트     300         300         NaN   
58    대림에이엠씨  대림제6호부산우암동기업형임대위탁관리부동산투자회사      81          81         NaN   
59    대림오토바이                        대림산업       3           3         NaN   
60     대림피앤피                        대림산업    1157        1157       52652   

      국내매출액    해외매출액  
0   6110881  1236867  
1   6110881  1236867  
2   6110881  1236867  
3   6110881  1236867  
4   6110881  1236867  
..      ...      ...  
56    43511      NaN  
57      300      NaN  
58       81      NaN  
59    64290     5988  
60   149191   248414  

[61 rows x 7 columns]

In [319]:
dfLong = api.getTransTable('DL', 2021)
dfLong

매출회사                        매입회사     매출액 국내계열사계(매출액)  \
0                          디엘                          대림    4843      677705   
1                          디엘                        디엘건설   83395      677705   
2                          디엘                      대림씨엔에스       3      677705   
3                          디엘                   글래드호텔앤리조트    1809      677705   
4                          디엘                        송도파워       2      677705   
5                          디엘                       에코술이홀      88      677705   
6                          디엘                    험프리에스엘큐원     100      677705   
7                          디엘                    제주항공우주호텔       9      677705   
8                          디엘                   덕송내각고속화도로    2562      677705   
9                          디엘                       디엘에너지      79      677705   
10                         디엘                    청진이삼자산관리       1      677705   
11                         디엘                      대림에이엠씨     106      677705   
12                         디엘                      대림오토바이       3      677705   
13                         디엘                       대림피앤피  584319      677705   
14                         디엘   대림제7호마산회원기업형임대위탁관리부동산투자회사     324      677705   
15                         디엘                      디엘에프엔씨       8      677705   
16                         디엘                     효제피에프브이      54      677705   
17                         대림                          디엘  156276      228674   
18                         대림                        디엘건설   36141      228674   
19                         대림                       디엘모터스    1265      228674   
20                         대림                      대림씨엔에스     172      228674   
21                         대림                   글래드호텔앤리조트    3364      228674   
22                         대림                        포천파워      93      228674   
23                         대림                    험프리에스엘큐원    1091      228674   
24                         대림                      에이플러스디     583      228674   
25                         대림                       디엘에너지     304      228674   
26                         대림                      대림에이엠씨      61      228674   
27                         대림                      대림오토바이     459      228674   
28                         대림                       대림피앤피   26438      228674   
29                         대림                      디엘에프엔씨    2427      228674   
30                       디엘건설                          디엘  186136      227138   
31                       디엘건설                          대림   41002      227138   
32                      디엘모터스                      대림오토바이     609         609   
33                  글래드호텔앤리조트                          디엘    1095        4619   
34                  글래드호텔앤리조트                          대림    2822        4619   
35                  글래드호텔앤리조트                        디엘건설      27        4619   
36                  글래드호텔앤리조트                      에이플러스디     641        4619   
37                  글래드호텔앤리조트                       디엘에너지      16        4619   
38                  글래드호텔앤리조트                       대림피앤피      18        4619   
39                     에이플러스디                          디엘     173         550   
40                     에이플러스디                          대림      45         550   
41                     에이플러스디                   글래드호텔앤리조트     330         550   
42                     에이플러스디                       대림피앤피       2         550   
43                   제주항공우주호텔                   글래드호텔앤리조트      13          13   
44                      디엘에너지                          디엘     170       11147   
45                      디엘에너지                        포천파워    1944       11147   
46                      디엘에너지                     밀머란에스피씨     100       11147   
47                      디엘에너지                      포승그린파워    8917       11147   
48                      디엘에너지                    코크레인에스피

# 카카오

In [313]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('카카오', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사      매입회사  매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액 해외매출액
0      카카오     그라운드원   43      105523        3912  1173925   NaN
1      카카오  다음글로벌홀딩스    7      105523        3912  1173925   NaN
2      카카오   디케이비즈니스   89      105523        3912  1173925   NaN
3      카카오    디케이서비스   26      105523        3912  1173925   NaN
4      카카오    디케이테크인   83      105523        3912  1173925   NaN
..     ...       ...  ...         ...         ...      ...   ...
188  카카오페이   카카오모빌리티   14       26776         NaN    69526   NaN
189  카카오페이   카카오브이엑스    1       26776         NaN    69526   NaN
190  카카오페이      픽셀허브    1       26776         NaN    69526   NaN
191  카카오페이   카카오아이엑스   59       26776         NaN    69526   NaN
192  카카오페이      카카오엠  285       26776         NaN    69526   NaN

[193 rows x 7 columns]

In [314]:
dfLong = api.getTransTable('카카오', 2020)
dfLong

매출회사            매입회사  매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액 해외매출액
0      카카오           그라운드원  132      191859        1065  1506276  7195
1      카카오            그레이고    0      191859        1065  1506276  7195
2      카카오        다음글로벌홀딩스    7      191859        1065  1506276  7195
3      카카오          디케이테크인   97      191859        1065  1506276  7195
4      카카오            링키지랩    9      191859        1065  1506276  7195
..     ...             ...  ...         ...         ...      ...   ...
246  카카오페이             하시스    1       53381          20   139879  1190
247  카카오페이         카카오아이엑스  284       53381          20   139879  1190
248  카카오페이          루프트코리아    1       53381          20   139879  1190
249  카카오페이            인바이유  109       53381          20   139879  1190
250  카카오페이  KlaytnPte.Ltd.   20       53381          20   139879  1190

[251 rows x 7 columns]

In [315]:
dfLong = api.getTransTable('카카오', 2021)
dfLong

매출회사      매입회사   매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  해외매출액
0         카카오    그라운드엑스   557      274704        2102  1772806   6389
1         카카오      그레이고     1      274704        2102  1772806   6389
2         카카오     네오바자르    12      274704        2102  1772806   6389
3         카카오  다음글로벌홀딩스     7      274704        2102  1772806   6389
4         카카오    디케이테크인   145      274704        2102  1772806   6389
..        ...       ...   ...         ...         ...      ...    ...
351     카카오페이   비미디어컴퍼니     1       88842         NaN   234893  10663
352     카카오페이   카카오페이증권  3650       88842         NaN   234893  10663
353     카카오페이  케이피보험서비스   412       88842         NaN   234893  10663
354   카카오페이증권     카카오페이  4619        4619         NaN    58527    NaN
355  케이피보험서비스     카카오페이    21          21         NaN     2452    NaN

[356 rows x 7 columns]

# 부영

In [310]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('부영', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사      매입회사   매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  해외매출액
0     부영      부영주택  6206        7684         NaN    48970    NaN
1     부영    동광주택산업    51        7684         NaN    48970    NaN
2     부영      동광주택  1084        7684         NaN    48970    NaN
3     부영      광영토건   160        7684         NaN    48970    NaN
4     부영    남광건설산업     1        7684         NaN    48970    NaN
..   ...       ...   ...         ...         ...      ...    ...
186   합계     한라일보사    78       57194       20357  2032907  20357
187   합계    에이치아이엠    51       57194       20357  2032907  20357
188   합계      인천일보   144       57194       20357  2032907  20357
189   합계     인천출판사    31       57194       20357  2032907  20357
190   합계  부영대부파이낸스   289       57194       20357  2032907  20357

[191 rows x 7 columns]

In [311]:
dfLong = api.getTransTable('부영', 2020)
dfLong

매출회사      매입회사   매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  해외매출액
0     부영      부영주택  3137        4156         NaN     6521    NaN
1     부영    동광주택산업    52        4156         NaN     6521    NaN
2     부영      동광주택   518        4156         NaN     6521    NaN
3     부영      광영토건   255        4156         NaN     6521    NaN
4     부영    남광건설산업    30        4156         NaN     6521    NaN
..   ...       ...   ...         ...         ...      ...    ...
126   합계     한라일보사    52        9130       28560  1340210  28560
127   합계    에이치아이엠    29        9130       28560  1340210  28560
128   합계      인천일보   122        9130       28560  1340210  28560
129   합계     인천출판사    39        9130       28560  1340210  28560
130   합계  부영대부파이낸스    15        9130       28560  1340210  28560

[131 rows x 7 columns]

In [312]:
dfLong = api.getTransTable('부영', 2021)
dfLong

매출회사      매입회사   매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  해외매출액
0     부영      부영주택  4653        5553         NaN   197597    NaN
1     부영    동광주택산업    52        5553         NaN   197597    NaN
2     부영      동광주택   480        5553         NaN   197597    NaN
3     부영      광영토건   221        5553         NaN   197597    NaN
4     부영    남광건설산업     1        5553         NaN   197597    NaN
..   ...       ...   ...         ...         ...      ...    ...
131   합계     한라일보사    36        8519       72530  2903321  72530
132   합계    에이치아이엠     0        8519       72530  2903321  72530
133   합계      인천일보   121        8519       72530  2903321  72530
134   합계     인천출판사    31        8519       72530  2903321  72530
135   합계  부영대부파이낸스    13        8519       72530  2903321  72530

[136 rows x 7 columns]

# LS

In [304]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('LS', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사           매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액  해외매출액
0        LS           LS전선  10201.0        33523         3299  238704   3299
1        LS           LS산전   6593.0        33523         3299  238704   3299
2        LS        LS니꼬동제련   7958.0        33523         3299  238704   3299
3        LS          LS엠트론   3615.0        33523         3299  238704   3299
4        LS             E1    227.0        33523         3299  238704   3299
..      ...            ...      ...          ...          ...     ...    ...
244   한성플랜지            예스코     15.0        20459            0   22521      0
245   한성플랜지         한성피씨건설    345.0        20459            0   22521      0
246   한성플랜지  거래업체수*LS는주석참조      0.0        20459            0   22521      0
247  한성피씨건설             한성    147.0          147            0  122733      0
248  한성피씨건설  거래업체수*LS는주석참조      0.0          147            0  122733      0

[249 rows x 7 columns]

In [305]:
dfLong = api.getTransTable('LS', 2020)
dfLong

매출회사           매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)       국내매출액  \
0        LS           LS전선  9849.0      33355.0         3342   83768.000   
1        LS         LS일렉트릭  6794.0      33355.0         3342   83768.000   
2        LS        LS니꼬동제련  8262.0      33355.0         3342   83768.000   
3        LS          LS엠트론  3180.0      33355.0         3342   83768.000   
4        LS             E1   246.0      33355.0         3342   83768.000   
..      ...            ...     ...          ...          ...         ...   
230   한성플랜지         한성피씨건설  1044.0      20491.0            0      22.399   
231   한성플랜지  거래업체수*LS는주석참조     0.0      20491.0            0      22.399   
232  한성피씨건설         LS일렉트릭   414.0          NaN          629  148990.000   
233  한성피씨건설             한성   215.0          NaN          629  148990.000   
234  한성피씨건설  거래업체수*LS는주석참조   629.0          NaN          629  148990.000   

     해외매출액  
0     3342  
1     3342  
2     3342  
3     3342  
4     3342  
..     ...  
230      0  
231      0  
232      0  
233      0  
234      0  

[235 rows x 7 columns]

In [306]:
dfLong = api.getTransTable('LS', 2021)
dfLong

매출회사           매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액  해외매출액
0        LS           LS전선  10595.0        33510         4380  161367   4380
1        LS         LS일렉트릭   6948.0        33510         4380  161367   4380
2        LS        LS니꼬동제련   8265.0        33510         4380  161367   4380
3        LS          LS엠트론   2915.0        33510         4380  161367   4380
4        LS             E1    248.0        33510         4380  161367   4380
..      ...            ...      ...          ...          ...     ...    ...
242   한성플랜지             한성      1.0        19625            0   21141      0
243   한성플랜지  거래업체수*LS는주석참조      0.0        19625            0   21141      0
244  한성피씨건설         LS일렉트릭   5341.0         5608            0  204493      0
245  한성피씨건설             한성    267.0         5608            0  204493      0
246  한성피씨건설  거래업체수*LS는주석참조      0.0         5608            0  204493      0

[247 rows x 7 columns]

# 두산

In [301]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('두산', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사                    매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액  \
0         두산                   두산중공업   60564      314417      612233  944924   
1         두산                 두산인프라코어  132634      314417      612233  944924   
2         두산                    두산밥캣    3765      314417      612233  944924   
3         두산                    두산건설   19153      314417      612233  944924   
4         두산                     오리콤    3369      314417      612233  944924   
..       ...                     ...     ...         ...         ...     ...   
166     밸류웍스                   두산메카텍    4933        5427         NaN    5427   
167  오성파워오엔엠                   두산중공업    3666        3666         NaN    3666   
168    네오플럭스            네오플럭스제1호사모투자   314.0       10350         NaN   22219   
169    네오플럭스  케이티씨엔피그로쓰챔프2011의2호사모투자  8425.0       10350         NaN   22219   
170    네오플럭스            네오플럭스제3호사모투자  1611.0       10350         NaN   22219   

       해외매출액  
0    1974477  
1    1974477  
2    1974477  
3    1974477  
4    1974477  
..       ...  
166      NaN  
167      NaN  
168      NaN  
169      NaN  
170      NaN  

[171 rows x 7 columns]

In [302]:
dfLong = api.getTransTable('두산', 2020)
dfLong

매출회사                    매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)  \
0            두산                   두산중공업   55269      230979      649368   
1            두산                 두산인프라코어  124486      230979      649368   
2            두산                    두산밥캣    5333      230979      649368   
3            두산                    두산건설    6481      230979      649368   
4            두산                     오리콤    4728      230979      649368   
..          ...                     ...     ...         ...         ...   
180     오성파워오엔엠                   두산중공업    4471        4471         NaN   
181  두산로지스틱스솔루션                      두산      92          92         NaN   
182       네오플럭스            네오플럭스제1호사모투자    3364        6749         NaN   
183       네오플럭스  케이티씨엔피그로쓰챔프2011의2호사모투자     583        6749         NaN   
184       네오플럭스            네오플럭스제3호사모투자    2802        6749         NaN   

      국내매출액    해외매출액  
0    808326  1252379  
1    808326  1252379  
2    808326  1252379  
3    808326  1252379  
4    808326  1252379  
..      ...      ...  
180    4631      NaN  
181    5655       36  
182   15486      NaN  
183   15486      NaN  
184   15486      NaN  

[185 rows x 7 columns]

In [303]:
dfLong = api.getTransTable('두산', 2021)
dfLong

매출회사     매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액   해외매출액
0            두산    두산중공업   51436      239332      686124  613498  935838
1            두산  두산인프라코어  108826      239332      686124  613498  935838
2            두산     두산밥캣    5199      239332      686124  613498  935838
3            두산     두산건설    6746      239332      686124  613498  935838
4            두산      오리콤    3518      239332      686124  613498  935838
..          ...      ...     ...         ...         ...     ...     ...
159          한컴       두산    2827        2910         NaN   50193      89
160          한컴      오리콤       6        2910         NaN   50193      89
161          한컴      디비씨      77        2910         NaN   50193      89
162     오성파워오엔엠    두산중공업    5461        5461         NaN    5547     NaN
163  두산로지스틱스솔루션       두산      17          17         NaN   13437     876

[164 rows x 7 columns]

# 한진칼

In [296]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('대한항공', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사                                    매입회사     매출액  국내계열사계(매출액)  \
0           대한항공                                     한진칼   178.0       199900   
1           대한항공                                      한진  3347.0       199900   
2           대한항공                                    한국공항  1003.0       199900   
3           대한항공                                    정석기업     1.0       199900   
4           대한항공                                    한진관광   112.0       199900   
..           ...                                     ...     ...          ...   
180     세계혼재항공화물                                    대한항공  3147.0         3147   
181     세계혼재항공화물              WACINT'LLOGISTICSHKLIMITED    92.0         3147   
182     세계혼재항공화물  WACINT'LLOGISTICSQINGDAOCOMPANYLIMITED    18.0         3147   
183  더블유에이씨항공서비스                                세계혼재항공화물   596.0          596   
184  더블유에이씨항공서비스  WACINT'LLOGISTICSQINGDAOCOMPANYLIMITED   216.0          596   

    해외계열사계(매출액)    국내매출액     해외매출액  
0          1714  1183532  11463333  
1          1714  1183532  11463333  
2          1714  1183532  11463333  
3          1714  1183532  11463333  
4          1714  1183532  11463333  
..          ...      ...       ...  
180         110     3470       NaN  
181         110     3470       NaN  
182         110     3470       NaN  
183         216      912       NaN  
184         216      912       NaN  

[185 rows x 7 columns]

In [298]:
dfLong = api.getTransTable('대한항공', 2020)
dfLong

매출회사                                    매입회사       매출액  \
0           대한항공                                     한진칼     721.0   
1           대한항공                                      한진     591.0   
2           대한항공                                    한국공항     739.0   
3           대한항공                                     진에어  167762.0   
4           대한항공                                  싸이버스카이      51.0   
..           ...                                     ...       ...   
185         청원냉장                                   태일캐터링     180.0   
186     세계혼재항공화물                                    대한항공    2754.0   
187     세계혼재항공화물              WACINT'LLOGISTICSHKLIMITED       3.0   
188     세계혼재항공화물  WACINT'LLOGISTICSQINGDAOCOMPANYLIMITED      74.0   
189  더블유에이씨항공서비스                                세계혼재항공화물     606.0   

     국내계열사계(매출액)  해외계열사계(매출액)    국내매출액     해외매출액  
0         190910          375  1328452  10963216  
1         190910          375  1328452  10963216  
2         190910          375  1328452  10963216  
3         190910          375  1328452  10963216  
4         190910          375  1328452  10963216  
..           ...          ...      ...       ...  
185          180            0      180       NaN  
186         2754           77     2754        77  
187         2754           77     2754        77  
188         2754           77     2754        77  
189          606            0      606       NaN  

[190 rows x 7 columns]

In [300]:
dfLong = api.getTransTable('대한항공', 2021)
dfLong

매출회사                                    매입회사      매출액 국내계열사계(매출액)  \
0           대한항공                                     한진칼   1158.0       65681   
1           대한항공                                      한진    709.0       65681   
2           대한항공                                     진에어  56518.0       65681   
3           대한항공                                    정석기업    141.0       65681   
4           대한항공                                 칼호텔네트워크    265.0       65681   
..           ...                                     ...      ...         ...   
181     세계혼재항공화물                                    대한항공    672.0        1627   
182     세계혼재항공화물                                      한진    955.0        1627   
183     세계혼재항공화물              WACINT'LLOGISTICSHKLIMITED     10.0        1627   
184     세계혼재항공화물  WACINT'LLOGISTICSQINGDAOCONPANYLIMITED    940.0        1627   
185  더블유에이씨항공서비스                                세계혼재항공화물    625.0         625   

    해외계열사계(매출액)   국내매출액    해외매출액  
0           318  736452  6668554  
1           318  736452  6668554  
2           318  736452  6668554  
3           318  736452  6668554  
4           318  736452  6668554  
..          ...     ...      ...  
181         950    3837     2275  
182         950    3837     2275  
183         950    3837     2275  
184         950    3837     2275  
185         NaN     719      NaN  

[186 rows x 7 columns]

# 신세계

In [289]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('신세계', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사      매입회사    매출액  국내계열사계(매출액) 해외계열사계(매출액)    국내매출액 해외매출액
0          신세계     광주신세계  13336       196180         NaN  1739327   NaN
1          신세계   신세계조선호텔   2674       196180         NaN  1739327   NaN
2          신세계     신세계건설    696       196180         NaN  1739327   NaN
3          신세계  신세계인터내셔날  42334       196180         NaN  1739327   NaN
4          신세계    신세계톰보이   3461       196180         NaN  1739327   NaN
..         ...       ...    ...          ...         ...      ...   ...
398       제주소주     이마트24    262          278         NaN     4299   NaN
399  신세계디에프글로벌       신세계      5         8253         NaN   204480   NaN
400  신세계디에프글로벌   신세계조선호텔      2         8253         NaN   204480   NaN
401  신세계디에프글로벌    신세계디에프   8246         8253         NaN   204480   NaN
402    에스에스지닷컴       이마트   2268         2268         NaN     2268   NaN

[403 rows x 7 columns]

In [291]:
dfLong = api.getTransTable('한국조선해양', 2020)
dfLong

매출회사                                          매입회사     매출액 국내계열사계(매출액)  \
0    한국조선해양                                        현대미포조선  154880      414937   
1    한국조선해양                                       현대삼호중공업  178734      414937   
2    한국조선해양                                        현대오일뱅크    2624      414937   
3    한국조선해양                                         현대이엔티     550      414937   
4    한국조선해양                                      현대중공업스포츠     411      414937   
..      ...                                           ...     ...         ...   
220   현대중공업  HyundaiConstructionEquipmentIndiaPrivateLtd.      35      528125   
221   현대중공업                                  현대강소공정기계유한공사      59      528125   
222   현대중공업                                  연대현대빙윤중공유한공사       8      528125   
223   현대중공업              HyundaiPowerTransformersUSA,Inc.     102      528125   
224   현대중공업                                  현대중공중국전기유한공사      15      528125   

    해외계열사계(매출액)    국내매출액    해외매출액  
0          1028   421747  2935082  
1          1028   421747  2935082  
2          1028   421747  2935082  
3          1028   421747  2935082  
4          1028   421747  2935082  
..          ...      ...      ...  
220         735  1434850  4021818  
221         735  1434850  4021818  
222         735  1434850  4021818  
223         735  1434850  4021818  
224         735  1434850  4021818  

[225 rows x 7 columns]

In [292]:
fLong = api.getTransTable('한국조선해양', 2021)
dfLong

매출회사                                          매입회사     매출액 국내계열사계(매출액)  \
0    한국조선해양                                        현대미포조선  154880      414937   
1    한국조선해양                                       현대삼호중공업  178734      414937   
2    한국조선해양                                        현대오일뱅크    2624      414937   
3    한국조선해양                                         현대이엔티     550      414937   
4    한국조선해양                                      현대중공업스포츠     411      414937   
..      ...                                           ...     ...         ...   
220   현대중공업  HyundaiConstructionEquipmentIndiaPrivateLtd.      35      528125   
221   현대중공업                                  현대강소공정기계유한공사      59      528125   
222   현대중공업                                  연대현대빙윤중공유한공사       8      528125   
223   현대중공업              HyundaiPowerTransformersUSA,Inc.     102      528125   
224   현대중공업                                  현대중공중국전기유한공사      15      528125   

    해외계열사계(매출액)    국내매출액    해외매출액  
0          1028   421747  2935082  
1          1028   421747  2935082  
2          1028   421747  2935082  
3          1028   421747  2935082  
4          1028   421747  2935082  
..          ...      ...      ...  
220         735  1434850  4021818  
221         735  1434850  4021818  
222         735  1434850  4021818  
223         735  1434850  4021818  
224         735  1434850  4021818  

[225 rows x 7 columns]

# 한국조선해양

In [286]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('한국조선해양', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사                                   매입회사     매출액 국내계열사계(매출액)  \
0            현대중공업                                 현대미포조선  220863      630274   
1            현대중공업                                현대삼호중공업  322204      630274   
2            현대중공업                                 현대오일뱅크   22980      630274   
3            현대중공업                                  현대이엔티     873      630274   
4            현대중공업                               현대중공업스포츠     432      630274   
..             ...                                    ...     ...         ...   
149  현대일렉트릭앤에너지시스템      HyundaiHeavyIndustriesCo.Bulgaria     812      172090   
150  현대일렉트릭앤에너지시스템                           현대중공중국전기유한공사    4495      172090   
151  현대일렉트릭앤에너지시스템  HDENEPOWERSOLUTIONINDIAPRIVATELIMITED     179      172090   
152     현대중공업파워시스템                                  현대중공업      46          46   
153           티앤에이                                 현대건설기계     211         211   

    해외계열사계(매출액)    국내매출액   해외매출액  
0          5318  7326175  740527  
1          5318  7326175  740527  
2          5318  7326175  740527  
3          5318  7326175  740527  
4          5318  7326175  740527  
..          ...      ...     ...  
149       34674  1026121  706493  
150       34674  1026121  706493  
151       34674  1026121  706493  
152         NaN      204     NaN  
153         NaN      459     604  

[154 rows x 7 columns]

In [287]:
dfLong = api.getTransTable('한국조선해양', 2020)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사                                          매입회사     매출액 국내계열사계(매출액)  \
0    한국조선해양                                        현대미포조선  154880      414937   
1    한국조선해양                                       현대삼호중공업  178734      414937   
2    한국조선해양                                        현대오일뱅크    2624      414937   
3    한국조선해양                                         현대이엔티     550      414937   
4    한국조선해양                                      현대중공업스포츠     411      414937   
..      ...                                           ...     ...         ...   
220   현대중공업  HyundaiConstructionEquipmentIndiaPrivateLtd.      35      528125   
221   현대중공업                                  현대강소공정기계유한공사      59      528125   
222   현대중공업                                  연대현대빙윤중공유한공사       8      528125   
223   현대중공업              HyundaiPowerTransformersUSA,Inc.     102      528125   
224   현대중공업                                  현대중공중국전기유한공사      15      528125   

    해외계열사계(매출액)    국내매출액    해외매출액  
0          1028   421747  2935082  
1          1028   421747  2935082  
2          1028   421747  2935082  
3          1028   421747  2935082  
4          1028   421747  2935082  
..          ...      ...      ...  
220         735  1434850  4021818  
221         735  1434850  4021818  
222         735  1434850  4021818  
223         735  1434850  4021818  
224         735  1434850  4021818  

[225 rows x 7 columns]

In [288]:
dfLong = api.getTransTable('한국조선해양', 2021)
dfLong

매출회사                                          매입회사   매출액 국내계열사계(매출액)  \
0    한국조선해양                                        현대미포조선  8053      121588   
1    한국조선해양                                       현대삼호중공업  4181      121588   
2    한국조선해양                                        현대오일뱅크  2903      121588   
3    한국조선해양                                         현대이엔티   214      121588   
4    한국조선해양                                      현대중공업스포츠   382      121588   
..      ...                                           ...   ...         ...   
237  현대로보틱스                                        현대엘앤에스   278       17604   
238  현대로보틱스                                  현대중공중국투자유한공사   120       17604   
239  현대로보틱스  HyundaiConstructionEquipmentIndiaPrivateLtd.   309       17604   
240  현대로보틱스                HYUNDAIROBOTICSSHANGHAICO.,LTD  9043       17604   
241  현대로보틱스         HAININGHAGONGHYUNDAIROBOTICS.CO.,LTD.   521       17604   

    해외계열사계(매출액)   국내매출액  해외매출액  
0           225  172158    225  
1           225  172158    225  
2           225  172158    225  
3           225  172158    225  
4           225  172158    225  
..          ...     ...    ...  
237        9993  136111  54875  
238        9993  136111  54875  
239        9993  136111  54875  
240        9993  136111  54875  
241        9993  136111  54875  

[242 rows x 7 columns]

# GS

In [282]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('GS', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사        매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액  해외매출액
0        지에스      지에스에너지  2494.0     121478.0          NaN  506038.0    NaN
1        지에스       지에스파워  3098.0     121478.0          NaN  506038.0    NaN
2        지에스      해양도시가스  1015.0     121478.0          NaN  506038.0    NaN
3        지에스     서라벌도시가스   240.0     121478.0          NaN  506038.0    NaN
4        지에스       지에스이엠     3.0     121478.0          NaN  506038.0    NaN
..       ...         ...     ...          ...          ...       ...    ...
291   지에스이앤알  지에스포천그린에너지   105.0       2853.0          NaN  878777.0    NaN
292   지에스이앤알    영양제2풍력발전   350.0       2853.0          NaN  878777.0    NaN
293   지에스이앤알        와이지이   300.0       2853.0          NaN  878777.0    NaN
294   지에스이앤알    영덕제1풍력발전   327.0       2853.0          NaN  878777.0    NaN
295  지에스풍력발전     지에스동해전력  7394.0       7394.0          NaN   38152.0    NaN

[296 rows x 7 columns]

In [283]:
dfLong = api.getTransTable('GS', 2020)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사       매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액  해외매출액
0        지에스     지에스에너지   3525.0     123148.0          NaN  365336.0    NaN
1        지에스      지에스파워   3535.0     123148.0          NaN  365336.0    NaN
2        지에스   지에스파크이십사    813.0     123148.0          NaN  365336.0    NaN
3        지에스     지에스칼텍스  51656.0     123148.0          NaN  365336.0    NaN
4        지에스     지에스엠비즈    466.0     123148.0          NaN  365336.0    NaN
..       ...        ...      ...          ...          ...       ...    ...
293   지에스이앤알   영양제2풍력발전    481.0       2198.0          NaN  804170.0    NaN
294   지에스이앤알   영덕제1풍력발전    381.0       2198.0          NaN  804170.0    NaN
295   지에스이앤알  삼척그린에너지파크    148.0       2198.0          NaN  804170.0    NaN
296   지에스이앤알  동해국제자원터미널     55.0       2198.0          NaN  804170.0    NaN
297  지에스풍력발전    지에스동해전력   6647.0       6647.0          NaN   33084.0    NaN

[298 rows x 7 columns]

In [284]:
dfLong = api.getTransTable('GS', 2021)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사       매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액  해외매출액
0        지에스     지에스에너지   3213.0     110367.0          NaN  306286.0    NaN
1        지에스      지에스파워   3377.0     110367.0          NaN  306286.0    NaN
2        지에스   지에스파크이십사    592.0     110367.0          NaN  306286.0    NaN
3        지에스     지에스칼텍스  41837.0     110367.0          NaN  306286.0    NaN
4        지에스     지에스엠비즈    397.0     110367.0          NaN  306286.0    NaN
..       ...        ...      ...          ...          ...       ...    ...
297   지에스이앤알   영덕제1풍력발전    453.0       1921.0          NaN  572548.0    NaN
298   지에스이앤알  삼척그린에너지파크    123.0       1921.0          NaN  572548.0    NaN
299   지에스이앤알  동해국제자원터미널    134.0       1921.0          NaN  572548.0    NaN
300  지에스풍력발전    지에스동해전력   8544.0       8544.0          NaN   32835.0    NaN
301  동해그린에너지    지에스동해전력    271.0        271.0          NaN     385.0    NaN

[302 rows x 7 columns]

In [271]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfWide = api.getTransTableTest('GS', 2019)
dfWide

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
  매출회사(소속회사)＼매입자                  국내 계열회사                                   \
  매출회사(소속회사)＼매입자                    비금융회사                                    
  매출회사(소속회사)＼매입자 매출회사(소속회사)＼매입자.1    ㈜지에스 지에스에너지㈜  지에스파워㈜ ㈜해양도시가스 서라벌도시가스㈜   
0          비금융회사             ㈜지에스     NaN  2494.0  3098.0  1015.0    240.0   
1          비금융회사          지에스에너지㈜     NaN     NaN  6609.0     NaN      NaN   
2          비금융회사           지에스파워㈜   660.0  1231.0     NaN     NaN      NaN   
3          비금융회사          지에스나노텍㈜     NaN     NaN     NaN     NaN      NaN   
4          비금융회사         살데비다코리아㈜     NaN     NaN     NaN     NaN      NaN   

                              ...    해외 계열회사                     \
                              ...      비금융회사                      
  보령엘엔지터미널㈜ 지에스이엠㈜ 지에스파크이십사㈜  ... 청도삼양피혁유한공사 SY ENERGY PTE.LTD.   
0       NaN    3.0    1026.0  ... 

매출회사(소속회사)＼매입자                  국내 계열회사                                     \
   매출회사(소속회사)＼매입자                    비금융회사                                      
   매출회사(소속회사)＼매입자 매출회사(소속회사)＼매입자.1    ㈜지에스 지에스에너지㈜    지에스파워㈜ ㈜해양도시가스 서라벌도시가스㈜   
0           비금융회사             ㈜지에스     NaN  2494.0    3098.0  1015.0    240.0   
1           비금융회사          지에스에너지㈜     NaN     NaN    6609.0     NaN      NaN   
2           비금융회사           지에스파워㈜   660.0  1231.0       NaN     NaN      NaN   
3           비금융회사          지에스나노텍㈜     NaN     NaN       NaN     NaN      NaN   
4           비금융회사         살데비다코리아㈜     NaN     NaN       NaN     NaN      NaN   
..            ...              ...     ...     ...       ...     ...      ...   
61          비금융회사        ㈜영덕제1풍력발전     NaN     NaN       NaN     NaN      NaN   
62          비금융회사       ㈜삼척그린에너지파크     NaN     NaN       NaN     NaN      NaN   
63          비금융회사            ㈜프로케어     NaN     NaN       NaN     NaN      NaN   
64          비금융회사               소계   986.0  5698.0  112562.0  1024.0    243.0   
65             합계               합계   986.0  5698.0  112562.0  1024.0    243.0   

                               ...    해외 계열회사                     \
                               ...      비금융회사                      
   보령엘엔지터미널㈜ 지에스이엠㈜ 지에스파크이십사㈜  ... 청도삼양피혁유한공사 SY ENERGY PTE.LTD.   
0        NaN    3.0    1026.0  ...        NaN                NaN   
1       76.0    NaN       NaN  ...        NaN                NaN   
2        NaN    NaN       NaN  ...        NaN                NaN   
3        NaN    NaN       NaN  ...        NaN                NaN   
4        NaN    NaN       NaN  ...        NaN                NaN   
..       ...    ...       ...  ...        ...                ...   
61       NaN    NaN       NaN  ...        NaN                NaN   
62       NaN    NaN       NaN  ...        NaN                NaN   
63       NaN    NaN       NaN  ...        NaN                NaN   
64   35724.0    3.0    2707.0  ...     1084.0            93064.0   
65   35724.0    3.0    2707.0  ...     1084.0            93064.0   

                                                                             \
                                                                       금융회사   
   GS Aromatics Pte. Ltd. Qingdao Lidong Chemical Co., Ltd.         소계   소계   
0                     NaN                               NaN        NaN  NaN   
1                     NaN                               NaN        NaN  NaN   
2                     NaN                               NaN        NaN  NaN   
3                     NaN                               NaN        NaN  NaN   
4                     NaN                               NaN        NaN  NaN   
..                    ...                               ...        ...  ...   
61                    NaN                               NaN        NaN  NaN   
62                    NaN                               NaN        NaN  NaN   
63                    NaN                               NaN        NaN  NaN   
64                94564.0                         1067470.0  9336305.0  NaN   
65                94564.0                         1067470.0  9336305.0  NaN   

   해외계열사 계 (매출액)      매출액 총계                          
   해외계열사 계 (매출액)      매출액 총계                          
   해외계열사 계 (매출액)           계      국내 매출액      해외 매출액  
0            NaN    506038.0    506038.0         NaN  
1            NaN    352289.0    345021.0      7268.0  
2            NaN    879119.0    879119.0         NaN  
3            NaN         NaN         NaN         NaN  
4            NaN         NaN         NaN         NaN  
..           ...         ...         ...         ...  
61           NaN         NaN         NaN         NaN  
62           NaN         NaN         NaN         NaN  
63           NaN     11529.0     11529.0         NaN  
64     9336305.0  67880865.0  34851364.0  33029501.0  
65     9336305.0  67880865.0  34851364.0  33029501.0  

[66 rows x 101 columns]

In [272]:
# exclude a financial dummy variable for sales company
dfWideT = dfWide.T
dfWideV2 = dfWideT[~dfWideT[0].replace(' ', '', regex=True).isin(['금융회사', '비금융회사'])]
dfWideV2 = dfWideV2.T

# convert multiIndex column names to normal column names
dfWideV2.columns = convColNm(dfWideV2.columns)

# clean column names
dfWideV2.columns = cleanColNm(dfWideV2.columns)

In [267]:
dfWideT = dfWide.T
dfWideV2 = dfWideT[~dfWideT[0].replaceisin(['금융회사', '비금융회사'])]
dfWideV2 = dfWideV2.T

# convert multiIndex column names to normal column names
dfWideV2.columns = convColNm(dfWideV2.columns)

# clean column names
dfWideV2.columns = cleanColNm(dfWideV2.columns)
dfWideV2

매출회사    지에스  지에스에너지     지에스파워  해양도시가스 서라벌도시가스 보령엘엔지터미널 지에스이엠  \
0         ㈜지에스    NaN  2494.0    3098.0  1015.0   240.0      NaN   3.0   
1      지에스에너지㈜    NaN     NaN    6609.0     NaN     NaN     76.0   NaN   
2       지에스파워㈜  660.0  1231.0       NaN     NaN     NaN      NaN   NaN   
3      지에스나노텍㈜    NaN     NaN       NaN     NaN     NaN      NaN   NaN   
4     살데비다코리아㈜    NaN     NaN       NaN     NaN     NaN      NaN   NaN   
..         ...    ...     ...       ...     ...     ...      ...   ...   
61   ㈜영덕제1풍력발전    NaN     NaN       NaN     NaN     NaN      NaN   NaN   
62  ㈜삼척그린에너지파크    NaN     NaN       NaN     NaN     NaN      NaN   NaN   
63       ㈜프로케어    NaN     NaN       NaN     NaN     NaN      NaN   NaN   
64          소계  986.0  5698.0  112562.0  1024.0   243.0  35724.0   3.0   
65          합계  986.0  5698.0  112562.0  1024.0   243.0  35724.0   3.0   

   지에스파크이십사 인천종합에너지  ... 청도삼양피혁유한공사 SYENERGYPTE.LTD. GSAromaticsPte.Ltd.  \
0    1026.0     NaN  ...        NaN              NaN                 NaN   
1       NaN     NaN  ...        NaN              NaN                 NaN   
2       NaN    92.0  ...        NaN              NaN                 NaN   
3       NaN     NaN  ...        NaN              NaN                 NaN   
4       NaN     NaN  ...        NaN              NaN                 NaN   
..      ...     ...  ...        ...              ...                 ...   
61      NaN     NaN  ...        NaN              NaN                 NaN   
62      NaN     NaN  ...        NaN              NaN                 NaN   
63      NaN     NaN  ...        NaN              NaN                 NaN   
64   2707.0    92.0  ...     1084.0          93064.0             94564.0   
65   2707.0    92.0  ...     1084.0          93064.0             94564.0   

   QingdaoLidongChemicalCo.,Ltd.         소계   소계 해외계열사계(매출액)           계  \
0                            NaN        NaN  NaN         NaN    506038.0   
1                            NaN        NaN  NaN         NaN    352289.0   
2                            NaN        NaN  NaN         NaN    879119.0   
3                            NaN        NaN  NaN         NaN         NaN   
4                            NaN        NaN  NaN         NaN         NaN   
..                           ...        ...  ...         ...         ...   
61                           NaN        NaN  NaN         NaN         NaN   
62                           NaN        NaN  NaN         NaN         NaN   
63                           NaN        NaN  NaN         NaN     11529.0   
64                     1067470.0  9336305.0  NaN   9336305.0  67880865.0   
65                     1067470.0  9336305.0  NaN   9336305.0  67880865.0   

         국내매출액       해외매출액  
0     506038.0         NaN  
1     345021.0      7268.0  
2     879119.0         NaN  
3          NaN         NaN  
4          NaN         NaN  
..         ...         ...  
61         NaN         NaN  
62         NaN         NaN  
63     11529.0         NaN  
64  34851364.0  33029501.0  
65  34851364.0  33029501.0  

[66 rows x 100 columns]

In [279]:
isinstance(dfWideV2['매출회사'], pd.DataFrame)

True

In [276]:
len(dfWideV2['매출회사'].is

1

In [274]:
dfWideV2['매출회사']

0           ㈜지에스
1        지에스에너지㈜
2         지에스파워㈜
3        지에스나노텍㈜
4       살데비다코리아㈜
         ...    
61     ㈜영덕제1풍력발전
62    ㈜삼척그린에너지파크
63         ㈜프로케어
64            소계
65            합계
Name: 매출회사, Length: 66, dtype: object

In [273]:
# if there are multiple columns with the same name '매출회사' drop the others except the first one
srSales = dfWideV2['매출회사'].iloc[:,0]
dfWideV2 = dfWideV2.drop(columns=['매출회사'])
dfWideV2['매출회사'] = srSales

IndexingError: Too many indexers

# 한화

In [260]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('한화', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사         매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)      국내매출액  \
0          한화        한화케미칼  17813.0       317992       298664  2953044.0   
1          한화       한화갤러리아     1664       317992       298664  2953044.0   
2          한화   한화갤러리아타임월드   1258.0       317992       298664  2953044.0   
3          한화       한화컴파운드   1392.0       317992       298664  2953044.0   
4          한화        한화비앤비     13.0       317992       298664  2953044.0   
..        ...          ...      ...          ...          ...        ...   
782    한화투자증권       한화정밀기계     20.0         1025            0  1875074.0   
783    한화투자증권       한화생명보험    110.0         1025            0  1875074.0   
784    한화투자증권       한화손해보험     35.0         1025            0  1875074.0   
785    한화투자증권  해외계열사계매출액한화        0         1025            0  1875074.0   
786  드림플러스아시아  해외계열사계매출액한화        0            0            0      360.0   

         해외매출액  
0    2271204.0  
1    2271204.0  
2    2271204.0  
3    2271204.0  
4    2271204.0  
..         ...  
782        NaN  
783        NaN  
784        NaN  
785        NaN  
786        NaN  

[787 rows x 7 columns]

In [258]:
dfLong = api.getTransTable('한화', 2020)
dfLong

매출회사        매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)      국내매출액    해외매출액
0        한화       한화솔루션  26971.0       329259          NaN  2742051.0  1691112
1        한화      한화갤러리아   1469.0       329259          NaN  2742051.0  1691112
2        한화  한화갤러리아타임월드   1147.0       329259          NaN  2742051.0  1691112
3        한화      한화컴파운드   3240.0       329259          NaN  2742051.0  1691112
4        한화       한화비앤비     10.0       329259          NaN  2742051.0  1691112
..      ...         ...      ...          ...          ...        ...      ...
594  한화투자증권        한화건설      7.0          822          0.0  1547499.0      NaN
595  한화투자증권   한화에어로스페이스    150.0          822          0.0  1547499.0      NaN
596  한화투자증권      한화저축은행     46.0          822          0.0  1547499.0      NaN
597  한화투자증권      한화생명보험    452.0          822          0.0  1547499.0      NaN
598  캐롯손해보험      한화생명보험     13.0           13          0.0      725.0        0

[599 rows x 7 columns]

In [259]:
dfLong = api.getTransTable('한화', 2021)
dfLong

매출회사        매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액    해외매출액
0        한화       한화솔루션  66995.0       274472          NaN  2624121  1376654
1        한화      한화갤러리아   1122.0       274472          NaN  2624121  1376654
2        한화  한화갤러리아타임월드    642.0       274472          NaN  2624121  1376654
3        한화      한화컴파운드   2738.0       274472          NaN  2624121  1376654
4        한화       한화비앤비      3.0       274472          NaN  2624121  1376654
..      ...         ...      ...          ...          ...      ...      ...
543  캐롯손해보험        한화역사      0.4         1248          NaN    39584        0
544  캐롯손해보험   한화호텔앤드리조트    200.0         1248          NaN    39584        0
545  캐롯손해보험       한화에너지    146.0         1248          NaN    39584        0
546  캐롯손해보험      한화생명보험    178.0         1248          NaN    39584        0
547  캐롯손해보험      한화투자증권     11.0         1248          NaN    39584        0

[548 rows x 7 columns]

In [250]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfWide = api.getTransTableTest('한화', 2019)
dfWide

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
  매입자 ＼ 매출회사                     국내 계열회사                                  \
  매입자 ＼ 매출회사                       비금융회사                                   
  매입자 ＼ 매출회사      매입자 ＼ 매출회사.1       ㈜한화   한화케미칼㈜ ㈜한화 갤러리아 ㈜한화 갤러리아 타임월드   
0  비 금 융 회 사             (주)한화       NaN  17813.0     1664        1258.0   
1  비 금 융 회 사          한화케미칼(주)  110015.0      NaN      395           0.0   
2  비 금 융 회 사         (주)한화갤러리아    3203.0    990.0        -       35237.0   
3  비 금 융 회 사     (주)한화갤러리아타임월드       0.0     10.0     3118           0.0   
4  비 금 융 회 사  한화케미칼 오버시즈홀딩스(주)       NaN      NaN      NaN           NaN   

                                                                       \
                                                                        
  한화케미칼 오버시즈 홀딩스(주)  한화컴파운드㈜ (주)한화 비앤비   한화큐셀㈜ (주)한화 도시개발 (주)아산 테크노밸리   
0               NaN   1392.0    

(매입자 ＼ 매출회사, 매입자 ＼ 매출회사, 매입자 ＼ 매출회사)  \
0                             비 금 융 회 사   
1                             비 금 융 회 사   
2                             비 금 융 회 사   
3                             비 금 융 회 사   
4                             비 금 융 회 사   
..                                  ...   
73                              금 융 회 사   
74                              금 융 회 사   
75                              금 융 회 사   
76                              금 융 회 사   
77                                   합계   

   (매입자 ＼ 매출회사, 매입자 ＼ 매출회사, 매입자 ＼ 매출회사.1)  (국내 계열회사, 비금융회사, ㈜한화)  \
0                                   (주)한화                    NaN   
1                                한화케미칼(주)               110015.0   
2                               (주)한화갤러리아                 3203.0   
3                           (주)한화갤러리아타임월드                    0.0   
4                        한화케미칼 오버시즈홀딩스(주)                    NaN   
..                                    ...                    ...   
73                              한화인베스트먼트㈜                    NaN   
74                                한화투자증권㈜                    NaN   
75                            드림플러스아시아(유)                    NaN   
76                                     소계                14832.0   
77                                     합계               379899.0   

    (국내 계열회사, 비금융회사, 한화케미칼㈜) (국내 계열회사, 비금융회사, ㈜한화 갤러리아)  \
0                    17813.0                       1664   
1                        NaN                        395   
2                      990.0                          -   
3                       10.0                       3118   
4                        NaN                        NaN   
..                       ...                        ...   
73                       NaN                        NaN   
74                       NaN                          1   
75                       NaN                        NaN   
76                   12311.0                       2906   
77                  361052.0                      72964   

    (국내 계열회사, 비금융회사, ㈜한화 갤러리아 타임월드) (국내 계열회사, 비금융회사, 한화케미칼 오버시즈 홀딩스(주))  \
0                            1258.0                                 NaN   
1                               0.0                                 NaN   
2                           35237.0                                   -   
3                               0.0                                   0   
4                               NaN                                 NaN   
..                              ...                                 ...   
73                              NaN                                 NaN   
74                              NaN                                 NaN   
75                              NaN                                 NaN   
76                           9457.0                                   0   
77                          54340.0                                   0   

    (국내 계열회사, 비금융회사, 한화컴파운드㈜)  (국내 계열회사, 비금융회사, (주)한화 비앤비)  \
0                      1392.0                         13.0   
1                     49100.0                          0.0   
2                        71.0                        408.0   
3                         0.0                        128.0   
4                         NaN                          NaN   
..                        ...                          ...   
73                        NaN                          NaN   
74                        NaN                          NaN   
75                        NaN                          NaN   
76                      217.0                        183.0   
77                    64126.0                       4205.0   

    (국내 계열회사, 비금융회사, 한화큐셀㈜)  ...  (해외 계열회사, 한화화학(상해)유한공사)  \
0                    1532.0  ...                      1.0   
1                     888.0  ...                      NaN   
2                      41.0  ...                      NaN   
3                       0.0  ...                      NaN   
4        

In [248]:
dfWideV2 = dfWideT[~dfWideT[0].replace(' ', '', regex=True).isin(['금융회사', '비금융회사'])].T
dfWideV2

(매입자 ＼ 매출회사, 매입자 ＼ 매출회사, 매입자 ＼ 매출회사.1) (국내 계열회사, 비금융회사, ㈜한화)  \
0                                   (주)한화                   NaN   
1                                한화케미칼(주)              110015.0   
2                               (주)한화갤러리아                3203.0   
3                           (주)한화갤러리아타임월드                   0.0   
4                        한화케미칼 오버시즈홀딩스(주)                   NaN   
..                                    ...                   ...   
73                              한화인베스트먼트㈜                   NaN   
74                                한화투자증권㈜                   NaN   
75                            드림플러스아시아(유)                   NaN   
76                                     소계               14832.0   
77                                     합계              379899.0   

   (국내 계열회사, 비금융회사, 한화케미칼㈜) (국내 계열회사, 비금융회사, ㈜한화 갤러리아)  \
0                   17813.0                       1664   
1                       NaN                        395   
2                     990.0                          -   
3                      10.0                       3118   
4                       NaN                        NaN   
..                      ...                        ...   
73                      NaN                        NaN   
74                      NaN                          1   
75                      NaN                        NaN   
76                  12311.0                       2906   
77                 361052.0                      72964   

   (국내 계열회사, 비금융회사, ㈜한화 갤러리아 타임월드) (국내 계열회사, 비금융회사, 한화케미칼 오버시즈 홀딩스(주))  \
0                           1258.0                                 NaN   
1                              0.0                                 NaN   
2                          35237.0                                   -   
3                              0.0                                   0   
4                              NaN                                 NaN   
..                             ...                                 ...   
73                             NaN                                 NaN   
74                             NaN                                 NaN   
75                             NaN                                 NaN   
76                          9457.0                                   0   
77                         54340.0                                   0   

   (국내 계열회사, 비금융회사, 한화컴파운드㈜) (국내 계열회사, 비금융회사, (주)한화 비앤비)  \
0                     1392.0                        13.0   
1                    49100.0                         0.0   
2                       71.0                       408.0   
3                        0.0                       128.0   
4                        NaN                         NaN   
..                       ...                         ...   
73                       NaN                         NaN   
74                       NaN                         NaN   
75                       NaN                         NaN   
76                     217.0                       183.0   
77                   64126.0                      4205.0   

   (국내 계열회사, 비금융회사, 한화큐셀㈜) (국내 계열회사, 비금융회사, (주)한화 도시개발)  ...  \
0                   1532.0                         50.0  ...   
1                    888.0                          NaN  ...   
2                     41.0                         27.0  ...   
3                      0.0                          0.0  ...   
4                      NaN                          NaN  ...   
..                     ...                          ...  ...   
73                     NaN                          NaN  ...   
74                     NaN                          NaN  ...   
75                     NaN                          NaN  ...   
76                  1081.0                        190.0  ...   
77                  4744.0                       1064.0  ...   

   (해외 계열회사, 한화화학(상해)유한공사) (해외 계열회사, Hanwha Resources (USA) Corp.)  \
0                      1.0                          

# 롯데

In [199]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('롯데지주', 2019)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사         매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액 해외매출액
0         롯데지주㈜         호텔롯데  12284      167669         NaN  255644  7296
1         롯데지주㈜      롯데면세점제주   1140      167669         NaN  255644  7296
2         롯데지주㈜       부산롯데호텔   1073      167669         NaN  255644  7296
3         롯데지주㈜         롯데물산    913      167669         NaN  255644  7296
4         롯데지주㈜  (구)롯데디에프글로벌      1      167669         NaN  255644  7296
...         ...          ...    ...         ...         ...     ...   ...
1707     ㈜한페이시스         이비카드   3121        4060         NaN    7562     0
1708     ㈜한페이시스          마이비    120        4060         NaN    7562     0
1709  롯데액셀러레이터㈜       롯데칠성음료     37         142         NaN     917     0
1710  롯데액셀러레이터㈜        롯데멤버스     70         142         NaN     917     0
1711  롯데액셀러레이터㈜       롯데정보통신     35         142         NaN     917     0

[1712 rows x 7 columns]

In [200]:
dfLong = api.getTransTable('롯데지주', 2020)
dfLong

매출회사     매입회사    매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액 해외매출액
0         롯데지주㈜     호텔롯데  13286      162481         322  388658  1053
1         롯데지주㈜  롯데면세점제주   1133      162481         322  388658  1053
2         롯데지주㈜   부산롯데호텔    944      162481         322  388658  1053
3         롯데지주㈜     롯데물산   1026      162481         322  388658  1053
4         롯데지주㈜    스마일위드      3      162481         322  388658  1053
...         ...      ...    ...         ...         ...     ...   ...
1510    롯데오토리스㈜  (구)롯데카드     14        7046         NaN  133867     0
1511  롯데액셀러레이터㈜     롯데지주     51         165         NaN    1661     0
1512  롯데액셀러레이터㈜   롯데칠성음료     44         165         NaN    1661     0
1513  롯데액셀러레이터㈜     롯데쇼핑     35         165         NaN    1661     0
1514  롯데액셀러레이터㈜   롯데정보통신     35         165         NaN    1661     0

[1515 rows x 7 columns]

In [201]:
dfLong = api.getTransTable('롯데지주', 2021)
dfLong

Null value found at (nan, '금융회사', '롯데캐피탈㈜').Null value found at (nan, '금융회사', '롯데오토리스㈜').Null value found at (nan, '금융회사', '롯데액셀러레이터㈜').Null value found at (nan, '금융회사', '소계').Null value found at (nan, '국내계열사 계 (매출액)', '국내계열사 계 (매출액)').

매출회사     매입회사   매출액 국내계열사계(매출액) 해외계열사계(매출액)   국내매출액 해외매출액
0         롯데지주㈜     호텔롯데  9006      134945         177  290321   177
1         롯데지주㈜  롯데면세점제주   317      134945         177  290321   177
2         롯데지주㈜   부산롯데호텔   438      134945         177  290321   177
3         롯데지주㈜     롯데물산   983      134945         177  290321   177
4         롯데지주㈜    스마일위드     6      134945         177  290321   177
...         ...      ...   ...         ...         ...     ...   ...
1329    롯데오토리스㈜   롯데정보통신     7       13756         NaN  123517     0
1330  롯데액셀러레이터㈜     롯데제과     3          54         NaN    2534     0
1331  롯데액셀러레이터㈜   롯데칠성음료     7          54         NaN    2534     0
1332  롯데액셀러레이터㈜     롯데쇼핑    35          54         NaN    2534     0
1333  롯데액셀러레이터㈜   롯데정보통신     8          54         NaN    2534     0

[1334 rows x 7 columns]

# LG

In [186]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfLong = api.getTransTable('LG', 2021)
dfLong

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사      매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액  \
0             (주)엘지      엘지화학  75584.0       396649         19.0  1016183   
1             (주)엘지  엘지에너지솔루션   2058.0       396649         19.0  1016183   
2             (주)엘지    엘지하우시스  12756.0       396649         19.0  1016183   
3             (주)엘지   엘지토스템비엠      0.0       396649         19.0  1016183   
4             (주)엘지    엘지생활건강  17216.0       396649         19.0  1016183   
..              ...       ...      ...          ...          ...      ...   
724  (주)판토스부산신항물류센터    엘지생활건강      4.0         8167          0.0    11368   
725  (주)판토스부산신항물류센터      엘지전자     31.0         8167          0.0    11368   
726  (주)판토스부산신항물류센터       판토스   8100.0         8167          0.0    11368   
727       (주)헬리스타항공       판토스  20986.0        20986       1458.0    62373   
728           ㈜한울타리       판토스    632.0          632          0.0      632   

    해외매출액  
0      19  
1      19  
2      19  
3      19  
4      19  
..    ...  
724     9  
725     9  
726     9  
727  1458  
728     0  

[729 rows x 7 columns]

In [24]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfWide = api.getTransTable('LG', 2021)
dfWide

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사      매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액 해외매출액
0             엘지      엘지화학  75584.0       396649         19.0  1016183    19
1             엘지  엘지에너지솔루션   2058.0       396649         19.0  1016183    19
2             엘지    엘지하우시스  12756.0       396649         19.0  1016183    19
3             엘지   엘지토스템비엠      0.0       396649         19.0  1016183    19
4             엘지    엘지생활건강  17216.0       396649         19.0  1016183    19
..           ...       ...      ...          ...          ...      ...   ...
724  판토스부산신항물류센터    엘지생활건강      4.0         8167          0.0    11368     9
725  판토스부산신항물류센터      엘지전자     31.0         8167          0.0    11368     9
726  판토스부산신항물류센터       판토스   8100.0         8167          0.0    11368     9
727       헬리스타항공       판토스  20986.0        20986       1458.0    62373  1458
728         한울타리       판토스    632.0          632          0.0      632     0

[729 rows x 7 columns]

In [25]:
dfWide = api.getTransTable('LG', 2020)
dfWide

매출회사     매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액  해외매출액
0             엘지     엘지화학  73171.0     382224.0         18.0  874654   18.0
1             엘지   엘지하우시스   6773.0     382224.0         18.0  874654   18.0
2             엘지  엘지토스템비엠      1.0     382224.0         18.0  874654   18.0
3             엘지   엘지생활건강  16922.0     382224.0         18.0  874654   18.0
4             엘지    더페이스샵     59.0     382224.0         18.0  874654   18.0
..           ...      ...      ...          ...          ...     ...    ...
701  판토스부산신항물류센터   엘지생활건강      4.0       5486.0          0.0    9096   22.0
702  판토스부산신항물류센터     엘지전자     72.0       5486.0          0.0    9096   22.0
703  판토스부산신항물류센터      판토스     5312       5486.0          0.0    9096   22.0
704       헬리스타항공      판토스    27816      27816.0          0.0   39472    0.0
705         한울타리      판토스      449        449.0          0.0     449    0.0

[706 rows x 7 columns]

In [26]:
dfWide = api.getTransTable('LG', 2019)
dfWide

매출회사     매입회사      매출액  국내계열사계(매출액) 해외계열사계(매출액)     국내매출액 해외매출액
0             엘지     엘지화학  69969.0       372226          16  757180.0    16
1             엘지   엘지하우시스   6644.0       372226          16  757180.0    16
2             엘지  엘지토스템비엠     24.0       372226          16  757180.0    16
3             엘지   엘지생활건강  16283.0       372226          16  757180.0    16
4             엘지    더페이스샵     65.0       372226          16  757180.0    16
..           ...      ...      ...          ...         ...       ...   ...
661  판토스부산신항물류센터    더페이스샵      5.0         4030         NaN    7540.0    15
662  판토스부산신항물류센터     엘지전자     31.0         4030         NaN    7540.0    15
663  판토스부산신항물류센터      판토스   3880.0         4030         NaN    7540.0    15
664       헬리스타항공      판토스  29677.0        29677         NaN   40094.0     0
665         한울타리      판토스    209.0          209         NaN     209.0   NaN

[666 rows x 7 columns]

# SK

In [202]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfWide = api.getTransTable('SK', 2019)
dfWide

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사       매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액   해외매출액
0      에스케이㈜    대한송유관공사     276     1403811       77406  2829872  163941
1      에스케이㈜     부산도시가스    2147     1403811       77406  2829872  163941
2      에스케이㈜      에스엠코어      56     1403811       77406  2829872  163941
3      에스케이㈜     에이디티캡스      69     1403811       77406  2829872  163941
4      에스케이㈜     엔에스오케이     153     1403811       77406  2829872  163941
...      ...        ...     ...         ...         ...      ...     ...
1471   행복모아㈜   에스케이하이닉스    8360        8360         NaN     8360     NaN
1472  홈앤서비스㈜     에이디티캡스      62      325168         NaN   325177     NaN
1473  홈앤서비스㈜     엔에스오케이    1725      325168         NaN   325177     NaN
1474  홈앤서비스㈜     에스케이매직      75      325168         NaN   325177     NaN
1475  홈앤서비스㈜  에스케이브로드밴드  323306      325168         NaN   325177     NaN

[1476 rows x 7 columns]

In [203]:
dfWide = api.getTransTable('SK', 2020)
dfWide

매출회사            매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액   해외매출액
0      에스케이㈜         대한송유관공사     506     1401318       85604  3023603  221913
1      에스케이㈜  드림어스컴퍼니(舊아이리버)      76     1401318       85604  3023603  221913
2      에스케이㈜          부산도시가스    2012     1401318       85604  3023603  221913
3      에스케이㈜           에스엠코어     271     1401318       85604  3023603  221913
4      에스케이㈜          에이디티캡스    2926     1401318       85604  3023603  221913
...      ...             ...     ...         ...         ...      ...     ...
1611   행복한울㈜         에스케이텔레콤     252         252         NaN      252     NaN
1612  홈앤서비스㈜          에이디티캡스     354      351112         NaN   351154     NaN
1613  홈앤서비스㈜          에스케이매직      21      351112         NaN   351154     NaN
1614  홈앤서비스㈜       에스케이브로드밴드  350727      351112         NaN   351154     NaN
1615  홈앤서비스㈜             유빈스      10      351112         NaN   351154     NaN

[1616 rows x 7 columns]

In [204]:
dfWide = api.getTransTable('SK', 2021)
dfWide

매출회사               매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)    국내매출액  \
0      에스케이㈜            대한송유관공사    2174     1451273      101150  3365216   
1      에스케이㈜            드림어스컴퍼니     129     1451273      101150  3365216   
2      에스케이㈜             부산도시가스    1957     1451273      101150  3365216   
3      에스케이㈜              에스엠코어     150     1451273      101150  3365216   
4      에스케이㈜  에이디티캡스(舊.에스케이인포섹)    3419     1451273      101150  3365216   
...      ...                ...     ...         ...         ...      ...   
1525  홈앤서비스㈜             에스케이매직      19      397743         NaN   397754   
1526  홈앤서비스㈜          에스케이브로드밴드  390467      397743         NaN   397754   
1527  홈앤서비스㈜            에스케이텔레콤       2      397743         NaN   397754   
1528  홈앤서비스㈜                유빈스      11      397743         NaN   397754   
1529  홈앤서비스㈜            舊에이디티캡스    1083      397743         NaN   397754   

       해외매출액  
0     108768  
1     108768  
2     108768  
3     108768  
4     108768  
...      ...  
1525     NaN  
1526     NaN  
1527     NaN  
1528     NaN  
1529     NaN  

[1530 rows x 7 columns]

# 삼성

In [83]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
api = Api(apiKey)
dfWide = api.getTransTable('삼성전자', 2019)
dfWide

No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml


매출회사            매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)     국내매출액  \
0         삼성전자            삼성물산   78676     2269950   149436938  16821275   
1         삼성전자        삼성바이오에피스       1     2269950   149436938  16821275   
2         삼성전자         삼성디스플레이  164404     2269950   149436938  16821275   
3         삼성전자             세메스       5     2269950   149436938  16821275   
4         삼성전자         삼성전자서비스  456109     2269950   149436938  16821275   
..         ...             ...     ...         ...         ...       ...   
827       삼성카드  삼성생명금융서비스보험대리점      30      141801          13   3253706   
828       삼성카드            삼성증권    1187      141801          13   3253706   
829       삼성카드       삼성카드고객서비스    1244      141801          13   3253706   
830       삼성카드          삼성벤처투자     274      141801          13   3253706   
831  삼성카드고객서비스            삼성카드  114361      114361         NaN    114361   

         해외매출액  
0    153560595  
1    153560595  
2    153560595  
3    153560595  
4    153560595  
..         ...  
827        NaN  
828        NaN  
829        NaN  
830        NaN  
831        NaN  

[832 rows x 7 columns]

In [84]:
dfWide = api.getTransTable('삼성전자', 2020)
dfWide

매출회사            매입회사     매출액 국내계열사계(매출액) 해외계열사계(매출액)     국내매출액  \
0         삼성전자            삼성물산  110238     2422149   130563681  20300939   
1         삼성전자        삼성바이오에피스       4     2422149   130563681  20300939   
2         삼성전자         삼성디스플레이  279363     2422149   130563681  20300939   
3         삼성전자             세메스       7     2422149   130563681  20300939   
4         삼성전자         삼성전자서비스  453084     2422149   130563681  20300939   
..         ...             ...     ...         ...         ...       ...   
825       삼성카드  삼성생명금융서비스보험대리점      24      137178          13   3218275   
826       삼성카드            삼성증권    1121      137178          13   3218275   
827       삼성카드       삼성카드고객서비스    1139      137178          13   3218275   
828       삼성카드          삼성벤처투자     177      137178          13   3218275   
829  삼성카드고객서비스            삼성카드  112011      112011         NaN    112063   

         해외매출액  
0    134471920  
1    134471920  
2    134471920  
3    134471920  
4    134471920  
..         ...  
825        NaN  
826        NaN  
827        NaN  
828        NaN  
829        NaN  

[830 rows x 7 columns]

In [85]:
dfWide = api.getTransTable('삼성전자', 2021)
dfWide

매출회사            매입회사      매출액 국내계열사계(매출액) 해외계열사계(매출액)     국내매출액  \
0         삼성전자            삼성물산    86705     3142513   141902290  19833135   
1         삼성전자        삼성바이오에피스        0     3142513   141902290  19833135   
2         삼성전자         삼성디스플레이   196823     3142513   141902290  19833135   
3         삼성전자         삼성전자서비스   465778     3142513   141902290  19833135   
4         삼성전자          삼성전자판매  2225711     3142513   141902290  19833135   
..         ...             ...      ...         ...         ...       ...   
823       삼성카드  삼성생명금융서비스보험대리점       44      143713          30   3233170   
824       삼성카드            삼성증권      718      143713          30   3233170   
825       삼성카드       삼성카드고객서비스     1094      143713          30   3233170   
826       삼성카드          삼성벤처투자      215      143713          30   3233170   
827  삼성카드고객서비스            삼성카드   112053      112053         NaN    112139   

         해외매출액  
0    146478056  
1    146478056  
2    146478056  
3    146478056  
4    146478056  
..         ...  
823        NaN  
824        NaN  
825        NaN  
826        NaN  
827        NaN  

[828 rows x 7 columns]

# 현대자동차

In [86]:
dfWide = api.getTransTable('현대자동차', 2019)
dfWide

매출회사    매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액       해외매출액
0    현대자동차   기아자동차  705647.0    1996769.0   15510199.0  21329921  21830188.0
1    현대자동차    현대제철   14034.0    1996769.0   15510199.0  21329921  21830188.0
2    현대자동차   현대모비스  214836.0    1996769.0   15510199.0  21329921  21830188.0
3    현대자동차    현대건설     172.0    1996769.0   15510199.0  21329921  21830188.0
4    현대자동차  현대글로비스   42838.0    1996769.0   15510199.0  21329921  21830188.0
..     ...     ...       ...          ...          ...       ...         ...
490  현대차증권   현대파워텍      61.0      12922.0          NaN    592807         NaN
491  현대차증권   현대캐피탈       0.0      12922.0          NaN    592807         NaN
492  현대차증권    현대카드      20.0      12922.0          NaN    592807         NaN
493   블루월넛   현대모비스      46.0        814.0          NaN      3999         NaN
494   블루월넛    현대카드     768.0        814.0          NaN      3999         NaN

[495 rows x 7 columns]

In [87]:
dfWide = api.getTransTable('현대자동차', 2020)
dfWide

매출회사    매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액       해외매출액
0    현대자동차   기아자동차  498823.0    1891132.0   20450440.0  22861342  26294351.0
1    현대자동차    현대제철      5254    1891132.0   20450440.0  22861342  26294351.0
2    현대자동차   현대모비스  251993.0    1891132.0   20450440.0  22861342  26294351.0
3    현대자동차    현대건설    1160.0    1891132.0   20450440.0  22861342  26294351.0
4    현대자동차  현대글로비스   88262.0    1891132.0   20450440.0  22861342  26294351.0
..     ...     ...       ...          ...          ...       ...         ...
478   블루월넛   기아자동차   19846.0      23470.0          NaN     27474         NaN
479   블루월넛   현대모비스      52.0      23470.0          NaN     27474         NaN
480   블루월넛  현대오토에버    1014.0      23470.0          NaN     27474         NaN
481   블루월넛   현대캐피탈      15.0      23470.0          NaN     27474         NaN
482   블루월넛    현대카드    2256.0      23470.0          NaN     27474         NaN

[483 rows x 7 columns]

In [88]:
dfWide = api.getTransTable('현대자동차', 2019)
dfWide

매출회사    매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액       해외매출액
0    현대자동차   기아자동차  705647.0    1996769.0   15510199.0  21329921  21830188.0
1    현대자동차    현대제철   14034.0    1996769.0   15510199.0  21329921  21830188.0
2    현대자동차   현대모비스  214836.0    1996769.0   15510199.0  21329921  21830188.0
3    현대자동차    현대건설     172.0    1996769.0   15510199.0  21329921  21830188.0
4    현대자동차  현대글로비스   42838.0    1996769.0   15510199.0  21329921  21830188.0
..     ...     ...       ...          ...          ...       ...         ...
490  현대차증권   현대파워텍      61.0      12922.0          NaN    592807         NaN
491  현대차증권   현대캐피탈       0.0      12922.0          NaN    592807         NaN
492  현대차증권    현대카드      20.0      12922.0          NaN    592807         NaN
493   블루월넛   현대모비스      46.0        814.0          NaN      3999         NaN
494   블루월넛    현대카드     768.0        814.0          NaN      3999         NaN

[495 rows x 7 columns]